In [1]:
#Import Modules

import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils import column_index_from_string
import re

In [2]:
#Load files. The file names must match exactly the names of the application data files. Manual Input Required.

term_1_file_application_comparative = "Application Comparative-Summer 2025 as of May 16, 2025.xlsx"
term_1_file_application_details = "Applications Reports Details-IR-New-Summer 2025 as of May 01, 2025 (Fake Data).xlsx"

term_2_file_application_comparative = "Application Comparative-Fall 2025 as of May 16, 2025.xlsx"
term_2_file_application_details = "Applications Reports Details-IR-New-Fall 2025 as of May 01, 2025 (Fake Data).xlsx"

term_3_file_application_comparative = "Application Comparative-Winter 2026 as of May 16, 2025.xlsx"
term_3_file_application_details = "Applications Reports Details-IR-New-Winter 2026 as of May 01, 2025 (Fake Data).xlsx"

term_4_file_application_comparative = "Application Comparative-Spring 2026 as of May 16, 2025.xlsx"
term_4_file_application_details = "Applications Reports Details-IR-New-Spring 2026 as of May 01, 2025 (Fake Data).xlsx"

variable_Extraction_date = "May 16, 2025" #Enter date of data extraction. Format: "Month DD, YYYY"
variable_Current_applications_academic_year = "2025-26" #Enter the current academic year. Format: "YYYY-YY"

In [3]:
#Course Code List (must match all courses on template, using course codes in Power Campus)

#List all Business Administration courses on template
course_ID_Business_Administration = [
    'BABUSC',
    'BAACTD',
    'BABUSD',
    'BAESPM',
    'BAHRMD',
    'GSCMD',
    'DIGMD',
    'BAATCO',
    'BABSCO',
    'NAO',
    'ALO',
    'HOSMNG',
    'BAHRMC',
    'OFAC'
]

#List all Education and Academic Upgrading courses on template
course_ID_Education_and_Academic_Upgrading = [
    'EAP',
    'HSAFA',
    'HSAD',
    'HSGD',
    'HSNP',
    'UTELEM',
    'UTSEC'
]

#List all University Studies and Environmental Science courses on template
course_ID_University_Studies_and_Environmental_Science = [
    'ENVTD',
    'ENVNC',
    'UTARTS',
    'UTCOMM',
    'UTENG',
    'UTSCGS',
    'UTSCWK',
    'OPEN',
    'GOVNSC',
    'COMSCC',
    'COMSCD',
    'GNARTC',
    'GNSCIC',
    'ANCPDC',
    'NSPDC'
]

#List all Language Instruction for Newcomers to Canada courses on template
course_ID_Language_Instruction_for_Newcomers_to_Canada = [
    'LINC13'
]

#List all Childhood Studies and Human Services courses on template
course_ID_Childhood_Studies_and_Human_Services = [
    'AELCC',
    'ECCERT',
    'ECDIP',
    'CYCD',
    'ECEAC',
    'SOCWKD'
]

#List all Nursing and Allied Health courses on template
course_ID_Nursing_and_Allied_Health = [
    'HCAC',
    'PCPC',
    'ACP',
    'PNR',
    'BSCN',
    'BSCACN',
    'BSCAEE',
    'BSCAFT',
    'IENCN'
]

#List all Pre-employment courses on template
course_ID_Pre_employment = [
    'PAELEC',
    'PAWELD',
    'PAHET',
    'PAPLUM'
]

#List all Power Engineering courses on template
course_ID_Power_Engineering = [
    'POWCM4',
    'POWCM3',
    'POWCP4',
    'POWCO4',
    'POWCO3',
    'PPET'
]


In [4]:
#Customized Math Formula Functions

#Function for subtracting current_year_value by previous_year_value, then dividing by previous_year_value, then multiplying by 100, rounding to 1 decimal place
def percent_difference_value_calculator_round_1(current_year_value, previous_year_value):
    try:
        return round((current_year_value - previous_year_value) / previous_year_value * 100, 1)
    except ZeroDivisionError:
        return "N/A"
    except TypeError:
        return "N/A"
    
#Function for dividing numerator by denominator, then multiplying by 100, rounding to 1 decimal place
def division_calculator(numerator, denominator): 
    try:
        return round(numerator / denominator * 100, 1)
    except ZeroDivisionError:
        return "N/A"
    except TypeError:
        return "N/A"


In [5]:
#Data Check (Will add later to ensure data is properly cleaned)

In [6]:
#Application Comparative Term 1
# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)  # Replace with your actual file name
ws = wb.active  # Use ws = wb["SheetName"] if known

# Define regex pattern to find the term and year (handles cases like "Spring2024" or "Spring 2024")
pattern = r"\b(Summer|Fall|Winter|Spring)\s*(\d{4})\b"  # Now correctly separates term & year

# Initialize term variable
term_1_term_year_current = None

# Iterate through all cells in the worksheet
for row in ws.iter_rows():
    for cell in row:
        if isinstance(cell.value, str):  # Ensure the cell contains text
            ##print(f"Checking cell: {cell.coordinate}, Value: '{cell.value}'")  # Debugging: print each cell
            
            match = re.search(pattern, cell.value, re.IGNORECASE)  # Case-insensitive search
            if match:
                term_name = match.group(1).capitalize()  # Capitalize first letter (e.g., "Spring")
                term_year = match.group(2)  # Extract the year (e.g., "2024")

                term_1_term_year_current = f"{term_name} {term_year}"  # Format as "Spring 2024"
                ##print(f"Match found: {match.group(0)}")  # Debug print
                break  # Stop searching once found
    if term_1_term_year_current:
        break  # Stop outer loop if term is found

#create variables for term year and years
term_1_year_current = int(re.search(r"\d{4}", term_1_term_year_current).group())
term_1_year_minus_one = term_1_year_current - 1
term_1_year_minus_two = term_1_year_current - 2
term_1_year_minus_three = term_1_year_current - 3

#Extract location of 'Report Summary'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_1_Report_Summary_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "Report Summary"
    if top_left_cell.value == "Report Summary":
        term_1_Report_Summary_range = merged_range
        ##print(f"Merged cell containing 'Report Summary' spans: {term_1_Report_Summary_range}")

# Convert merged range to string
range_str_Report_Summary = str(term_1_Report_Summary_range)  # "A45:AA45"

# Extract the first part before the number
term_1_Report_Summary_first_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[0]))

##print("First column:", term_1_Report_Summary_first_column)  # Output: "A"

# Convert merged range to string
range_str_Report_Summary = str(term_1_Report_Summary_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_Report_Summary_last_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[1]))

##print("Last column:", term_1_Report_Summary_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Report_Summary = str(term_1_Report_Summary_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Report_Summary.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_Report_Summary_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_1_Report_Summary_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Report_Summary = str(term_1_Report_Summary_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Report_Summary.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_Report_Summary_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_1_Report_Summary_last_row)
#Extract location of 'All Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_1_All_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "All Applicants":
        term_1_All_Applicants_range = merged_range
        ##print(f"Merged cell containing 'All Applicants' spans: {term_1_All_Applicants_range}")

# Convert merged range to string
range_str_All_Applicants = str(term_1_All_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_1_All_Applicants_first_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[0]))

##print("First column:", term_1_All_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_All_Applicants = str(term_1_All_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_All_Applicants_last_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[1]))

##print("Last column:", term_1_All_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_All_Applicants = str(term_1_All_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_All_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_All_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_1_All_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_All_Applicants = str(term_1_All_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_All_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_All_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_1_All_Applicants_last_row)
#Extract location of 'International Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_1_International_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "International Applicants":
        term_1_International_Applicants_range = merged_range
        #print(f"Merged cell containing 'International Applicants' spans: {term_1_International_Applicants_range}")

# Convert merged range to string
range_str_International_Applicants = str(term_1_International_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_1_International_Applicants_first_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[0]))

##print("First column:", term_1_International_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_International_Applicants = str(term_1_International_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_International_Applicants_last_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[1]))

##print("Last column:", term_1_International_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_International_Applicants = str(term_1_International_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_International_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_International_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_1_International_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_International_Applicants = str(term_1_International_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_International_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_International_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_1_International_Applicants_last_row)
#Extract location of 'Domestic Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_1_Domestic_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "Domestic Applicants":
        term_1_Domestic_Applicants_range = merged_range
        #print(f"Merged cell containing 'Domestic Applicants' spans: {term_1_Domestic_Applicants_range}")

# Convert merged range to string
range_str_Domestic_Applicants = str(term_1_Domestic_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_1_Domestic_Applicants_first_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[0]))

##print("First column:", term_1_Domestic_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_1_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_Domestic_Applicants_last_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[1]))

##print("Last column:", term_1_Domestic_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_1_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Domestic_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_Domestic_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_1_Domestic_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Domestic_Applicants = str(term_1_Domestic_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Domestic_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_Domestic_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_1_Domestic_Applicants_last_row)
#Extract Location of 2023 everything 

# Define your search boundaries using column letters
min_search_row = term_1_Report_Summary_last_row + 1
max_search_row = term_1_All_Applicants_first_row - 1
min_search_col_letter = term_1_Report_Summary_first_column  # Column A
max_search_col_letter = term_1_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_one_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_1_year_minus_one:
            term_1_year_minus_one_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_one_merged_range}")

# Now term_1_year_minus_one_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_1_year_minus_one_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_one_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_1_year_minus_one_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_one_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_1_year_minus_one_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_one_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_1_year_minus_one_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_1_year_minus_one_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_one_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_1_year_minus_one_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_1_year_minus_one_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_one_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_1_year_minus_one_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_one_Unique_merged_range}")

# Now term_1_year_minus_one_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_1_year_minus_one_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_one_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_1_year_minus_one_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_one_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_1_year_minus_one_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_one_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_one_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_1_year_minus_one_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_one_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_one_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_1_year_minus_one_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_one_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_1_year_minus_one_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_one_Total_Apps_merged_range}")

# Now term_1_year_minus_one_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_1_year_minus_one_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_one_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

#print("First column:", term_1_year_minus_one_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_one_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

#print("Last column:", term_1_year_minus_one_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_one_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_one_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_1_year_minus_one_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_one_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_one_Total_Apps_last_row)
#Extract Location of 2022 everything 

# Define your search boundaries using column letters
min_search_row = term_1_Report_Summary_last_row + 1
max_search_row = term_1_All_Applicants_first_row - 1
min_search_col_letter = term_1_Report_Summary_first_column  # Column A
max_search_col_letter = term_1_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_two_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_1_year_minus_two:
            term_1_year_minus_two_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_two_merged_range}")

# Now term_1_year_minus_two_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_1_year_minus_two_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_two_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_1_year_minus_two_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_two_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_1_year_minus_two_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_two_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_two_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_1_year_minus_two_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_two_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_two_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_1_year_minus_two_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_two_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_1_year_minus_two_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_two_Unique_merged_range}")

# Now term_1_year_minus_two_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_1_year_minus_two_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_two_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_1_year_minus_two_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_two_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_1_year_minus_two_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_two_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_two_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_1_year_minus_two_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_two_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_two_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_1_year_minus_two_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_two_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_1_year_minus_two_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_two_Total_Apps_merged_range}")

# Now term_1_year_minus_two_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_1_year_minus_two_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_two_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_1_year_minus_two_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_two_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_1_year_minus_two_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_two_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_two_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_1_year_minus_two_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_two_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_two_Total_Apps_last_row)
#Extract Location of 2021 everything 

# Define your search boundaries using column letters
min_search_row = term_1_Report_Summary_last_row + 1
max_search_row = term_1_All_Applicants_first_row - 1
min_search_col_letter = term_1_Report_Summary_first_column  # Column A
max_search_col_letter = term_1_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_three_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_1_year_minus_three:
            term_1_year_minus_three_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_three_merged_range}")

# Now term_1_year_minus_three_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_1_year_minus_three_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_three_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_1_year_minus_three_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_three_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_1_year_minus_three_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_three_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_three_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_1_year_minus_three_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_three_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_three_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_1_year_minus_three_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_three_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_1_year_minus_three_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_three_Unique_merged_range}")

# Now term_1_year_minus_three_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_1_year_minus_three_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_three_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_1_year_minus_three_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_three_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_1_year_minus_three_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_three_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_three_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_1_year_minus_three_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_three_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_three_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_1_year_minus_three_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_three_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_1_year_minus_three_Total_Apps_merged_range = merged_range
            ##print(f"Merged cell containing '2023' spans: {term_1_year_minus_three_Total_Apps_merged_range}")

# Now term_1_year_minus_three_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_1_year_minus_three_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_three_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

##print("First column:", term_1_year_minus_three_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_three_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

##print("Last column:", term_1_year_minus_three_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_three_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_1_year_minus_three_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_1_year_minus_three_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_three_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_1_year_minus_three_Total_Apps_last_row)
#All 2023 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_1_All_Applicants_first_row
max_search_row = term_1_All_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_one_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_one_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_one_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_1_International_Applicants_first_row
max_search_row = term_1_International_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_one_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_one_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_one_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_1_Domestic_Applicants_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_one_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_one_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_one_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_1_All_Applicants_first_row
max_search_row = term_1_All_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_one_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_one_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_one_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_1_International_Applicants_first_row
max_search_row = term_1_International_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_one_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_one_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_one_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_1_Domestic_Applicants_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_one_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_one_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_one_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_1_year_minus_one_Unique_All_Applicants,",term_1_year_minus_one_Unique_International_Applicants,",term_1_year_minus_one_Unique_Domestic_Applicants)
##print("Extracted Number:", term_1_year_minus_one_Total_Apps_All_Applicants,",term_1_year_minus_one_Total_Apps_International_Applicants,",term_1_year_minus_one_Total_Apps_Domestic_Applicants)


#All 2022 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_1_All_Applicants_first_row
max_search_row = term_1_All_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_two_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_two_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_two_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_1_International_Applicants_first_row
max_search_row = term_1_International_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_two_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_two_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_two_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_1_Domestic_Applicants_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_two_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_two_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_two_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_1_All_Applicants_first_row
max_search_row = term_1_All_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_two_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_two_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_two_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_1_International_Applicants_first_row
max_search_row = term_1_International_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_two_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_two_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_two_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_1_Domestic_Applicants_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_two_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_two_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_two_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_1_year_minus_two_Unique_All_Applicants,",term_1_year_minus_two_Unique_International_Applicants,",term_1_year_minus_two_Unique_Domestic_Applicants)
##print("Extracted Number:", term_1_year_minus_two_Total_Apps_All_Applicants,",term_1_year_minus_two_Total_Apps_International_Applicants,",term_1_year_minus_two_Total_Apps_Domestic_Applicants)


#All 2021 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_1_All_Applicants_first_row
max_search_row = term_1_All_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_three_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_three_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_three_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_1_International_Applicants_first_row
max_search_row = term_1_International_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_three_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_three_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_three_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_1_Domestic_Applicants_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_three_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_three_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_three_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_1_All_Applicants_first_row
max_search_row = term_1_All_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_three_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_three_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_three_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_1_International_Applicants_first_row
max_search_row = term_1_International_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_three_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_three_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_three_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK-------------------------------------------------------------------------------term_1--------------------3
# Define your search boundaries using column letters
min_search_row = term_1_Domestic_Applicants_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_three_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_three_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_three_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

#print("Extracted Number:", term_1_year_minus_three_Unique_All_Applicants,",term_1_year_minus_three_Unique_International_Applicants,",term_1_year_minus_three_Unique_Domestic_Applicants)
#print("Extracted Number:", term_1_year_minus_three_Total_Apps_All_Applicants,",term_1_year_minus_three_Total_Apps_International_Applicants,",term_1_year_minus_three_Total_Apps_Domestic_Applicants)

# Extract the term and year 
term_match = re.match(r"([A-Za-z]+)\s+(\d{4})", term_1_term_year_current)

if term_match:
    term_1_term = term_match.group(1)
    term_1_year_current = int(term_match.group(2))

    # Subtract one year
    term_1_year_minus_one = term_1_year_current - 1

    # Construct the previous term-year string
    term_1_term_year_minus_one = f"{term_1_term} {term_1_year_minus_one}"

#Variables created
print("Current Term and Year:", term_1_term_year_current)
print("Current Term and Year Minus 1:", term_1_term_year_minus_one)
print("\n")
print(f"{term_1_year_minus_one}, Unique, All Applicants:", term_1_year_minus_one_Unique_All_Applicants)
print(f"{term_1_year_minus_one}, Unique, International Applicants:", term_1_year_minus_one_Unique_International_Applicants)
print(f"{term_1_year_minus_one}, Unique, Domestic Applicants:", term_1_year_minus_one_Unique_Domestic_Applicants)
print(f"{term_1_year_minus_one}, Total Apps, All Applicants:", term_1_year_minus_one_Total_Apps_All_Applicants)
print(f"{term_1_year_minus_one}, Total Apps, International Applicants:", term_1_year_minus_one_Total_Apps_International_Applicants)
print(f"{term_1_year_minus_one}, Total Apps, Domestic Applicants:", term_1_year_minus_one_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_1_year_minus_two}, Unique, All Applicants:", term_1_year_minus_two_Unique_All_Applicants)
print(f"{term_1_year_minus_two}, Unique, International Applicants:", term_1_year_minus_two_Unique_International_Applicants)
print(f"{term_1_year_minus_two}, Unique, Domestic Applicants:", term_1_year_minus_two_Unique_Domestic_Applicants)
print(f"{term_1_year_minus_two}, Total Apps, All Applicants:", term_1_year_minus_two_Total_Apps_All_Applicants)
print(f"{term_1_year_minus_two}, Total Apps, International Applicants:", term_1_year_minus_two_Total_Apps_International_Applicants)
print(f"{term_1_year_minus_two}, Total Apps, Domestic Applicants:", term_1_year_minus_two_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_1_year_minus_three}, Unique, All Applicants:", term_1_year_minus_three_Unique_All_Applicants)
print(f"{term_1_year_minus_three}, Unique, International Applicants:", term_1_year_minus_three_Unique_International_Applicants)
print(f"{term_1_year_minus_three}, Unique, Domestic Applicants:", term_1_year_minus_three_Unique_Domestic_Applicants)
print(f"{term_1_year_minus_three}, Total Apps, All Applicants:", term_1_year_minus_three_Total_Apps_All_Applicants)
print(f"{term_1_year_minus_three}, Total Apps, International Applicants:", term_1_year_minus_three_Total_Apps_International_Applicants)
print(f"{term_1_year_minus_three}, Total Apps, Domestic Applicants:", term_1_year_minus_three_Total_Apps_Domestic_Applicants)
      


FileNotFoundError: [Errno 2] No such file or directory: 'Application Comparative-Summer 2025 as of May 16, 2025.xlsx'

In [ ]:
#Application Comparative Term 2
# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)  # Replace with your actual file name
ws = wb.active  # Use ws = wb["SheetName"] if known

# Define regex pattern to find the term and year (handles cases like "Spring2024" or "Spring 2024")
pattern = r"\b(Summer|Fall|Winter|Spring)\s*(\d{4})\b"  # Now correctly separates term & year

# Initialize term variable
term_2_term_year_current = None

# Iterate through all cells in the worksheet
for row in ws.iter_rows():
    for cell in row:
        if isinstance(cell.value, str):  # Ensure the cell contains text
            ##print(f"Checking cell: {cell.coordinate}, Value: '{cell.value}'")  # Debugging: print each cell
            
            match = re.search(pattern, cell.value, re.IGNORECASE)  # Case-insensitive search
            if match:
                term_name = match.group(1).capitalize()  # Capitalize first letter (e.g., "Spring")
                term_year = match.group(2)  # Extract the year (e.g., "2024")

                term_2_term_year_current = f"{term_name} {term_year}"  # Format as "Spring 2024"
                ##print(f"Match found: {match.group(0)}")  # Debug print
                break  # Stop searching once found
    if term_2_term_year_current:
        break  # Stop outer loop if term is found

#create variables for term year and years
term_2_year_current = int(re.search(r"\d{4}", term_2_term_year_current).group())
term_2_year_minus_one = term_2_year_current - 1
term_2_year_minus_two = term_2_year_current - 2
term_2_year_minus_three = term_2_year_current - 3

#Extract location of 'Report Summary'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_2_Report_Summary_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "Report Summary"
    if top_left_cell.value == "Report Summary":
        term_2_Report_Summary_range = merged_range
        ##print(f"Merged cell containing 'Report Summary' spans: {term_2_Report_Summary_range}")

# Convert merged range to string
range_str_Report_Summary = str(term_2_Report_Summary_range)  # "A45:AA45"

# Extract the first part before the number
term_2_Report_Summary_first_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[0]))

##print("First column:", term_2_Report_Summary_first_column)  # Output: "A"

# Convert merged range to string
range_str_Report_Summary = str(term_2_Report_Summary_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_Report_Summary_last_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[1]))

##print("Last column:", term_2_Report_Summary_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Report_Summary = str(term_2_Report_Summary_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Report_Summary.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_Report_Summary_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_2_Report_Summary_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Report_Summary = str(term_2_Report_Summary_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Report_Summary.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_Report_Summary_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_2_Report_Summary_last_row)
#Extract location of 'All Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_2_All_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "All Applicants":
        term_2_All_Applicants_range = merged_range
        ##print(f"Merged cell containing 'All Applicants' spans: {term_2_All_Applicants_range}")

# Convert merged range to string
range_str_All_Applicants = str(term_2_All_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_2_All_Applicants_first_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[0]))

##print("First column:", term_2_All_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_All_Applicants = str(term_2_All_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_All_Applicants_last_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[1]))

##print("Last column:", term_2_All_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_All_Applicants = str(term_2_All_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_All_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_All_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_2_All_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_All_Applicants = str(term_2_All_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_All_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_All_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_2_All_Applicants_last_row)
#Extract location of 'International Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_2_International_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "International Applicants":
        term_2_International_Applicants_range = merged_range
        #print(f"Merged cell containing 'International Applicants' spans: {term_2_International_Applicants_range}")

# Convert merged range to string
range_str_International_Applicants = str(term_2_International_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_2_International_Applicants_first_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[0]))

##print("First column:", term_2_International_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_International_Applicants = str(term_2_International_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_International_Applicants_last_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[1]))

##print("Last column:", term_2_International_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_International_Applicants = str(term_2_International_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_International_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_International_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_2_International_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_International_Applicants = str(term_2_International_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_International_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_International_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_2_International_Applicants_last_row)
#Extract location of 'Domestic Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_2_Domestic_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "Domestic Applicants":
        term_2_Domestic_Applicants_range = merged_range
        #print(f"Merged cell containing 'Domestic Applicants' spans: {term_2_Domestic_Applicants_range}")

# Convert merged range to string
range_str_Domestic_Applicants = str(term_2_Domestic_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_2_Domestic_Applicants_first_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[0]))

##print("First column:", term_2_Domestic_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_2_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_Domestic_Applicants_last_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[1]))

##print("Last column:", term_2_Domestic_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_2_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Domestic_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_Domestic_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_2_Domestic_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Domestic_Applicants = str(term_2_Domestic_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Domestic_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_Domestic_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_2_Domestic_Applicants_last_row)
#Extract Location of 2023 everything 

# Define your search boundaries using column letters
min_search_row = term_2_Report_Summary_last_row + 1
max_search_row = term_2_All_Applicants_first_row - 1
min_search_col_letter = term_2_Report_Summary_first_column  # Column A
max_search_col_letter = term_2_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_one_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_2_year_minus_one:
            term_2_year_minus_one_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_one_merged_range}")

# Now term_2_year_minus_one_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_2_year_minus_one_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_one_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_2_year_minus_one_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_one_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_2_year_minus_one_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_one_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_2_year_minus_one_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_2_year_minus_one_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_one_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_2_year_minus_one_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_2_year_minus_one_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_one_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_2_year_minus_one_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_one_Unique_merged_range}")

# Now term_2_year_minus_one_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_2_year_minus_one_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_one_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_2_year_minus_one_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_one_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_2_year_minus_one_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_one_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_one_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_2_year_minus_one_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_one_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_one_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_2_year_minus_one_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_one_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_2_year_minus_one_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_one_Total_Apps_merged_range}")

# Now term_2_year_minus_one_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_2_year_minus_one_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_one_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

#print("First column:", term_2_year_minus_one_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_one_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

#print("Last column:", term_2_year_minus_one_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_one_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_one_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_2_year_minus_one_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_one_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_one_Total_Apps_last_row)
#Extract Location of 2022 everything 

# Define your search boundaries using column letters
min_search_row = term_2_Report_Summary_last_row + 1
max_search_row = term_2_All_Applicants_first_row - 1
min_search_col_letter = term_2_Report_Summary_first_column  # Column A
max_search_col_letter = term_2_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_two_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_2_year_minus_two:
            term_2_year_minus_two_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_two_merged_range}")

# Now term_2_year_minus_two_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_2_year_minus_two_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_two_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_2_year_minus_two_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_two_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_2_year_minus_two_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_two_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_two_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_2_year_minus_two_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_two_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_two_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_2_year_minus_two_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_two_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_2_year_minus_two_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_two_Unique_merged_range}")

# Now term_2_year_minus_two_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_2_year_minus_two_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_two_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_2_year_minus_two_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_two_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_2_year_minus_two_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_two_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_two_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_2_year_minus_two_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_two_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_two_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_2_year_minus_two_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_two_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_2_year_minus_two_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_two_Total_Apps_merged_range}")

# Now term_2_year_minus_two_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_2_year_minus_two_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_two_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_2_year_minus_two_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_two_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_2_year_minus_two_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_two_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_two_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_2_year_minus_two_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_two_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_two_Total_Apps_last_row)
#Extract Location of 2021 everything 

# Define your search boundaries using column letters
min_search_row = term_2_Report_Summary_last_row + 1
max_search_row = term_2_All_Applicants_first_row - 1
min_search_col_letter = term_2_Report_Summary_first_column  # Column A
max_search_col_letter = term_2_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_three_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_2_year_minus_three:
            term_2_year_minus_three_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_three_merged_range}")

# Now term_2_year_minus_three_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_2_year_minus_three_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_three_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_2_year_minus_three_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_three_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_2_year_minus_three_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_three_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_three_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_2_year_minus_three_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_three_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_three_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_2_year_minus_three_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_three_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_2_year_minus_three_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_three_Unique_merged_range}")

# Now term_2_year_minus_three_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_2_year_minus_three_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_three_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_2_year_minus_three_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_three_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_2_year_minus_three_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_three_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_three_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_2_year_minus_three_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_three_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_three_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_2_year_minus_three_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_three_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_2_year_minus_three_Total_Apps_merged_range = merged_range
            ##print(f"Merged cell containing '2023' spans: {term_2_year_minus_three_Total_Apps_merged_range}")

# Now term_2_year_minus_three_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_2_year_minus_three_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_three_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

##print("First column:", term_2_year_minus_three_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_three_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

##print("Last column:", term_2_year_minus_three_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_three_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_2_year_minus_three_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_2_year_minus_three_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_three_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_2_year_minus_three_Total_Apps_last_row)
#All 2023 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_2_All_Applicants_first_row
max_search_row = term_2_All_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_one_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_one_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_one_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_2_International_Applicants_first_row
max_search_row = term_2_International_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_one_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_one_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_one_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_2_Domestic_Applicants_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_one_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_one_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_one_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_2_All_Applicants_first_row
max_search_row = term_2_All_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_one_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_one_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_one_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_2_International_Applicants_first_row
max_search_row = term_2_International_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_one_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_one_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_one_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_2_Domestic_Applicants_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_one_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_one_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_one_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_2_year_minus_one_Unique_All_Applicants,",term_2_year_minus_one_Unique_International_Applicants,",term_2_year_minus_one_Unique_Domestic_Applicants)
##print("Extracted Number:", term_2_year_minus_one_Total_Apps_All_Applicants,",term_2_year_minus_one_Total_Apps_International_Applicants,",term_2_year_minus_one_Total_Apps_Domestic_Applicants)


#All 2022 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_2_All_Applicants_first_row
max_search_row = term_2_All_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_two_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_two_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_two_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_2_International_Applicants_first_row
max_search_row = term_2_International_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_two_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_two_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_two_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_2_Domestic_Applicants_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_two_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_two_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_two_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_2_All_Applicants_first_row
max_search_row = term_2_All_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_two_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_two_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_two_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_2_International_Applicants_first_row
max_search_row = term_2_International_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_two_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_two_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_two_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_2_Domestic_Applicants_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_two_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_two_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_two_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_2_year_minus_two_Unique_All_Applicants,",term_2_year_minus_two_Unique_International_Applicants,",term_2_year_minus_two_Unique_Domestic_Applicants)
##print("Extracted Number:", term_2_year_minus_two_Total_Apps_All_Applicants,",term_2_year_minus_two_Total_Apps_International_Applicants,",term_2_year_minus_two_Total_Apps_Domestic_Applicants)


#All 2021 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_2_All_Applicants_first_row
max_search_row = term_2_All_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_three_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_three_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_three_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_2_International_Applicants_first_row
max_search_row = term_2_International_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_three_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_three_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_three_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_2_Domestic_Applicants_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_three_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_three_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_three_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_2_All_Applicants_first_row
max_search_row = term_2_All_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_three_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_three_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_three_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_2_International_Applicants_first_row
max_search_row = term_2_International_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_three_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_three_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_three_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_2_Domestic_Applicants_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_three_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_three_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_three_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

#print("Extracted Number:", term_2_year_minus_three_Unique_All_Applicants,",term_2_year_minus_three_Unique_International_Applicants,",term_2_year_minus_three_Unique_Domestic_Applicants)
#print("Extracted Number:", term_2_year_minus_three_Total_Apps_All_Applicants,",term_2_year_minus_three_Total_Apps_International_Applicants,",term_2_year_minus_three_Total_Apps_Domestic_Applicants)

# Extract the term and year 
term_match = re.match(r"([A-Za-z]+)\s+(\d{4})", term_2_term_year_current)

if term_match:
    term_2_term = term_match.group(1)
    term_2_year_current = int(term_match.group(2))

    # Subtract one year
    term_2_year_minus_one = term_2_year_current - 1

    # Construct the previous term-year string
    term_2_term_year_minus_one = f"{term_2_term} {term_2_year_minus_one}"

#Variables created
print("Current Term and Year:", term_2_term_year_current)
print("Current Term and Year Minus 1:", term_2_term_year_minus_one)
print("\n")
print(f"{term_2_year_minus_one}, Unique, All Applicants:", term_2_year_minus_one_Unique_All_Applicants)
print(f"{term_2_year_minus_one}, Unique, International Applicants:", term_2_year_minus_one_Unique_International_Applicants)
print(f"{term_2_year_minus_one}, Unique, Domestic Applicants:", term_2_year_minus_one_Unique_Domestic_Applicants)
print(f"{term_2_year_minus_one}, Total Apps, All Applicants:", term_2_year_minus_one_Total_Apps_All_Applicants)
print(f"{term_2_year_minus_one}, Total Apps, International Applicants:", term_2_year_minus_one_Total_Apps_International_Applicants)
print(f"{term_2_year_minus_one}, Total Apps, Domestic Applicants:", term_2_year_minus_one_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_2_year_minus_two}, Unique, All Applicants:", term_2_year_minus_two_Unique_All_Applicants)
print(f"{term_2_year_minus_two}, Unique, International Applicants:", term_2_year_minus_two_Unique_International_Applicants)
print(f"{term_2_year_minus_two}, Unique, Domestic Applicants:", term_2_year_minus_two_Unique_Domestic_Applicants)
print(f"{term_2_year_minus_two}, Total Apps, All Applicants:", term_2_year_minus_two_Total_Apps_All_Applicants)
print(f"{term_2_year_minus_two}, Total Apps, International Applicants:", term_2_year_minus_two_Total_Apps_International_Applicants)
print(f"{term_2_year_minus_two}, Total Apps, Domestic Applicants:", term_2_year_minus_two_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_2_year_minus_three}, Unique, All Applicants:", term_2_year_minus_three_Unique_All_Applicants)
print(f"{term_2_year_minus_three}, Unique, International Applicants:", term_2_year_minus_three_Unique_International_Applicants)
print(f"{term_2_year_minus_three}, Unique, Domestic Applicants:", term_2_year_minus_three_Unique_Domestic_Applicants)
print(f"{term_2_year_minus_three}, Total Apps, All Applicants:", term_2_year_minus_three_Total_Apps_All_Applicants)
print(f"{term_2_year_minus_three}, Total Apps, International Applicants:", term_2_year_minus_three_Total_Apps_International_Applicants)
print(f"{term_2_year_minus_three}, Total Apps, Domestic Applicants:", term_2_year_minus_three_Total_Apps_Domestic_Applicants)
      


Current Term and Year: Fall 2025
Current Term and Year Minus 1: Fall 2024


2024, Unique, All Applicants: 4301
2024, Unique, International Applicants: 3492
2024, Unique, Domestic Applicants: 809
2024, Total Apps, All Applicants: 4577
2024, Total Apps, International Applicants: 3667
2024, Total Apps, Domestic Applicants: 910


2023, Unique, All Applicants: 2247
2023, Unique, International Applicants: 1529
2023, Unique, Domestic Applicants: 718
2023, Total Apps, All Applicants: 2454
2023, Total Apps, International Applicants: 1640
2023, Total Apps, Domestic Applicants: 814


2022, Unique, All Applicants: 1552
2022, Unique, International Applicants: 799
2022, Unique, Domestic Applicants: 753
2022, Total Apps, All Applicants: 1673
2022, Total Apps, International Applicants: 848
2022, Total Apps, Domestic Applicants: 825


In [ ]:
#Application Comparative Term 3
# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)  # Replace with your actual file name
ws = wb.active  # Use ws = wb["SheetName"] if known

# Define regex pattern to find the term and year (handles cases like "Spring2024" or "Spring 2024")
pattern = r"\b(Summer|Fall|Winter|Spring)\s*(\d{4})\b"  # Now correctly separates term & year

# Initialize term variable
term_3_term_year_current = None

# Iterate through all cells in the worksheet
for row in ws.iter_rows():
    for cell in row:
        if isinstance(cell.value, str):  # Ensure the cell contains text
            ##print(f"Checking cell: {cell.coordinate}, Value: '{cell.value}'")  # Debugging: print each cell
            
            match = re.search(pattern, cell.value, re.IGNORECASE)  # Case-insensitive search
            if match:
                term_name = match.group(1).capitalize()  # Capitalize first letter (e.g., "Spring")
                term_year = match.group(2)  # Extract the year (e.g., "2024")

                term_3_term_year_current = f"{term_name} {term_year}"  # Format as "Spring 2024"
                ##print(f"Match found: {match.group(0)}")  # Debug print
                break  # Stop searching once found
    if term_3_term_year_current:
        break  # Stop outer loop if term is found

#create variables for term year and years
term_3_year_current = int(re.search(r"\d{4}", term_3_term_year_current).group())
term_3_year_minus_one = term_3_year_current - 1
term_3_year_minus_two = term_3_year_current - 2
term_3_year_minus_three = term_3_year_current - 3

#Extract location of 'Report Summary'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_3_Report_Summary_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "Report Summary"
    if top_left_cell.value == "Report Summary":
        term_3_Report_Summary_range = merged_range
        ##print(f"Merged cell containing 'Report Summary' spans: {term_3_Report_Summary_range}")

# Convert merged range to string
range_str_Report_Summary = str(term_3_Report_Summary_range)  # "A45:AA45"

# Extract the first part before the number
term_3_Report_Summary_first_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[0]))

##print("First column:", term_3_Report_Summary_first_column)  # Output: "A"

# Convert merged range to string
range_str_Report_Summary = str(term_3_Report_Summary_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_Report_Summary_last_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[1]))

##print("Last column:", term_3_Report_Summary_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Report_Summary = str(term_3_Report_Summary_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Report_Summary.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_Report_Summary_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_3_Report_Summary_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Report_Summary = str(term_3_Report_Summary_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Report_Summary.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_Report_Summary_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_3_Report_Summary_last_row)
#Extract location of 'All Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_3_All_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "All Applicants":
        term_3_All_Applicants_range = merged_range
        ##print(f"Merged cell containing 'All Applicants' spans: {term_3_All_Applicants_range}")

# Convert merged range to string
range_str_All_Applicants = str(term_3_All_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_3_All_Applicants_first_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[0]))

##print("First column:", term_3_All_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_All_Applicants = str(term_3_All_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_All_Applicants_last_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[1]))

##print("Last column:", term_3_All_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_All_Applicants = str(term_3_All_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_All_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_All_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_3_All_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_All_Applicants = str(term_3_All_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_All_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_All_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_3_All_Applicants_last_row)
#Extract location of 'International Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_3_International_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "International Applicants":
        term_3_International_Applicants_range = merged_range
        #print(f"Merged cell containing 'International Applicants' spans: {term_3_International_Applicants_range}")

# Convert merged range to string
range_str_International_Applicants = str(term_3_International_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_3_International_Applicants_first_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[0]))

##print("First column:", term_3_International_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_International_Applicants = str(term_3_International_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_International_Applicants_last_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[1]))

##print("Last column:", term_3_International_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_International_Applicants = str(term_3_International_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_International_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_International_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_3_International_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_International_Applicants = str(term_3_International_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_International_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_International_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_3_International_Applicants_last_row)
#Extract location of 'Domestic Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_3_Domestic_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "Domestic Applicants":
        term_3_Domestic_Applicants_range = merged_range
        #print(f"Merged cell containing 'Domestic Applicants' spans: {term_3_Domestic_Applicants_range}")

# Convert merged range to string
range_str_Domestic_Applicants = str(term_3_Domestic_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_3_Domestic_Applicants_first_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[0]))

##print("First column:", term_3_Domestic_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_3_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_Domestic_Applicants_last_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[1]))

##print("Last column:", term_3_Domestic_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_3_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Domestic_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_Domestic_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_3_Domestic_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Domestic_Applicants = str(term_3_Domestic_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Domestic_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_Domestic_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_3_Domestic_Applicants_last_row)
#Extract Location of 2023 everything 

# Define your search boundaries using column letters
min_search_row = term_3_Report_Summary_last_row + 1
max_search_row = term_3_All_Applicants_first_row - 1
min_search_col_letter = term_3_Report_Summary_first_column  # Column A
max_search_col_letter = term_3_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_one_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_3_year_minus_one:
            term_3_year_minus_one_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_one_merged_range}")

# Now term_3_year_minus_one_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_3_year_minus_one_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_one_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_3_year_minus_one_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_one_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_3_year_minus_one_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_one_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_3_year_minus_one_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_3_year_minus_one_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_one_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_3_year_minus_one_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_3_year_minus_one_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_one_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_3_year_minus_one_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_one_Unique_merged_range}")

# Now term_3_year_minus_one_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_3_year_minus_one_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_one_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_3_year_minus_one_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_one_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_3_year_minus_one_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_one_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_one_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_3_year_minus_one_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_one_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_one_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_3_year_minus_one_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_one_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_3_year_minus_one_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_one_Total_Apps_merged_range}")

# Now term_3_year_minus_one_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_3_year_minus_one_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_one_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

#print("First column:", term_3_year_minus_one_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_one_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

#print("Last column:", term_3_year_minus_one_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_one_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_one_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_3_year_minus_one_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_one_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_one_Total_Apps_last_row)
#Extract Location of 2022 everything 

# Define your search boundaries using column letters
min_search_row = term_3_Report_Summary_last_row + 1
max_search_row = term_3_All_Applicants_first_row - 1
min_search_col_letter = term_3_Report_Summary_first_column  # Column A
max_search_col_letter = term_3_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_two_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_3_year_minus_two:
            term_3_year_minus_two_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_two_merged_range}")

# Now term_3_year_minus_two_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_3_year_minus_two_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_two_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_3_year_minus_two_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_two_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_3_year_minus_two_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_two_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_two_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_3_year_minus_two_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_two_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_two_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_3_year_minus_two_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_two_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_3_year_minus_two_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_two_Unique_merged_range}")

# Now term_3_year_minus_two_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_3_year_minus_two_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_two_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_3_year_minus_two_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_two_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_3_year_minus_two_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_two_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_two_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_3_year_minus_two_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_two_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_two_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_3_year_minus_two_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_two_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_3_year_minus_two_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_two_Total_Apps_merged_range}")

# Now term_3_year_minus_two_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_3_year_minus_two_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_two_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_3_year_minus_two_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_two_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_3_year_minus_two_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_two_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_two_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_3_year_minus_two_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_two_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_two_Total_Apps_last_row)
#Extract Location of 2021 everything 

# Define your search boundaries using column letters
min_search_row = term_3_Report_Summary_last_row + 1
max_search_row = term_3_All_Applicants_first_row - 1
min_search_col_letter = term_3_Report_Summary_first_column  # Column A
max_search_col_letter = term_3_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_three_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_3_year_minus_three:
            term_3_year_minus_three_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_three_merged_range}")

# Now term_3_year_minus_three_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_3_year_minus_three_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_three_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_3_year_minus_three_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_three_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_3_year_minus_three_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_three_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_three_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_3_year_minus_three_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_three_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_three_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_3_year_minus_three_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_three_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_3_year_minus_three_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_three_Unique_merged_range}")

# Now term_3_year_minus_three_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_3_year_minus_three_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_three_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_3_year_minus_three_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_three_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_3_year_minus_three_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_three_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_three_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_3_year_minus_three_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_three_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_three_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_3_year_minus_three_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_three_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_3_year_minus_three_Total_Apps_merged_range = merged_range
            ##print(f"Merged cell containing '2023' spans: {term_3_year_minus_three_Total_Apps_merged_range}")

# Now term_3_year_minus_three_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_3_year_minus_three_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_three_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

##print("First column:", term_3_year_minus_three_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_three_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

##print("Last column:", term_3_year_minus_three_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_three_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_3_year_minus_three_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_3_year_minus_three_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_three_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_3_year_minus_three_Total_Apps_last_row)
#All 2023 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_3_All_Applicants_first_row
max_search_row = term_3_All_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_one_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_one_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_one_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_3_International_Applicants_first_row
max_search_row = term_3_International_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_one_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_one_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_one_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_3_Domestic_Applicants_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_one_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_one_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_one_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_3_All_Applicants_first_row
max_search_row = term_3_All_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_one_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_one_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_one_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_3_International_Applicants_first_row
max_search_row = term_3_International_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_one_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_one_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_one_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_3_Domestic_Applicants_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_one_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_one_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_one_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_3_year_minus_one_Unique_All_Applicants,",term_3_year_minus_one_Unique_International_Applicants,",term_3_year_minus_one_Unique_Domestic_Applicants)
##print("Extracted Number:", term_3_year_minus_one_Total_Apps_All_Applicants,",term_3_year_minus_one_Total_Apps_International_Applicants,",term_3_year_minus_one_Total_Apps_Domestic_Applicants)


#All 2022 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_3_All_Applicants_first_row
max_search_row = term_3_All_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_two_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_two_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_two_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_3_International_Applicants_first_row
max_search_row = term_3_International_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_two_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_two_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_two_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_3_Domestic_Applicants_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_two_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_two_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_two_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_3_All_Applicants_first_row
max_search_row = term_3_All_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_two_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_two_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_two_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_3_International_Applicants_first_row
max_search_row = term_3_International_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_two_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_two_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_two_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_3_Domestic_Applicants_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_two_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_two_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_two_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_3_year_minus_two_Unique_All_Applicants,",term_3_year_minus_two_Unique_International_Applicants,",term_3_year_minus_two_Unique_Domestic_Applicants)
##print("Extracted Number:", term_3_year_minus_two_Total_Apps_All_Applicants,",term_3_year_minus_two_Total_Apps_International_Applicants,",term_3_year_minus_two_Total_Apps_Domestic_Applicants)


#All 2021 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_3_All_Applicants_first_row
max_search_row = term_3_All_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_three_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_three_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_three_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_3_International_Applicants_first_row
max_search_row = term_3_International_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_three_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_three_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_three_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_3_Domestic_Applicants_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_three_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_three_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_three_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_3_All_Applicants_first_row
max_search_row = term_3_All_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_three_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_three_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_three_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_3_International_Applicants_first_row
max_search_row = term_3_International_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_three_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_three_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_three_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_3_Domestic_Applicants_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_three_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_three_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_three_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

#print("Extracted Number:", term_3_year_minus_three_Unique_All_Applicants,",term_3_year_minus_three_Unique_International_Applicants,",term_3_year_minus_three_Unique_Domestic_Applicants)
#print("Extracted Number:", term_3_year_minus_three_Total_Apps_All_Applicants,",term_3_year_minus_three_Total_Apps_International_Applicants,",term_3_year_minus_three_Total_Apps_Domestic_Applicants)

# Extract the term and year 
term_match = re.match(r"([A-Za-z]+)\s+(\d{4})", term_3_term_year_current)

if term_match:
    term_3_term = term_match.group(1)
    term_3_year_current = int(term_match.group(2))

    # Subtract one year
    term_3_year_minus_one = term_3_year_current - 1

    # Construct the previous term-year string
    term_3_term_year_minus_one = f"{term_3_term} {term_3_year_minus_one}"

#Variables created
print("Current Term and Year:", term_3_term_year_current)
print("Current Term and Year Minus 1:", term_3_term_year_minus_one)
print("\n")
print(f"{term_3_year_minus_one}, Unique, All Applicants:", term_3_year_minus_one_Unique_All_Applicants)
print(f"{term_3_year_minus_one}, Unique, International Applicants:", term_3_year_minus_one_Unique_International_Applicants)
print(f"{term_3_year_minus_one}, Unique, Domestic Applicants:", term_3_year_minus_one_Unique_Domestic_Applicants)
print(f"{term_3_year_minus_one}, Total Apps, All Applicants:", term_3_year_minus_one_Total_Apps_All_Applicants)
print(f"{term_3_year_minus_one}, Total Apps, International Applicants:", term_3_year_minus_one_Total_Apps_International_Applicants)
print(f"{term_3_year_minus_one}, Total Apps, Domestic Applicants:", term_3_year_minus_one_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_3_year_minus_two}, Unique, All Applicants:", term_3_year_minus_two_Unique_All_Applicants)
print(f"{term_3_year_minus_two}, Unique, International Applicants:", term_3_year_minus_two_Unique_International_Applicants)
print(f"{term_3_year_minus_two}, Unique, Domestic Applicants:", term_3_year_minus_two_Unique_Domestic_Applicants)
print(f"{term_3_year_minus_two}, Total Apps, All Applicants:", term_3_year_minus_two_Total_Apps_All_Applicants)
print(f"{term_3_year_minus_two}, Total Apps, International Applicants:", term_3_year_minus_two_Total_Apps_International_Applicants)
print(f"{term_3_year_minus_two}, Total Apps, Domestic Applicants:", term_3_year_minus_two_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_3_year_minus_three}, Unique, All Applicants:", term_3_year_minus_three_Unique_All_Applicants)
print(f"{term_3_year_minus_three}, Unique, International Applicants:", term_3_year_minus_three_Unique_International_Applicants)
print(f"{term_3_year_minus_three}, Unique, Domestic Applicants:", term_3_year_minus_three_Unique_Domestic_Applicants)
print(f"{term_3_year_minus_three}, Total Apps, All Applicants:", term_3_year_minus_three_Total_Apps_All_Applicants)
print(f"{term_3_year_minus_three}, Total Apps, International Applicants:", term_3_year_minus_three_Total_Apps_International_Applicants)
print(f"{term_3_year_minus_three}, Total Apps, Domestic Applicants:", term_3_year_minus_three_Total_Apps_Domestic_Applicants)
      


Current Term and Year: Winter 2026
Current Term and Year Minus 1: Winter 2025


2025, Unique, All Applicants: 913
2025, Unique, International Applicants: 896
2025, Unique, Domestic Applicants: 17
2025, Total Apps, All Applicants: 926
2025, Total Apps, International Applicants: 909
2025, Total Apps, Domestic Applicants: 17


2024, Unique, All Applicants: 240
2024, Unique, International Applicants: 213
2024, Unique, Domestic Applicants: 27
2024, Total Apps, All Applicants: 245
2024, Total Apps, International Applicants: 217
2024, Total Apps, Domestic Applicants: 28


2023, Unique, All Applicants: 92
2023, Unique, International Applicants: 53
2023, Unique, Domestic Applicants: 39
2023, Total Apps, All Applicants: 93
2023, Total Apps, International Applicants: 54
2023, Total Apps, Domestic Applicants: 39


In [ ]:
#Application Comparative Term 4
# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)  # Replace with your actual file name
ws = wb.active  # Use ws = wb["SheetName"] if known

# Define regex pattern to find the term and year (handles cases like "Spring2024" or "Spring 2024")
pattern = r"\b(Summer|Fall|Winter|Spring)\s*(\d{4})\b"  # Now correctly separates term & year

# Initialize term variable
term_4_term_year_current = None

# Iterate through all cells in the worksheet
for row in ws.iter_rows():
    for cell in row:
        if isinstance(cell.value, str):  # Ensure the cell contains text
            ##print(f"Checking cell: {cell.coordinate}, Value: '{cell.value}'")  # Debugging: print each cell
            
            match = re.search(pattern, cell.value, re.IGNORECASE)  # Case-insensitive search
            if match:
                term_name = match.group(1).capitalize()  # Capitalize first letter (e.g., "Spring")
                term_year = match.group(2)  # Extract the year (e.g., "2024")

                term_4_term_year_current = f"{term_name} {term_year}"  # Format as "Spring 2024"
                ##print(f"Match found: {match.group(0)}")  # Debug print
                break  # Stop searching once found
    if term_4_term_year_current:
        break  # Stop outer loop if term is found

#create variables for term year and years
term_4_year_current = int(re.search(r"\d{4}", term_4_term_year_current).group())
term_4_year_minus_one = term_4_year_current - 1
term_4_year_minus_two = term_4_year_current - 2
term_4_year_minus_three = term_4_year_current - 3

#Extract location of 'Report Summary'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_4_Report_Summary_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "Report Summary"
    if top_left_cell.value == "Report Summary":
        term_4_Report_Summary_range = merged_range
        ##print(f"Merged cell containing 'Report Summary' spans: {term_4_Report_Summary_range}")

# Convert merged range to string
range_str_Report_Summary = str(term_4_Report_Summary_range)  # "A45:AA45"

# Extract the first part before the number
term_4_Report_Summary_first_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[0]))

##print("First column:", term_4_Report_Summary_first_column)  # Output: "A"

# Convert merged range to string
range_str_Report_Summary = str(term_4_Report_Summary_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_Report_Summary_last_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[1]))

##print("Last column:", term_4_Report_Summary_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Report_Summary = str(term_4_Report_Summary_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Report_Summary.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_Report_Summary_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_4_Report_Summary_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Report_Summary = str(term_4_Report_Summary_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Report_Summary.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_Report_Summary_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_4_Report_Summary_last_row)
#Extract location of 'All Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_4_All_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "All Applicants":
        term_4_All_Applicants_range = merged_range
        ##print(f"Merged cell containing 'All Applicants' spans: {term_4_All_Applicants_range}")

# Convert merged range to string
range_str_All_Applicants = str(term_4_All_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_4_All_Applicants_first_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[0]))

##print("First column:", term_4_All_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_All_Applicants = str(term_4_All_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_All_Applicants_last_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[1]))

##print("Last column:", term_4_All_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_All_Applicants = str(term_4_All_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_All_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_All_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_4_All_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_All_Applicants = str(term_4_All_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_All_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_All_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_4_All_Applicants_last_row)
#Extract location of 'International Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_4_International_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "International Applicants":
        term_4_International_Applicants_range = merged_range
        #print(f"Merged cell containing 'International Applicants' spans: {term_4_International_Applicants_range}")

# Convert merged range to string
range_str_International_Applicants = str(term_4_International_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_4_International_Applicants_first_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[0]))

##print("First column:", term_4_International_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_International_Applicants = str(term_4_International_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_International_Applicants_last_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[1]))

##print("Last column:", term_4_International_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_International_Applicants = str(term_4_International_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_International_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_International_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_4_International_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_International_Applicants = str(term_4_International_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_International_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_International_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_4_International_Applicants_last_row)
#Extract location of 'Domestic Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_4_Domestic_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "Domestic Applicants":
        term_4_Domestic_Applicants_range = merged_range
        #print(f"Merged cell containing 'Domestic Applicants' spans: {term_4_Domestic_Applicants_range}")

# Convert merged range to string
range_str_Domestic_Applicants = str(term_4_Domestic_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_4_Domestic_Applicants_first_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[0]))

##print("First column:", term_4_Domestic_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_4_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_Domestic_Applicants_last_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[1]))

##print("Last column:", term_4_Domestic_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_4_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Domestic_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_Domestic_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_4_Domestic_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Domestic_Applicants = str(term_4_Domestic_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Domestic_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_Domestic_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_4_Domestic_Applicants_last_row)
#Extract Location of 2023 everything 

# Define your search boundaries using column letters
min_search_row = term_4_Report_Summary_last_row + 1
max_search_row = term_4_All_Applicants_first_row - 1
min_search_col_letter = term_4_Report_Summary_first_column  # Column A
max_search_col_letter = term_4_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_one_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_4_year_minus_one:
            term_4_year_minus_one_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_one_merged_range}")

# Now term_4_year_minus_one_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_4_year_minus_one_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_one_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_4_year_minus_one_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_one_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_4_year_minus_one_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_one_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_4_year_minus_one_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_4_year_minus_one_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_one_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_4_year_minus_one_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_4_year_minus_one_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_one_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_4_year_minus_one_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_one_Unique_merged_range}")

# Now term_4_year_minus_one_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_4_year_minus_one_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_one_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_4_year_minus_one_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_one_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_4_year_minus_one_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_one_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_one_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_4_year_minus_one_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_one_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_one_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_4_year_minus_one_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_one_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_4_year_minus_one_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_one_Total_Apps_merged_range}")

# Now term_4_year_minus_one_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_4_year_minus_one_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_one_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

#print("First column:", term_4_year_minus_one_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_one_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

#print("Last column:", term_4_year_minus_one_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_one_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_one_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_4_year_minus_one_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_one_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_one_Total_Apps_last_row)
#Extract Location of 2022 everything 

# Define your search boundaries using column letters
min_search_row = term_4_Report_Summary_last_row + 1
max_search_row = term_4_All_Applicants_first_row - 1
min_search_col_letter = term_4_Report_Summary_first_column  # Column A
max_search_col_letter = term_4_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_two_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_4_year_minus_two:
            term_4_year_minus_two_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_two_merged_range}")

# Now term_4_year_minus_two_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_4_year_minus_two_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_two_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_4_year_minus_two_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_two_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_4_year_minus_two_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_two_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_two_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_4_year_minus_two_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_two_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_two_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_4_year_minus_two_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_two_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_4_year_minus_two_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_two_Unique_merged_range}")

# Now term_4_year_minus_two_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_4_year_minus_two_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_two_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_4_year_minus_two_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_two_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_4_year_minus_two_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_two_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_two_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_4_year_minus_two_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_two_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_two_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_4_year_minus_two_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_two_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_4_year_minus_two_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_two_Total_Apps_merged_range}")

# Now term_4_year_minus_two_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_4_year_minus_two_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_two_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_4_year_minus_two_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_two_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_4_year_minus_two_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_two_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_two_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_4_year_minus_two_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_two_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_two_Total_Apps_last_row)
#Extract Location of 2021 everything 

# Define your search boundaries using column letters
min_search_row = term_4_Report_Summary_last_row + 1
max_search_row = term_4_All_Applicants_first_row - 1
min_search_col_letter = term_4_Report_Summary_first_column  # Column A
max_search_col_letter = term_4_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_three_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_4_year_minus_three:
            term_4_year_minus_three_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_three_merged_range}")

# Now term_4_year_minus_three_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_4_year_minus_three_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_three_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_4_year_minus_three_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_three_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_4_year_minus_three_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_three_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_three_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_4_year_minus_three_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_three_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_three_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_4_year_minus_three_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_three_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_4_year_minus_three_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_three_Unique_merged_range}")

# Now term_4_year_minus_three_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_4_year_minus_three_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_three_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_4_year_minus_three_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_three_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_4_year_minus_three_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_three_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_three_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_4_year_minus_three_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_three_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_three_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_4_year_minus_three_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_three_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_4_year_minus_three_Total_Apps_merged_range = merged_range
            ##print(f"Merged cell containing '2023' spans: {term_4_year_minus_three_Total_Apps_merged_range}")

# Now term_4_year_minus_three_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_4_year_minus_three_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_three_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

##print("First column:", term_4_year_minus_three_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_three_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

##print("Last column:", term_4_year_minus_three_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_three_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_4_year_minus_three_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_4_year_minus_three_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_three_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_4_year_minus_three_Total_Apps_last_row)
#All 2023 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_4_All_Applicants_first_row
max_search_row = term_4_All_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_one_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_one_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_one_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_4_International_Applicants_first_row
max_search_row = term_4_International_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_one_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_one_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_one_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_4_Domestic_Applicants_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_one_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_one_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_one_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_4_All_Applicants_first_row
max_search_row = term_4_All_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_one_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_one_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_one_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_4_International_Applicants_first_row
max_search_row = term_4_International_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_one_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_one_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_one_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_4_Domestic_Applicants_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_one_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_one_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_one_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_4_year_minus_one_Unique_All_Applicants,",term_4_year_minus_one_Unique_International_Applicants,",term_4_year_minus_one_Unique_Domestic_Applicants)
##print("Extracted Number:", term_4_year_minus_one_Total_Apps_All_Applicants,",term_4_year_minus_one_Total_Apps_International_Applicants,",term_4_year_minus_one_Total_Apps_Domestic_Applicants)


#All 2022 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_4_All_Applicants_first_row
max_search_row = term_4_All_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_two_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_two_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_two_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_4_International_Applicants_first_row
max_search_row = term_4_International_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_two_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_two_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_two_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_4_Domestic_Applicants_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_two_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_two_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_two_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_4_All_Applicants_first_row
max_search_row = term_4_All_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_two_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_two_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_two_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_4_International_Applicants_first_row
max_search_row = term_4_International_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_two_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_two_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_two_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_4_Domestic_Applicants_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_two_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_two_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_two_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_4_year_minus_two_Unique_All_Applicants,",term_4_year_minus_two_Unique_International_Applicants,",term_4_year_minus_two_Unique_Domestic_Applicants)
##print("Extracted Number:", term_4_year_minus_two_Total_Apps_All_Applicants,",term_4_year_minus_two_Total_Apps_International_Applicants,",term_4_year_minus_two_Total_Apps_Domestic_Applicants)


#All 2021 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_4_All_Applicants_first_row
max_search_row = term_4_All_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_three_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_three_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_three_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_4_International_Applicants_first_row
max_search_row = term_4_International_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_three_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_three_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_three_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_4_Domestic_Applicants_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_three_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_three_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_three_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_4_All_Applicants_first_row
max_search_row = term_4_All_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_three_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_three_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_three_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_4_International_Applicants_first_row
max_search_row = term_4_International_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_three_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_three_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_three_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_4_Domestic_Applicants_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_three_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_three_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_three_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

#print("Extracted Number:", term_4_year_minus_three_Unique_All_Applicants,",term_4_year_minus_three_Unique_International_Applicants,",term_4_year_minus_three_Unique_Domestic_Applicants)
#print("Extracted Number:", term_4_year_minus_three_Total_Apps_All_Applicants,",term_4_year_minus_three_Total_Apps_International_Applicants,",term_4_year_minus_three_Total_Apps_Domestic_Applicants)

# Extract the term and year 
term_match = re.match(r"([A-Za-z]+)\s+(\d{4})", term_4_term_year_current)

if term_match:
    term_4_term = term_match.group(1)
    term_4_year_current = int(term_match.group(2))

    # Subtract one year
    term_4_year_minus_one = term_4_year_current - 1

    # Construct the previous term-year string
    term_4_term_year_minus_one = f"{term_4_term} {term_4_year_minus_one}"

#Variables created
print("Current Term and Year:", term_4_term_year_current)
print("Current Term and Year Minus 1:", term_4_term_year_minus_one)
print("\n")
print(f"{term_4_year_minus_one}, Unique, All Applicants:", term_4_year_minus_one_Unique_All_Applicants)
print(f"{term_4_year_minus_one}, Unique, International Applicants:", term_4_year_minus_one_Unique_International_Applicants)
print(f"{term_4_year_minus_one}, Unique, Domestic Applicants:", term_4_year_minus_one_Unique_Domestic_Applicants)
print(f"{term_4_year_minus_one}, Total Apps, All Applicants:", term_4_year_minus_one_Total_Apps_All_Applicants)
print(f"{term_4_year_minus_one}, Total Apps, International Applicants:", term_4_year_minus_one_Total_Apps_International_Applicants)
print(f"{term_4_year_minus_one}, Total Apps, Domestic Applicants:", term_4_year_minus_one_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_4_year_minus_two}, Unique, All Applicants:", term_4_year_minus_two_Unique_All_Applicants)
print(f"{term_4_year_minus_two}, Unique, International Applicants:", term_4_year_minus_two_Unique_International_Applicants)
print(f"{term_4_year_minus_two}, Unique, Domestic Applicants:", term_4_year_minus_two_Unique_Domestic_Applicants)
print(f"{term_4_year_minus_two}, Total Apps, All Applicants:", term_4_year_minus_two_Total_Apps_All_Applicants)
print(f"{term_4_year_minus_two}, Total Apps, International Applicants:", term_4_year_minus_two_Total_Apps_International_Applicants)
print(f"{term_4_year_minus_two}, Total Apps, Domestic Applicants:", term_4_year_minus_two_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_4_year_minus_three}, Unique, All Applicants:", term_4_year_minus_three_Unique_All_Applicants)
print(f"{term_4_year_minus_three}, Unique, International Applicants:", term_4_year_minus_three_Unique_International_Applicants)
print(f"{term_4_year_minus_three}, Unique, Domestic Applicants:", term_4_year_minus_three_Unique_Domestic_Applicants)
print(f"{term_4_year_minus_three}, Total Apps, All Applicants:", term_4_year_minus_three_Total_Apps_All_Applicants)
print(f"{term_4_year_minus_three}, Total Apps, International Applicants:", term_4_year_minus_three_Total_Apps_International_Applicants)
print(f"{term_4_year_minus_three}, Total Apps, Domestic Applicants:", term_4_year_minus_three_Total_Apps_Domestic_Applicants)
      


Current Term and Year: Spring 2026
Current Term and Year Minus 1: Spring 2025


2025, Unique, All Applicants: 2
2025, Unique, International Applicants: 0
2025, Unique, Domestic Applicants: 2
2025, Total Apps, All Applicants: 2
2025, Total Apps, International Applicants: 0
2025, Total Apps, Domestic Applicants: 2


2024, Unique, All Applicants: 2
2024, Unique, International Applicants: 2
2024, Unique, Domestic Applicants: 0
2024, Total Apps, All Applicants: 2
2024, Total Apps, International Applicants: 2
2024, Total Apps, Domestic Applicants: 0


2023, Unique, All Applicants: 2
2023, Unique, International Applicants: 1
2023, Unique, Domestic Applicants: 1
2023, Total Apps, All Applicants: 2
2023, Total Apps, International Applicants: 1
2023, Total Apps, Domestic Applicants: 1


In [ ]:
#Application Details Term 1
# Load the workbook
wb = openpyxl.load_workbook(term_1_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(", ")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_1_details = pd.read_excel(term_1_file_application_details, sheet_name=matched_sheet_name)

# Count total applications (assuming no empty cells)
term_1_details_total_applications = df_term_1_details['Applicants'].count()

# Count unique applicants (assuming the column contains identifiers for each applicant)
term_1_details_total_unique_applicants = df_term_1_details['Applicants'].nunique()

# Output the results
print(f"Total applications: {term_1_details_total_applications}")
print(f"Total unique applicants: {term_1_details_total_unique_applicants}")

# Count how many "Domestic" applications
term_1_details_domestic_applications = df_term_1_details[df_term_1_details['Population'] == 'Domestic'].shape[0]

# Count how many "International" applications
term_1_details_international_applications = df_term_1_details[df_term_1_details['Population'] == 'International'].shape[0]

# Output the results
print(f"Total Domestic applications: {term_1_details_domestic_applications}")
print(f"Total International applications: {term_1_details_international_applications}")

column_Accepted_values = ["ACPT", "COND", "ACLN"]
column_Deferred_Application_values = ["ACPT", "COND"]

# Apply the filters
filtered_df_term_1_details = df_term_1_details[
    df_term_1_details["Accepted"].isin(column_Accepted_values) &
    ~df_term_1_details["Deferred Application"].isin(column_Deferred_Application_values)
]

#Count how many Accepted applications
term_1_details_accepted_applications = len(filtered_df_term_1_details)
print(f"Valid application count: {term_1_details_accepted_applications}")

Total applications: 48
Total unique applicants: 48
Total Domestic applications: 24
Total International applications: 25
Valid application count: 43


In [ ]:
#Application Details Term 2
# Load the workbook
wb = openpyxl.load_workbook(term_2_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(", ")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_2_details = pd.read_excel(term_2_file_application_details, sheet_name=matched_sheet_name)

# Count total applications (assuming no empty cells)
term_2_details_total_applications = df_term_2_details['Applicants'].count()

# Count unique applicants (assuming the column contains identifiers for each applicant)
term_2_details_total_unique_applicants = df_term_2_details['Applicants'].nunique()

# Output the results
print(f"Total applications: {term_2_details_total_applications}")
print(f"Total unique applicants: {term_2_details_total_unique_applicants}")

# Count how many "Domestic" applications
term_2_details_domestic_applications = df_term_2_details[df_term_2_details['Population'] == 'Domestic'].shape[0]

# Count how many "International" applications
term_2_details_international_applications = df_term_2_details[df_term_2_details['Population'] == 'International'].shape[0]

# Output the results
print(f"Total Domestic applications: {term_2_details_domestic_applications}")
print(f"Total International applications: {term_2_details_international_applications}")

column_Accepted_values = ["ACPT", "COND", "ACLN"]
column_Deferred_Application_values = ["ACPT", "COND"]

# Apply the filters
filtered_df_term_2_details = df_term_2_details[
    df_term_2_details["Accepted"].isin(column_Accepted_values) &
    ~df_term_2_details["Deferred Application"].isin(column_Deferred_Application_values)
]

#Count how many Accepted applications
term_2_details_accepted_applications = len(filtered_df_term_2_details)
print(f"Valid application count: {term_2_details_accepted_applications}")

Total applications: 1144
Total unique applicants: 1144
Total Domestic applications: 750
Total International applications: 394
Valid application count: 393


In [ ]:
#Application Details Term 3
# Load the workbook
wb = openpyxl.load_workbook(term_3_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(", ")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_3_details = pd.read_excel(term_3_file_application_details, sheet_name=matched_sheet_name)

# Count total applications (assuming no empty cells)
term_3_details_total_applications = df_term_3_details['Applicants'].count()

# Count unique applicants (assuming the column contains identifiers for each applicant)
term_3_details_total_unique_applicants = df_term_3_details['Applicants'].nunique()

# Output the results
print(f"Total applications: {term_3_details_total_applications}")
print(f"Total unique applicants: {term_3_details_total_unique_applicants}")

# Count how many "Domestic" applications
term_3_details_domestic_applications = df_term_3_details[df_term_3_details['Population'] == 'Domestic'].shape[0]

# Count how many "International" applications
term_3_details_international_applications = df_term_3_details[df_term_3_details['Population'] == 'International'].shape[0]

# Output the results
print(f"Total Domestic applications: {term_3_details_domestic_applications}")
print(f"Total International applications: {term_3_details_international_applications}")

column_Accepted_values = ["ACPT", "COND", "ACLN"]
column_Deferred_Application_values = ["ACPT", "COND"]

# Apply the filters
filtered_df_term_3_details = df_term_3_details[
    df_term_3_details["Accepted"].isin(column_Accepted_values) &
    ~df_term_3_details["Deferred Application"].isin(column_Deferred_Application_values)
]

#Count how many Accepted applications
term_3_details_accepted_applications = len(filtered_df_term_3_details)
print(f"Valid application count: {term_3_details_accepted_applications}")

Total applications: 16
Total unique applicants: 16
Total Domestic applications: 5
Total International applications: 11
Valid application count: 6


In [ ]:
#Application Details Term 4
# Load the workbook
wb = openpyxl.load_workbook(term_4_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(", ")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_4_details = pd.read_excel(term_4_file_application_details, sheet_name=matched_sheet_name)

# Count total applications (assuming no empty cells)
term_4_details_total_applications = df_term_4_details['Applicants'].count()

# Count unique applicants (assuming the column contains identifiers for each applicant)
term_4_details_total_unique_applicants = df_term_4_details['Applicants'].nunique()

# Output the results
print(f"Total applications: {term_4_details_total_applications}")
print(f"Total unique applicants: {term_4_details_total_unique_applicants}")

# Count how many "Domestic" applications
term_4_details_domestic_applications = df_term_4_details[df_term_4_details['Population'] == 'Domestic'].shape[0]

# Count how many "International" applications
term_4_details_international_applications = df_term_4_details[df_term_4_details['Population'] == 'International'].shape[0]

# Output the results
print(f"Total Domestic applications: {term_4_details_domestic_applications}")
print(f"Total International applications: {term_4_details_international_applications}")

column_Accepted_values = ["ACPT", "COND", "ACLN"]
column_Deferred_Application_values = ["ACPT", "COND"]

# Apply the filters
filtered_df_term_4_details = df_term_4_details[
    df_term_4_details["Accepted"].isin(column_Accepted_values) &
    ~df_term_4_details["Deferred Application"].isin(column_Deferred_Application_values)
]

#Count how many Accepted applications
term_4_details_accepted_applications = len(filtered_df_term_4_details)
print(f"Valid application count: {term_4_details_accepted_applications}")

Total applications: 1
Total unique applicants: 1
Total Domestic applications: 1
Total International applications: 0
Valid application count: 0


In [ ]:
#Applications Calculations

#Calculate % Change for term 1 Total Applications
term_1_total_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_1_details_total_applications, term_1_year_minus_one_Total_Apps_All_Applicants)}%"
term_1_total_applications_percent_change_number = percent_difference_value_calculator_round_1(term_1_details_total_applications, term_1_year_minus_one_Total_Apps_All_Applicants)
try:
    term_1_total_applications_percent_change_number_absolute = np.abs(term_1_total_applications_percent_change_number)
except (TypeError, ValueError):
    term_1_total_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 1 Unique Applications
term_1_total_applications_percent_change_unique = f"{percent_difference_value_calculator_round_1(term_1_details_total_unique_applicants, term_1_year_minus_one_Unique_All_Applicants)}%"
term_1_total_applications_percent_change_number_unique = percent_difference_value_calculator_round_1(term_1_details_total_unique_applicants, term_1_year_minus_one_Unique_All_Applicants)
try:
    term_1_total_applications_percent_change_number_unique_absolute = np.abs(term_1_total_applications_percent_change_number_unique)
except (TypeError, ValueError):
    term_1_total_applications_percent_change_number_unique_absolute = "N/A"

#Calculate % Change for term 1 International Applications
term_1_international_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_1_details_international_applications, term_1_year_minus_one_Total_Apps_International_Applicants)}%"
term_1_international_applications_percent_change_number = percent_difference_value_calculator_round_1(term_1_details_international_applications, term_1_year_minus_one_Total_Apps_International_Applicants)
try:
    term_1_international_applications_percent_change_number_absolute = np.abs(term_1_international_applications_percent_change_number)
except (TypeError, ValueError):
    term_1_international_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 1 Domestic Applications
term_1_domestic_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_1_details_domestic_applications, term_1_year_minus_one_Total_Apps_Domestic_Applicants)}%"
term_1_domestic_applications_percent_change_number = percent_difference_value_calculator_round_1(term_1_details_domestic_applications, term_1_year_minus_one_Total_Apps_Domestic_Applicants)
try:
    term_1_domestic_applications_percent_change_number_absolute = np.abs(term_1_domestic_applications_percent_change_number)
except (TypeError, ValueError):
    term_1_domestic_applications_percent_change_number_absolute = "N/A"

#Calculation for {{higher_lower_1}}
try:
    if term_1_total_applications_percent_change_number_unique >= 0:
        higher_lower_1 = "higher"
    else:
        higher_lower_1 = "lower"
except (TypeError, ValueError): 
    higher_lower_1 = "FAILED TO CALCULATE"

#Calculation for {{term_1_acceptance_percentage_number}}

term_1_acceptance_percentage_number = division_calculator(term_1_details_accepted_applications, term_1_details_total_applications)

#Calculation for {{term_1_international_percentage_number}}

term_1_international_percentage_number = division_calculator(term_1_details_international_applications, term_1_details_total_applications)

#Calculate % Change for term 2 Total Applications
term_2_total_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_2_details_total_applications, term_2_year_minus_one_Total_Apps_All_Applicants)}%"
term_2_total_applications_percent_change_number = percent_difference_value_calculator_round_1(term_2_details_total_applications, term_2_year_minus_one_Total_Apps_All_Applicants)
try:
    term_2_total_applications_percent_change_number_absolute = np.abs(term_2_total_applications_percent_change_number)
except (TypeError, ValueError):
    term_2_total_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 2 Unique Applications
term_2_total_applications_percent_change_unique = f"{percent_difference_value_calculator_round_1(term_2_details_total_unique_applicants, term_2_year_minus_one_Unique_All_Applicants)}%"
term_2_total_applications_percent_change_number_unique = percent_difference_value_calculator_round_1(term_2_details_total_unique_applicants, term_2_year_minus_one_Unique_All_Applicants)
try:
    term_2_total_applications_percent_change_number_unique_absolute = np.abs(term_2_total_applications_percent_change_number_unique)
except (TypeError, ValueError):
    term_2_total_applications_percent_change_number_unique_absolute = "N/A"

#Calculate % Change for term 2 International Applications
term_2_international_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_2_details_international_applications, term_2_year_minus_one_Total_Apps_International_Applicants)}%"
term_2_international_applications_percent_change_number = percent_difference_value_calculator_round_1(term_2_details_international_applications, term_2_year_minus_one_Total_Apps_International_Applicants)
try:
    term_2_international_applications_percent_change_number_absolute = np.abs(term_2_international_applications_percent_change_number)
except (TypeError, ValueError):
    term_2_international_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 2 Domestic Applications
term_2_domestic_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_2_details_domestic_applications, term_2_year_minus_one_Total_Apps_Domestic_Applicants)}%"
term_2_domestic_applications_percent_change_number = percent_difference_value_calculator_round_1(term_2_details_domestic_applications, term_2_year_minus_one_Total_Apps_Domestic_Applicants)
try:
    term_2_domestic_applications_percent_change_number_absolute = np.abs(term_2_domestic_applications_percent_change_number)
except (TypeError, ValueError):
    term_2_domestic_applications_percent_change_number_absolute = "N/A"

#Calculation for {{higher_lower_2}}
try:
    if term_2_total_applications_percent_change_number_unique >= 0:
        higher_lower_2 = "higher"
    else:
        higher_lower_2 = "lower"
except (TypeError, ValueError): 
    higher_lower_2 = "FAILED TO CALCULATE"

#Calculation for {{term_2_acceptance_percentage_number}}

term_2_acceptance_percentage_number = division_calculator(term_2_details_accepted_applications, term_2_details_total_applications)

#Calculation for {{term_2_international_percentage_number}}

term_2_international_percentage_number = division_calculator(term_2_details_international_applications, term_2_details_total_applications)

#Calculate % Change for term 3 Total Applications
term_3_total_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_3_details_total_applications, term_3_year_minus_one_Total_Apps_All_Applicants)}%"
term_3_total_applications_percent_change_number = percent_difference_value_calculator_round_1(term_3_details_total_applications, term_3_year_minus_one_Total_Apps_All_Applicants)
try:
    term_3_total_applications_percent_change_number_absolute = np.abs(term_3_total_applications_percent_change_number)
except (TypeError, ValueError):
    term_3_total_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 3 Unique Applications
term_3_total_applications_percent_change_unique = f"{percent_difference_value_calculator_round_1(term_3_details_total_unique_applicants, term_3_year_minus_one_Unique_All_Applicants)}%"
term_3_total_applications_percent_change_number_unique = percent_difference_value_calculator_round_1(term_3_details_total_unique_applicants, term_3_year_minus_one_Unique_All_Applicants)
try:
    term_3_total_applications_percent_change_number_unique_absolute = np.abs(term_3_total_applications_percent_change_number_unique)
except (TypeError, ValueError):
    term_3_total_applications_percent_change_number_unique_absolute = "N/A"

#Calculate % Change for term 3 International Applications
term_3_international_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_3_details_international_applications, term_3_year_minus_one_Total_Apps_International_Applicants)}%"
term_3_international_applications_percent_change_number = percent_difference_value_calculator_round_1(term_3_details_international_applications, term_3_year_minus_one_Total_Apps_International_Applicants)
try:
    term_3_international_applications_percent_change_number_absolute = np.abs(term_3_international_applications_percent_change_number)
except (TypeError, ValueError):
    term_3_international_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 3 Domestic Applications
term_3_domestic_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_3_details_domestic_applications, term_3_year_minus_one_Total_Apps_Domestic_Applicants)}%"
term_3_domestic_applications_percent_change_number = percent_difference_value_calculator_round_1(term_3_details_domestic_applications, term_3_year_minus_one_Total_Apps_Domestic_Applicants)
try:
    term_3_domestic_applications_percent_change_number_absolute = np.abs(term_3_domestic_applications_percent_change_number)
except (TypeError, ValueError):
    term_3_domestic_applications_percent_change_number_absolute = "N/A"

#Calculation for {{higher_lower_3}}
try:
    if term_3_total_applications_percent_change_number_unique >= 0:
        higher_lower_3 = "higher"
    else:
        higher_lower_3 = "lower"

except (TypeError, ValueError): 
    higher_lower_3 = "FAILED TO CALCULATE"

#Calculation for {{term_3_acceptance_percentage_number}}

term_3_acceptance_percentage_number = division_calculator(term_3_details_accepted_applications, term_3_details_total_applications)

#Calculation for {{term_3_international_percentage_number}}

term_3_international_percentage_number = division_calculator(term_3_details_international_applications, term_3_details_total_applications)

#Calculate % Change for term 4 Total Applications
term_4_total_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_4_details_total_applications, term_4_year_minus_one_Total_Apps_All_Applicants)}%"
term_4_total_applications_percent_change_number = percent_difference_value_calculator_round_1(term_4_details_total_applications, term_4_year_minus_one_Total_Apps_All_Applicants)
try:
    term_4_total_applications_percent_change_number_absolute = np.abs(term_4_total_applications_percent_change_number)
except (TypeError, ValueError):
    term_4_total_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 4 Unique Applications
term_4_total_applications_percent_change_unique = f"{percent_difference_value_calculator_round_1(term_4_details_total_unique_applicants, term_4_year_minus_one_Unique_All_Applicants)}%"
term_4_total_applications_percent_change_number_unique = percent_difference_value_calculator_round_1(term_4_details_total_unique_applicants, term_4_year_minus_one_Unique_All_Applicants)
try:
    term_4_total_applications_percent_change_number_unique_absolute = np.abs(term_4_total_applications_percent_change_number_unique)
except (TypeError, ValueError):
    term_4_total_applications_percent_change_number_unique_absolute = "N/A"

#Calculate % Change for term 4 International Applications
term_4_international_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_4_details_international_applications, term_4_year_minus_one_Total_Apps_International_Applicants)}%"
term_4_international_applications_percent_change_number = percent_difference_value_calculator_round_1(term_4_details_international_applications, term_4_year_minus_one_Total_Apps_International_Applicants)
try:
    term_4_international_applications_percent_change_number_absolute = np.abs(term_4_international_applications_percent_change_number)
except (TypeError, ValueError):
    term_4_international_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 4 Domestic Applications
term_4_domestic_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_4_details_domestic_applications, term_4_year_minus_one_Total_Apps_Domestic_Applicants)}%"
term_4_domestic_applications_percent_change_number = percent_difference_value_calculator_round_1(term_4_details_domestic_applications, term_4_year_minus_one_Total_Apps_Domestic_Applicants)
try:
    term_4_domestic_applications_percent_change_number_absolute = np.abs(term_4_domestic_applications_percent_change_number)
except (TypeError, ValueError):
    term_4_domestic_applications_percent_change_number_absolute = "N/A"



In [ ]:
# Fill in Applications KPI Excel Document
# Load Excel template
template_path = "KC 2025-26 Applications KPI_TEMPLATE.xlsx"
wb = openpyxl.load_workbook(template_path)

# Map
variable_map = {
    #Testing variables, to be removed later
    "Test_1": 0.3432453252345,
    "Test_2": 57,

    #Year variables
    "SuYr": term_1_year_current,
    "FYr": term_2_year_current,
    "WYr": term_3_year_current,
    "SpYr": term_4_year_current,

    #App Comparative Report Summary sheet
    "variable_Current_applications_academic_year": variable_Current_applications_academic_year,
    "AppSummerYr1": term_1_year_current,
    "AppSummerYr2": term_1_year_minus_one,
    "AppSummerYr3": term_1_year_minus_two,
    "AppSummerYr4": term_1_year_minus_three,
    "AppFallYr1": term_2_year_current,
    "AppFallYr2": term_2_year_minus_one,
    "AppFallYr3": term_2_year_minus_two,
    "AppFallYr4": term_2_year_minus_three,
    "AppWinterYr1": term_3_year_current,
    "AppWinterYr2": term_3_year_minus_one,
    "AppWinterYr3": term_3_year_minus_two,
    "AppWinterYr4": term_3_year_minus_three,
    "AppSpringYr1": term_4_year_current,
    "AppSpringYr2": term_4_year_minus_one,
    "AppSpringYr3": term_4_year_minus_two,
    "AppSpringYr4": term_4_year_minus_three,
    "App1": 18,
    "App2": 18,
    "App3": "PEND",
    "App4": 18,
    "App5": 18,
    "App6": 18,
    "App7": 18,
    "App8": 18,
    "App9": 18,
    "App10": 18,
    "App11": 18,
    "App12": "PEND",
    "App13": 18,
    "App14": 18,
    "App15": 18,
    "App16": 18,
    "App17": 18,
    "App18": 18,
    "App19": 18,
    "App20": 18,
    "App21": "PEND",
    "App22": 18,
    "App23": 18,
    "App24": 18,
    "App25": 18,
    "App26": 18,
    "App27": 18,
    "App28": 18,
    "App29": 18,
    "App30": "PEND",
    "App31": 18,
    "App32": 18,
    "App33": 18,
    "App34": 18,
    "App35": 18,
    "App36": 18,

    #Term1Yr YYYY Apps By School sheet
    "term_1_BABUSC_DOMESTIC_Year_1_Applications": x,
    "term_1_BABUSC_DOMESTIC_Year_1_Accepted": x,
    "term_1_BABUSC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_BABUSC_DOMESTIC_Year_1_Denied": x,
    "term_1_BABUSC_DOMESTIC_Year_1_Declined": x,
    "term_1_BABUSC_DOMESTIC_Year_1_Pending": x,
    "term_1_BABUSC_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_BABUSC_DOMESTIC_Year_1_Enrolled": x,
    "term_1_BABUSC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_BABUSC_INTERNATIONAL_Year_1_Applications": x,
    "term_1_BABUSC_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_BABUSC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_BABUSC_INTERNATIONAL_Year_1_Denied": x,
    "term_1_BABUSC_INTERNATIONAL_Year_1_Declined": x,
    "term_1_BABUSC_INTERNATIONAL_Year_1_Pending": x,
    "term_1_BABUSC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_BABUSC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_BABUSC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_BAACTD_DOMESTIC_Year_1_Applications": x,
    "term_1_BAACTD_DOMESTIC_Year_1_Accepted": x,
    "term_1_BAACTD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_BAACTD_DOMESTIC_Year_1_Denied": x,
    "term_1_BAACTD_DOMESTIC_Year_1_Declined": x,
    "term_1_BAACTD_DOMESTIC_Year_1_Pending": x,
    "term_1_BAACTD_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_BAACTD_DOMESTIC_Year_1_Enrolled": x,
    "term_1_BAACTD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_BAACTD_INTERNATIONAL_Year_1_Applications": x,
    "term_1_BAACTD_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_BAACTD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_BAACTD_INTERNATIONAL_Year_1_Denied": x,
    "term_1_BAACTD_INTERNATIONAL_Year_1_Declined": x,
    "term_1_BAACTD_INTERNATIONAL_Year_1_Pending": x,
    "term_1_BAACTD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_BAACTD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_BAACTD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_BAACTD_DOMESTIC_Year_2_Applications": x,
    "term_1_BAACTD_DOMESTIC_Year_2_Accepted": x,
    "term_1_BAACTD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_BAACTD_DOMESTIC_Year_2_Denied": x,
    "term_1_BAACTD_DOMESTIC_Year_2_Declined": x,
    "term_1_BAACTD_DOMESTIC_Year_2_Pending": x,
    "term_1_BAACTD_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_BAACTD_DOMESTIC_Year_2_Enrolled": x,
    "term_1_BAACTD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_BAACTD_INTERNATIONAL_Year_2_Applications": x,
    "term_1_BAACTD_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_BAACTD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_BAACTD_INTERNATIONAL_Year_2_Denied": x,
    "term_1_BAACTD_INTERNATIONAL_Year_2_Declined": x,
    "term_1_BAACTD_INTERNATIONAL_Year_2_Pending": x,
    "term_1_BAACTD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_BAACTD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_BAACTD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_BABUSD_DOMESTIC_Year_1_Applications": x,
    "term_1_BABUSD_DOMESTIC_Year_1_Accepted": x,
    "term_1_BABUSD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_BABUSD_DOMESTIC_Year_1_Denied": x,
    "term_1_BABUSD_DOMESTIC_Year_1_Declined": x,
    "term_1_BABUSD_DOMESTIC_Year_1_Pending": x,
    "term_1_BABUSD_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_BABUSD_DOMESTIC_Year_1_Enrolled": x,
    "term_1_BABUSD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_BABUSD_INTERNATIONAL_Year_1_Applications": x,
    "term_1_BABUSD_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_BABUSD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_BABUSD_INTERNATIONAL_Year_1_Denied": x,
    "term_1_BABUSD_INTERNATIONAL_Year_1_Declined": x,
    "term_1_BABUSD_INTERNATIONAL_Year_1_Pending": x,
    "term_1_BABUSD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_BABUSD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_BABUSD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_BABUSD_DOMESTIC_Year_2_Applications": x,
    "term_1_BABUSD_DOMESTIC_Year_2_Accepted": x,
    "term_1_BABUSD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_BABUSD_DOMESTIC_Year_2_Denied": x,
    "term_1_BABUSD_DOMESTIC_Year_2_Declined": x,
    "term_1_BABUSD_DOMESTIC_Year_2_Pending": x,
    "term_1_BABUSD_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_BABUSD_DOMESTIC_Year_2_Enrolled": x,
    "term_1_BABUSD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_BABUSD_INTERNATIONAL_Year_2_Applications": x,
    "term_1_BABUSD_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_BABUSD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_BABUSD_INTERNATIONAL_Year_2_Denied": x,
    "term_1_BABUSD_INTERNATIONAL_Year_2_Declined": x,
    "term_1_BABUSD_INTERNATIONAL_Year_2_Pending": x,
    "term_1_BABUSD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_BABUSD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_BABUSD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_BAESPM_DOMESTIC_Year_1_Applications": x,
    "term_1_BAESPM_DOMESTIC_Year_1_Accepted": x,
    "term_1_BAESPM_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_BAESPM_DOMESTIC_Year_1_Denied": x,
    "term_1_BAESPM_DOMESTIC_Year_1_Declined": x,
    "term_1_BAESPM_DOMESTIC_Year_1_Pending": x,
    "term_1_BAESPM_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_BAESPM_DOMESTIC_Year_1_Enrolled": x,
    "term_1_BAESPM_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_BAESPM_INTERNATIONAL_Year_1_Applications": x,
    "term_1_BAESPM_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_BAESPM_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_BAESPM_INTERNATIONAL_Year_1_Denied": x,
    "term_1_BAESPM_INTERNATIONAL_Year_1_Declined": x,
    "term_1_BAESPM_INTERNATIONAL_Year_1_Pending": x,
    "term_1_BAESPM_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_BAESPM_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_BAESPM_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_BAESPM_DOMESTIC_Year_2_Applications": x,
    "term_1_BAESPM_DOMESTIC_Year_2_Accepted": x,
    "term_1_BAESPM_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_BAESPM_DOMESTIC_Year_2_Denied": x,
    "term_1_BAESPM_DOMESTIC_Year_2_Declined": x,
    "term_1_BAESPM_DOMESTIC_Year_2_Pending": x,
    "term_1_BAESPM_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_BAESPM_DOMESTIC_Year_2_Enrolled": x,
    "term_1_BAESPM_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_BAESPM_INTERNATIONAL_Year_2_Applications": x,
    "term_1_BAESPM_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_BAESPM_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_BAESPM_INTERNATIONAL_Year_2_Denied": x,
    "term_1_BAESPM_INTERNATIONAL_Year_2_Declined": x,
    "term_1_BAESPM_INTERNATIONAL_Year_2_Pending": x,
    "term_1_BAESPM_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_BAESPM_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_BAESPM_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_BAHRMD_DOMESTIC_Year_1_Applications": x,
    "term_1_BAHRMD_DOMESTIC_Year_1_Accepted": x,
    "term_1_BAHRMD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_BAHRMD_DOMESTIC_Year_1_Denied": x,
    "term_1_BAHRMD_DOMESTIC_Year_1_Declined": x,
    "term_1_BAHRMD_DOMESTIC_Year_1_Pending": x,
    "term_1_BAHRMD_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_BAHRMD_DOMESTIC_Year_1_Enrolled": x,
    "term_1_BAHRMD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_BAHRMD_INTERNATIONAL_Year_1_Applications": x,
    "term_1_BAHRMD_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_BAHRMD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_BAHRMD_INTERNATIONAL_Year_1_Denied": x,
    "term_1_BAHRMD_INTERNATIONAL_Year_1_Declined": x,
    "term_1_BAHRMD_INTERNATIONAL_Year_1_Pending": x,
    "term_1_BAHRMD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_BAHRMD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_BAHRMD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_BAHRMD_DOMESTIC_Year_2_Applications": x,
    "term_1_BAHRMD_DOMESTIC_Year_2_Accepted": x,
    "term_1_BAHRMD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_BAHRMD_DOMESTIC_Year_2_Denied": x,
    "term_1_BAHRMD_DOMESTIC_Year_2_Declined": x,
    "term_1_BAHRMD_DOMESTIC_Year_2_Pending": x,
    "term_1_BAHRMD_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_BAHRMD_DOMESTIC_Year_2_Enrolled": x,
    "term_1_BAHRMD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_BAHRMD_INTERNATIONAL_Year_2_Applications": x,
    "term_1_BAHRMD_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_BAHRMD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_BAHRMD_INTERNATIONAL_Year_2_Denied": x,
    "term_1_BAHRMD_INTERNATIONAL_Year_2_Declined": x,
    "term_1_BAHRMD_INTERNATIONAL_Year_2_Pending": x,
    "term_1_BAHRMD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_BAHRMD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_BAHRMD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_GSCMD_DOMESTIC_Year_1_Applications": x,
    "term_1_GSCMD_DOMESTIC_Year_1_Accepted": x,
    "term_1_GSCMD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_GSCMD_DOMESTIC_Year_1_Denied": x,
    "term_1_GSCMD_DOMESTIC_Year_1_Declined": x,
    "term_1_GSCMD_DOMESTIC_Year_1_Pending": x,
    "term_1_GSCMD_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_GSCMD_DOMESTIC_Year_1_Enrolled": x,
    "term_1_GSCMD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_GSCMD_INTERNATIONAL_Year_1_Applications": x,
    "term_1_GSCMD_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_GSCMD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_GSCMD_INTERNATIONAL_Year_1_Denied": x,
    "term_1_GSCMD_INTERNATIONAL_Year_1_Declined": x,
    "term_1_GSCMD_INTERNATIONAL_Year_1_Pending": x,
    "term_1_GSCMD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_GSCMD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_GSCMD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_GSCMD_DOMESTIC_Year_2_Applications": x,
    "term_1_GSCMD_DOMESTIC_Year_2_Accepted": x,
    "term_1_GSCMD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_GSCMD_DOMESTIC_Year_2_Denied": x,
    "term_1_GSCMD_DOMESTIC_Year_2_Declined": x,
    "term_1_GSCMD_DOMESTIC_Year_2_Pending": x,
    "term_1_GSCMD_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_GSCMD_DOMESTIC_Year_2_Enrolled": x,
    "term_1_GSCMD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_GSCMD_INTERNATIONAL_Year_2_Applications": x,
    "term_1_GSCMD_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_GSCMD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_GSCMD_INTERNATIONAL_Year_2_Denied": x,
    "term_1_GSCMD_INTERNATIONAL_Year_2_Declined": x,
    "term_1_GSCMD_INTERNATIONAL_Year_2_Pending": x,
    "term_1_GSCMD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_GSCMD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_GSCMD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_DIGMD_DOMESTIC_Year_1_Applications": x,
    "term_1_DIGMD_DOMESTIC_Year_1_Accepted": x,
    "term_1_DIGMD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_DIGMD_DOMESTIC_Year_1_Denied": x,
    "term_1_DIGMD_DOMESTIC_Year_1_Declined": x,
    "term_1_DIGMD_DOMESTIC_Year_1_Pending": x,
    "term_1_DIGMD_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_DIGMD_DOMESTIC_Year_1_Enrolled": x,
    "term_1_DIGMD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_DIGMD_INTERNATIONAL_Year_1_Applications": x,
    "term_1_DIGMD_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_DIGMD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_DIGMD_INTERNATIONAL_Year_1_Denied": x,
    "term_1_DIGMD_INTERNATIONAL_Year_1_Declined": x,
    "term_1_DIGMD_INTERNATIONAL_Year_1_Pending": x,
    "term_1_DIGMD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_DIGMD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_DIGMD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_DIGMD_DOMESTIC_Year_2_Applications": x,
    "term_1_DIGMD_DOMESTIC_Year_2_Accepted": x,
    "term_1_DIGMD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_DIGMD_DOMESTIC_Year_2_Denied": x,
    "term_1_DIGMD_DOMESTIC_Year_2_Declined": x,
    "term_1_DIGMD_DOMESTIC_Year_2_Pending": x,
    "term_1_DIGMD_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_DIGMD_DOMESTIC_Year_2_Enrolled": x,
    "term_1_DIGMD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_DIGMD_INTERNATIONAL_Year_2_Applications": x,
    "term_1_DIGMD_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_DIGMD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_DIGMD_INTERNATIONAL_Year_2_Denied": x,
    "term_1_DIGMD_INTERNATIONAL_Year_2_Declined": x,
    "term_1_DIGMD_INTERNATIONAL_Year_2_Pending": x,
    "term_1_DIGMD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_DIGMD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_DIGMD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_BAATCO_DOMESTIC_Year_1_Applications": x,
    "term_1_BAATCO_DOMESTIC_Year_1_Accepted": x,
    "term_1_BAATCO_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_BAATCO_DOMESTIC_Year_1_Denied": x,
    "term_1_BAATCO_DOMESTIC_Year_1_Declined": x,
    "term_1_BAATCO_DOMESTIC_Year_1_Pending": x,
    "term_1_BAATCO_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_BAATCO_DOMESTIC_Year_1_Enrolled": x,
    "term_1_BAATCO_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_BAATCO_INTERNATIONAL_Year_1_Applications": x,
    "term_1_BAATCO_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_BAATCO_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_BAATCO_INTERNATIONAL_Year_1_Denied": x,
    "term_1_BAATCO_INTERNATIONAL_Year_1_Declined": x,
    "term_1_BAATCO_INTERNATIONAL_Year_1_Pending": x,
    "term_1_BAATCO_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_BAATCO_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_BAATCO_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_BAATCO_DOMESTIC_Year_2_Applications": x,
    "term_1_BAATCO_DOMESTIC_Year_2_Accepted": x,
    "term_1_BAATCO_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_BAATCO_DOMESTIC_Year_2_Denied": x,
    "term_1_BAATCO_DOMESTIC_Year_2_Declined": x,
    "term_1_BAATCO_DOMESTIC_Year_2_Pending": x,
    "term_1_BAATCO_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_BAATCO_DOMESTIC_Year_2_Enrolled": x,
    "term_1_BAATCO_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_BAATCO_INTERNATIONAL_Year_2_Applications": x,
    "term_1_BAATCO_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_BAATCO_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_BAATCO_INTERNATIONAL_Year_2_Denied": x,
    "term_1_BAATCO_INTERNATIONAL_Year_2_Declined": x,
    "term_1_BAATCO_INTERNATIONAL_Year_2_Pending": x,
    "term_1_BAATCO_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_BAATCO_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_BAATCO_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_BABSCO_DOMESTIC_Year_1_Applications": x,
    "term_1_BABSCO_DOMESTIC_Year_1_Accepted": x,
    "term_1_BABSCO_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_BABSCO_DOMESTIC_Year_1_Denied": x,
    "term_1_BABSCO_DOMESTIC_Year_1_Declined": x,
    "term_1_BABSCO_DOMESTIC_Year_1_Pending": x,
    "term_1_BABSCO_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_BABSCO_DOMESTIC_Year_1_Enrolled": x,
    "term_1_BABSCO_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_BABSCO_INTERNATIONAL_Year_1_Applications": x,
    "term_1_BABSCO_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_BABSCO_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_BABSCO_INTERNATIONAL_Year_1_Denied": x,
    "term_1_BABSCO_INTERNATIONAL_Year_1_Declined": x,
    "term_1_BABSCO_INTERNATIONAL_Year_1_Pending": x,
    "term_1_BABSCO_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_BABSCO_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_BABSCO_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_BABSCO_DOMESTIC_Year_2_Applications": x,
    "term_1_BABSCO_DOMESTIC_Year_2_Accepted": x,
    "term_1_BABSCO_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_BABSCO_DOMESTIC_Year_2_Denied": x,
    "term_1_BABSCO_DOMESTIC_Year_2_Declined": x,
    "term_1_BABSCO_DOMESTIC_Year_2_Pending": x,
    "term_1_BABSCO_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_BABSCO_DOMESTIC_Year_2_Enrolled": x,
    "term_1_BABSCO_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_BABSCO_INTERNATIONAL_Year_2_Applications": x,
    "term_1_BABSCO_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_BABSCO_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_BABSCO_INTERNATIONAL_Year_2_Denied": x,
    "term_1_BABSCO_INTERNATIONAL_Year_2_Declined": x,
    "term_1_BABSCO_INTERNATIONAL_Year_2_Pending": x,
    "term_1_BABSCO_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_BABSCO_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_BABSCO_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_NAO_DOMESTIC_Year_1_Applications": x,
    "term_1_NAO_DOMESTIC_Year_1_Accepted": x,
    "term_1_NAO_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_NAO_DOMESTIC_Year_1_Denied": x,
    "term_1_NAO_DOMESTIC_Year_1_Declined": x,
    "term_1_NAO_DOMESTIC_Year_1_Pending": x,
    "term_1_NAO_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_NAO_DOMESTIC_Year_1_Enrolled": x,
    "term_1_NAO_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_NAO_INTERNATIONAL_Year_1_Applications": x,
    "term_1_NAO_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_NAO_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_NAO_INTERNATIONAL_Year_1_Denied": x,
    "term_1_NAO_INTERNATIONAL_Year_1_Declined": x,
    "term_1_NAO_INTERNATIONAL_Year_1_Pending": x,
    "term_1_NAO_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_NAO_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_NAO_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_NAO_DOMESTIC_Year_2_Applications": x,
    "term_1_NAO_DOMESTIC_Year_2_Accepted": x,
    "term_1_NAO_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_NAO_DOMESTIC_Year_2_Denied": x,
    "term_1_NAO_DOMESTIC_Year_2_Declined": x,
    "term_1_NAO_DOMESTIC_Year_2_Pending": x,
    "term_1_NAO_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_NAO_DOMESTIC_Year_2_Enrolled": x,
    "term_1_NAO_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_NAO_INTERNATIONAL_Year_2_Applications": x,
    "term_1_NAO_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_NAO_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_NAO_INTERNATIONAL_Year_2_Denied": x,
    "term_1_NAO_INTERNATIONAL_Year_2_Declined": x,
    "term_1_NAO_INTERNATIONAL_Year_2_Pending": x,
    "term_1_NAO_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_NAO_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_NAO_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_ALO_DOMESTIC_Year_1_Applications": x,
    "term_1_ALO_DOMESTIC_Year_1_Accepted": x,
    "term_1_ALO_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_ALO_DOMESTIC_Year_1_Denied": x,
    "term_1_ALO_DOMESTIC_Year_1_Declined": x,
    "term_1_ALO_DOMESTIC_Year_1_Pending": x,
    "term_1_ALO_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_ALO_DOMESTIC_Year_1_Enrolled": x,
    "term_1_ALO_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_ALO_INTERNATIONAL_Year_1_Applications": x,
    "term_1_ALO_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_ALO_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_ALO_INTERNATIONAL_Year_1_Denied": x,
    "term_1_ALO_INTERNATIONAL_Year_1_Declined": x,
    "term_1_ALO_INTERNATIONAL_Year_1_Pending": x,
    "term_1_ALO_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_ALO_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_ALO_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_ALO_DOMESTIC_Year_2_Applications": x,
    "term_1_ALO_DOMESTIC_Year_2_Accepted": x,
    "term_1_ALO_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_ALO_DOMESTIC_Year_2_Denied": x,
    "term_1_ALO_DOMESTIC_Year_2_Declined": x,
    "term_1_ALO_DOMESTIC_Year_2_Pending": x,
    "term_1_ALO_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_ALO_DOMESTIC_Year_2_Enrolled": x,
    "term_1_ALO_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_ALO_INTERNATIONAL_Year_2_Applications": x,
    "term_1_ALO_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_ALO_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_ALO_INTERNATIONAL_Year_2_Denied": x,
    "term_1_ALO_INTERNATIONAL_Year_2_Declined": x,
    "term_1_ALO_INTERNATIONAL_Year_2_Pending": x,
    "term_1_ALO_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_ALO_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_ALO_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_HOSMNG_DOMESTIC_Year_1_Applications": x,
    "term_1_HOSMNG_DOMESTIC_Year_1_Accepted": x,
    "term_1_HOSMNG_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_HOSMNG_DOMESTIC_Year_1_Denied": x,
    "term_1_HOSMNG_DOMESTIC_Year_1_Declined": x,
    "term_1_HOSMNG_DOMESTIC_Year_1_Pending": x,
    "term_1_HOSMNG_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_HOSMNG_DOMESTIC_Year_1_Enrolled": x,
    "term_1_HOSMNG_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_HOSMNG_INTERNATIONAL_Year_1_Applications": x,
    "term_1_HOSMNG_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_HOSMNG_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_HOSMNG_INTERNATIONAL_Year_1_Denied": x,
    "term_1_HOSMNG_INTERNATIONAL_Year_1_Declined": x,
    "term_1_HOSMNG_INTERNATIONAL_Year_1_Pending": x,
    "term_1_HOSMNG_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_HOSMNG_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_HOSMNG_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_HOSMNG_DOMESTIC_Year_2_Applications": x,
    "term_1_HOSMNG_DOMESTIC_Year_2_Accepted": x,
    "term_1_HOSMNG_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_HOSMNG_DOMESTIC_Year_2_Denied": x,
    "term_1_HOSMNG_DOMESTIC_Year_2_Declined": x,
    "term_1_HOSMNG_DOMESTIC_Year_2_Pending": x,
    "term_1_HOSMNG_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_HOSMNG_DOMESTIC_Year_2_Enrolled": x,
    "term_1_HOSMNG_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_HOSMNG_INTERNATIONAL_Year_2_Applications": x,
    "term_1_HOSMNG_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_HOSMNG_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_HOSMNG_INTERNATIONAL_Year_2_Denied": x,
    "term_1_HOSMNG_INTERNATIONAL_Year_2_Declined": x,
    "term_1_HOSMNG_INTERNATIONAL_Year_2_Pending": x,
    "term_1_HOSMNG_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_HOSMNG_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_HOSMNG_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_BAHRMC_DOMESTIC_Year_1_Applications": x,
    "term_1_BAHRMC_DOMESTIC_Year_1_Accepted": x,
    "term_1_BAHRMC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_BAHRMC_DOMESTIC_Year_1_Denied": x,
    "term_1_BAHRMC_DOMESTIC_Year_1_Declined": x,
    "term_1_BAHRMC_DOMESTIC_Year_1_Pending": x,
    "term_1_BAHRMC_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_BAHRMC_DOMESTIC_Year_1_Enrolled": x,
    "term_1_BAHRMC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_BAHRMC_INTERNATIONAL_Year_1_Applications": x,
    "term_1_BAHRMC_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_BAHRMC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_BAHRMC_INTERNATIONAL_Year_1_Denied": x,
    "term_1_BAHRMC_INTERNATIONAL_Year_1_Declined": x,
    "term_1_BAHRMC_INTERNATIONAL_Year_1_Pending": x,
    "term_1_BAHRMC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_BAHRMC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_BAHRMC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_OFAC_DOMESTIC_Year_1_Applications": x,
    "term_1_OFAC_DOMESTIC_Year_1_Accepted": x,
    "term_1_OFAC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_OFAC_DOMESTIC_Year_1_Denied": x,
    "term_1_OFAC_DOMESTIC_Year_1_Declined": x,
    "term_1_OFAC_DOMESTIC_Year_1_Pending": x,
    "term_1_OFAC_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_OFAC_DOMESTIC_Year_1_Enrolled": x,
    "term_1_OFAC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_OFAC_INTERNATIONAL_Year_1_Applications": x,
    "term_1_OFAC_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_OFAC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_OFAC_INTERNATIONAL_Year_1_Denied": x,
    "term_1_OFAC_INTERNATIONAL_Year_1_Declined": x,
    "term_1_OFAC_INTERNATIONAL_Year_1_Pending": x,
    "term_1_OFAC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_OFAC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_OFAC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_EAP_DOMESTIC_Year_1_Applications": x,
    "term_1_EAP_DOMESTIC_Year_1_Accepted": x,
    "term_1_EAP_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_EAP_DOMESTIC_Year_1_Denied": x,
    "term_1_EAP_DOMESTIC_Year_1_Declined": x,
    "term_1_EAP_DOMESTIC_Year_1_Pending": x,
    "term_1_EAP_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_EAP_DOMESTIC_Year_1_Enrolled": x,
    "term_1_EAP_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_EAP_INTERNATIONAL_Year_1_Applications": x,
    "term_1_EAP_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_EAP_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_EAP_INTERNATIONAL_Year_1_Denied": x,
    "term_1_EAP_INTERNATIONAL_Year_1_Declined": x,
    "term_1_EAP_INTERNATIONAL_Year_1_Pending": x,
    "term_1_EAP_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_EAP_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_EAP_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_HSAFA_DOMESTIC_Year_1_Applications": x,
    "term_1_HSAFA_DOMESTIC_Year_1_Accepted": x,
    "term_1_HSAFA_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_HSAFA_DOMESTIC_Year_1_Denied": x,
    "term_1_HSAFA_DOMESTIC_Year_1_Declined": x,
    "term_1_HSAFA_DOMESTIC_Year_1_Pending": x,
    "term_1_HSAFA_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_HSAFA_DOMESTIC_Year_1_Enrolled": x,
    "term_1_HSAFA_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_HSAFA_INTERNATIONAL_Year_1_Applications": x,
    "term_1_HSAFA_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_HSAFA_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_HSAFA_INTERNATIONAL_Year_1_Denied": x,
    "term_1_HSAFA_INTERNATIONAL_Year_1_Declined": x,
    "term_1_HSAFA_INTERNATIONAL_Year_1_Pending": x,
    "term_1_HSAFA_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_HSAFA_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_HSAFA_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_HSAD_DOMESTIC_Year_1_Applications": x,
    "term_1_HSAD_DOMESTIC_Year_1_Accepted": x,
    "term_1_HSAD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_HSAD_DOMESTIC_Year_1_Denied": x,
    "term_1_HSAD_DOMESTIC_Year_1_Declined": x,
    "term_1_HSAD_DOMESTIC_Year_1_Pending": x,
    "term_1_HSAD_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_HSAD_DOMESTIC_Year_1_Enrolled": x,
    "term_1_HSAD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_HSAD_INTERNATIONAL_Year_1_Applications": x,
    "term_1_HSAD_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_HSAD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_HSAD_INTERNATIONAL_Year_1_Denied": x,
    "term_1_HSAD_INTERNATIONAL_Year_1_Declined": x,
    "term_1_HSAD_INTERNATIONAL_Year_1_Pending": x,
    "term_1_HSAD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_HSAD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_HSAD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_HSGD_DOMESTIC_Year_1_Applications": x,
    "term_1_HSGD_DOMESTIC_Year_1_Accepted": x,
    "term_1_HSGD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_HSGD_DOMESTIC_Year_1_Denied": x,
    "term_1_HSGD_DOMESTIC_Year_1_Declined": x,
    "term_1_HSGD_DOMESTIC_Year_1_Pending": x,
    "term_1_HSGD_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_HSGD_DOMESTIC_Year_1_Enrolled": x,
    "term_1_HSGD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_HSGD_INTERNATIONAL_Year_1_Applications": x,
    "term_1_HSGD_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_HSGD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_HSGD_INTERNATIONAL_Year_1_Denied": x,
    "term_1_HSGD_INTERNATIONAL_Year_1_Declined": x,
    "term_1_HSGD_INTERNATIONAL_Year_1_Pending": x,
    "term_1_HSGD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_HSGD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_HSGD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_HSNP_DOMESTIC_Year_1_Applications": x,
    "term_1_HSNP_DOMESTIC_Year_1_Accepted": x,
    "term_1_HSNP_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_HSNP_DOMESTIC_Year_1_Denied": x,
    "term_1_HSNP_DOMESTIC_Year_1_Declined": x,
    "term_1_HSNP_DOMESTIC_Year_1_Pending": x,
    "term_1_HSNP_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_HSNP_DOMESTIC_Year_1_Enrolled": x,
    "term_1_HSNP_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_HSNP_INTERNATIONAL_Year_1_Applications": x,
    "term_1_HSNP_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_HSNP_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_HSNP_INTERNATIONAL_Year_1_Denied": x,
    "term_1_HSNP_INTERNATIONAL_Year_1_Declined": x,
    "term_1_HSNP_INTERNATIONAL_Year_1_Pending": x,
    "term_1_HSNP_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_HSNP_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_HSNP_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_UTELEM_DOMESTIC_Year_1_Applications": x,
    "term_1_UTELEM_DOMESTIC_Year_1_Accepted": x,
    "term_1_UTELEM_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_UTELEM_DOMESTIC_Year_1_Denied": x,
    "term_1_UTELEM_DOMESTIC_Year_1_Declined": x,
    "term_1_UTELEM_DOMESTIC_Year_1_Pending": x,
    "term_1_UTELEM_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_UTELEM_DOMESTIC_Year_1_Enrolled": x,
    "term_1_UTELEM_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_UTELEM_INTERNATIONAL_Year_1_Applications": x,
    "term_1_UTELEM_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_UTELEM_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_UTELEM_INTERNATIONAL_Year_1_Denied": x,
    "term_1_UTELEM_INTERNATIONAL_Year_1_Declined": x,
    "term_1_UTELEM_INTERNATIONAL_Year_1_Pending": x,
    "term_1_UTELEM_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_UTELEM_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_UTELEM_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_UTELEM_DOMESTIC_Year_2_Applications": x,
    "term_1_UTELEM_DOMESTIC_Year_2_Accepted": x,
    "term_1_UTELEM_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_UTELEM_DOMESTIC_Year_2_Denied": x,
    "term_1_UTELEM_DOMESTIC_Year_2_Declined": x,
    "term_1_UTELEM_DOMESTIC_Year_2_Pending": x,
    "term_1_UTELEM_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_UTELEM_DOMESTIC_Year_2_Enrolled": x,
    "term_1_UTELEM_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_UTELEM_INTERNATIONAL_Year_2_Applications": x,
    "term_1_UTELEM_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_UTELEM_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_UTELEM_INTERNATIONAL_Year_2_Denied": x,
    "term_1_UTELEM_INTERNATIONAL_Year_2_Declined": x,
    "term_1_UTELEM_INTERNATIONAL_Year_2_Pending": x,
    "term_1_UTELEM_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_UTELEM_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_UTELEM_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_UTSEC_DOMESTIC_Year_1_Applications": x,
    "term_1_UTSEC_DOMESTIC_Year_1_Accepted": x,
    "term_1_UTSEC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_UTSEC_DOMESTIC_Year_1_Denied": x,
    "term_1_UTSEC_DOMESTIC_Year_1_Declined": x,
    "term_1_UTSEC_DOMESTIC_Year_1_Pending": x,
    "term_1_UTSEC_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_UTSEC_DOMESTIC_Year_1_Enrolled": x,
    "term_1_UTSEC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_UTSEC_INTERNATIONAL_Year_1_Applications": x,
    "term_1_UTSEC_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_UTSEC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_UTSEC_INTERNATIONAL_Year_1_Denied": x,
    "term_1_UTSEC_INTERNATIONAL_Year_1_Declined": x,
    "term_1_UTSEC_INTERNATIONAL_Year_1_Pending": x,
    "term_1_UTSEC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_UTSEC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_UTSEC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_UTSEC_DOMESTIC_Year_2_Applications": x,
    "term_1_UTSEC_DOMESTIC_Year_2_Accepted": x,
    "term_1_UTSEC_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_UTSEC_DOMESTIC_Year_2_Denied": x,
    "term_1_UTSEC_DOMESTIC_Year_2_Declined": x,
    "term_1_UTSEC_DOMESTIC_Year_2_Pending": x,
    "term_1_UTSEC_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_UTSEC_DOMESTIC_Year_2_Enrolled": x,
    "term_1_UTSEC_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_UTSEC_INTERNATIONAL_Year_2_Applications": x,
    "term_1_UTSEC_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_UTSEC_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_UTSEC_INTERNATIONAL_Year_2_Denied": x,
    "term_1_UTSEC_INTERNATIONAL_Year_2_Declined": x,
    "term_1_UTSEC_INTERNATIONAL_Year_2_Pending": x,
    "term_1_UTSEC_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_UTSEC_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_UTSEC_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_ENVTD_DOMESTIC_Year_1_Applications": x,
    "term_1_ENVTD_DOMESTIC_Year_1_Accepted": x,
    "term_1_ENVTD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_ENVTD_DOMESTIC_Year_1_Denied": x,
    "term_1_ENVTD_DOMESTIC_Year_1_Declined": x,
    "term_1_ENVTD_DOMESTIC_Year_1_Pending": x,
    "term_1_ENVTD_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_ENVTD_DOMESTIC_Year_1_Enrolled": x,
    "term_1_ENVTD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_ENVTD_INTERNATIONAL_Year_1_Applications": x,
    "term_1_ENVTD_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_ENVTD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_ENVTD_INTERNATIONAL_Year_1_Denied": x,
    "term_1_ENVTD_INTERNATIONAL_Year_1_Declined": x,
    "term_1_ENVTD_INTERNATIONAL_Year_1_Pending": x,
    "term_1_ENVTD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_ENVTD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_ENVTD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_ENVTD_DOMESTIC_Year_2_Applications": x,
    "term_1_ENVTD_DOMESTIC_Year_2_Accepted": x,
    "term_1_ENVTD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_ENVTD_DOMESTIC_Year_2_Denied": x,
    "term_1_ENVTD_DOMESTIC_Year_2_Declined": x,
    "term_1_ENVTD_DOMESTIC_Year_2_Pending": x,
    "term_1_ENVTD_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_ENVTD_DOMESTIC_Year_2_Enrolled": x,
    "term_1_ENVTD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_ENVTD_INTERNATIONAL_Year_2_Applications": x,
    "term_1_ENVTD_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_ENVTD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_ENVTD_INTERNATIONAL_Year_2_Denied": x,
    "term_1_ENVTD_INTERNATIONAL_Year_2_Declined": x,
    "term_1_ENVTD_INTERNATIONAL_Year_2_Pending": x,
    "term_1_ENVTD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_ENVTD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_ENVTD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_ENVNC_DOMESTIC_Year_1_Applications": x,
    "term_1_ENVNC_DOMESTIC_Year_1_Accepted": x,
    "term_1_ENVNC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_ENVNC_DOMESTIC_Year_1_Denied": x,
    "term_1_ENVNC_DOMESTIC_Year_1_Declined": x,
    "term_1_ENVNC_DOMESTIC_Year_1_Pending": x,
    "term_1_ENVNC_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_ENVNC_DOMESTIC_Year_1_Enrolled": x,
    "term_1_ENVNC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_ENVNC_INTERNATIONAL_Year_1_Applications": x,
    "term_1_ENVNC_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_ENVNC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_ENVNC_INTERNATIONAL_Year_1_Denied": x,
    "term_1_ENVNC_INTERNATIONAL_Year_1_Declined": x,
    "term_1_ENVNC_INTERNATIONAL_Year_1_Pending": x,
    "term_1_ENVNC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_ENVNC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_ENVNC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_ENVNC_DOMESTIC_Year_2_Applications": x,
    "term_1_ENVNC_DOMESTIC_Year_2_Accepted": x,
    "term_1_ENVNC_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_ENVNC_DOMESTIC_Year_2_Denied": x,
    "term_1_ENVNC_DOMESTIC_Year_2_Declined": x,
    "term_1_ENVNC_DOMESTIC_Year_2_Pending": x,
    "term_1_ENVNC_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_ENVNC_DOMESTIC_Year_2_Enrolled": x,
    "term_1_ENVNC_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_ENVNC_INTERNATIONAL_Year_2_Applications": x,
    "term_1_ENVNC_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_ENVNC_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_ENVNC_INTERNATIONAL_Year_2_Denied": x,
    "term_1_ENVNC_INTERNATIONAL_Year_2_Declined": x,
    "term_1_ENVNC_INTERNATIONAL_Year_2_Pending": x,
    "term_1_ENVNC_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_ENVNC_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_ENVNC_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_UTARTS_DOMESTIC_Year_1_Applications": x,
    "term_1_UTARTS_DOMESTIC_Year_1_Accepted": x,
    "term_1_UTARTS_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_UTARTS_DOMESTIC_Year_1_Denied": x,
    "term_1_UTARTS_DOMESTIC_Year_1_Declined": x,
    "term_1_UTARTS_DOMESTIC_Year_1_Pending": x,
    "term_1_UTARTS_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_UTARTS_DOMESTIC_Year_1_Enrolled": x,
    "term_1_UTARTS_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_UTARTS_INTERNATIONAL_Year_1_Applications": x,
    "term_1_UTARTS_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_UTARTS_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_UTARTS_INTERNATIONAL_Year_1_Denied": x,
    "term_1_UTARTS_INTERNATIONAL_Year_1_Declined": x,
    "term_1_UTARTS_INTERNATIONAL_Year_1_Pending": x,
    "term_1_UTARTS_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_UTARTS_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_UTARTS_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_UTARTS_DOMESTIC_Year_2_Applications": x,
    "term_1_UTARTS_DOMESTIC_Year_2_Accepted": x,
    "term_1_UTARTS_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_UTARTS_DOMESTIC_Year_2_Denied": x,
    "term_1_UTARTS_DOMESTIC_Year_2_Declined": x,
    "term_1_UTARTS_DOMESTIC_Year_2_Pending": x,
    "term_1_UTARTS_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_UTARTS_DOMESTIC_Year_2_Enrolled": x,
    "term_1_UTARTS_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_UTARTS_INTERNATIONAL_Year_2_Applications": x,
    "term_1_UTARTS_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_UTARTS_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_UTARTS_INTERNATIONAL_Year_2_Denied": x,
    "term_1_UTARTS_INTERNATIONAL_Year_2_Declined": x,
    "term_1_UTARTS_INTERNATIONAL_Year_2_Pending": x,
    "term_1_UTARTS_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_UTARTS_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_UTARTS_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_UTCOMM_DOMESTIC_Year_1_Applications": x,
    "term_1_UTCOMM_DOMESTIC_Year_1_Accepted": x,
    "term_1_UTCOMM_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_UTCOMM_DOMESTIC_Year_1_Denied": x,
    "term_1_UTCOMM_DOMESTIC_Year_1_Declined": x,
    "term_1_UTCOMM_DOMESTIC_Year_1_Pending": x,
    "term_1_UTCOMM_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_UTCOMM_DOMESTIC_Year_1_Enrolled": x,
    "term_1_UTCOMM_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_UTCOMM_INTERNATIONAL_Year_1_Applications": x,
    "term_1_UTCOMM_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_UTCOMM_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_UTCOMM_INTERNATIONAL_Year_1_Denied": x,
    "term_1_UTCOMM_INTERNATIONAL_Year_1_Declined": x,
    "term_1_UTCOMM_INTERNATIONAL_Year_1_Pending": x,
    "term_1_UTCOMM_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_UTCOMM_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_UTCOMM_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_UTCOMM_DOMESTIC_Year_2_Applications": x,
    "term_1_UTCOMM_DOMESTIC_Year_2_Accepted": x,
    "term_1_UTCOMM_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_UTCOMM_DOMESTIC_Year_2_Denied": x,
    "term_1_UTCOMM_DOMESTIC_Year_2_Declined": x,
    "term_1_UTCOMM_DOMESTIC_Year_2_Pending": x,
    "term_1_UTCOMM_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_UTCOMM_DOMESTIC_Year_2_Enrolled": x,
    "term_1_UTCOMM_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_UTCOMM_INTERNATIONAL_Year_2_Applications": x,
    "term_1_UTCOMM_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_UTCOMM_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_UTCOMM_INTERNATIONAL_Year_2_Denied": x,
    "term_1_UTCOMM_INTERNATIONAL_Year_2_Declined": x,
    "term_1_UTCOMM_INTERNATIONAL_Year_2_Pending": x,
    "term_1_UTCOMM_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_UTCOMM_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_UTCOMM_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_UTENG_DOMESTIC_Year_1_Applications": x,
    "term_1_UTENG_DOMESTIC_Year_1_Accepted": x,
    "term_1_UTENG_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_UTENG_DOMESTIC_Year_1_Denied": x,
    "term_1_UTENG_DOMESTIC_Year_1_Declined": x,
    "term_1_UTENG_DOMESTIC_Year_1_Pending": x,
    "term_1_UTENG_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_UTENG_DOMESTIC_Year_1_Enrolled": x,
    "term_1_UTENG_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_UTENG_INTERNATIONAL_Year_1_Applications": x,
    "term_1_UTENG_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_UTENG_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_UTENG_INTERNATIONAL_Year_1_Denied": x,
    "term_1_UTENG_INTERNATIONAL_Year_1_Declined": x,
    "term_1_UTENG_INTERNATIONAL_Year_1_Pending": x,
    "term_1_UTENG_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_UTENG_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_UTENG_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_UTSCGS_DOMESTIC_Year_1_Applications": x,
    "term_1_UTSCGS_DOMESTIC_Year_1_Accepted": x,
    "term_1_UTSCGS_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_UTSCGS_DOMESTIC_Year_1_Denied": x,
    "term_1_UTSCGS_DOMESTIC_Year_1_Declined": x,
    "term_1_UTSCGS_DOMESTIC_Year_1_Pending": x,
    "term_1_UTSCGS_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_UTSCGS_DOMESTIC_Year_1_Enrolled": x,
    "term_1_UTSCGS_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_UTSCGS_INTERNATIONAL_Year_1_Applications": x,
    "term_1_UTSCGS_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_UTSCGS_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_UTSCGS_INTERNATIONAL_Year_1_Denied": x,
    "term_1_UTSCGS_INTERNATIONAL_Year_1_Declined": x,
    "term_1_UTSCGS_INTERNATIONAL_Year_1_Pending": x,
    "term_1_UTSCGS_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_UTSCGS_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_UTSCGS_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_UTSCGS_DOMESTIC_Year_2_Applications": x,
    "term_1_UTSCGS_DOMESTIC_Year_2_Accepted": x,
    "term_1_UTSCGS_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_UTSCGS_DOMESTIC_Year_2_Denied": x,
    "term_1_UTSCGS_DOMESTIC_Year_2_Declined": x,
    "term_1_UTSCGS_DOMESTIC_Year_2_Pending": x,
    "term_1_UTSCGS_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_UTSCGS_DOMESTIC_Year_2_Enrolled": x,
    "term_1_UTSCGS_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_UTSCGS_INTERNATIONAL_Year_2_Applications": x,
    "term_1_UTSCGS_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_UTSCGS_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_UTSCGS_INTERNATIONAL_Year_2_Denied": x,
    "term_1_UTSCGS_INTERNATIONAL_Year_2_Declined": x,
    "term_1_UTSCGS_INTERNATIONAL_Year_2_Pending": x,
    "term_1_UTSCGS_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_UTSCGS_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_UTSCGS_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_UTSCWK_DOMESTIC_Year_1_Applications": x,
    "term_1_UTSCWK_DOMESTIC_Year_1_Accepted": x,
    "term_1_UTSCWK_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_UTSCWK_DOMESTIC_Year_1_Denied": x,
    "term_1_UTSCWK_DOMESTIC_Year_1_Declined": x,
    "term_1_UTSCWK_DOMESTIC_Year_1_Pending": x,
    "term_1_UTSCWK_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_UTSCWK_DOMESTIC_Year_1_Enrolled": x,
    "term_1_UTSCWK_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_UTSCWK_INTERNATIONAL_Year_1_Applications": x,
    "term_1_UTSCWK_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_UTSCWK_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_UTSCWK_INTERNATIONAL_Year_1_Denied": x,
    "term_1_UTSCWK_INTERNATIONAL_Year_1_Declined": x,
    "term_1_UTSCWK_INTERNATIONAL_Year_1_Pending": x,
    "term_1_UTSCWK_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_UTSCWK_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_UTSCWK_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_UTSCWK_DOMESTIC_Year_2_Applications": x,
    "term_1_UTSCWK_DOMESTIC_Year_2_Accepted": x,
    "term_1_UTSCWK_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_UTSCWK_DOMESTIC_Year_2_Denied": x,
    "term_1_UTSCWK_DOMESTIC_Year_2_Declined": x,
    "term_1_UTSCWK_DOMESTIC_Year_2_Pending": x,
    "term_1_UTSCWK_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_UTSCWK_DOMESTIC_Year_2_Enrolled": x,
    "term_1_UTSCWK_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_UTSCWK_INTERNATIONAL_Year_2_Applications": x,
    "term_1_UTSCWK_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_UTSCWK_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_UTSCWK_INTERNATIONAL_Year_2_Denied": x,
    "term_1_UTSCWK_INTERNATIONAL_Year_2_Declined": x,
    "term_1_UTSCWK_INTERNATIONAL_Year_2_Pending": x,
    "term_1_UTSCWK_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_UTSCWK_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_UTSCWK_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_OPEN_DOMESTIC_Year_1_Applications": x,
    "term_1_OPEN_DOMESTIC_Year_1_Accepted": x,
    "term_1_OPEN_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_OPEN_DOMESTIC_Year_1_Denied": x,
    "term_1_OPEN_DOMESTIC_Year_1_Declined": x,
    "term_1_OPEN_DOMESTIC_Year_1_Pending": x,
    "term_1_OPEN_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_OPEN_DOMESTIC_Year_1_Enrolled": x,
    "term_1_OPEN_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_OPEN_INTERNATIONAL_Year_1_Applications": x,
    "term_1_OPEN_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_OPEN_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_OPEN_INTERNATIONAL_Year_1_Denied": x,
    "term_1_OPEN_INTERNATIONAL_Year_1_Declined": x,
    "term_1_OPEN_INTERNATIONAL_Year_1_Pending": x,
    "term_1_OPEN_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_OPEN_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_OPEN_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_GOVNSC_DOMESTIC_Year_1_Applications": x,
    "term_1_GOVNSC_DOMESTIC_Year_1_Accepted": x,
    "term_1_GOVNSC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_GOVNSC_DOMESTIC_Year_1_Denied": x,
    "term_1_GOVNSC_DOMESTIC_Year_1_Declined": x,
    "term_1_GOVNSC_DOMESTIC_Year_1_Pending": x,
    "term_1_GOVNSC_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_GOVNSC_DOMESTIC_Year_1_Enrolled": x,
    "term_1_GOVNSC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_GOVNSC_INTERNATIONAL_Year_1_Applications": x,
    "term_1_GOVNSC_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_GOVNSC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_GOVNSC_INTERNATIONAL_Year_1_Denied": x,
    "term_1_GOVNSC_INTERNATIONAL_Year_1_Declined": x,
    "term_1_GOVNSC_INTERNATIONAL_Year_1_Pending": x,
    "term_1_GOVNSC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_GOVNSC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_GOVNSC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_GOVNSC_DOMESTIC_Year_2_Applications": x,
    "term_1_GOVNSC_DOMESTIC_Year_2_Accepted": x,
    "term_1_GOVNSC_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_GOVNSC_DOMESTIC_Year_2_Denied": x,
    "term_1_GOVNSC_DOMESTIC_Year_2_Declined": x,
    "term_1_GOVNSC_DOMESTIC_Year_2_Pending": x,
    "term_1_GOVNSC_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_GOVNSC_DOMESTIC_Year_2_Enrolled": x,
    "term_1_GOVNSC_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_GOVNSC_INTERNATIONAL_Year_2_Applications": x,
    "term_1_GOVNSC_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_GOVNSC_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_GOVNSC_INTERNATIONAL_Year_2_Denied": x,
    "term_1_GOVNSC_INTERNATIONAL_Year_2_Declined": x,
    "term_1_GOVNSC_INTERNATIONAL_Year_2_Pending": x,
    "term_1_GOVNSC_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_GOVNSC_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_GOVNSC_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_COMSCC_DOMESTIC_Year_1_Applications": x,
    "term_1_COMSCC_DOMESTIC_Year_1_Accepted": x,
    "term_1_COMSCC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_COMSCC_DOMESTIC_Year_1_Denied": x,
    "term_1_COMSCC_DOMESTIC_Year_1_Declined": x,
    "term_1_COMSCC_DOMESTIC_Year_1_Pending": x,
    "term_1_COMSCC_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_COMSCC_DOMESTIC_Year_1_Enrolled": x,
    "term_1_COMSCC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_COMSCC_INTERNATIONAL_Year_1_Applications": x,
    "term_1_COMSCC_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_COMSCC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_COMSCC_INTERNATIONAL_Year_1_Denied": x,
    "term_1_COMSCC_INTERNATIONAL_Year_1_Declined": x,
    "term_1_COMSCC_INTERNATIONAL_Year_1_Pending": x,
    "term_1_COMSCC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_COMSCC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_COMSCC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_COMSCD_DOMESTIC_Year_1_Applications": x,
    "term_1_COMSCD_DOMESTIC_Year_1_Accepted": x,
    "term_1_COMSCD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_COMSCD_DOMESTIC_Year_1_Denied": x,
    "term_1_COMSCD_DOMESTIC_Year_1_Declined": x,
    "term_1_COMSCD_DOMESTIC_Year_1_Pending": x,
    "term_1_COMSCD_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_COMSCD_DOMESTIC_Year_1_Enrolled": x,
    "term_1_COMSCD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_COMSCD_INTERNATIONAL_Year_1_Applications": x,
    "term_1_COMSCD_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_COMSCD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_COMSCD_INTERNATIONAL_Year_1_Denied": x,
    "term_1_COMSCD_INTERNATIONAL_Year_1_Declined": x,
    "term_1_COMSCD_INTERNATIONAL_Year_1_Pending": x,
    "term_1_COMSCD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_COMSCD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_COMSCD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_COMSCD_DOMESTIC_Year_2_Applications": x,
    "term_1_COMSCD_DOMESTIC_Year_2_Accepted": x,
    "term_1_COMSCD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_COMSCD_DOMESTIC_Year_2_Denied": x,
    "term_1_COMSCD_DOMESTIC_Year_2_Declined": x,
    "term_1_COMSCD_DOMESTIC_Year_2_Pending": x,
    "term_1_COMSCD_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_COMSCD_DOMESTIC_Year_2_Enrolled": x,
    "term_1_COMSCD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_COMSCD_INTERNATIONAL_Year_2_Applications": x,
    "term_1_COMSCD_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_COMSCD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_COMSCD_INTERNATIONAL_Year_2_Denied": x,
    "term_1_COMSCD_INTERNATIONAL_Year_2_Declined": x,
    "term_1_COMSCD_INTERNATIONAL_Year_2_Pending": x,
    "term_1_COMSCD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_COMSCD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_COMSCD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_GNARTC_DOMESTIC_Year_1_Applications": x,
    "term_1_GNARTC_DOMESTIC_Year_1_Accepted": x,
    "term_1_GNARTC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_GNARTC_DOMESTIC_Year_1_Denied": x,
    "term_1_GNARTC_DOMESTIC_Year_1_Declined": x,
    "term_1_GNARTC_DOMESTIC_Year_1_Pending": x,
    "term_1_GNARTC_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_GNARTC_DOMESTIC_Year_1_Enrolled": x,
    "term_1_GNARTC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_GNARTC_INTERNATIONAL_Year_1_Applications": x,
    "term_1_GNARTC_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_GNARTC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_GNARTC_INTERNATIONAL_Year_1_Denied": x,
    "term_1_GNARTC_INTERNATIONAL_Year_1_Declined": x,
    "term_1_GNARTC_INTERNATIONAL_Year_1_Pending": x,
    "term_1_GNARTC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_GNARTC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_GNARTC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_GNSCIC_DOMESTIC_Year_1_Applications": x,
    "term_1_GNSCIC_DOMESTIC_Year_1_Accepted": x,
    "term_1_GNSCIC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_GNSCIC_DOMESTIC_Year_1_Denied": x,
    "term_1_GNSCIC_DOMESTIC_Year_1_Declined": x,
    "term_1_GNSCIC_DOMESTIC_Year_1_Pending": x,
    "term_1_GNSCIC_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_GNSCIC_DOMESTIC_Year_1_Enrolled": x,
    "term_1_GNSCIC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_GNSCIC_INTERNATIONAL_Year_1_Applications": x,
    "term_1_GNSCIC_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_GNSCIC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_GNSCIC_INTERNATIONAL_Year_1_Denied": x,
    "term_1_GNSCIC_INTERNATIONAL_Year_1_Declined": x,
    "term_1_GNSCIC_INTERNATIONAL_Year_1_Pending": x,
    "term_1_GNSCIC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_GNSCIC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_GNSCIC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_ANCPDC_DOMESTIC_Year_1_Applications": x,
    "term_1_ANCPDC_DOMESTIC_Year_1_Accepted": x,
    "term_1_ANCPDC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_ANCPDC_DOMESTIC_Year_1_Denied": x,
    "term_1_ANCPDC_DOMESTIC_Year_1_Declined": x,
    "term_1_ANCPDC_DOMESTIC_Year_1_Pending": x,
    "term_1_ANCPDC_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_ANCPDC_DOMESTIC_Year_1_Enrolled": x,
    "term_1_ANCPDC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_ANCPDC_INTERNATIONAL_Year_1_Applications": x,
    "term_1_ANCPDC_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_ANCPDC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_ANCPDC_INTERNATIONAL_Year_1_Denied": x,
    "term_1_ANCPDC_INTERNATIONAL_Year_1_Declined": x,
    "term_1_ANCPDC_INTERNATIONAL_Year_1_Pending": x,
    "term_1_ANCPDC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_ANCPDC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_ANCPDC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_NSPDC_DOMESTIC_Year_1_Applications": x,
    "term_1_NSPDC_DOMESTIC_Year_1_Accepted": x,
    "term_1_NSPDC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_NSPDC_DOMESTIC_Year_1_Denied": x,
    "term_1_NSPDC_DOMESTIC_Year_1_Declined": x,
    "term_1_NSPDC_DOMESTIC_Year_1_Pending": x,
    "term_1_NSPDC_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_NSPDC_DOMESTIC_Year_1_Enrolled": x,
    "term_1_NSPDC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_NSPDC_INTERNATIONAL_Year_1_Applications": x,
    "term_1_NSPDC_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_NSPDC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_NSPDC_INTERNATIONAL_Year_1_Denied": x,
    "term_1_NSPDC_INTERNATIONAL_Year_1_Declined": x,
    "term_1_NSPDC_INTERNATIONAL_Year_1_Pending": x,
    "term_1_NSPDC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_NSPDC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_NSPDC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1__DOMESTIC_Year_1_Applications": x,
    "term_1__DOMESTIC_Year_1_Accepted": x,
    "term_1__DOMESTIC_Year_1_Payment_Pending": x,
    "term_1__DOMESTIC_Year_1_Denied": x,
    "term_1__DOMESTIC_Year_1_Declined": x,
    "term_1__DOMESTIC_Year_1_Pending": x,
    "term_1__DOMESTIC_Year_1_Waitlisted": x,
    "term_1__DOMESTIC_Year_1_Enrolled": x,
    "term_1__DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1__INTERNATIONAL_Year_1_Applications": x,
    "term_1__INTERNATIONAL_Year_1_Accepted": x,
    "term_1__INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1__INTERNATIONAL_Year_1_Denied": x,
    "term_1__INTERNATIONAL_Year_1_Declined": x,
    "term_1__INTERNATIONAL_Year_1_Pending": x,
    "term_1__INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1__INTERNATIONAL_Year_1_Enrolled": x,
    "term_1__INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_LINC13_DOMESTIC_Year_1_Applications": x,
    "term_1_LINC13_DOMESTIC_Year_1_Accepted": x,
    "term_1_LINC13_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_LINC13_DOMESTIC_Year_1_Denied": x,
    "term_1_LINC13_DOMESTIC_Year_1_Declined": x,
    "term_1_LINC13_DOMESTIC_Year_1_Pending": x,
    "term_1_LINC13_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_LINC13_DOMESTIC_Year_1_Enrolled": x,
    "term_1_LINC13_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_LINC13_INTERNATIONAL_Year_1_Applications": x,
    "term_1_LINC13_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_LINC13_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_LINC13_INTERNATIONAL_Year_1_Denied": x,
    "term_1_LINC13_INTERNATIONAL_Year_1_Declined": x,
    "term_1_LINC13_INTERNATIONAL_Year_1_Pending": x,
    "term_1_LINC13_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_LINC13_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_LINC13_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_AELCC_DOMESTIC_Year_1_Applications": x,
    "term_1_AELCC_DOMESTIC_Year_1_Accepted": x,
    "term_1_AELCC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_AELCC_DOMESTIC_Year_1_Denied": x,
    "term_1_AELCC_DOMESTIC_Year_1_Declined": x,
    "term_1_AELCC_DOMESTIC_Year_1_Pending": x,
    "term_1_AELCC_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_AELCC_DOMESTIC_Year_1_Enrolled": x,
    "term_1_AELCC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_AELCC_INTERNATIONAL_Year_1_Applications": x,
    "term_1_AELCC_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_AELCC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_AELCC_INTERNATIONAL_Year_1_Denied": x,
    "term_1_AELCC_INTERNATIONAL_Year_1_Declined": x,
    "term_1_AELCC_INTERNATIONAL_Year_1_Pending": x,
    "term_1_AELCC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_AELCC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_AELCC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_ECCERT_DOMESTIC_Year_1_Applications": x,
    "term_1_ECCERT_DOMESTIC_Year_1_Accepted": x,
    "term_1_ECCERT_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_ECCERT_DOMESTIC_Year_1_Denied": x,
    "term_1_ECCERT_DOMESTIC_Year_1_Declined": x,
    "term_1_ECCERT_DOMESTIC_Year_1_Pending": x,
    "term_1_ECCERT_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_ECCERT_DOMESTIC_Year_1_Enrolled": x,
    "term_1_ECCERT_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_ECCERT_INTERNATIONAL_Year_1_Applications": x,
    "term_1_ECCERT_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_ECCERT_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_ECCERT_INTERNATIONAL_Year_1_Denied": x,
    "term_1_ECCERT_INTERNATIONAL_Year_1_Declined": x,
    "term_1_ECCERT_INTERNATIONAL_Year_1_Pending": x,
    "term_1_ECCERT_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_ECCERT_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_ECCERT_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_ECDIP_DOMESTIC_Year_1_Applications": x,
    "term_1_ECDIP_DOMESTIC_Year_1_Accepted": x,
    "term_1_ECDIP_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_ECDIP_DOMESTIC_Year_1_Denied": x,
    "term_1_ECDIP_DOMESTIC_Year_1_Declined": x,
    "term_1_ECDIP_DOMESTIC_Year_1_Pending": x,
    "term_1_ECDIP_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_ECDIP_DOMESTIC_Year_1_Enrolled": x,
    "term_1_ECDIP_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_ECDIP_INTERNATIONAL_Year_1_Applications": x,
    "term_1_ECDIP_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_ECDIP_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_ECDIP_INTERNATIONAL_Year_1_Denied": x,
    "term_1_ECDIP_INTERNATIONAL_Year_1_Declined": x,
    "term_1_ECDIP_INTERNATIONAL_Year_1_Pending": x,
    "term_1_ECDIP_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_ECDIP_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_ECDIP_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_ECDIP_DOMESTIC_Year_2_Applications": x,
    "term_1_ECDIP_DOMESTIC_Year_2_Accepted": x,
    "term_1_ECDIP_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_ECDIP_DOMESTIC_Year_2_Denied": x,
    "term_1_ECDIP_DOMESTIC_Year_2_Declined": x,
    "term_1_ECDIP_DOMESTIC_Year_2_Pending": x,
    "term_1_ECDIP_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_ECDIP_DOMESTIC_Year_2_Enrolled": x,
    "term_1_ECDIP_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_ECDIP_INTERNATIONAL_Year_2_Applications": x,
    "term_1_ECDIP_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_ECDIP_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_ECDIP_INTERNATIONAL_Year_2_Denied": x,
    "term_1_ECDIP_INTERNATIONAL_Year_2_Declined": x,
    "term_1_ECDIP_INTERNATIONAL_Year_2_Pending": x,
    "term_1_ECDIP_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_ECDIP_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_ECDIP_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_CYCD_DOMESTIC_Year_1_Applications": x,
    "term_1_CYCD_DOMESTIC_Year_1_Accepted": x,
    "term_1_CYCD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_CYCD_DOMESTIC_Year_1_Denied": x,
    "term_1_CYCD_DOMESTIC_Year_1_Declined": x,
    "term_1_CYCD_DOMESTIC_Year_1_Pending": x,
    "term_1_CYCD_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_CYCD_DOMESTIC_Year_1_Enrolled": x,
    "term_1_CYCD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_CYCD_INTERNATIONAL_Year_1_Applications": x,
    "term_1_CYCD_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_CYCD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_CYCD_INTERNATIONAL_Year_1_Denied": x,
    "term_1_CYCD_INTERNATIONAL_Year_1_Declined": x,
    "term_1_CYCD_INTERNATIONAL_Year_1_Pending": x,
    "term_1_CYCD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_CYCD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_CYCD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_CYCD_DOMESTIC_Year_2_Applications": x,
    "term_1_CYCD_DOMESTIC_Year_2_Accepted": x,
    "term_1_CYCD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_CYCD_DOMESTIC_Year_2_Denied": x,
    "term_1_CYCD_DOMESTIC_Year_2_Declined": x,
    "term_1_CYCD_DOMESTIC_Year_2_Pending": x,
    "term_1_CYCD_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_CYCD_DOMESTIC_Year_2_Enrolled": x,
    "term_1_CYCD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_CYCD_INTERNATIONAL_Year_2_Applications": x,
    "term_1_CYCD_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_CYCD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_CYCD_INTERNATIONAL_Year_2_Denied": x,
    "term_1_CYCD_INTERNATIONAL_Year_2_Declined": x,
    "term_1_CYCD_INTERNATIONAL_Year_2_Pending": x,
    "term_1_CYCD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_CYCD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_CYCD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_ECEAC_DOMESTIC_Year_1_Applications": x,
    "term_1_ECEAC_DOMESTIC_Year_1_Accepted": x,
    "term_1_ECEAC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_ECEAC_DOMESTIC_Year_1_Denied": x,
    "term_1_ECEAC_DOMESTIC_Year_1_Declined": x,
    "term_1_ECEAC_DOMESTIC_Year_1_Pending": x,
    "term_1_ECEAC_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_ECEAC_DOMESTIC_Year_1_Enrolled": x,
    "term_1_ECEAC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_ECEAC_INTERNATIONAL_Year_1_Applications": x,
    "term_1_ECEAC_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_ECEAC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_ECEAC_INTERNATIONAL_Year_1_Denied": x,
    "term_1_ECEAC_INTERNATIONAL_Year_1_Declined": x,
    "term_1_ECEAC_INTERNATIONAL_Year_1_Pending": x,
    "term_1_ECEAC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_ECEAC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_ECEAC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_SOCWKD_DOMESTIC_Year_1_Applications": x,
    "term_1_SOCWKD_DOMESTIC_Year_1_Accepted": x,
    "term_1_SOCWKD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_SOCWKD_DOMESTIC_Year_1_Denied": x,
    "term_1_SOCWKD_DOMESTIC_Year_1_Declined": x,
    "term_1_SOCWKD_DOMESTIC_Year_1_Pending": x,
    "term_1_SOCWKD_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_SOCWKD_DOMESTIC_Year_1_Enrolled": x,
    "term_1_SOCWKD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_SOCWKD_INTERNATIONAL_Year_1_Applications": x,
    "term_1_SOCWKD_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_SOCWKD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_SOCWKD_INTERNATIONAL_Year_1_Denied": x,
    "term_1_SOCWKD_INTERNATIONAL_Year_1_Declined": x,
    "term_1_SOCWKD_INTERNATIONAL_Year_1_Pending": x,
    "term_1_SOCWKD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_SOCWKD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_SOCWKD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_SOCWKD_DOMESTIC_Year_2_Applications": x,
    "term_1_SOCWKD_DOMESTIC_Year_2_Accepted": x,
    "term_1_SOCWKD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_SOCWKD_DOMESTIC_Year_2_Denied": x,
    "term_1_SOCWKD_DOMESTIC_Year_2_Declined": x,
    "term_1_SOCWKD_DOMESTIC_Year_2_Pending": x,
    "term_1_SOCWKD_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_SOCWKD_DOMESTIC_Year_2_Enrolled": x,
    "term_1_SOCWKD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_SOCWKD_INTERNATIONAL_Year_2_Applications": x,
    "term_1_SOCWKD_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_SOCWKD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_SOCWKD_INTERNATIONAL_Year_2_Denied": x,
    "term_1_SOCWKD_INTERNATIONAL_Year_2_Declined": x,
    "term_1_SOCWKD_INTERNATIONAL_Year_2_Pending": x,
    "term_1_SOCWKD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_SOCWKD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_SOCWKD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_HCAC_DOMESTIC_Year_1_Applications": x,
    "term_1_HCAC_DOMESTIC_Year_1_Accepted": x,
    "term_1_HCAC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_HCAC_DOMESTIC_Year_1_Denied": x,
    "term_1_HCAC_DOMESTIC_Year_1_Declined": x,
    "term_1_HCAC_DOMESTIC_Year_1_Pending": x,
    "term_1_HCAC_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_HCAC_DOMESTIC_Year_1_Enrolled": x,
    "term_1_HCAC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_HCAC_INTERNATIONAL_Year_1_Applications": x,
    "term_1_HCAC_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_HCAC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_HCAC_INTERNATIONAL_Year_1_Denied": x,
    "term_1_HCAC_INTERNATIONAL_Year_1_Declined": x,
    "term_1_HCAC_INTERNATIONAL_Year_1_Pending": x,
    "term_1_HCAC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_HCAC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_HCAC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_PCPC_DOMESTIC_Year_1_Applications": x,
    "term_1_PCPC_DOMESTIC_Year_1_Accepted": x,
    "term_1_PCPC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_PCPC_DOMESTIC_Year_1_Denied": x,
    "term_1_PCPC_DOMESTIC_Year_1_Declined": x,
    "term_1_PCPC_DOMESTIC_Year_1_Pending": x,
    "term_1_PCPC_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_PCPC_DOMESTIC_Year_1_Enrolled": x,
    "term_1_PCPC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_PCPC_INTERNATIONAL_Year_1_Applications": x,
    "term_1_PCPC_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_PCPC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_PCPC_INTERNATIONAL_Year_1_Denied": x,
    "term_1_PCPC_INTERNATIONAL_Year_1_Declined": x,
    "term_1_PCPC_INTERNATIONAL_Year_1_Pending": x,
    "term_1_PCPC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_PCPC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_PCPC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_ACP_DOMESTIC_Year_1_Applications": x,
    "term_1_ACP_DOMESTIC_Year_1_Accepted": x,
    "term_1_ACP_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_ACP_DOMESTIC_Year_1_Denied": x,
    "term_1_ACP_DOMESTIC_Year_1_Declined": x,
    "term_1_ACP_DOMESTIC_Year_1_Pending": x,
    "term_1_ACP_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_ACP_DOMESTIC_Year_1_Enrolled": x,
    "term_1_ACP_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_ACP_INTERNATIONAL_Year_1_Applications": x,
    "term_1_ACP_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_ACP_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_ACP_INTERNATIONAL_Year_1_Denied": x,
    "term_1_ACP_INTERNATIONAL_Year_1_Declined": x,
    "term_1_ACP_INTERNATIONAL_Year_1_Pending": x,
    "term_1_ACP_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_ACP_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_ACP_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_ACP_DOMESTIC_Year_2_Applications": x,
    "term_1_ACP_DOMESTIC_Year_2_Accepted": x,
    "term_1_ACP_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_ACP_DOMESTIC_Year_2_Denied": x,
    "term_1_ACP_DOMESTIC_Year_2_Declined": x,
    "term_1_ACP_DOMESTIC_Year_2_Pending": x,
    "term_1_ACP_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_ACP_DOMESTIC_Year_2_Enrolled": x,
    "term_1_ACP_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_ACP_INTERNATIONAL_Year_2_Applications": x,
    "term_1_ACP_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_ACP_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_ACP_INTERNATIONAL_Year_2_Denied": x,
    "term_1_ACP_INTERNATIONAL_Year_2_Declined": x,
    "term_1_ACP_INTERNATIONAL_Year_2_Pending": x,
    "term_1_ACP_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_ACP_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_ACP_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_PNR_DOMESTIC_Year_1_Applications": x,
    "term_1_PNR_DOMESTIC_Year_1_Accepted": x,
    "term_1_PNR_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_PNR_DOMESTIC_Year_1_Denied": x,
    "term_1_PNR_DOMESTIC_Year_1_Declined": x,
    "term_1_PNR_DOMESTIC_Year_1_Pending": x,
    "term_1_PNR_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_PNR_DOMESTIC_Year_1_Enrolled": x,
    "term_1_PNR_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_PNR_INTERNATIONAL_Year_1_Applications": x,
    "term_1_PNR_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_PNR_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_PNR_INTERNATIONAL_Year_1_Denied": x,
    "term_1_PNR_INTERNATIONAL_Year_1_Declined": x,
    "term_1_PNR_INTERNATIONAL_Year_1_Pending": x,
    "term_1_PNR_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_PNR_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_PNR_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_PNR_DOMESTIC_Year_2_Applications": x,
    "term_1_PNR_DOMESTIC_Year_2_Accepted": x,
    "term_1_PNR_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_PNR_DOMESTIC_Year_2_Denied": x,
    "term_1_PNR_DOMESTIC_Year_2_Declined": x,
    "term_1_PNR_DOMESTIC_Year_2_Pending": x,
    "term_1_PNR_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_PNR_DOMESTIC_Year_2_Enrolled": x,
    "term_1_PNR_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_PNR_INTERNATIONAL_Year_2_Applications": x,
    "term_1_PNR_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_PNR_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_PNR_INTERNATIONAL_Year_2_Denied": x,
    "term_1_PNR_INTERNATIONAL_Year_2_Declined": x,
    "term_1_PNR_INTERNATIONAL_Year_2_Pending": x,
    "term_1_PNR_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_PNR_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_PNR_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_BSCN_DOMESTIC_Year_1_Applications": x,
    "term_1_BSCN_DOMESTIC_Year_1_Accepted": x,
    "term_1_BSCN_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_BSCN_DOMESTIC_Year_1_Denied": x,
    "term_1_BSCN_DOMESTIC_Year_1_Declined": x,
    "term_1_BSCN_DOMESTIC_Year_1_Pending": x,
    "term_1_BSCN_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_BSCN_DOMESTIC_Year_1_Enrolled": x,
    "term_1_BSCN_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_BSCN_INTERNATIONAL_Year_1_Applications": x,
    "term_1_BSCN_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_BSCN_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_BSCN_INTERNATIONAL_Year_1_Denied": x,
    "term_1_BSCN_INTERNATIONAL_Year_1_Declined": x,
    "term_1_BSCN_INTERNATIONAL_Year_1_Pending": x,
    "term_1_BSCN_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_BSCN_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_BSCN_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_BSCN_DOMESTIC_Year_2_Applications": x,
    "term_1_BSCN_DOMESTIC_Year_2_Accepted": x,
    "term_1_BSCN_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_BSCN_DOMESTIC_Year_2_Denied": x,
    "term_1_BSCN_DOMESTIC_Year_2_Declined": x,
    "term_1_BSCN_DOMESTIC_Year_2_Pending": x,
    "term_1_BSCN_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_BSCN_DOMESTIC_Year_2_Enrolled": x,
    "term_1_BSCN_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_BSCN_INTERNATIONAL_Year_2_Applications": x,
    "term_1_BSCN_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_BSCN_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_BSCN_INTERNATIONAL_Year_2_Denied": x,
    "term_1_BSCN_INTERNATIONAL_Year_2_Declined": x,
    "term_1_BSCN_INTERNATIONAL_Year_2_Pending": x,
    "term_1_BSCN_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_BSCN_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_BSCN_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_1_BSCN_DOMESTIC_Year_3_Applications": x,
    "term_1_BSCN_DOMESTIC_Year_3_Accepted": x,
    "term_1_BSCN_DOMESTIC_Year_3_Payment_Pending": x,
    "term_1_BSCN_DOMESTIC_Year_3_Denied": x,
    "term_1_BSCN_DOMESTIC_Year_3_Declined": x,
    "term_1_BSCN_DOMESTIC_Year_3_Pending": x,
    "term_1_BSCN_DOMESTIC_Year_3_Waitlisted": x,
    "term_1_BSCN_DOMESTIC_Year_3_Enrolled": x,
    "term_1_BSCN_DOMESTIC_Year_3_Conversion_Rate": x,
    "term_1_BSCN_INTERNATIONAL_Year_3_Applications": x,
    "term_1_BSCN_INTERNATIONAL_Year_3_Accepted": x,
    "term_1_BSCN_INTERNATIONAL_Year_3_Payment_Pending": x,
    "term_1_BSCN_INTERNATIONAL_Year_3_Denied": x,
    "term_1_BSCN_INTERNATIONAL_Year_3_Declined": x,
    "term_1_BSCN_INTERNATIONAL_Year_3_Pending": x,
    "term_1_BSCN_INTERNATIONAL_Year_3_Waitlisted": x,
    "term_1_BSCN_INTERNATIONAL_Year_3_Enrolled": x,
    "term_1_BSCN_INTERNATIONAL_Year_3_Conversion_Rate": x,
    "term_1_IENCN_DOMESTIC_Year_1_Applications": x,
    "term_1_IENCN_DOMESTIC_Year_1_Accepted": x,
    "term_1_IENCN_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_IENCN_DOMESTIC_Year_1_Denied": x,
    "term_1_IENCN_DOMESTIC_Year_1_Declined": x,
    "term_1_IENCN_DOMESTIC_Year_1_Pending": x,
    "term_1_IENCN_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_IENCN_DOMESTIC_Year_1_Enrolled": x,
    "term_1_IENCN_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_IENCN_INTERNATIONAL_Year_1_Applications": x,
    "term_1_IENCN_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_IENCN_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_IENCN_INTERNATIONAL_Year_1_Denied": x,
    "term_1_IENCN_INTERNATIONAL_Year_1_Declined": x,
    "term_1_IENCN_INTERNATIONAL_Year_1_Pending": x,
    "term_1_IENCN_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_IENCN_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_IENCN_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_PAELEC_DOMESTIC_Year_1_Applications": x,
    "term_1_PAELEC_DOMESTIC_Year_1_Accepted": x,
    "term_1_PAELEC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_PAELEC_DOMESTIC_Year_1_Denied": x,
    "term_1_PAELEC_DOMESTIC_Year_1_Declined": x,
    "term_1_PAELEC_DOMESTIC_Year_1_Pending": x,
    "term_1_PAELEC_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_PAELEC_DOMESTIC_Year_1_Enrolled": x,
    "term_1_PAELEC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_PAELEC_INTERNATIONAL_Year_1_Applications": x,
    "term_1_PAELEC_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_PAELEC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_PAELEC_INTERNATIONAL_Year_1_Denied": x,
    "term_1_PAELEC_INTERNATIONAL_Year_1_Declined": x,
    "term_1_PAELEC_INTERNATIONAL_Year_1_Pending": x,
    "term_1_PAELEC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_PAELEC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_PAELEC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_PAWELD_DOMESTIC_Year_1_Applications": x,
    "term_1_PAWELD_DOMESTIC_Year_1_Accepted": x,
    "term_1_PAWELD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_PAWELD_DOMESTIC_Year_1_Denied": x,
    "term_1_PAWELD_DOMESTIC_Year_1_Declined": x,
    "term_1_PAWELD_DOMESTIC_Year_1_Pending": x,
    "term_1_PAWELD_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_PAWELD_DOMESTIC_Year_1_Enrolled": x,
    "term_1_PAWELD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_PAWELD_INTERNATIONAL_Year_1_Applications": x,
    "term_1_PAWELD_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_PAWELD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_PAWELD_INTERNATIONAL_Year_1_Denied": x,
    "term_1_PAWELD_INTERNATIONAL_Year_1_Declined": x,
    "term_1_PAWELD_INTERNATIONAL_Year_1_Pending": x,
    "term_1_PAWELD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_PAWELD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_PAWELD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_PAHET_DOMESTIC_Year_1_Applications": x,
    "term_1_PAHET_DOMESTIC_Year_1_Accepted": x,
    "term_1_PAHET_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_PAHET_DOMESTIC_Year_1_Denied": x,
    "term_1_PAHET_DOMESTIC_Year_1_Declined": x,
    "term_1_PAHET_DOMESTIC_Year_1_Pending": x,
    "term_1_PAHET_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_PAHET_DOMESTIC_Year_1_Enrolled": x,
    "term_1_PAHET_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_PAHET_INTERNATIONAL_Year_1_Applications": x,
    "term_1_PAHET_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_PAHET_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_PAHET_INTERNATIONAL_Year_1_Denied": x,
    "term_1_PAHET_INTERNATIONAL_Year_1_Declined": x,
    "term_1_PAHET_INTERNATIONAL_Year_1_Pending": x,
    "term_1_PAHET_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_PAHET_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_PAHET_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_PAPLUM_DOMESTIC_Year_1_Applications": x,
    "term_1_PAPLUM_DOMESTIC_Year_1_Accepted": x,
    "term_1_PAPLUM_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_PAPLUM_DOMESTIC_Year_1_Denied": x,
    "term_1_PAPLUM_DOMESTIC_Year_1_Declined": x,
    "term_1_PAPLUM_DOMESTIC_Year_1_Pending": x,
    "term_1_PAPLUM_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_PAPLUM_DOMESTIC_Year_1_Enrolled": x,
    "term_1_PAPLUM_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_PAPLUM_INTERNATIONAL_Year_1_Applications": x,
    "term_1_PAPLUM_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_PAPLUM_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_PAPLUM_INTERNATIONAL_Year_1_Denied": x,
    "term_1_PAPLUM_INTERNATIONAL_Year_1_Declined": x,
    "term_1_PAPLUM_INTERNATIONAL_Year_1_Pending": x,
    "term_1_PAPLUM_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_PAPLUM_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_PAPLUM_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_POWCM4_DOMESTIC_Year_1_Applications": x,
    "term_1_POWCM4_DOMESTIC_Year_1_Accepted": x,
    "term_1_POWCM4_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_POWCM4_DOMESTIC_Year_1_Denied": x,
    "term_1_POWCM4_DOMESTIC_Year_1_Declined": x,
    "term_1_POWCM4_DOMESTIC_Year_1_Pending": x,
    "term_1_POWCM4_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_POWCM4_DOMESTIC_Year_1_Enrolled": x,
    "term_1_POWCM4_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_POWCM4_INTERNATIONAL_Year_1_Applications": x,
    "term_1_POWCM4_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_POWCM4_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_POWCM4_INTERNATIONAL_Year_1_Denied": x,
    "term_1_POWCM4_INTERNATIONAL_Year_1_Declined": x,
    "term_1_POWCM4_INTERNATIONAL_Year_1_Pending": x,
    "term_1_POWCM4_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_POWCM4_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_POWCM4_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_POWCM3_DOMESTIC_Year_1_Applications": x,
    "term_1_POWCM3_DOMESTIC_Year_1_Accepted": x,
    "term_1_POWCM3_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_POWCM3_DOMESTIC_Year_1_Denied": x,
    "term_1_POWCM3_DOMESTIC_Year_1_Declined": x,
    "term_1_POWCM3_DOMESTIC_Year_1_Pending": x,
    "term_1_POWCM3_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_POWCM3_DOMESTIC_Year_1_Enrolled": x,
    "term_1_POWCM3_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_POWCM3_INTERNATIONAL_Year_1_Applications": x,
    "term_1_POWCM3_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_POWCM3_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_POWCM3_INTERNATIONAL_Year_1_Denied": x,
    "term_1_POWCM3_INTERNATIONAL_Year_1_Declined": x,
    "term_1_POWCM3_INTERNATIONAL_Year_1_Pending": x,
    "term_1_POWCM3_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_POWCM3_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_POWCM3_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_POWCP4_DOMESTIC_Year_1_Applications": x,
    "term_1_POWCP4_DOMESTIC_Year_1_Accepted": x,
    "term_1_POWCP4_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_POWCP4_DOMESTIC_Year_1_Denied": x,
    "term_1_POWCP4_DOMESTIC_Year_1_Declined": x,
    "term_1_POWCP4_DOMESTIC_Year_1_Pending": x,
    "term_1_POWCP4_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_POWCP4_DOMESTIC_Year_1_Enrolled": x,
    "term_1_POWCP4_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_POWCP4_INTERNATIONAL_Year_1_Applications": x,
    "term_1_POWCP4_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_POWCP4_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_POWCP4_INTERNATIONAL_Year_1_Denied": x,
    "term_1_POWCP4_INTERNATIONAL_Year_1_Declined": x,
    "term_1_POWCP4_INTERNATIONAL_Year_1_Pending": x,
    "term_1_POWCP4_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_POWCP4_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_POWCP4_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_POWCO4_DOMESTIC_Year_1_Applications": x,
    "term_1_POWCO4_DOMESTIC_Year_1_Accepted": x,
    "term_1_POWCO4_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_POWCO4_DOMESTIC_Year_1_Denied": x,
    "term_1_POWCO4_DOMESTIC_Year_1_Declined": x,
    "term_1_POWCO4_DOMESTIC_Year_1_Pending": x,
    "term_1_POWCO4_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_POWCO4_DOMESTIC_Year_1_Enrolled": x,
    "term_1_POWCO4_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_POWCO4_INTERNATIONAL_Year_1_Applications": x,
    "term_1_POWCO4_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_POWCO4_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_POWCO4_INTERNATIONAL_Year_1_Denied": x,
    "term_1_POWCO4_INTERNATIONAL_Year_1_Declined": x,
    "term_1_POWCO4_INTERNATIONAL_Year_1_Pending": x,
    "term_1_POWCO4_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_POWCO4_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_POWCO4_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_POWCO3_DOMESTIC_Year_1_Applications": x,
    "term_1_POWCO3_DOMESTIC_Year_1_Accepted": x,
    "term_1_POWCO3_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_POWCO3_DOMESTIC_Year_1_Denied": x,
    "term_1_POWCO3_DOMESTIC_Year_1_Declined": x,
    "term_1_POWCO3_DOMESTIC_Year_1_Pending": x,
    "term_1_POWCO3_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_POWCO3_DOMESTIC_Year_1_Enrolled": x,
    "term_1_POWCO3_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_POWCO3_INTERNATIONAL_Year_1_Applications": x,
    "term_1_POWCO3_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_POWCO3_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_POWCO3_INTERNATIONAL_Year_1_Denied": x,
    "term_1_POWCO3_INTERNATIONAL_Year_1_Declined": x,
    "term_1_POWCO3_INTERNATIONAL_Year_1_Pending": x,
    "term_1_POWCO3_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_POWCO3_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_POWCO3_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_PPET_DOMESTIC_Year_1_Applications": x,
    "term_1_PPET_DOMESTIC_Year_1_Accepted": x,
    "term_1_PPET_DOMESTIC_Year_1_Payment_Pending": x,
    "term_1_PPET_DOMESTIC_Year_1_Denied": x,
    "term_1_PPET_DOMESTIC_Year_1_Declined": x,
    "term_1_PPET_DOMESTIC_Year_1_Pending": x,
    "term_1_PPET_DOMESTIC_Year_1_Waitlisted": x,
    "term_1_PPET_DOMESTIC_Year_1_Enrolled": x,
    "term_1_PPET_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_1_PPET_INTERNATIONAL_Year_1_Applications": x,
    "term_1_PPET_INTERNATIONAL_Year_1_Accepted": x,
    "term_1_PPET_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_1_PPET_INTERNATIONAL_Year_1_Denied": x,
    "term_1_PPET_INTERNATIONAL_Year_1_Declined": x,
    "term_1_PPET_INTERNATIONAL_Year_1_Pending": x,
    "term_1_PPET_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_1_PPET_INTERNATIONAL_Year_1_Enrolled": x,
    "term_1_PPET_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_1_PPET_DOMESTIC_Year_2_Applications": x,
    "term_1_PPET_DOMESTIC_Year_2_Accepted": x,
    "term_1_PPET_DOMESTIC_Year_2_Payment_Pending": x,
    "term_1_PPET_DOMESTIC_Year_2_Denied": x,
    "term_1_PPET_DOMESTIC_Year_2_Declined": x,
    "term_1_PPET_DOMESTIC_Year_2_Pending": x,
    "term_1_PPET_DOMESTIC_Year_2_Waitlisted": x,
    "term_1_PPET_DOMESTIC_Year_2_Enrolled": x,
    "term_1_PPET_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_1_PPET_INTERNATIONAL_Year_2_Applications": x,
    "term_1_PPET_INTERNATIONAL_Year_2_Accepted": x,
    "term_1_PPET_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_1_PPET_INTERNATIONAL_Year_2_Denied": x,
    "term_1_PPET_INTERNATIONAL_Year_2_Declined": x,
    "term_1_PPET_INTERNATIONAL_Year_2_Pending": x,
    "term_1_PPET_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_1_PPET_INTERNATIONAL_Year_2_Enrolled": x,
    "term_1_PPET_INTERNATIONAL_Year_2_Conversion_Rate": x,

    #Term2Yr YYYY Apps By School sheet
    "term_2_BABUSC_DOMESTIC_Year_1_Applications": x,
    "term_2_BABUSC_DOMESTIC_Year_1_Accepted": x,
    "term_2_BABUSC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_BABUSC_DOMESTIC_Year_1_Denied": x,
    "term_2_BABUSC_DOMESTIC_Year_1_Declined": x,
    "term_2_BABUSC_DOMESTIC_Year_1_Pending": x,
    "term_2_BABUSC_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_BABUSC_DOMESTIC_Year_1_Enrolled": x,
    "term_2_BABUSC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_BABUSC_INTERNATIONAL_Year_1_Applications": x,
    "term_2_BABUSC_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_BABUSC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_BABUSC_INTERNATIONAL_Year_1_Denied": x,
    "term_2_BABUSC_INTERNATIONAL_Year_1_Declined": x,
    "term_2_BABUSC_INTERNATIONAL_Year_1_Pending": x,
    "term_2_BABUSC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_BABUSC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_BABUSC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_BAACTD_DOMESTIC_Year_1_Applications": x,
    "term_2_BAACTD_DOMESTIC_Year_1_Accepted": x,
    "term_2_BAACTD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_BAACTD_DOMESTIC_Year_1_Denied": x,
    "term_2_BAACTD_DOMESTIC_Year_1_Declined": x,
    "term_2_BAACTD_DOMESTIC_Year_1_Pending": x,
    "term_2_BAACTD_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_BAACTD_DOMESTIC_Year_1_Enrolled": x,
    "term_2_BAACTD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_BAACTD_INTERNATIONAL_Year_1_Applications": x,
    "term_2_BAACTD_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_BAACTD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_BAACTD_INTERNATIONAL_Year_1_Denied": x,
    "term_2_BAACTD_INTERNATIONAL_Year_1_Declined": x,
    "term_2_BAACTD_INTERNATIONAL_Year_1_Pending": x,
    "term_2_BAACTD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_BAACTD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_BAACTD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_BAACTD_DOMESTIC_Year_2_Applications": x,
    "term_2_BAACTD_DOMESTIC_Year_2_Accepted": x,
    "term_2_BAACTD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_BAACTD_DOMESTIC_Year_2_Denied": x,
    "term_2_BAACTD_DOMESTIC_Year_2_Declined": x,
    "term_2_BAACTD_DOMESTIC_Year_2_Pending": x,
    "term_2_BAACTD_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_BAACTD_DOMESTIC_Year_2_Enrolled": x,
    "term_2_BAACTD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_BAACTD_INTERNATIONAL_Year_2_Applications": x,
    "term_2_BAACTD_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_BAACTD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_BAACTD_INTERNATIONAL_Year_2_Denied": x,
    "term_2_BAACTD_INTERNATIONAL_Year_2_Declined": x,
    "term_2_BAACTD_INTERNATIONAL_Year_2_Pending": x,
    "term_2_BAACTD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_BAACTD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_BAACTD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_BABUSD_DOMESTIC_Year_1_Applications": x,
    "term_2_BABUSD_DOMESTIC_Year_1_Accepted": x,
    "term_2_BABUSD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_BABUSD_DOMESTIC_Year_1_Denied": x,
    "term_2_BABUSD_DOMESTIC_Year_1_Declined": x,
    "term_2_BABUSD_DOMESTIC_Year_1_Pending": x,
    "term_2_BABUSD_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_BABUSD_DOMESTIC_Year_1_Enrolled": x,
    "term_2_BABUSD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_BABUSD_INTERNATIONAL_Year_1_Applications": x,
    "term_2_BABUSD_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_BABUSD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_BABUSD_INTERNATIONAL_Year_1_Denied": x,
    "term_2_BABUSD_INTERNATIONAL_Year_1_Declined": x,
    "term_2_BABUSD_INTERNATIONAL_Year_1_Pending": x,
    "term_2_BABUSD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_BABUSD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_BABUSD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_BABUSD_DOMESTIC_Year_2_Applications": x,
    "term_2_BABUSD_DOMESTIC_Year_2_Accepted": x,
    "term_2_BABUSD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_BABUSD_DOMESTIC_Year_2_Denied": x,
    "term_2_BABUSD_DOMESTIC_Year_2_Declined": x,
    "term_2_BABUSD_DOMESTIC_Year_2_Pending": x,
    "term_2_BABUSD_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_BABUSD_DOMESTIC_Year_2_Enrolled": x,
    "term_2_BABUSD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_BABUSD_INTERNATIONAL_Year_2_Applications": x,
    "term_2_BABUSD_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_BABUSD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_BABUSD_INTERNATIONAL_Year_2_Denied": x,
    "term_2_BABUSD_INTERNATIONAL_Year_2_Declined": x,
    "term_2_BABUSD_INTERNATIONAL_Year_2_Pending": x,
    "term_2_BABUSD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_BABUSD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_BABUSD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_BAESPM_DOMESTIC_Year_1_Applications": x,
    "term_2_BAESPM_DOMESTIC_Year_1_Accepted": x,
    "term_2_BAESPM_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_BAESPM_DOMESTIC_Year_1_Denied": x,
    "term_2_BAESPM_DOMESTIC_Year_1_Declined": x,
    "term_2_BAESPM_DOMESTIC_Year_1_Pending": x,
    "term_2_BAESPM_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_BAESPM_DOMESTIC_Year_1_Enrolled": x,
    "term_2_BAESPM_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_BAESPM_INTERNATIONAL_Year_1_Applications": x,
    "term_2_BAESPM_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_BAESPM_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_BAESPM_INTERNATIONAL_Year_1_Denied": x,
    "term_2_BAESPM_INTERNATIONAL_Year_1_Declined": x,
    "term_2_BAESPM_INTERNATIONAL_Year_1_Pending": x,
    "term_2_BAESPM_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_BAESPM_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_BAESPM_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_BAESPM_DOMESTIC_Year_2_Applications": x,
    "term_2_BAESPM_DOMESTIC_Year_2_Accepted": x,
    "term_2_BAESPM_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_BAESPM_DOMESTIC_Year_2_Denied": x,
    "term_2_BAESPM_DOMESTIC_Year_2_Declined": x,
    "term_2_BAESPM_DOMESTIC_Year_2_Pending": x,
    "term_2_BAESPM_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_BAESPM_DOMESTIC_Year_2_Enrolled": x,
    "term_2_BAESPM_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_BAESPM_INTERNATIONAL_Year_2_Applications": x,
    "term_2_BAESPM_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_BAESPM_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_BAESPM_INTERNATIONAL_Year_2_Denied": x,
    "term_2_BAESPM_INTERNATIONAL_Year_2_Declined": x,
    "term_2_BAESPM_INTERNATIONAL_Year_2_Pending": x,
    "term_2_BAESPM_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_BAESPM_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_BAESPM_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_BAHRMD_DOMESTIC_Year_1_Applications": x,
    "term_2_BAHRMD_DOMESTIC_Year_1_Accepted": x,
    "term_2_BAHRMD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_BAHRMD_DOMESTIC_Year_1_Denied": x,
    "term_2_BAHRMD_DOMESTIC_Year_1_Declined": x,
    "term_2_BAHRMD_DOMESTIC_Year_1_Pending": x,
    "term_2_BAHRMD_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_BAHRMD_DOMESTIC_Year_1_Enrolled": x,
    "term_2_BAHRMD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_BAHRMD_INTERNATIONAL_Year_1_Applications": x,
    "term_2_BAHRMD_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_BAHRMD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_BAHRMD_INTERNATIONAL_Year_1_Denied": x,
    "term_2_BAHRMD_INTERNATIONAL_Year_1_Declined": x,
    "term_2_BAHRMD_INTERNATIONAL_Year_1_Pending": x,
    "term_2_BAHRMD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_BAHRMD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_BAHRMD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_BAHRMD_DOMESTIC_Year_2_Applications": x,
    "term_2_BAHRMD_DOMESTIC_Year_2_Accepted": x,
    "term_2_BAHRMD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_BAHRMD_DOMESTIC_Year_2_Denied": x,
    "term_2_BAHRMD_DOMESTIC_Year_2_Declined": x,
    "term_2_BAHRMD_DOMESTIC_Year_2_Pending": x,
    "term_2_BAHRMD_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_BAHRMD_DOMESTIC_Year_2_Enrolled": x,
    "term_2_BAHRMD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_BAHRMD_INTERNATIONAL_Year_2_Applications": x,
    "term_2_BAHRMD_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_BAHRMD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_BAHRMD_INTERNATIONAL_Year_2_Denied": x,
    "term_2_BAHRMD_INTERNATIONAL_Year_2_Declined": x,
    "term_2_BAHRMD_INTERNATIONAL_Year_2_Pending": x,
    "term_2_BAHRMD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_BAHRMD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_BAHRMD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_GSCMD_DOMESTIC_Year_1_Applications": x,
    "term_2_GSCMD_DOMESTIC_Year_1_Accepted": x,
    "term_2_GSCMD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_GSCMD_DOMESTIC_Year_1_Denied": x,
    "term_2_GSCMD_DOMESTIC_Year_1_Declined": x,
    "term_2_GSCMD_DOMESTIC_Year_1_Pending": x,
    "term_2_GSCMD_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_GSCMD_DOMESTIC_Year_1_Enrolled": x,
    "term_2_GSCMD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_GSCMD_INTERNATIONAL_Year_1_Applications": x,
    "term_2_GSCMD_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_GSCMD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_GSCMD_INTERNATIONAL_Year_1_Denied": x,
    "term_2_GSCMD_INTERNATIONAL_Year_1_Declined": x,
    "term_2_GSCMD_INTERNATIONAL_Year_1_Pending": x,
    "term_2_GSCMD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_GSCMD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_GSCMD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_GSCMD_DOMESTIC_Year_2_Applications": x,
    "term_2_GSCMD_DOMESTIC_Year_2_Accepted": x,
    "term_2_GSCMD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_GSCMD_DOMESTIC_Year_2_Denied": x,
    "term_2_GSCMD_DOMESTIC_Year_2_Declined": x,
    "term_2_GSCMD_DOMESTIC_Year_2_Pending": x,
    "term_2_GSCMD_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_GSCMD_DOMESTIC_Year_2_Enrolled": x,
    "term_2_GSCMD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_GSCMD_INTERNATIONAL_Year_2_Applications": x,
    "term_2_GSCMD_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_GSCMD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_GSCMD_INTERNATIONAL_Year_2_Denied": x,
    "term_2_GSCMD_INTERNATIONAL_Year_2_Declined": x,
    "term_2_GSCMD_INTERNATIONAL_Year_2_Pending": x,
    "term_2_GSCMD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_GSCMD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_GSCMD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_DIGMD_DOMESTIC_Year_1_Applications": x,
    "term_2_DIGMD_DOMESTIC_Year_1_Accepted": x,
    "term_2_DIGMD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_DIGMD_DOMESTIC_Year_1_Denied": x,
    "term_2_DIGMD_DOMESTIC_Year_1_Declined": x,
    "term_2_DIGMD_DOMESTIC_Year_1_Pending": x,
    "term_2_DIGMD_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_DIGMD_DOMESTIC_Year_1_Enrolled": x,
    "term_2_DIGMD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_DIGMD_INTERNATIONAL_Year_1_Applications": x,
    "term_2_DIGMD_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_DIGMD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_DIGMD_INTERNATIONAL_Year_1_Denied": x,
    "term_2_DIGMD_INTERNATIONAL_Year_1_Declined": x,
    "term_2_DIGMD_INTERNATIONAL_Year_1_Pending": x,
    "term_2_DIGMD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_DIGMD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_DIGMD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_DIGMD_DOMESTIC_Year_2_Applications": x,
    "term_2_DIGMD_DOMESTIC_Year_2_Accepted": x,
    "term_2_DIGMD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_DIGMD_DOMESTIC_Year_2_Denied": x,
    "term_2_DIGMD_DOMESTIC_Year_2_Declined": x,
    "term_2_DIGMD_DOMESTIC_Year_2_Pending": x,
    "term_2_DIGMD_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_DIGMD_DOMESTIC_Year_2_Enrolled": x,
    "term_2_DIGMD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_DIGMD_INTERNATIONAL_Year_2_Applications": x,
    "term_2_DIGMD_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_DIGMD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_DIGMD_INTERNATIONAL_Year_2_Denied": x,
    "term_2_DIGMD_INTERNATIONAL_Year_2_Declined": x,
    "term_2_DIGMD_INTERNATIONAL_Year_2_Pending": x,
    "term_2_DIGMD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_DIGMD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_DIGMD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_BAATCO_DOMESTIC_Year_1_Applications": x,
    "term_2_BAATCO_DOMESTIC_Year_1_Accepted": x,
    "term_2_BAATCO_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_BAATCO_DOMESTIC_Year_1_Denied": x,
    "term_2_BAATCO_DOMESTIC_Year_1_Declined": x,
    "term_2_BAATCO_DOMESTIC_Year_1_Pending": x,
    "term_2_BAATCO_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_BAATCO_DOMESTIC_Year_1_Enrolled": x,
    "term_2_BAATCO_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_BAATCO_INTERNATIONAL_Year_1_Applications": x,
    "term_2_BAATCO_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_BAATCO_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_BAATCO_INTERNATIONAL_Year_1_Denied": x,
    "term_2_BAATCO_INTERNATIONAL_Year_1_Declined": x,
    "term_2_BAATCO_INTERNATIONAL_Year_1_Pending": x,
    "term_2_BAATCO_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_BAATCO_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_BAATCO_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_BAATCO_DOMESTIC_Year_2_Applications": x,
    "term_2_BAATCO_DOMESTIC_Year_2_Accepted": x,
    "term_2_BAATCO_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_BAATCO_DOMESTIC_Year_2_Denied": x,
    "term_2_BAATCO_DOMESTIC_Year_2_Declined": x,
    "term_2_BAATCO_DOMESTIC_Year_2_Pending": x,
    "term_2_BAATCO_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_BAATCO_DOMESTIC_Year_2_Enrolled": x,
    "term_2_BAATCO_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_BAATCO_INTERNATIONAL_Year_2_Applications": x,
    "term_2_BAATCO_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_BAATCO_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_BAATCO_INTERNATIONAL_Year_2_Denied": x,
    "term_2_BAATCO_INTERNATIONAL_Year_2_Declined": x,
    "term_2_BAATCO_INTERNATIONAL_Year_2_Pending": x,
    "term_2_BAATCO_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_BAATCO_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_BAATCO_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_BABSCO_DOMESTIC_Year_1_Applications": x,
    "term_2_BABSCO_DOMESTIC_Year_1_Accepted": x,
    "term_2_BABSCO_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_BABSCO_DOMESTIC_Year_1_Denied": x,
    "term_2_BABSCO_DOMESTIC_Year_1_Declined": x,
    "term_2_BABSCO_DOMESTIC_Year_1_Pending": x,
    "term_2_BABSCO_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_BABSCO_DOMESTIC_Year_1_Enrolled": x,
    "term_2_BABSCO_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_BABSCO_INTERNATIONAL_Year_1_Applications": x,
    "term_2_BABSCO_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_BABSCO_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_BABSCO_INTERNATIONAL_Year_1_Denied": x,
    "term_2_BABSCO_INTERNATIONAL_Year_1_Declined": x,
    "term_2_BABSCO_INTERNATIONAL_Year_1_Pending": x,
    "term_2_BABSCO_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_BABSCO_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_BABSCO_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_BABSCO_DOMESTIC_Year_2_Applications": x,
    "term_2_BABSCO_DOMESTIC_Year_2_Accepted": x,
    "term_2_BABSCO_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_BABSCO_DOMESTIC_Year_2_Denied": x,
    "term_2_BABSCO_DOMESTIC_Year_2_Declined": x,
    "term_2_BABSCO_DOMESTIC_Year_2_Pending": x,
    "term_2_BABSCO_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_BABSCO_DOMESTIC_Year_2_Enrolled": x,
    "term_2_BABSCO_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_BABSCO_INTERNATIONAL_Year_2_Applications": x,
    "term_2_BABSCO_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_BABSCO_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_BABSCO_INTERNATIONAL_Year_2_Denied": x,
    "term_2_BABSCO_INTERNATIONAL_Year_2_Declined": x,
    "term_2_BABSCO_INTERNATIONAL_Year_2_Pending": x,
    "term_2_BABSCO_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_BABSCO_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_BABSCO_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_NAO_DOMESTIC_Year_1_Applications": x,
    "term_2_NAO_DOMESTIC_Year_1_Accepted": x,
    "term_2_NAO_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_NAO_DOMESTIC_Year_1_Denied": x,
    "term_2_NAO_DOMESTIC_Year_1_Declined": x,
    "term_2_NAO_DOMESTIC_Year_1_Pending": x,
    "term_2_NAO_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_NAO_DOMESTIC_Year_1_Enrolled": x,
    "term_2_NAO_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_NAO_INTERNATIONAL_Year_1_Applications": x,
    "term_2_NAO_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_NAO_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_NAO_INTERNATIONAL_Year_1_Denied": x,
    "term_2_NAO_INTERNATIONAL_Year_1_Declined": x,
    "term_2_NAO_INTERNATIONAL_Year_1_Pending": x,
    "term_2_NAO_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_NAO_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_NAO_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_NAO_DOMESTIC_Year_2_Applications": x,
    "term_2_NAO_DOMESTIC_Year_2_Accepted": x,
    "term_2_NAO_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_NAO_DOMESTIC_Year_2_Denied": x,
    "term_2_NAO_DOMESTIC_Year_2_Declined": x,
    "term_2_NAO_DOMESTIC_Year_2_Pending": x,
    "term_2_NAO_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_NAO_DOMESTIC_Year_2_Enrolled": x,
    "term_2_NAO_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_NAO_INTERNATIONAL_Year_2_Applications": x,
    "term_2_NAO_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_NAO_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_NAO_INTERNATIONAL_Year_2_Denied": x,
    "term_2_NAO_INTERNATIONAL_Year_2_Declined": x,
    "term_2_NAO_INTERNATIONAL_Year_2_Pending": x,
    "term_2_NAO_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_NAO_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_NAO_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_ALO_DOMESTIC_Year_1_Applications": x,
    "term_2_ALO_DOMESTIC_Year_1_Accepted": x,
    "term_2_ALO_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_ALO_DOMESTIC_Year_1_Denied": x,
    "term_2_ALO_DOMESTIC_Year_1_Declined": x,
    "term_2_ALO_DOMESTIC_Year_1_Pending": x,
    "term_2_ALO_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_ALO_DOMESTIC_Year_1_Enrolled": x,
    "term_2_ALO_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_ALO_INTERNATIONAL_Year_1_Applications": x,
    "term_2_ALO_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_ALO_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_ALO_INTERNATIONAL_Year_1_Denied": x,
    "term_2_ALO_INTERNATIONAL_Year_1_Declined": x,
    "term_2_ALO_INTERNATIONAL_Year_1_Pending": x,
    "term_2_ALO_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_ALO_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_ALO_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_ALO_DOMESTIC_Year_2_Applications": x,
    "term_2_ALO_DOMESTIC_Year_2_Accepted": x,
    "term_2_ALO_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_ALO_DOMESTIC_Year_2_Denied": x,
    "term_2_ALO_DOMESTIC_Year_2_Declined": x,
    "term_2_ALO_DOMESTIC_Year_2_Pending": x,
    "term_2_ALO_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_ALO_DOMESTIC_Year_2_Enrolled": x,
    "term_2_ALO_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_ALO_INTERNATIONAL_Year_2_Applications": x,
    "term_2_ALO_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_ALO_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_ALO_INTERNATIONAL_Year_2_Denied": x,
    "term_2_ALO_INTERNATIONAL_Year_2_Declined": x,
    "term_2_ALO_INTERNATIONAL_Year_2_Pending": x,
    "term_2_ALO_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_ALO_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_ALO_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_HOSMNG_DOMESTIC_Year_1_Applications": x,
    "term_2_HOSMNG_DOMESTIC_Year_1_Accepted": x,
    "term_2_HOSMNG_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_HOSMNG_DOMESTIC_Year_1_Denied": x,
    "term_2_HOSMNG_DOMESTIC_Year_1_Declined": x,
    "term_2_HOSMNG_DOMESTIC_Year_1_Pending": x,
    "term_2_HOSMNG_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_HOSMNG_DOMESTIC_Year_1_Enrolled": x,
    "term_2_HOSMNG_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_HOSMNG_INTERNATIONAL_Year_1_Applications": x,
    "term_2_HOSMNG_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_HOSMNG_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_HOSMNG_INTERNATIONAL_Year_1_Denied": x,
    "term_2_HOSMNG_INTERNATIONAL_Year_1_Declined": x,
    "term_2_HOSMNG_INTERNATIONAL_Year_1_Pending": x,
    "term_2_HOSMNG_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_HOSMNG_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_HOSMNG_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_HOSMNG_DOMESTIC_Year_2_Applications": x,
    "term_2_HOSMNG_DOMESTIC_Year_2_Accepted": x,
    "term_2_HOSMNG_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_HOSMNG_DOMESTIC_Year_2_Denied": x,
    "term_2_HOSMNG_DOMESTIC_Year_2_Declined": x,
    "term_2_HOSMNG_DOMESTIC_Year_2_Pending": x,
    "term_2_HOSMNG_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_HOSMNG_DOMESTIC_Year_2_Enrolled": x,
    "term_2_HOSMNG_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_HOSMNG_INTERNATIONAL_Year_2_Applications": x,
    "term_2_HOSMNG_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_HOSMNG_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_HOSMNG_INTERNATIONAL_Year_2_Denied": x,
    "term_2_HOSMNG_INTERNATIONAL_Year_2_Declined": x,
    "term_2_HOSMNG_INTERNATIONAL_Year_2_Pending": x,
    "term_2_HOSMNG_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_HOSMNG_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_HOSMNG_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_BAHRMC_DOMESTIC_Year_1_Applications": x,
    "term_2_BAHRMC_DOMESTIC_Year_1_Accepted": x,
    "term_2_BAHRMC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_BAHRMC_DOMESTIC_Year_1_Denied": x,
    "term_2_BAHRMC_DOMESTIC_Year_1_Declined": x,
    "term_2_BAHRMC_DOMESTIC_Year_1_Pending": x,
    "term_2_BAHRMC_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_BAHRMC_DOMESTIC_Year_1_Enrolled": x,
    "term_2_BAHRMC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_BAHRMC_INTERNATIONAL_Year_1_Applications": x,
    "term_2_BAHRMC_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_BAHRMC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_BAHRMC_INTERNATIONAL_Year_1_Denied": x,
    "term_2_BAHRMC_INTERNATIONAL_Year_1_Declined": x,
    "term_2_BAHRMC_INTERNATIONAL_Year_1_Pending": x,
    "term_2_BAHRMC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_BAHRMC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_BAHRMC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_OFAC_DOMESTIC_Year_1_Applications": x,
    "term_2_OFAC_DOMESTIC_Year_1_Accepted": x,
    "term_2_OFAC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_OFAC_DOMESTIC_Year_1_Denied": x,
    "term_2_OFAC_DOMESTIC_Year_1_Declined": x,
    "term_2_OFAC_DOMESTIC_Year_1_Pending": x,
    "term_2_OFAC_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_OFAC_DOMESTIC_Year_1_Enrolled": x,
    "term_2_OFAC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_OFAC_INTERNATIONAL_Year_1_Applications": x,
    "term_2_OFAC_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_OFAC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_OFAC_INTERNATIONAL_Year_1_Denied": x,
    "term_2_OFAC_INTERNATIONAL_Year_1_Declined": x,
    "term_2_OFAC_INTERNATIONAL_Year_1_Pending": x,
    "term_2_OFAC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_OFAC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_OFAC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_EAP_DOMESTIC_Year_1_Applications": x,
    "term_2_EAP_DOMESTIC_Year_1_Accepted": x,
    "term_2_EAP_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_EAP_DOMESTIC_Year_1_Denied": x,
    "term_2_EAP_DOMESTIC_Year_1_Declined": x,
    "term_2_EAP_DOMESTIC_Year_1_Pending": x,
    "term_2_EAP_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_EAP_DOMESTIC_Year_1_Enrolled": x,
    "term_2_EAP_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_EAP_INTERNATIONAL_Year_1_Applications": x,
    "term_2_EAP_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_EAP_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_EAP_INTERNATIONAL_Year_1_Denied": x,
    "term_2_EAP_INTERNATIONAL_Year_1_Declined": x,
    "term_2_EAP_INTERNATIONAL_Year_1_Pending": x,
    "term_2_EAP_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_EAP_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_EAP_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_HSAFA_DOMESTIC_Year_1_Applications": x,
    "term_2_HSAFA_DOMESTIC_Year_1_Accepted": x,
    "term_2_HSAFA_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_HSAFA_DOMESTIC_Year_1_Denied": x,
    "term_2_HSAFA_DOMESTIC_Year_1_Declined": x,
    "term_2_HSAFA_DOMESTIC_Year_1_Pending": x,
    "term_2_HSAFA_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_HSAFA_DOMESTIC_Year_1_Enrolled": x,
    "term_2_HSAFA_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_HSAFA_INTERNATIONAL_Year_1_Applications": x,
    "term_2_HSAFA_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_HSAFA_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_HSAFA_INTERNATIONAL_Year_1_Denied": x,
    "term_2_HSAFA_INTERNATIONAL_Year_1_Declined": x,
    "term_2_HSAFA_INTERNATIONAL_Year_1_Pending": x,
    "term_2_HSAFA_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_HSAFA_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_HSAFA_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_HSAD_DOMESTIC_Year_1_Applications": x,
    "term_2_HSAD_DOMESTIC_Year_1_Accepted": x,
    "term_2_HSAD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_HSAD_DOMESTIC_Year_1_Denied": x,
    "term_2_HSAD_DOMESTIC_Year_1_Declined": x,
    "term_2_HSAD_DOMESTIC_Year_1_Pending": x,
    "term_2_HSAD_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_HSAD_DOMESTIC_Year_1_Enrolled": x,
    "term_2_HSAD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_HSAD_INTERNATIONAL_Year_1_Applications": x,
    "term_2_HSAD_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_HSAD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_HSAD_INTERNATIONAL_Year_1_Denied": x,
    "term_2_HSAD_INTERNATIONAL_Year_1_Declined": x,
    "term_2_HSAD_INTERNATIONAL_Year_1_Pending": x,
    "term_2_HSAD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_HSAD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_HSAD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_HSGD_DOMESTIC_Year_1_Applications": x,
    "term_2_HSGD_DOMESTIC_Year_1_Accepted": x,
    "term_2_HSGD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_HSGD_DOMESTIC_Year_1_Denied": x,
    "term_2_HSGD_DOMESTIC_Year_1_Declined": x,
    "term_2_HSGD_DOMESTIC_Year_1_Pending": x,
    "term_2_HSGD_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_HSGD_DOMESTIC_Year_1_Enrolled": x,
    "term_2_HSGD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_HSGD_INTERNATIONAL_Year_1_Applications": x,
    "term_2_HSGD_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_HSGD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_HSGD_INTERNATIONAL_Year_1_Denied": x,
    "term_2_HSGD_INTERNATIONAL_Year_1_Declined": x,
    "term_2_HSGD_INTERNATIONAL_Year_1_Pending": x,
    "term_2_HSGD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_HSGD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_HSGD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_HSNP_DOMESTIC_Year_1_Applications": x,
    "term_2_HSNP_DOMESTIC_Year_1_Accepted": x,
    "term_2_HSNP_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_HSNP_DOMESTIC_Year_1_Denied": x,
    "term_2_HSNP_DOMESTIC_Year_1_Declined": x,
    "term_2_HSNP_DOMESTIC_Year_1_Pending": x,
    "term_2_HSNP_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_HSNP_DOMESTIC_Year_1_Enrolled": x,
    "term_2_HSNP_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_HSNP_INTERNATIONAL_Year_1_Applications": x,
    "term_2_HSNP_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_HSNP_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_HSNP_INTERNATIONAL_Year_1_Denied": x,
    "term_2_HSNP_INTERNATIONAL_Year_1_Declined": x,
    "term_2_HSNP_INTERNATIONAL_Year_1_Pending": x,
    "term_2_HSNP_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_HSNP_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_HSNP_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_UTELEM_DOMESTIC_Year_1_Applications": x,
    "term_2_UTELEM_DOMESTIC_Year_1_Accepted": x,
    "term_2_UTELEM_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_UTELEM_DOMESTIC_Year_1_Denied": x,
    "term_2_UTELEM_DOMESTIC_Year_1_Declined": x,
    "term_2_UTELEM_DOMESTIC_Year_1_Pending": x,
    "term_2_UTELEM_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_UTELEM_DOMESTIC_Year_1_Enrolled": x,
    "term_2_UTELEM_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_UTELEM_INTERNATIONAL_Year_1_Applications": x,
    "term_2_UTELEM_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_UTELEM_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_UTELEM_INTERNATIONAL_Year_1_Denied": x,
    "term_2_UTELEM_INTERNATIONAL_Year_1_Declined": x,
    "term_2_UTELEM_INTERNATIONAL_Year_1_Pending": x,
    "term_2_UTELEM_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_UTELEM_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_UTELEM_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_UTELEM_DOMESTIC_Year_2_Applications": x,
    "term_2_UTELEM_DOMESTIC_Year_2_Accepted": x,
    "term_2_UTELEM_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_UTELEM_DOMESTIC_Year_2_Denied": x,
    "term_2_UTELEM_DOMESTIC_Year_2_Declined": x,
    "term_2_UTELEM_DOMESTIC_Year_2_Pending": x,
    "term_2_UTELEM_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_UTELEM_DOMESTIC_Year_2_Enrolled": x,
    "term_2_UTELEM_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_UTELEM_INTERNATIONAL_Year_2_Applications": x,
    "term_2_UTELEM_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_UTELEM_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_UTELEM_INTERNATIONAL_Year_2_Denied": x,
    "term_2_UTELEM_INTERNATIONAL_Year_2_Declined": x,
    "term_2_UTELEM_INTERNATIONAL_Year_2_Pending": x,
    "term_2_UTELEM_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_UTELEM_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_UTELEM_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_UTSEC_DOMESTIC_Year_1_Applications": x,
    "term_2_UTSEC_DOMESTIC_Year_1_Accepted": x,
    "term_2_UTSEC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_UTSEC_DOMESTIC_Year_1_Denied": x,
    "term_2_UTSEC_DOMESTIC_Year_1_Declined": x,
    "term_2_UTSEC_DOMESTIC_Year_1_Pending": x,
    "term_2_UTSEC_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_UTSEC_DOMESTIC_Year_1_Enrolled": x,
    "term_2_UTSEC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_UTSEC_INTERNATIONAL_Year_1_Applications": x,
    "term_2_UTSEC_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_UTSEC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_UTSEC_INTERNATIONAL_Year_1_Denied": x,
    "term_2_UTSEC_INTERNATIONAL_Year_1_Declined": x,
    "term_2_UTSEC_INTERNATIONAL_Year_1_Pending": x,
    "term_2_UTSEC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_UTSEC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_UTSEC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_UTSEC_DOMESTIC_Year_2_Applications": x,
    "term_2_UTSEC_DOMESTIC_Year_2_Accepted": x,
    "term_2_UTSEC_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_UTSEC_DOMESTIC_Year_2_Denied": x,
    "term_2_UTSEC_DOMESTIC_Year_2_Declined": x,
    "term_2_UTSEC_DOMESTIC_Year_2_Pending": x,
    "term_2_UTSEC_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_UTSEC_DOMESTIC_Year_2_Enrolled": x,
    "term_2_UTSEC_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_UTSEC_INTERNATIONAL_Year_2_Applications": x,
    "term_2_UTSEC_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_UTSEC_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_UTSEC_INTERNATIONAL_Year_2_Denied": x,
    "term_2_UTSEC_INTERNATIONAL_Year_2_Declined": x,
    "term_2_UTSEC_INTERNATIONAL_Year_2_Pending": x,
    "term_2_UTSEC_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_UTSEC_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_UTSEC_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_ENVTD_DOMESTIC_Year_1_Applications": x,
    "term_2_ENVTD_DOMESTIC_Year_1_Accepted": x,
    "term_2_ENVTD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_ENVTD_DOMESTIC_Year_1_Denied": x,
    "term_2_ENVTD_DOMESTIC_Year_1_Declined": x,
    "term_2_ENVTD_DOMESTIC_Year_1_Pending": x,
    "term_2_ENVTD_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_ENVTD_DOMESTIC_Year_1_Enrolled": x,
    "term_2_ENVTD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_ENVTD_INTERNATIONAL_Year_1_Applications": x,
    "term_2_ENVTD_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_ENVTD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_ENVTD_INTERNATIONAL_Year_1_Denied": x,
    "term_2_ENVTD_INTERNATIONAL_Year_1_Declined": x,
    "term_2_ENVTD_INTERNATIONAL_Year_1_Pending": x,
    "term_2_ENVTD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_ENVTD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_ENVTD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_ENVTD_DOMESTIC_Year_2_Applications": x,
    "term_2_ENVTD_DOMESTIC_Year_2_Accepted": x,
    "term_2_ENVTD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_ENVTD_DOMESTIC_Year_2_Denied": x,
    "term_2_ENVTD_DOMESTIC_Year_2_Declined": x,
    "term_2_ENVTD_DOMESTIC_Year_2_Pending": x,
    "term_2_ENVTD_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_ENVTD_DOMESTIC_Year_2_Enrolled": x,
    "term_2_ENVTD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_ENVTD_INTERNATIONAL_Year_2_Applications": x,
    "term_2_ENVTD_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_ENVTD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_ENVTD_INTERNATIONAL_Year_2_Denied": x,
    "term_2_ENVTD_INTERNATIONAL_Year_2_Declined": x,
    "term_2_ENVTD_INTERNATIONAL_Year_2_Pending": x,
    "term_2_ENVTD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_ENVTD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_ENVTD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_ENVNC_DOMESTIC_Year_1_Applications": x,
    "term_2_ENVNC_DOMESTIC_Year_1_Accepted": x,
    "term_2_ENVNC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_ENVNC_DOMESTIC_Year_1_Denied": x,
    "term_2_ENVNC_DOMESTIC_Year_1_Declined": x,
    "term_2_ENVNC_DOMESTIC_Year_1_Pending": x,
    "term_2_ENVNC_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_ENVNC_DOMESTIC_Year_1_Enrolled": x,
    "term_2_ENVNC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_ENVNC_INTERNATIONAL_Year_1_Applications": x,
    "term_2_ENVNC_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_ENVNC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_ENVNC_INTERNATIONAL_Year_1_Denied": x,
    "term_2_ENVNC_INTERNATIONAL_Year_1_Declined": x,
    "term_2_ENVNC_INTERNATIONAL_Year_1_Pending": x,
    "term_2_ENVNC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_ENVNC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_ENVNC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_ENVNC_DOMESTIC_Year_2_Applications": x,
    "term_2_ENVNC_DOMESTIC_Year_2_Accepted": x,
    "term_2_ENVNC_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_ENVNC_DOMESTIC_Year_2_Denied": x,
    "term_2_ENVNC_DOMESTIC_Year_2_Declined": x,
    "term_2_ENVNC_DOMESTIC_Year_2_Pending": x,
    "term_2_ENVNC_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_ENVNC_DOMESTIC_Year_2_Enrolled": x,
    "term_2_ENVNC_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_ENVNC_INTERNATIONAL_Year_2_Applications": x,
    "term_2_ENVNC_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_ENVNC_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_ENVNC_INTERNATIONAL_Year_2_Denied": x,
    "term_2_ENVNC_INTERNATIONAL_Year_2_Declined": x,
    "term_2_ENVNC_INTERNATIONAL_Year_2_Pending": x,
    "term_2_ENVNC_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_ENVNC_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_ENVNC_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_UTARTS_DOMESTIC_Year_1_Applications": x,
    "term_2_UTARTS_DOMESTIC_Year_1_Accepted": x,
    "term_2_UTARTS_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_UTARTS_DOMESTIC_Year_1_Denied": x,
    "term_2_UTARTS_DOMESTIC_Year_1_Declined": x,
    "term_2_UTARTS_DOMESTIC_Year_1_Pending": x,
    "term_2_UTARTS_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_UTARTS_DOMESTIC_Year_1_Enrolled": x,
    "term_2_UTARTS_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_UTARTS_INTERNATIONAL_Year_1_Applications": x,
    "term_2_UTARTS_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_UTARTS_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_UTARTS_INTERNATIONAL_Year_1_Denied": x,
    "term_2_UTARTS_INTERNATIONAL_Year_1_Declined": x,
    "term_2_UTARTS_INTERNATIONAL_Year_1_Pending": x,
    "term_2_UTARTS_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_UTARTS_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_UTARTS_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_UTARTS_DOMESTIC_Year_2_Applications": x,
    "term_2_UTARTS_DOMESTIC_Year_2_Accepted": x,
    "term_2_UTARTS_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_UTARTS_DOMESTIC_Year_2_Denied": x,
    "term_2_UTARTS_DOMESTIC_Year_2_Declined": x,
    "term_2_UTARTS_DOMESTIC_Year_2_Pending": x,
    "term_2_UTARTS_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_UTARTS_DOMESTIC_Year_2_Enrolled": x,
    "term_2_UTARTS_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_UTARTS_INTERNATIONAL_Year_2_Applications": x,
    "term_2_UTARTS_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_UTARTS_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_UTARTS_INTERNATIONAL_Year_2_Denied": x,
    "term_2_UTARTS_INTERNATIONAL_Year_2_Declined": x,
    "term_2_UTARTS_INTERNATIONAL_Year_2_Pending": x,
    "term_2_UTARTS_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_UTARTS_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_UTARTS_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_UTCOMM_DOMESTIC_Year_1_Applications": x,
    "term_2_UTCOMM_DOMESTIC_Year_1_Accepted": x,
    "term_2_UTCOMM_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_UTCOMM_DOMESTIC_Year_1_Denied": x,
    "term_2_UTCOMM_DOMESTIC_Year_1_Declined": x,
    "term_2_UTCOMM_DOMESTIC_Year_1_Pending": x,
    "term_2_UTCOMM_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_UTCOMM_DOMESTIC_Year_1_Enrolled": x,
    "term_2_UTCOMM_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_UTCOMM_INTERNATIONAL_Year_1_Applications": x,
    "term_2_UTCOMM_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_UTCOMM_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_UTCOMM_INTERNATIONAL_Year_1_Denied": x,
    "term_2_UTCOMM_INTERNATIONAL_Year_1_Declined": x,
    "term_2_UTCOMM_INTERNATIONAL_Year_1_Pending": x,
    "term_2_UTCOMM_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_UTCOMM_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_UTCOMM_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_UTCOMM_DOMESTIC_Year_2_Applications": x,
    "term_2_UTCOMM_DOMESTIC_Year_2_Accepted": x,
    "term_2_UTCOMM_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_UTCOMM_DOMESTIC_Year_2_Denied": x,
    "term_2_UTCOMM_DOMESTIC_Year_2_Declined": x,
    "term_2_UTCOMM_DOMESTIC_Year_2_Pending": x,
    "term_2_UTCOMM_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_UTCOMM_DOMESTIC_Year_2_Enrolled": x,
    "term_2_UTCOMM_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_UTCOMM_INTERNATIONAL_Year_2_Applications": x,
    "term_2_UTCOMM_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_UTCOMM_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_UTCOMM_INTERNATIONAL_Year_2_Denied": x,
    "term_2_UTCOMM_INTERNATIONAL_Year_2_Declined": x,
    "term_2_UTCOMM_INTERNATIONAL_Year_2_Pending": x,
    "term_2_UTCOMM_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_UTCOMM_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_UTCOMM_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_UTENG_DOMESTIC_Year_1_Applications": x,
    "term_2_UTENG_DOMESTIC_Year_1_Accepted": x,
    "term_2_UTENG_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_UTENG_DOMESTIC_Year_1_Denied": x,
    "term_2_UTENG_DOMESTIC_Year_1_Declined": x,
    "term_2_UTENG_DOMESTIC_Year_1_Pending": x,
    "term_2_UTENG_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_UTENG_DOMESTIC_Year_1_Enrolled": x,
    "term_2_UTENG_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_UTENG_INTERNATIONAL_Year_1_Applications": x,
    "term_2_UTENG_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_UTENG_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_UTENG_INTERNATIONAL_Year_1_Denied": x,
    "term_2_UTENG_INTERNATIONAL_Year_1_Declined": x,
    "term_2_UTENG_INTERNATIONAL_Year_1_Pending": x,
    "term_2_UTENG_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_UTENG_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_UTENG_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_UTSCGS_DOMESTIC_Year_1_Applications": x,
    "term_2_UTSCGS_DOMESTIC_Year_1_Accepted": x,
    "term_2_UTSCGS_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_UTSCGS_DOMESTIC_Year_1_Denied": x,
    "term_2_UTSCGS_DOMESTIC_Year_1_Declined": x,
    "term_2_UTSCGS_DOMESTIC_Year_1_Pending": x,
    "term_2_UTSCGS_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_UTSCGS_DOMESTIC_Year_1_Enrolled": x,
    "term_2_UTSCGS_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_UTSCGS_INTERNATIONAL_Year_1_Applications": x,
    "term_2_UTSCGS_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_UTSCGS_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_UTSCGS_INTERNATIONAL_Year_1_Denied": x,
    "term_2_UTSCGS_INTERNATIONAL_Year_1_Declined": x,
    "term_2_UTSCGS_INTERNATIONAL_Year_1_Pending": x,
    "term_2_UTSCGS_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_UTSCGS_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_UTSCGS_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_UTSCGS_DOMESTIC_Year_2_Applications": x,
    "term_2_UTSCGS_DOMESTIC_Year_2_Accepted": x,
    "term_2_UTSCGS_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_UTSCGS_DOMESTIC_Year_2_Denied": x,
    "term_2_UTSCGS_DOMESTIC_Year_2_Declined": x,
    "term_2_UTSCGS_DOMESTIC_Year_2_Pending": x,
    "term_2_UTSCGS_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_UTSCGS_DOMESTIC_Year_2_Enrolled": x,
    "term_2_UTSCGS_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_UTSCGS_INTERNATIONAL_Year_2_Applications": x,
    "term_2_UTSCGS_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_UTSCGS_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_UTSCGS_INTERNATIONAL_Year_2_Denied": x,
    "term_2_UTSCGS_INTERNATIONAL_Year_2_Declined": x,
    "term_2_UTSCGS_INTERNATIONAL_Year_2_Pending": x,
    "term_2_UTSCGS_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_UTSCGS_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_UTSCGS_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_UTSCWK_DOMESTIC_Year_1_Applications": x,
    "term_2_UTSCWK_DOMESTIC_Year_1_Accepted": x,
    "term_2_UTSCWK_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_UTSCWK_DOMESTIC_Year_1_Denied": x,
    "term_2_UTSCWK_DOMESTIC_Year_1_Declined": x,
    "term_2_UTSCWK_DOMESTIC_Year_1_Pending": x,
    "term_2_UTSCWK_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_UTSCWK_DOMESTIC_Year_1_Enrolled": x,
    "term_2_UTSCWK_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_UTSCWK_INTERNATIONAL_Year_1_Applications": x,
    "term_2_UTSCWK_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_UTSCWK_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_UTSCWK_INTERNATIONAL_Year_1_Denied": x,
    "term_2_UTSCWK_INTERNATIONAL_Year_1_Declined": x,
    "term_2_UTSCWK_INTERNATIONAL_Year_1_Pending": x,
    "term_2_UTSCWK_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_UTSCWK_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_UTSCWK_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_UTSCWK_DOMESTIC_Year_2_Applications": x,
    "term_2_UTSCWK_DOMESTIC_Year_2_Accepted": x,
    "term_2_UTSCWK_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_UTSCWK_DOMESTIC_Year_2_Denied": x,
    "term_2_UTSCWK_DOMESTIC_Year_2_Declined": x,
    "term_2_UTSCWK_DOMESTIC_Year_2_Pending": x,
    "term_2_UTSCWK_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_UTSCWK_DOMESTIC_Year_2_Enrolled": x,
    "term_2_UTSCWK_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_UTSCWK_INTERNATIONAL_Year_2_Applications": x,
    "term_2_UTSCWK_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_UTSCWK_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_UTSCWK_INTERNATIONAL_Year_2_Denied": x,
    "term_2_UTSCWK_INTERNATIONAL_Year_2_Declined": x,
    "term_2_UTSCWK_INTERNATIONAL_Year_2_Pending": x,
    "term_2_UTSCWK_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_UTSCWK_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_UTSCWK_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_OPEN_DOMESTIC_Year_1_Applications": x,
    "term_2_OPEN_DOMESTIC_Year_1_Accepted": x,
    "term_2_OPEN_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_OPEN_DOMESTIC_Year_1_Denied": x,
    "term_2_OPEN_DOMESTIC_Year_1_Declined": x,
    "term_2_OPEN_DOMESTIC_Year_1_Pending": x,
    "term_2_OPEN_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_OPEN_DOMESTIC_Year_1_Enrolled": x,
    "term_2_OPEN_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_OPEN_INTERNATIONAL_Year_1_Applications": x,
    "term_2_OPEN_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_OPEN_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_OPEN_INTERNATIONAL_Year_1_Denied": x,
    "term_2_OPEN_INTERNATIONAL_Year_1_Declined": x,
    "term_2_OPEN_INTERNATIONAL_Year_1_Pending": x,
    "term_2_OPEN_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_OPEN_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_OPEN_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_GOVNSC_DOMESTIC_Year_1_Applications": x,
    "term_2_GOVNSC_DOMESTIC_Year_1_Accepted": x,
    "term_2_GOVNSC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_GOVNSC_DOMESTIC_Year_1_Denied": x,
    "term_2_GOVNSC_DOMESTIC_Year_1_Declined": x,
    "term_2_GOVNSC_DOMESTIC_Year_1_Pending": x,
    "term_2_GOVNSC_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_GOVNSC_DOMESTIC_Year_1_Enrolled": x,
    "term_2_GOVNSC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_GOVNSC_INTERNATIONAL_Year_1_Applications": x,
    "term_2_GOVNSC_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_GOVNSC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_GOVNSC_INTERNATIONAL_Year_1_Denied": x,
    "term_2_GOVNSC_INTERNATIONAL_Year_1_Declined": x,
    "term_2_GOVNSC_INTERNATIONAL_Year_1_Pending": x,
    "term_2_GOVNSC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_GOVNSC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_GOVNSC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_GOVNSC_DOMESTIC_Year_2_Applications": x,
    "term_2_GOVNSC_DOMESTIC_Year_2_Accepted": x,
    "term_2_GOVNSC_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_GOVNSC_DOMESTIC_Year_2_Denied": x,
    "term_2_GOVNSC_DOMESTIC_Year_2_Declined": x,
    "term_2_GOVNSC_DOMESTIC_Year_2_Pending": x,
    "term_2_GOVNSC_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_GOVNSC_DOMESTIC_Year_2_Enrolled": x,
    "term_2_GOVNSC_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_GOVNSC_INTERNATIONAL_Year_2_Applications": x,
    "term_2_GOVNSC_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_GOVNSC_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_GOVNSC_INTERNATIONAL_Year_2_Denied": x,
    "term_2_GOVNSC_INTERNATIONAL_Year_2_Declined": x,
    "term_2_GOVNSC_INTERNATIONAL_Year_2_Pending": x,
    "term_2_GOVNSC_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_GOVNSC_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_GOVNSC_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_COMSCC_DOMESTIC_Year_1_Applications": x,
    "term_2_COMSCC_DOMESTIC_Year_1_Accepted": x,
    "term_2_COMSCC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_COMSCC_DOMESTIC_Year_1_Denied": x,
    "term_2_COMSCC_DOMESTIC_Year_1_Declined": x,
    "term_2_COMSCC_DOMESTIC_Year_1_Pending": x,
    "term_2_COMSCC_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_COMSCC_DOMESTIC_Year_1_Enrolled": x,
    "term_2_COMSCC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_COMSCC_INTERNATIONAL_Year_1_Applications": x,
    "term_2_COMSCC_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_COMSCC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_COMSCC_INTERNATIONAL_Year_1_Denied": x,
    "term_2_COMSCC_INTERNATIONAL_Year_1_Declined": x,
    "term_2_COMSCC_INTERNATIONAL_Year_1_Pending": x,
    "term_2_COMSCC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_COMSCC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_COMSCC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_COMSCD_DOMESTIC_Year_1_Applications": x,
    "term_2_COMSCD_DOMESTIC_Year_1_Accepted": x,
    "term_2_COMSCD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_COMSCD_DOMESTIC_Year_1_Denied": x,
    "term_2_COMSCD_DOMESTIC_Year_1_Declined": x,
    "term_2_COMSCD_DOMESTIC_Year_1_Pending": x,
    "term_2_COMSCD_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_COMSCD_DOMESTIC_Year_1_Enrolled": x,
    "term_2_COMSCD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_COMSCD_INTERNATIONAL_Year_1_Applications": x,
    "term_2_COMSCD_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_COMSCD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_COMSCD_INTERNATIONAL_Year_1_Denied": x,
    "term_2_COMSCD_INTERNATIONAL_Year_1_Declined": x,
    "term_2_COMSCD_INTERNATIONAL_Year_1_Pending": x,
    "term_2_COMSCD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_COMSCD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_COMSCD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_COMSCD_DOMESTIC_Year_2_Applications": x,
    "term_2_COMSCD_DOMESTIC_Year_2_Accepted": x,
    "term_2_COMSCD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_COMSCD_DOMESTIC_Year_2_Denied": x,
    "term_2_COMSCD_DOMESTIC_Year_2_Declined": x,
    "term_2_COMSCD_DOMESTIC_Year_2_Pending": x,
    "term_2_COMSCD_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_COMSCD_DOMESTIC_Year_2_Enrolled": x,
    "term_2_COMSCD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_COMSCD_INTERNATIONAL_Year_2_Applications": x,
    "term_2_COMSCD_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_COMSCD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_COMSCD_INTERNATIONAL_Year_2_Denied": x,
    "term_2_COMSCD_INTERNATIONAL_Year_2_Declined": x,
    "term_2_COMSCD_INTERNATIONAL_Year_2_Pending": x,
    "term_2_COMSCD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_COMSCD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_COMSCD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_GNARTC_DOMESTIC_Year_1_Applications": x,
    "term_2_GNARTC_DOMESTIC_Year_1_Accepted": x,
    "term_2_GNARTC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_GNARTC_DOMESTIC_Year_1_Denied": x,
    "term_2_GNARTC_DOMESTIC_Year_1_Declined": x,
    "term_2_GNARTC_DOMESTIC_Year_1_Pending": x,
    "term_2_GNARTC_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_GNARTC_DOMESTIC_Year_1_Enrolled": x,
    "term_2_GNARTC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_GNARTC_INTERNATIONAL_Year_1_Applications": x,
    "term_2_GNARTC_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_GNARTC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_GNARTC_INTERNATIONAL_Year_1_Denied": x,
    "term_2_GNARTC_INTERNATIONAL_Year_1_Declined": x,
    "term_2_GNARTC_INTERNATIONAL_Year_1_Pending": x,
    "term_2_GNARTC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_GNARTC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_GNARTC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_GNSCIC_DOMESTIC_Year_1_Applications": x,
    "term_2_GNSCIC_DOMESTIC_Year_1_Accepted": x,
    "term_2_GNSCIC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_GNSCIC_DOMESTIC_Year_1_Denied": x,
    "term_2_GNSCIC_DOMESTIC_Year_1_Declined": x,
    "term_2_GNSCIC_DOMESTIC_Year_1_Pending": x,
    "term_2_GNSCIC_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_GNSCIC_DOMESTIC_Year_1_Enrolled": x,
    "term_2_GNSCIC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_GNSCIC_INTERNATIONAL_Year_1_Applications": x,
    "term_2_GNSCIC_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_GNSCIC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_GNSCIC_INTERNATIONAL_Year_1_Denied": x,
    "term_2_GNSCIC_INTERNATIONAL_Year_1_Declined": x,
    "term_2_GNSCIC_INTERNATIONAL_Year_1_Pending": x,
    "term_2_GNSCIC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_GNSCIC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_GNSCIC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_ANCPDC_DOMESTIC_Year_1_Applications": x,
    "term_2_ANCPDC_DOMESTIC_Year_1_Accepted": x,
    "term_2_ANCPDC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_ANCPDC_DOMESTIC_Year_1_Denied": x,
    "term_2_ANCPDC_DOMESTIC_Year_1_Declined": x,
    "term_2_ANCPDC_DOMESTIC_Year_1_Pending": x,
    "term_2_ANCPDC_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_ANCPDC_DOMESTIC_Year_1_Enrolled": x,
    "term_2_ANCPDC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_ANCPDC_INTERNATIONAL_Year_1_Applications": x,
    "term_2_ANCPDC_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_ANCPDC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_ANCPDC_INTERNATIONAL_Year_1_Denied": x,
    "term_2_ANCPDC_INTERNATIONAL_Year_1_Declined": x,
    "term_2_ANCPDC_INTERNATIONAL_Year_1_Pending": x,
    "term_2_ANCPDC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_ANCPDC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_ANCPDC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_NSPDC_DOMESTIC_Year_1_Applications": x,
    "term_2_NSPDC_DOMESTIC_Year_1_Accepted": x,
    "term_2_NSPDC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_NSPDC_DOMESTIC_Year_1_Denied": x,
    "term_2_NSPDC_DOMESTIC_Year_1_Declined": x,
    "term_2_NSPDC_DOMESTIC_Year_1_Pending": x,
    "term_2_NSPDC_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_NSPDC_DOMESTIC_Year_1_Enrolled": x,
    "term_2_NSPDC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_NSPDC_INTERNATIONAL_Year_1_Applications": x,
    "term_2_NSPDC_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_NSPDC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_NSPDC_INTERNATIONAL_Year_1_Denied": x,
    "term_2_NSPDC_INTERNATIONAL_Year_1_Declined": x,
    "term_2_NSPDC_INTERNATIONAL_Year_1_Pending": x,
    "term_2_NSPDC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_NSPDC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_NSPDC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2__DOMESTIC_Year_1_Applications": x,
    "term_2__DOMESTIC_Year_1_Accepted": x,
    "term_2__DOMESTIC_Year_1_Payment_Pending": x,
    "term_2__DOMESTIC_Year_1_Denied": x,
    "term_2__DOMESTIC_Year_1_Declined": x,
    "term_2__DOMESTIC_Year_1_Pending": x,
    "term_2__DOMESTIC_Year_1_Waitlisted": x,
    "term_2__DOMESTIC_Year_1_Enrolled": x,
    "term_2__DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2__INTERNATIONAL_Year_1_Applications": x,
    "term_2__INTERNATIONAL_Year_1_Accepted": x,
    "term_2__INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2__INTERNATIONAL_Year_1_Denied": x,
    "term_2__INTERNATIONAL_Year_1_Declined": x,
    "term_2__INTERNATIONAL_Year_1_Pending": x,
    "term_2__INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2__INTERNATIONAL_Year_1_Enrolled": x,
    "term_2__INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_LINC13_DOMESTIC_Year_1_Applications": x,
    "term_2_LINC13_DOMESTIC_Year_1_Accepted": x,
    "term_2_LINC13_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_LINC13_DOMESTIC_Year_1_Denied": x,
    "term_2_LINC13_DOMESTIC_Year_1_Declined": x,
    "term_2_LINC13_DOMESTIC_Year_1_Pending": x,
    "term_2_LINC13_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_LINC13_DOMESTIC_Year_1_Enrolled": x,
    "term_2_LINC13_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_LINC13_INTERNATIONAL_Year_1_Applications": x,
    "term_2_LINC13_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_LINC13_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_LINC13_INTERNATIONAL_Year_1_Denied": x,
    "term_2_LINC13_INTERNATIONAL_Year_1_Declined": x,
    "term_2_LINC13_INTERNATIONAL_Year_1_Pending": x,
    "term_2_LINC13_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_LINC13_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_LINC13_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_AELCC_DOMESTIC_Year_1_Applications": x,
    "term_2_AELCC_DOMESTIC_Year_1_Accepted": x,
    "term_2_AELCC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_AELCC_DOMESTIC_Year_1_Denied": x,
    "term_2_AELCC_DOMESTIC_Year_1_Declined": x,
    "term_2_AELCC_DOMESTIC_Year_1_Pending": x,
    "term_2_AELCC_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_AELCC_DOMESTIC_Year_1_Enrolled": x,
    "term_2_AELCC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_AELCC_INTERNATIONAL_Year_1_Applications": x,
    "term_2_AELCC_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_AELCC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_AELCC_INTERNATIONAL_Year_1_Denied": x,
    "term_2_AELCC_INTERNATIONAL_Year_1_Declined": x,
    "term_2_AELCC_INTERNATIONAL_Year_1_Pending": x,
    "term_2_AELCC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_AELCC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_AELCC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_ECCERT_DOMESTIC_Year_1_Applications": x,
    "term_2_ECCERT_DOMESTIC_Year_1_Accepted": x,
    "term_2_ECCERT_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_ECCERT_DOMESTIC_Year_1_Denied": x,
    "term_2_ECCERT_DOMESTIC_Year_1_Declined": x,
    "term_2_ECCERT_DOMESTIC_Year_1_Pending": x,
    "term_2_ECCERT_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_ECCERT_DOMESTIC_Year_1_Enrolled": x,
    "term_2_ECCERT_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_ECCERT_INTERNATIONAL_Year_1_Applications": x,
    "term_2_ECCERT_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_ECCERT_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_ECCERT_INTERNATIONAL_Year_1_Denied": x,
    "term_2_ECCERT_INTERNATIONAL_Year_1_Declined": x,
    "term_2_ECCERT_INTERNATIONAL_Year_1_Pending": x,
    "term_2_ECCERT_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_ECCERT_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_ECCERT_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_ECDIP_DOMESTIC_Year_1_Applications": x,
    "term_2_ECDIP_DOMESTIC_Year_1_Accepted": x,
    "term_2_ECDIP_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_ECDIP_DOMESTIC_Year_1_Denied": x,
    "term_2_ECDIP_DOMESTIC_Year_1_Declined": x,
    "term_2_ECDIP_DOMESTIC_Year_1_Pending": x,
    "term_2_ECDIP_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_ECDIP_DOMESTIC_Year_1_Enrolled": x,
    "term_2_ECDIP_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_ECDIP_INTERNATIONAL_Year_1_Applications": x,
    "term_2_ECDIP_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_ECDIP_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_ECDIP_INTERNATIONAL_Year_1_Denied": x,
    "term_2_ECDIP_INTERNATIONAL_Year_1_Declined": x,
    "term_2_ECDIP_INTERNATIONAL_Year_1_Pending": x,
    "term_2_ECDIP_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_ECDIP_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_ECDIP_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_ECDIP_DOMESTIC_Year_2_Applications": x,
    "term_2_ECDIP_DOMESTIC_Year_2_Accepted": x,
    "term_2_ECDIP_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_ECDIP_DOMESTIC_Year_2_Denied": x,
    "term_2_ECDIP_DOMESTIC_Year_2_Declined": x,
    "term_2_ECDIP_DOMESTIC_Year_2_Pending": x,
    "term_2_ECDIP_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_ECDIP_DOMESTIC_Year_2_Enrolled": x,
    "term_2_ECDIP_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_ECDIP_INTERNATIONAL_Year_2_Applications": x,
    "term_2_ECDIP_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_ECDIP_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_ECDIP_INTERNATIONAL_Year_2_Denied": x,
    "term_2_ECDIP_INTERNATIONAL_Year_2_Declined": x,
    "term_2_ECDIP_INTERNATIONAL_Year_2_Pending": x,
    "term_2_ECDIP_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_ECDIP_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_ECDIP_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_CYCD_DOMESTIC_Year_1_Applications": x,
    "term_2_CYCD_DOMESTIC_Year_1_Accepted": x,
    "term_2_CYCD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_CYCD_DOMESTIC_Year_1_Denied": x,
    "term_2_CYCD_DOMESTIC_Year_1_Declined": x,
    "term_2_CYCD_DOMESTIC_Year_1_Pending": x,
    "term_2_CYCD_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_CYCD_DOMESTIC_Year_1_Enrolled": x,
    "term_2_CYCD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_CYCD_INTERNATIONAL_Year_1_Applications": x,
    "term_2_CYCD_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_CYCD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_CYCD_INTERNATIONAL_Year_1_Denied": x,
    "term_2_CYCD_INTERNATIONAL_Year_1_Declined": x,
    "term_2_CYCD_INTERNATIONAL_Year_1_Pending": x,
    "term_2_CYCD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_CYCD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_CYCD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_CYCD_DOMESTIC_Year_2_Applications": x,
    "term_2_CYCD_DOMESTIC_Year_2_Accepted": x,
    "term_2_CYCD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_CYCD_DOMESTIC_Year_2_Denied": x,
    "term_2_CYCD_DOMESTIC_Year_2_Declined": x,
    "term_2_CYCD_DOMESTIC_Year_2_Pending": x,
    "term_2_CYCD_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_CYCD_DOMESTIC_Year_2_Enrolled": x,
    "term_2_CYCD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_CYCD_INTERNATIONAL_Year_2_Applications": x,
    "term_2_CYCD_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_CYCD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_CYCD_INTERNATIONAL_Year_2_Denied": x,
    "term_2_CYCD_INTERNATIONAL_Year_2_Declined": x,
    "term_2_CYCD_INTERNATIONAL_Year_2_Pending": x,
    "term_2_CYCD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_CYCD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_CYCD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_ECEAC_DOMESTIC_Year_1_Applications": x,
    "term_2_ECEAC_DOMESTIC_Year_1_Accepted": x,
    "term_2_ECEAC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_ECEAC_DOMESTIC_Year_1_Denied": x,
    "term_2_ECEAC_DOMESTIC_Year_1_Declined": x,
    "term_2_ECEAC_DOMESTIC_Year_1_Pending": x,
    "term_2_ECEAC_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_ECEAC_DOMESTIC_Year_1_Enrolled": x,
    "term_2_ECEAC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_ECEAC_INTERNATIONAL_Year_1_Applications": x,
    "term_2_ECEAC_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_ECEAC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_ECEAC_INTERNATIONAL_Year_1_Denied": x,
    "term_2_ECEAC_INTERNATIONAL_Year_1_Declined": x,
    "term_2_ECEAC_INTERNATIONAL_Year_1_Pending": x,
    "term_2_ECEAC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_ECEAC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_ECEAC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_SOCWKD_DOMESTIC_Year_1_Applications": x,
    "term_2_SOCWKD_DOMESTIC_Year_1_Accepted": x,
    "term_2_SOCWKD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_SOCWKD_DOMESTIC_Year_1_Denied": x,
    "term_2_SOCWKD_DOMESTIC_Year_1_Declined": x,
    "term_2_SOCWKD_DOMESTIC_Year_1_Pending": x,
    "term_2_SOCWKD_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_SOCWKD_DOMESTIC_Year_1_Enrolled": x,
    "term_2_SOCWKD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_SOCWKD_INTERNATIONAL_Year_1_Applications": x,
    "term_2_SOCWKD_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_SOCWKD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_SOCWKD_INTERNATIONAL_Year_1_Denied": x,
    "term_2_SOCWKD_INTERNATIONAL_Year_1_Declined": x,
    "term_2_SOCWKD_INTERNATIONAL_Year_1_Pending": x,
    "term_2_SOCWKD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_SOCWKD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_SOCWKD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_SOCWKD_DOMESTIC_Year_2_Applications": x,
    "term_2_SOCWKD_DOMESTIC_Year_2_Accepted": x,
    "term_2_SOCWKD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_SOCWKD_DOMESTIC_Year_2_Denied": x,
    "term_2_SOCWKD_DOMESTIC_Year_2_Declined": x,
    "term_2_SOCWKD_DOMESTIC_Year_2_Pending": x,
    "term_2_SOCWKD_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_SOCWKD_DOMESTIC_Year_2_Enrolled": x,
    "term_2_SOCWKD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_SOCWKD_INTERNATIONAL_Year_2_Applications": x,
    "term_2_SOCWKD_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_SOCWKD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_SOCWKD_INTERNATIONAL_Year_2_Denied": x,
    "term_2_SOCWKD_INTERNATIONAL_Year_2_Declined": x,
    "term_2_SOCWKD_INTERNATIONAL_Year_2_Pending": x,
    "term_2_SOCWKD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_SOCWKD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_SOCWKD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_HCAC_DOMESTIC_Year_1_Applications": x,
    "term_2_HCAC_DOMESTIC_Year_1_Accepted": x,
    "term_2_HCAC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_HCAC_DOMESTIC_Year_1_Denied": x,
    "term_2_HCAC_DOMESTIC_Year_1_Declined": x,
    "term_2_HCAC_DOMESTIC_Year_1_Pending": x,
    "term_2_HCAC_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_HCAC_DOMESTIC_Year_1_Enrolled": x,
    "term_2_HCAC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_HCAC_INTERNATIONAL_Year_1_Applications": x,
    "term_2_HCAC_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_HCAC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_HCAC_INTERNATIONAL_Year_1_Denied": x,
    "term_2_HCAC_INTERNATIONAL_Year_1_Declined": x,
    "term_2_HCAC_INTERNATIONAL_Year_1_Pending": x,
    "term_2_HCAC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_HCAC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_HCAC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_PCPC_DOMESTIC_Year_1_Applications": x,
    "term_2_PCPC_DOMESTIC_Year_1_Accepted": x,
    "term_2_PCPC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_PCPC_DOMESTIC_Year_1_Denied": x,
    "term_2_PCPC_DOMESTIC_Year_1_Declined": x,
    "term_2_PCPC_DOMESTIC_Year_1_Pending": x,
    "term_2_PCPC_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_PCPC_DOMESTIC_Year_1_Enrolled": x,
    "term_2_PCPC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_PCPC_INTERNATIONAL_Year_1_Applications": x,
    "term_2_PCPC_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_PCPC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_PCPC_INTERNATIONAL_Year_1_Denied": x,
    "term_2_PCPC_INTERNATIONAL_Year_1_Declined": x,
    "term_2_PCPC_INTERNATIONAL_Year_1_Pending": x,
    "term_2_PCPC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_PCPC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_PCPC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_ACP_DOMESTIC_Year_1_Applications": x,
    "term_2_ACP_DOMESTIC_Year_1_Accepted": x,
    "term_2_ACP_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_ACP_DOMESTIC_Year_1_Denied": x,
    "term_2_ACP_DOMESTIC_Year_1_Declined": x,
    "term_2_ACP_DOMESTIC_Year_1_Pending": x,
    "term_2_ACP_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_ACP_DOMESTIC_Year_1_Enrolled": x,
    "term_2_ACP_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_ACP_INTERNATIONAL_Year_1_Applications": x,
    "term_2_ACP_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_ACP_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_ACP_INTERNATIONAL_Year_1_Denied": x,
    "term_2_ACP_INTERNATIONAL_Year_1_Declined": x,
    "term_2_ACP_INTERNATIONAL_Year_1_Pending": x,
    "term_2_ACP_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_ACP_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_ACP_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_ACP_DOMESTIC_Year_2_Applications": x,
    "term_2_ACP_DOMESTIC_Year_2_Accepted": x,
    "term_2_ACP_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_ACP_DOMESTIC_Year_2_Denied": x,
    "term_2_ACP_DOMESTIC_Year_2_Declined": x,
    "term_2_ACP_DOMESTIC_Year_2_Pending": x,
    "term_2_ACP_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_ACP_DOMESTIC_Year_2_Enrolled": x,
    "term_2_ACP_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_ACP_INTERNATIONAL_Year_2_Applications": x,
    "term_2_ACP_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_ACP_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_ACP_INTERNATIONAL_Year_2_Denied": x,
    "term_2_ACP_INTERNATIONAL_Year_2_Declined": x,
    "term_2_ACP_INTERNATIONAL_Year_2_Pending": x,
    "term_2_ACP_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_ACP_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_ACP_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_PNR_DOMESTIC_Year_1_Applications": x,
    "term_2_PNR_DOMESTIC_Year_1_Accepted": x,
    "term_2_PNR_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_PNR_DOMESTIC_Year_1_Denied": x,
    "term_2_PNR_DOMESTIC_Year_1_Declined": x,
    "term_2_PNR_DOMESTIC_Year_1_Pending": x,
    "term_2_PNR_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_PNR_DOMESTIC_Year_1_Enrolled": x,
    "term_2_PNR_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_PNR_INTERNATIONAL_Year_1_Applications": x,
    "term_2_PNR_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_PNR_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_PNR_INTERNATIONAL_Year_1_Denied": x,
    "term_2_PNR_INTERNATIONAL_Year_1_Declined": x,
    "term_2_PNR_INTERNATIONAL_Year_1_Pending": x,
    "term_2_PNR_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_PNR_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_PNR_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_PNR_DOMESTIC_Year_2_Applications": x,
    "term_2_PNR_DOMESTIC_Year_2_Accepted": x,
    "term_2_PNR_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_PNR_DOMESTIC_Year_2_Denied": x,
    "term_2_PNR_DOMESTIC_Year_2_Declined": x,
    "term_2_PNR_DOMESTIC_Year_2_Pending": x,
    "term_2_PNR_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_PNR_DOMESTIC_Year_2_Enrolled": x,
    "term_2_PNR_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_PNR_INTERNATIONAL_Year_2_Applications": x,
    "term_2_PNR_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_PNR_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_PNR_INTERNATIONAL_Year_2_Denied": x,
    "term_2_PNR_INTERNATIONAL_Year_2_Declined": x,
    "term_2_PNR_INTERNATIONAL_Year_2_Pending": x,
    "term_2_PNR_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_PNR_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_PNR_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_BSCN_DOMESTIC_Year_1_Applications": x,
    "term_2_BSCN_DOMESTIC_Year_1_Accepted": x,
    "term_2_BSCN_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_BSCN_DOMESTIC_Year_1_Denied": x,
    "term_2_BSCN_DOMESTIC_Year_1_Declined": x,
    "term_2_BSCN_DOMESTIC_Year_1_Pending": x,
    "term_2_BSCN_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_BSCN_DOMESTIC_Year_1_Enrolled": x,
    "term_2_BSCN_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_BSCN_INTERNATIONAL_Year_1_Applications": x,
    "term_2_BSCN_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_BSCN_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_BSCN_INTERNATIONAL_Year_1_Denied": x,
    "term_2_BSCN_INTERNATIONAL_Year_1_Declined": x,
    "term_2_BSCN_INTERNATIONAL_Year_1_Pending": x,
    "term_2_BSCN_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_BSCN_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_BSCN_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_BSCN_DOMESTIC_Year_2_Applications": x,
    "term_2_BSCN_DOMESTIC_Year_2_Accepted": x,
    "term_2_BSCN_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_BSCN_DOMESTIC_Year_2_Denied": x,
    "term_2_BSCN_DOMESTIC_Year_2_Declined": x,
    "term_2_BSCN_DOMESTIC_Year_2_Pending": x,
    "term_2_BSCN_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_BSCN_DOMESTIC_Year_2_Enrolled": x,
    "term_2_BSCN_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_BSCN_INTERNATIONAL_Year_2_Applications": x,
    "term_2_BSCN_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_BSCN_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_BSCN_INTERNATIONAL_Year_2_Denied": x,
    "term_2_BSCN_INTERNATIONAL_Year_2_Declined": x,
    "term_2_BSCN_INTERNATIONAL_Year_2_Pending": x,
    "term_2_BSCN_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_BSCN_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_BSCN_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_2_BSCN_DOMESTIC_Year_3_Applications": x,
    "term_2_BSCN_DOMESTIC_Year_3_Accepted": x,
    "term_2_BSCN_DOMESTIC_Year_3_Payment_Pending": x,
    "term_2_BSCN_DOMESTIC_Year_3_Denied": x,
    "term_2_BSCN_DOMESTIC_Year_3_Declined": x,
    "term_2_BSCN_DOMESTIC_Year_3_Pending": x,
    "term_2_BSCN_DOMESTIC_Year_3_Waitlisted": x,
    "term_2_BSCN_DOMESTIC_Year_3_Enrolled": x,
    "term_2_BSCN_DOMESTIC_Year_3_Conversion_Rate": x,
    "term_2_BSCN_INTERNATIONAL_Year_3_Applications": x,
    "term_2_BSCN_INTERNATIONAL_Year_3_Accepted": x,
    "term_2_BSCN_INTERNATIONAL_Year_3_Payment_Pending": x,
    "term_2_BSCN_INTERNATIONAL_Year_3_Denied": x,
    "term_2_BSCN_INTERNATIONAL_Year_3_Declined": x,
    "term_2_BSCN_INTERNATIONAL_Year_3_Pending": x,
    "term_2_BSCN_INTERNATIONAL_Year_3_Waitlisted": x,
    "term_2_BSCN_INTERNATIONAL_Year_3_Enrolled": x,
    "term_2_BSCN_INTERNATIONAL_Year_3_Conversion_Rate": x,
    "term_2_IENCN_DOMESTIC_Year_1_Applications": x,
    "term_2_IENCN_DOMESTIC_Year_1_Accepted": x,
    "term_2_IENCN_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_IENCN_DOMESTIC_Year_1_Denied": x,
    "term_2_IENCN_DOMESTIC_Year_1_Declined": x,
    "term_2_IENCN_DOMESTIC_Year_1_Pending": x,
    "term_2_IENCN_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_IENCN_DOMESTIC_Year_1_Enrolled": x,
    "term_2_IENCN_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_IENCN_INTERNATIONAL_Year_1_Applications": x,
    "term_2_IENCN_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_IENCN_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_IENCN_INTERNATIONAL_Year_1_Denied": x,
    "term_2_IENCN_INTERNATIONAL_Year_1_Declined": x,
    "term_2_IENCN_INTERNATIONAL_Year_1_Pending": x,
    "term_2_IENCN_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_IENCN_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_IENCN_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_PAELEC_DOMESTIC_Year_1_Applications": x,
    "term_2_PAELEC_DOMESTIC_Year_1_Accepted": x,
    "term_2_PAELEC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_PAELEC_DOMESTIC_Year_1_Denied": x,
    "term_2_PAELEC_DOMESTIC_Year_1_Declined": x,
    "term_2_PAELEC_DOMESTIC_Year_1_Pending": x,
    "term_2_PAELEC_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_PAELEC_DOMESTIC_Year_1_Enrolled": x,
    "term_2_PAELEC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_PAELEC_INTERNATIONAL_Year_1_Applications": x,
    "term_2_PAELEC_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_PAELEC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_PAELEC_INTERNATIONAL_Year_1_Denied": x,
    "term_2_PAELEC_INTERNATIONAL_Year_1_Declined": x,
    "term_2_PAELEC_INTERNATIONAL_Year_1_Pending": x,
    "term_2_PAELEC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_PAELEC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_PAELEC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_PAWELD_DOMESTIC_Year_1_Applications": x,
    "term_2_PAWELD_DOMESTIC_Year_1_Accepted": x,
    "term_2_PAWELD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_PAWELD_DOMESTIC_Year_1_Denied": x,
    "term_2_PAWELD_DOMESTIC_Year_1_Declined": x,
    "term_2_PAWELD_DOMESTIC_Year_1_Pending": x,
    "term_2_PAWELD_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_PAWELD_DOMESTIC_Year_1_Enrolled": x,
    "term_2_PAWELD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_PAWELD_INTERNATIONAL_Year_1_Applications": x,
    "term_2_PAWELD_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_PAWELD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_PAWELD_INTERNATIONAL_Year_1_Denied": x,
    "term_2_PAWELD_INTERNATIONAL_Year_1_Declined": x,
    "term_2_PAWELD_INTERNATIONAL_Year_1_Pending": x,
    "term_2_PAWELD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_PAWELD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_PAWELD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_PAHET_DOMESTIC_Year_1_Applications": x,
    "term_2_PAHET_DOMESTIC_Year_1_Accepted": x,
    "term_2_PAHET_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_PAHET_DOMESTIC_Year_1_Denied": x,
    "term_2_PAHET_DOMESTIC_Year_1_Declined": x,
    "term_2_PAHET_DOMESTIC_Year_1_Pending": x,
    "term_2_PAHET_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_PAHET_DOMESTIC_Year_1_Enrolled": x,
    "term_2_PAHET_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_PAHET_INTERNATIONAL_Year_1_Applications": x,
    "term_2_PAHET_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_PAHET_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_PAHET_INTERNATIONAL_Year_1_Denied": x,
    "term_2_PAHET_INTERNATIONAL_Year_1_Declined": x,
    "term_2_PAHET_INTERNATIONAL_Year_1_Pending": x,
    "term_2_PAHET_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_PAHET_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_PAHET_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_PAPLUM_DOMESTIC_Year_1_Applications": x,
    "term_2_PAPLUM_DOMESTIC_Year_1_Accepted": x,
    "term_2_PAPLUM_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_PAPLUM_DOMESTIC_Year_1_Denied": x,
    "term_2_PAPLUM_DOMESTIC_Year_1_Declined": x,
    "term_2_PAPLUM_DOMESTIC_Year_1_Pending": x,
    "term_2_PAPLUM_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_PAPLUM_DOMESTIC_Year_1_Enrolled": x,
    "term_2_PAPLUM_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_PAPLUM_INTERNATIONAL_Year_1_Applications": x,
    "term_2_PAPLUM_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_PAPLUM_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_PAPLUM_INTERNATIONAL_Year_1_Denied": x,
    "term_2_PAPLUM_INTERNATIONAL_Year_1_Declined": x,
    "term_2_PAPLUM_INTERNATIONAL_Year_1_Pending": x,
    "term_2_PAPLUM_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_PAPLUM_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_PAPLUM_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_POWCM4_DOMESTIC_Year_1_Applications": x,
    "term_2_POWCM4_DOMESTIC_Year_1_Accepted": x,
    "term_2_POWCM4_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_POWCM4_DOMESTIC_Year_1_Denied": x,
    "term_2_POWCM4_DOMESTIC_Year_1_Declined": x,
    "term_2_POWCM4_DOMESTIC_Year_1_Pending": x,
    "term_2_POWCM4_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_POWCM4_DOMESTIC_Year_1_Enrolled": x,
    "term_2_POWCM4_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_POWCM4_INTERNATIONAL_Year_1_Applications": x,
    "term_2_POWCM4_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_POWCM4_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_POWCM4_INTERNATIONAL_Year_1_Denied": x,
    "term_2_POWCM4_INTERNATIONAL_Year_1_Declined": x,
    "term_2_POWCM4_INTERNATIONAL_Year_1_Pending": x,
    "term_2_POWCM4_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_POWCM4_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_POWCM4_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_POWCM3_DOMESTIC_Year_1_Applications": x,
    "term_2_POWCM3_DOMESTIC_Year_1_Accepted": x,
    "term_2_POWCM3_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_POWCM3_DOMESTIC_Year_1_Denied": x,
    "term_2_POWCM3_DOMESTIC_Year_1_Declined": x,
    "term_2_POWCM3_DOMESTIC_Year_1_Pending": x,
    "term_2_POWCM3_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_POWCM3_DOMESTIC_Year_1_Enrolled": x,
    "term_2_POWCM3_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_POWCM3_INTERNATIONAL_Year_1_Applications": x,
    "term_2_POWCM3_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_POWCM3_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_POWCM3_INTERNATIONAL_Year_1_Denied": x,
    "term_2_POWCM3_INTERNATIONAL_Year_1_Declined": x,
    "term_2_POWCM3_INTERNATIONAL_Year_1_Pending": x,
    "term_2_POWCM3_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_POWCM3_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_POWCM3_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_POWCP4_DOMESTIC_Year_1_Applications": x,
    "term_2_POWCP4_DOMESTIC_Year_1_Accepted": x,
    "term_2_POWCP4_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_POWCP4_DOMESTIC_Year_1_Denied": x,
    "term_2_POWCP4_DOMESTIC_Year_1_Declined": x,
    "term_2_POWCP4_DOMESTIC_Year_1_Pending": x,
    "term_2_POWCP4_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_POWCP4_DOMESTIC_Year_1_Enrolled": x,
    "term_2_POWCP4_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_POWCP4_INTERNATIONAL_Year_1_Applications": x,
    "term_2_POWCP4_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_POWCP4_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_POWCP4_INTERNATIONAL_Year_1_Denied": x,
    "term_2_POWCP4_INTERNATIONAL_Year_1_Declined": x,
    "term_2_POWCP4_INTERNATIONAL_Year_1_Pending": x,
    "term_2_POWCP4_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_POWCP4_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_POWCP4_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_POWCO4_DOMESTIC_Year_1_Applications": x,
    "term_2_POWCO4_DOMESTIC_Year_1_Accepted": x,
    "term_2_POWCO4_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_POWCO4_DOMESTIC_Year_1_Denied": x,
    "term_2_POWCO4_DOMESTIC_Year_1_Declined": x,
    "term_2_POWCO4_DOMESTIC_Year_1_Pending": x,
    "term_2_POWCO4_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_POWCO4_DOMESTIC_Year_1_Enrolled": x,
    "term_2_POWCO4_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_POWCO4_INTERNATIONAL_Year_1_Applications": x,
    "term_2_POWCO4_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_POWCO4_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_POWCO4_INTERNATIONAL_Year_1_Denied": x,
    "term_2_POWCO4_INTERNATIONAL_Year_1_Declined": x,
    "term_2_POWCO4_INTERNATIONAL_Year_1_Pending": x,
    "term_2_POWCO4_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_POWCO4_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_POWCO4_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_POWCO3_DOMESTIC_Year_1_Applications": x,
    "term_2_POWCO3_DOMESTIC_Year_1_Accepted": x,
    "term_2_POWCO3_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_POWCO3_DOMESTIC_Year_1_Denied": x,
    "term_2_POWCO3_DOMESTIC_Year_1_Declined": x,
    "term_2_POWCO3_DOMESTIC_Year_1_Pending": x,
    "term_2_POWCO3_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_POWCO3_DOMESTIC_Year_1_Enrolled": x,
    "term_2_POWCO3_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_POWCO3_INTERNATIONAL_Year_1_Applications": x,
    "term_2_POWCO3_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_POWCO3_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_POWCO3_INTERNATIONAL_Year_1_Denied": x,
    "term_2_POWCO3_INTERNATIONAL_Year_1_Declined": x,
    "term_2_POWCO3_INTERNATIONAL_Year_1_Pending": x,
    "term_2_POWCO3_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_POWCO3_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_POWCO3_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_PPET_DOMESTIC_Year_1_Applications": x,
    "term_2_PPET_DOMESTIC_Year_1_Accepted": x,
    "term_2_PPET_DOMESTIC_Year_1_Payment_Pending": x,
    "term_2_PPET_DOMESTIC_Year_1_Denied": x,
    "term_2_PPET_DOMESTIC_Year_1_Declined": x,
    "term_2_PPET_DOMESTIC_Year_1_Pending": x,
    "term_2_PPET_DOMESTIC_Year_1_Waitlisted": x,
    "term_2_PPET_DOMESTIC_Year_1_Enrolled": x,
    "term_2_PPET_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_2_PPET_INTERNATIONAL_Year_1_Applications": x,
    "term_2_PPET_INTERNATIONAL_Year_1_Accepted": x,
    "term_2_PPET_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_2_PPET_INTERNATIONAL_Year_1_Denied": x,
    "term_2_PPET_INTERNATIONAL_Year_1_Declined": x,
    "term_2_PPET_INTERNATIONAL_Year_1_Pending": x,
    "term_2_PPET_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_2_PPET_INTERNATIONAL_Year_1_Enrolled": x,
    "term_2_PPET_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_2_PPET_DOMESTIC_Year_2_Applications": x,
    "term_2_PPET_DOMESTIC_Year_2_Accepted": x,
    "term_2_PPET_DOMESTIC_Year_2_Payment_Pending": x,
    "term_2_PPET_DOMESTIC_Year_2_Denied": x,
    "term_2_PPET_DOMESTIC_Year_2_Declined": x,
    "term_2_PPET_DOMESTIC_Year_2_Pending": x,
    "term_2_PPET_DOMESTIC_Year_2_Waitlisted": x,
    "term_2_PPET_DOMESTIC_Year_2_Enrolled": x,
    "term_2_PPET_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_2_PPET_INTERNATIONAL_Year_2_Applications": x,
    "term_2_PPET_INTERNATIONAL_Year_2_Accepted": x,
    "term_2_PPET_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_2_PPET_INTERNATIONAL_Year_2_Denied": x,
    "term_2_PPET_INTERNATIONAL_Year_2_Declined": x,
    "term_2_PPET_INTERNATIONAL_Year_2_Pending": x,
    "term_2_PPET_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_2_PPET_INTERNATIONAL_Year_2_Enrolled": x,
    "term_2_PPET_INTERNATIONAL_Year_2_Conversion_Rate": x,

    #Term3Yr YYYY Apps By School sheet
    "term_3_BABUSC_DOMESTIC_Year_1_Applications": x,
    "term_3_BABUSC_DOMESTIC_Year_1_Accepted": x,
    "term_3_BABUSC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_BABUSC_DOMESTIC_Year_1_Denied": x,
    "term_3_BABUSC_DOMESTIC_Year_1_Declined": x,
    "term_3_BABUSC_DOMESTIC_Year_1_Pending": x,
    "term_3_BABUSC_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_BABUSC_DOMESTIC_Year_1_Enrolled": x,
    "term_3_BABUSC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_BABUSC_INTERNATIONAL_Year_1_Applications": x,
    "term_3_BABUSC_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_BABUSC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_BABUSC_INTERNATIONAL_Year_1_Denied": x,
    "term_3_BABUSC_INTERNATIONAL_Year_1_Declined": x,
    "term_3_BABUSC_INTERNATIONAL_Year_1_Pending": x,
    "term_3_BABUSC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_BABUSC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_BABUSC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_BAACTD_DOMESTIC_Year_1_Applications": x,
    "term_3_BAACTD_DOMESTIC_Year_1_Accepted": x,
    "term_3_BAACTD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_BAACTD_DOMESTIC_Year_1_Denied": x,
    "term_3_BAACTD_DOMESTIC_Year_1_Declined": x,
    "term_3_BAACTD_DOMESTIC_Year_1_Pending": x,
    "term_3_BAACTD_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_BAACTD_DOMESTIC_Year_1_Enrolled": x,
    "term_3_BAACTD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_BAACTD_INTERNATIONAL_Year_1_Applications": x,
    "term_3_BAACTD_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_BAACTD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_BAACTD_INTERNATIONAL_Year_1_Denied": x,
    "term_3_BAACTD_INTERNATIONAL_Year_1_Declined": x,
    "term_3_BAACTD_INTERNATIONAL_Year_1_Pending": x,
    "term_3_BAACTD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_BAACTD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_BAACTD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_BAACTD_DOMESTIC_Year_2_Applications": x,
    "term_3_BAACTD_DOMESTIC_Year_2_Accepted": x,
    "term_3_BAACTD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_BAACTD_DOMESTIC_Year_2_Denied": x,
    "term_3_BAACTD_DOMESTIC_Year_2_Declined": x,
    "term_3_BAACTD_DOMESTIC_Year_2_Pending": x,
    "term_3_BAACTD_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_BAACTD_DOMESTIC_Year_2_Enrolled": x,
    "term_3_BAACTD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_BAACTD_INTERNATIONAL_Year_2_Applications": x,
    "term_3_BAACTD_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_BAACTD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_BAACTD_INTERNATIONAL_Year_2_Denied": x,
    "term_3_BAACTD_INTERNATIONAL_Year_2_Declined": x,
    "term_3_BAACTD_INTERNATIONAL_Year_2_Pending": x,
    "term_3_BAACTD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_BAACTD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_BAACTD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_BABUSD_DOMESTIC_Year_1_Applications": x,
    "term_3_BABUSD_DOMESTIC_Year_1_Accepted": x,
    "term_3_BABUSD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_BABUSD_DOMESTIC_Year_1_Denied": x,
    "term_3_BABUSD_DOMESTIC_Year_1_Declined": x,
    "term_3_BABUSD_DOMESTIC_Year_1_Pending": x,
    "term_3_BABUSD_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_BABUSD_DOMESTIC_Year_1_Enrolled": x,
    "term_3_BABUSD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_BABUSD_INTERNATIONAL_Year_1_Applications": x,
    "term_3_BABUSD_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_BABUSD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_BABUSD_INTERNATIONAL_Year_1_Denied": x,
    "term_3_BABUSD_INTERNATIONAL_Year_1_Declined": x,
    "term_3_BABUSD_INTERNATIONAL_Year_1_Pending": x,
    "term_3_BABUSD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_BABUSD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_BABUSD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_BABUSD_DOMESTIC_Year_2_Applications": x,
    "term_3_BABUSD_DOMESTIC_Year_2_Accepted": x,
    "term_3_BABUSD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_BABUSD_DOMESTIC_Year_2_Denied": x,
    "term_3_BABUSD_DOMESTIC_Year_2_Declined": x,
    "term_3_BABUSD_DOMESTIC_Year_2_Pending": x,
    "term_3_BABUSD_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_BABUSD_DOMESTIC_Year_2_Enrolled": x,
    "term_3_BABUSD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_BABUSD_INTERNATIONAL_Year_2_Applications": x,
    "term_3_BABUSD_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_BABUSD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_BABUSD_INTERNATIONAL_Year_2_Denied": x,
    "term_3_BABUSD_INTERNATIONAL_Year_2_Declined": x,
    "term_3_BABUSD_INTERNATIONAL_Year_2_Pending": x,
    "term_3_BABUSD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_BABUSD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_BABUSD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_BAESPM_DOMESTIC_Year_1_Applications": x,
    "term_3_BAESPM_DOMESTIC_Year_1_Accepted": x,
    "term_3_BAESPM_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_BAESPM_DOMESTIC_Year_1_Denied": x,
    "term_3_BAESPM_DOMESTIC_Year_1_Declined": x,
    "term_3_BAESPM_DOMESTIC_Year_1_Pending": x,
    "term_3_BAESPM_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_BAESPM_DOMESTIC_Year_1_Enrolled": x,
    "term_3_BAESPM_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_BAESPM_INTERNATIONAL_Year_1_Applications": x,
    "term_3_BAESPM_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_BAESPM_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_BAESPM_INTERNATIONAL_Year_1_Denied": x,
    "term_3_BAESPM_INTERNATIONAL_Year_1_Declined": x,
    "term_3_BAESPM_INTERNATIONAL_Year_1_Pending": x,
    "term_3_BAESPM_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_BAESPM_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_BAESPM_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_BAESPM_DOMESTIC_Year_2_Applications": x,
    "term_3_BAESPM_DOMESTIC_Year_2_Accepted": x,
    "term_3_BAESPM_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_BAESPM_DOMESTIC_Year_2_Denied": x,
    "term_3_BAESPM_DOMESTIC_Year_2_Declined": x,
    "term_3_BAESPM_DOMESTIC_Year_2_Pending": x,
    "term_3_BAESPM_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_BAESPM_DOMESTIC_Year_2_Enrolled": x,
    "term_3_BAESPM_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_BAESPM_INTERNATIONAL_Year_2_Applications": x,
    "term_3_BAESPM_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_BAESPM_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_BAESPM_INTERNATIONAL_Year_2_Denied": x,
    "term_3_BAESPM_INTERNATIONAL_Year_2_Declined": x,
    "term_3_BAESPM_INTERNATIONAL_Year_2_Pending": x,
    "term_3_BAESPM_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_BAESPM_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_BAESPM_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_BAHRMD_DOMESTIC_Year_1_Applications": x,
    "term_3_BAHRMD_DOMESTIC_Year_1_Accepted": x,
    "term_3_BAHRMD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_BAHRMD_DOMESTIC_Year_1_Denied": x,
    "term_3_BAHRMD_DOMESTIC_Year_1_Declined": x,
    "term_3_BAHRMD_DOMESTIC_Year_1_Pending": x,
    "term_3_BAHRMD_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_BAHRMD_DOMESTIC_Year_1_Enrolled": x,
    "term_3_BAHRMD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_BAHRMD_INTERNATIONAL_Year_1_Applications": x,
    "term_3_BAHRMD_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_BAHRMD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_BAHRMD_INTERNATIONAL_Year_1_Denied": x,
    "term_3_BAHRMD_INTERNATIONAL_Year_1_Declined": x,
    "term_3_BAHRMD_INTERNATIONAL_Year_1_Pending": x,
    "term_3_BAHRMD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_BAHRMD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_BAHRMD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_BAHRMD_DOMESTIC_Year_2_Applications": x,
    "term_3_BAHRMD_DOMESTIC_Year_2_Accepted": x,
    "term_3_BAHRMD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_BAHRMD_DOMESTIC_Year_2_Denied": x,
    "term_3_BAHRMD_DOMESTIC_Year_2_Declined": x,
    "term_3_BAHRMD_DOMESTIC_Year_2_Pending": x,
    "term_3_BAHRMD_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_BAHRMD_DOMESTIC_Year_2_Enrolled": x,
    "term_3_BAHRMD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_BAHRMD_INTERNATIONAL_Year_2_Applications": x,
    "term_3_BAHRMD_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_BAHRMD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_BAHRMD_INTERNATIONAL_Year_2_Denied": x,
    "term_3_BAHRMD_INTERNATIONAL_Year_2_Declined": x,
    "term_3_BAHRMD_INTERNATIONAL_Year_2_Pending": x,
    "term_3_BAHRMD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_BAHRMD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_BAHRMD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_GSCMD_DOMESTIC_Year_1_Applications": x,
    "term_3_GSCMD_DOMESTIC_Year_1_Accepted": x,
    "term_3_GSCMD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_GSCMD_DOMESTIC_Year_1_Denied": x,
    "term_3_GSCMD_DOMESTIC_Year_1_Declined": x,
    "term_3_GSCMD_DOMESTIC_Year_1_Pending": x,
    "term_3_GSCMD_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_GSCMD_DOMESTIC_Year_1_Enrolled": x,
    "term_3_GSCMD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_GSCMD_INTERNATIONAL_Year_1_Applications": x,
    "term_3_GSCMD_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_GSCMD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_GSCMD_INTERNATIONAL_Year_1_Denied": x,
    "term_3_GSCMD_INTERNATIONAL_Year_1_Declined": x,
    "term_3_GSCMD_INTERNATIONAL_Year_1_Pending": x,
    "term_3_GSCMD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_GSCMD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_GSCMD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_GSCMD_DOMESTIC_Year_2_Applications": x,
    "term_3_GSCMD_DOMESTIC_Year_2_Accepted": x,
    "term_3_GSCMD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_GSCMD_DOMESTIC_Year_2_Denied": x,
    "term_3_GSCMD_DOMESTIC_Year_2_Declined": x,
    "term_3_GSCMD_DOMESTIC_Year_2_Pending": x,
    "term_3_GSCMD_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_GSCMD_DOMESTIC_Year_2_Enrolled": x,
    "term_3_GSCMD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_GSCMD_INTERNATIONAL_Year_2_Applications": x,
    "term_3_GSCMD_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_GSCMD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_GSCMD_INTERNATIONAL_Year_2_Denied": x,
    "term_3_GSCMD_INTERNATIONAL_Year_2_Declined": x,
    "term_3_GSCMD_INTERNATIONAL_Year_2_Pending": x,
    "term_3_GSCMD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_GSCMD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_GSCMD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_DIGMD_DOMESTIC_Year_1_Applications": x,
    "term_3_DIGMD_DOMESTIC_Year_1_Accepted": x,
    "term_3_DIGMD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_DIGMD_DOMESTIC_Year_1_Denied": x,
    "term_3_DIGMD_DOMESTIC_Year_1_Declined": x,
    "term_3_DIGMD_DOMESTIC_Year_1_Pending": x,
    "term_3_DIGMD_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_DIGMD_DOMESTIC_Year_1_Enrolled": x,
    "term_3_DIGMD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_DIGMD_INTERNATIONAL_Year_1_Applications": x,
    "term_3_DIGMD_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_DIGMD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_DIGMD_INTERNATIONAL_Year_1_Denied": x,
    "term_3_DIGMD_INTERNATIONAL_Year_1_Declined": x,
    "term_3_DIGMD_INTERNATIONAL_Year_1_Pending": x,
    "term_3_DIGMD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_DIGMD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_DIGMD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_DIGMD_DOMESTIC_Year_2_Applications": x,
    "term_3_DIGMD_DOMESTIC_Year_2_Accepted": x,
    "term_3_DIGMD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_DIGMD_DOMESTIC_Year_2_Denied": x,
    "term_3_DIGMD_DOMESTIC_Year_2_Declined": x,
    "term_3_DIGMD_DOMESTIC_Year_2_Pending": x,
    "term_3_DIGMD_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_DIGMD_DOMESTIC_Year_2_Enrolled": x,
    "term_3_DIGMD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_DIGMD_INTERNATIONAL_Year_2_Applications": x,
    "term_3_DIGMD_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_DIGMD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_DIGMD_INTERNATIONAL_Year_2_Denied": x,
    "term_3_DIGMD_INTERNATIONAL_Year_2_Declined": x,
    "term_3_DIGMD_INTERNATIONAL_Year_2_Pending": x,
    "term_3_DIGMD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_DIGMD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_DIGMD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_BAATCO_DOMESTIC_Year_1_Applications": x,
    "term_3_BAATCO_DOMESTIC_Year_1_Accepted": x,
    "term_3_BAATCO_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_BAATCO_DOMESTIC_Year_1_Denied": x,
    "term_3_BAATCO_DOMESTIC_Year_1_Declined": x,
    "term_3_BAATCO_DOMESTIC_Year_1_Pending": x,
    "term_3_BAATCO_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_BAATCO_DOMESTIC_Year_1_Enrolled": x,
    "term_3_BAATCO_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_BAATCO_INTERNATIONAL_Year_1_Applications": x,
    "term_3_BAATCO_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_BAATCO_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_BAATCO_INTERNATIONAL_Year_1_Denied": x,
    "term_3_BAATCO_INTERNATIONAL_Year_1_Declined": x,
    "term_3_BAATCO_INTERNATIONAL_Year_1_Pending": x,
    "term_3_BAATCO_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_BAATCO_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_BAATCO_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_BAATCO_DOMESTIC_Year_2_Applications": x,
    "term_3_BAATCO_DOMESTIC_Year_2_Accepted": x,
    "term_3_BAATCO_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_BAATCO_DOMESTIC_Year_2_Denied": x,
    "term_3_BAATCO_DOMESTIC_Year_2_Declined": x,
    "term_3_BAATCO_DOMESTIC_Year_2_Pending": x,
    "term_3_BAATCO_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_BAATCO_DOMESTIC_Year_2_Enrolled": x,
    "term_3_BAATCO_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_BAATCO_INTERNATIONAL_Year_2_Applications": x,
    "term_3_BAATCO_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_BAATCO_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_BAATCO_INTERNATIONAL_Year_2_Denied": x,
    "term_3_BAATCO_INTERNATIONAL_Year_2_Declined": x,
    "term_3_BAATCO_INTERNATIONAL_Year_2_Pending": x,
    "term_3_BAATCO_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_BAATCO_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_BAATCO_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_BABSCO_DOMESTIC_Year_1_Applications": x,
    "term_3_BABSCO_DOMESTIC_Year_1_Accepted": x,
    "term_3_BABSCO_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_BABSCO_DOMESTIC_Year_1_Denied": x,
    "term_3_BABSCO_DOMESTIC_Year_1_Declined": x,
    "term_3_BABSCO_DOMESTIC_Year_1_Pending": x,
    "term_3_BABSCO_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_BABSCO_DOMESTIC_Year_1_Enrolled": x,
    "term_3_BABSCO_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_BABSCO_INTERNATIONAL_Year_1_Applications": x,
    "term_3_BABSCO_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_BABSCO_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_BABSCO_INTERNATIONAL_Year_1_Denied": x,
    "term_3_BABSCO_INTERNATIONAL_Year_1_Declined": x,
    "term_3_BABSCO_INTERNATIONAL_Year_1_Pending": x,
    "term_3_BABSCO_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_BABSCO_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_BABSCO_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_BABSCO_DOMESTIC_Year_2_Applications": x,
    "term_3_BABSCO_DOMESTIC_Year_2_Accepted": x,
    "term_3_BABSCO_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_BABSCO_DOMESTIC_Year_2_Denied": x,
    "term_3_BABSCO_DOMESTIC_Year_2_Declined": x,
    "term_3_BABSCO_DOMESTIC_Year_2_Pending": x,
    "term_3_BABSCO_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_BABSCO_DOMESTIC_Year_2_Enrolled": x,
    "term_3_BABSCO_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_BABSCO_INTERNATIONAL_Year_2_Applications": x,
    "term_3_BABSCO_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_BABSCO_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_BABSCO_INTERNATIONAL_Year_2_Denied": x,
    "term_3_BABSCO_INTERNATIONAL_Year_2_Declined": x,
    "term_3_BABSCO_INTERNATIONAL_Year_2_Pending": x,
    "term_3_BABSCO_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_BABSCO_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_BABSCO_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_NAO_DOMESTIC_Year_1_Applications": x,
    "term_3_NAO_DOMESTIC_Year_1_Accepted": x,
    "term_3_NAO_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_NAO_DOMESTIC_Year_1_Denied": x,
    "term_3_NAO_DOMESTIC_Year_1_Declined": x,
    "term_3_NAO_DOMESTIC_Year_1_Pending": x,
    "term_3_NAO_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_NAO_DOMESTIC_Year_1_Enrolled": x,
    "term_3_NAO_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_NAO_INTERNATIONAL_Year_1_Applications": x,
    "term_3_NAO_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_NAO_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_NAO_INTERNATIONAL_Year_1_Denied": x,
    "term_3_NAO_INTERNATIONAL_Year_1_Declined": x,
    "term_3_NAO_INTERNATIONAL_Year_1_Pending": x,
    "term_3_NAO_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_NAO_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_NAO_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_NAO_DOMESTIC_Year_2_Applications": x,
    "term_3_NAO_DOMESTIC_Year_2_Accepted": x,
    "term_3_NAO_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_NAO_DOMESTIC_Year_2_Denied": x,
    "term_3_NAO_DOMESTIC_Year_2_Declined": x,
    "term_3_NAO_DOMESTIC_Year_2_Pending": x,
    "term_3_NAO_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_NAO_DOMESTIC_Year_2_Enrolled": x,
    "term_3_NAO_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_NAO_INTERNATIONAL_Year_2_Applications": x,
    "term_3_NAO_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_NAO_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_NAO_INTERNATIONAL_Year_2_Denied": x,
    "term_3_NAO_INTERNATIONAL_Year_2_Declined": x,
    "term_3_NAO_INTERNATIONAL_Year_2_Pending": x,
    "term_3_NAO_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_NAO_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_NAO_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_ALO_DOMESTIC_Year_1_Applications": x,
    "term_3_ALO_DOMESTIC_Year_1_Accepted": x,
    "term_3_ALO_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_ALO_DOMESTIC_Year_1_Denied": x,
    "term_3_ALO_DOMESTIC_Year_1_Declined": x,
    "term_3_ALO_DOMESTIC_Year_1_Pending": x,
    "term_3_ALO_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_ALO_DOMESTIC_Year_1_Enrolled": x,
    "term_3_ALO_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_ALO_INTERNATIONAL_Year_1_Applications": x,
    "term_3_ALO_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_ALO_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_ALO_INTERNATIONAL_Year_1_Denied": x,
    "term_3_ALO_INTERNATIONAL_Year_1_Declined": x,
    "term_3_ALO_INTERNATIONAL_Year_1_Pending": x,
    "term_3_ALO_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_ALO_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_ALO_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_ALO_DOMESTIC_Year_2_Applications": x,
    "term_3_ALO_DOMESTIC_Year_2_Accepted": x,
    "term_3_ALO_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_ALO_DOMESTIC_Year_2_Denied": x,
    "term_3_ALO_DOMESTIC_Year_2_Declined": x,
    "term_3_ALO_DOMESTIC_Year_2_Pending": x,
    "term_3_ALO_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_ALO_DOMESTIC_Year_2_Enrolled": x,
    "term_3_ALO_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_ALO_INTERNATIONAL_Year_2_Applications": x,
    "term_3_ALO_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_ALO_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_ALO_INTERNATIONAL_Year_2_Denied": x,
    "term_3_ALO_INTERNATIONAL_Year_2_Declined": x,
    "term_3_ALO_INTERNATIONAL_Year_2_Pending": x,
    "term_3_ALO_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_ALO_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_ALO_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_HOSMNG_DOMESTIC_Year_1_Applications": x,
    "term_3_HOSMNG_DOMESTIC_Year_1_Accepted": x,
    "term_3_HOSMNG_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_HOSMNG_DOMESTIC_Year_1_Denied": x,
    "term_3_HOSMNG_DOMESTIC_Year_1_Declined": x,
    "term_3_HOSMNG_DOMESTIC_Year_1_Pending": x,
    "term_3_HOSMNG_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_HOSMNG_DOMESTIC_Year_1_Enrolled": x,
    "term_3_HOSMNG_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_HOSMNG_INTERNATIONAL_Year_1_Applications": x,
    "term_3_HOSMNG_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_HOSMNG_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_HOSMNG_INTERNATIONAL_Year_1_Denied": x,
    "term_3_HOSMNG_INTERNATIONAL_Year_1_Declined": x,
    "term_3_HOSMNG_INTERNATIONAL_Year_1_Pending": x,
    "term_3_HOSMNG_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_HOSMNG_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_HOSMNG_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_HOSMNG_DOMESTIC_Year_2_Applications": x,
    "term_3_HOSMNG_DOMESTIC_Year_2_Accepted": x,
    "term_3_HOSMNG_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_HOSMNG_DOMESTIC_Year_2_Denied": x,
    "term_3_HOSMNG_DOMESTIC_Year_2_Declined": x,
    "term_3_HOSMNG_DOMESTIC_Year_2_Pending": x,
    "term_3_HOSMNG_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_HOSMNG_DOMESTIC_Year_2_Enrolled": x,
    "term_3_HOSMNG_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_HOSMNG_INTERNATIONAL_Year_2_Applications": x,
    "term_3_HOSMNG_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_HOSMNG_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_HOSMNG_INTERNATIONAL_Year_2_Denied": x,
    "term_3_HOSMNG_INTERNATIONAL_Year_2_Declined": x,
    "term_3_HOSMNG_INTERNATIONAL_Year_2_Pending": x,
    "term_3_HOSMNG_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_HOSMNG_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_HOSMNG_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_BAHRMC_DOMESTIC_Year_1_Applications": x,
    "term_3_BAHRMC_DOMESTIC_Year_1_Accepted": x,
    "term_3_BAHRMC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_BAHRMC_DOMESTIC_Year_1_Denied": x,
    "term_3_BAHRMC_DOMESTIC_Year_1_Declined": x,
    "term_3_BAHRMC_DOMESTIC_Year_1_Pending": x,
    "term_3_BAHRMC_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_BAHRMC_DOMESTIC_Year_1_Enrolled": x,
    "term_3_BAHRMC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_BAHRMC_INTERNATIONAL_Year_1_Applications": x,
    "term_3_BAHRMC_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_BAHRMC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_BAHRMC_INTERNATIONAL_Year_1_Denied": x,
    "term_3_BAHRMC_INTERNATIONAL_Year_1_Declined": x,
    "term_3_BAHRMC_INTERNATIONAL_Year_1_Pending": x,
    "term_3_BAHRMC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_BAHRMC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_BAHRMC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_OFAC_DOMESTIC_Year_1_Applications": x,
    "term_3_OFAC_DOMESTIC_Year_1_Accepted": x,
    "term_3_OFAC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_OFAC_DOMESTIC_Year_1_Denied": x,
    "term_3_OFAC_DOMESTIC_Year_1_Declined": x,
    "term_3_OFAC_DOMESTIC_Year_1_Pending": x,
    "term_3_OFAC_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_OFAC_DOMESTIC_Year_1_Enrolled": x,
    "term_3_OFAC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_OFAC_INTERNATIONAL_Year_1_Applications": x,
    "term_3_OFAC_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_OFAC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_OFAC_INTERNATIONAL_Year_1_Denied": x,
    "term_3_OFAC_INTERNATIONAL_Year_1_Declined": x,
    "term_3_OFAC_INTERNATIONAL_Year_1_Pending": x,
    "term_3_OFAC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_OFAC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_OFAC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_EAP_DOMESTIC_Year_1_Applications": x,
    "term_3_EAP_DOMESTIC_Year_1_Accepted": x,
    "term_3_EAP_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_EAP_DOMESTIC_Year_1_Denied": x,
    "term_3_EAP_DOMESTIC_Year_1_Declined": x,
    "term_3_EAP_DOMESTIC_Year_1_Pending": x,
    "term_3_EAP_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_EAP_DOMESTIC_Year_1_Enrolled": x,
    "term_3_EAP_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_EAP_INTERNATIONAL_Year_1_Applications": x,
    "term_3_EAP_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_EAP_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_EAP_INTERNATIONAL_Year_1_Denied": x,
    "term_3_EAP_INTERNATIONAL_Year_1_Declined": x,
    "term_3_EAP_INTERNATIONAL_Year_1_Pending": x,
    "term_3_EAP_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_EAP_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_EAP_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_HSAFA_DOMESTIC_Year_1_Applications": x,
    "term_3_HSAFA_DOMESTIC_Year_1_Accepted": x,
    "term_3_HSAFA_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_HSAFA_DOMESTIC_Year_1_Denied": x,
    "term_3_HSAFA_DOMESTIC_Year_1_Declined": x,
    "term_3_HSAFA_DOMESTIC_Year_1_Pending": x,
    "term_3_HSAFA_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_HSAFA_DOMESTIC_Year_1_Enrolled": x,
    "term_3_HSAFA_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_HSAFA_INTERNATIONAL_Year_1_Applications": x,
    "term_3_HSAFA_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_HSAFA_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_HSAFA_INTERNATIONAL_Year_1_Denied": x,
    "term_3_HSAFA_INTERNATIONAL_Year_1_Declined": x,
    "term_3_HSAFA_INTERNATIONAL_Year_1_Pending": x,
    "term_3_HSAFA_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_HSAFA_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_HSAFA_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_HSAD_DOMESTIC_Year_1_Applications": x,
    "term_3_HSAD_DOMESTIC_Year_1_Accepted": x,
    "term_3_HSAD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_HSAD_DOMESTIC_Year_1_Denied": x,
    "term_3_HSAD_DOMESTIC_Year_1_Declined": x,
    "term_3_HSAD_DOMESTIC_Year_1_Pending": x,
    "term_3_HSAD_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_HSAD_DOMESTIC_Year_1_Enrolled": x,
    "term_3_HSAD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_HSAD_INTERNATIONAL_Year_1_Applications": x,
    "term_3_HSAD_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_HSAD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_HSAD_INTERNATIONAL_Year_1_Denied": x,
    "term_3_HSAD_INTERNATIONAL_Year_1_Declined": x,
    "term_3_HSAD_INTERNATIONAL_Year_1_Pending": x,
    "term_3_HSAD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_HSAD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_HSAD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_HSGD_DOMESTIC_Year_1_Applications": x,
    "term_3_HSGD_DOMESTIC_Year_1_Accepted": x,
    "term_3_HSGD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_HSGD_DOMESTIC_Year_1_Denied": x,
    "term_3_HSGD_DOMESTIC_Year_1_Declined": x,
    "term_3_HSGD_DOMESTIC_Year_1_Pending": x,
    "term_3_HSGD_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_HSGD_DOMESTIC_Year_1_Enrolled": x,
    "term_3_HSGD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_HSGD_INTERNATIONAL_Year_1_Applications": x,
    "term_3_HSGD_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_HSGD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_HSGD_INTERNATIONAL_Year_1_Denied": x,
    "term_3_HSGD_INTERNATIONAL_Year_1_Declined": x,
    "term_3_HSGD_INTERNATIONAL_Year_1_Pending": x,
    "term_3_HSGD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_HSGD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_HSGD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_HSNP_DOMESTIC_Year_1_Applications": x,
    "term_3_HSNP_DOMESTIC_Year_1_Accepted": x,
    "term_3_HSNP_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_HSNP_DOMESTIC_Year_1_Denied": x,
    "term_3_HSNP_DOMESTIC_Year_1_Declined": x,
    "term_3_HSNP_DOMESTIC_Year_1_Pending": x,
    "term_3_HSNP_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_HSNP_DOMESTIC_Year_1_Enrolled": x,
    "term_3_HSNP_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_HSNP_INTERNATIONAL_Year_1_Applications": x,
    "term_3_HSNP_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_HSNP_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_HSNP_INTERNATIONAL_Year_1_Denied": x,
    "term_3_HSNP_INTERNATIONAL_Year_1_Declined": x,
    "term_3_HSNP_INTERNATIONAL_Year_1_Pending": x,
    "term_3_HSNP_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_HSNP_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_HSNP_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_UTELEM_DOMESTIC_Year_1_Applications": x,
    "term_3_UTELEM_DOMESTIC_Year_1_Accepted": x,
    "term_3_UTELEM_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_UTELEM_DOMESTIC_Year_1_Denied": x,
    "term_3_UTELEM_DOMESTIC_Year_1_Declined": x,
    "term_3_UTELEM_DOMESTIC_Year_1_Pending": x,
    "term_3_UTELEM_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_UTELEM_DOMESTIC_Year_1_Enrolled": x,
    "term_3_UTELEM_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_UTELEM_INTERNATIONAL_Year_1_Applications": x,
    "term_3_UTELEM_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_UTELEM_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_UTELEM_INTERNATIONAL_Year_1_Denied": x,
    "term_3_UTELEM_INTERNATIONAL_Year_1_Declined": x,
    "term_3_UTELEM_INTERNATIONAL_Year_1_Pending": x,
    "term_3_UTELEM_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_UTELEM_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_UTELEM_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_UTELEM_DOMESTIC_Year_2_Applications": x,
    "term_3_UTELEM_DOMESTIC_Year_2_Accepted": x,
    "term_3_UTELEM_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_UTELEM_DOMESTIC_Year_2_Denied": x,
    "term_3_UTELEM_DOMESTIC_Year_2_Declined": x,
    "term_3_UTELEM_DOMESTIC_Year_2_Pending": x,
    "term_3_UTELEM_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_UTELEM_DOMESTIC_Year_2_Enrolled": x,
    "term_3_UTELEM_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_UTELEM_INTERNATIONAL_Year_2_Applications": x,
    "term_3_UTELEM_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_UTELEM_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_UTELEM_INTERNATIONAL_Year_2_Denied": x,
    "term_3_UTELEM_INTERNATIONAL_Year_2_Declined": x,
    "term_3_UTELEM_INTERNATIONAL_Year_2_Pending": x,
    "term_3_UTELEM_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_UTELEM_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_UTELEM_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_UTSEC_DOMESTIC_Year_1_Applications": x,
    "term_3_UTSEC_DOMESTIC_Year_1_Accepted": x,
    "term_3_UTSEC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_UTSEC_DOMESTIC_Year_1_Denied": x,
    "term_3_UTSEC_DOMESTIC_Year_1_Declined": x,
    "term_3_UTSEC_DOMESTIC_Year_1_Pending": x,
    "term_3_UTSEC_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_UTSEC_DOMESTIC_Year_1_Enrolled": x,
    "term_3_UTSEC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_UTSEC_INTERNATIONAL_Year_1_Applications": x,
    "term_3_UTSEC_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_UTSEC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_UTSEC_INTERNATIONAL_Year_1_Denied": x,
    "term_3_UTSEC_INTERNATIONAL_Year_1_Declined": x,
    "term_3_UTSEC_INTERNATIONAL_Year_1_Pending": x,
    "term_3_UTSEC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_UTSEC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_UTSEC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_UTSEC_DOMESTIC_Year_2_Applications": x,
    "term_3_UTSEC_DOMESTIC_Year_2_Accepted": x,
    "term_3_UTSEC_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_UTSEC_DOMESTIC_Year_2_Denied": x,
    "term_3_UTSEC_DOMESTIC_Year_2_Declined": x,
    "term_3_UTSEC_DOMESTIC_Year_2_Pending": x,
    "term_3_UTSEC_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_UTSEC_DOMESTIC_Year_2_Enrolled": x,
    "term_3_UTSEC_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_UTSEC_INTERNATIONAL_Year_2_Applications": x,
    "term_3_UTSEC_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_UTSEC_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_UTSEC_INTERNATIONAL_Year_2_Denied": x,
    "term_3_UTSEC_INTERNATIONAL_Year_2_Declined": x,
    "term_3_UTSEC_INTERNATIONAL_Year_2_Pending": x,
    "term_3_UTSEC_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_UTSEC_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_UTSEC_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_ENVTD_DOMESTIC_Year_1_Applications": x,
    "term_3_ENVTD_DOMESTIC_Year_1_Accepted": x,
    "term_3_ENVTD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_ENVTD_DOMESTIC_Year_1_Denied": x,
    "term_3_ENVTD_DOMESTIC_Year_1_Declined": x,
    "term_3_ENVTD_DOMESTIC_Year_1_Pending": x,
    "term_3_ENVTD_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_ENVTD_DOMESTIC_Year_1_Enrolled": x,
    "term_3_ENVTD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_ENVTD_INTERNATIONAL_Year_1_Applications": x,
    "term_3_ENVTD_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_ENVTD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_ENVTD_INTERNATIONAL_Year_1_Denied": x,
    "term_3_ENVTD_INTERNATIONAL_Year_1_Declined": x,
    "term_3_ENVTD_INTERNATIONAL_Year_1_Pending": x,
    "term_3_ENVTD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_ENVTD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_ENVTD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_ENVTD_DOMESTIC_Year_2_Applications": x,
    "term_3_ENVTD_DOMESTIC_Year_2_Accepted": x,
    "term_3_ENVTD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_ENVTD_DOMESTIC_Year_2_Denied": x,
    "term_3_ENVTD_DOMESTIC_Year_2_Declined": x,
    "term_3_ENVTD_DOMESTIC_Year_2_Pending": x,
    "term_3_ENVTD_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_ENVTD_DOMESTIC_Year_2_Enrolled": x,
    "term_3_ENVTD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_ENVTD_INTERNATIONAL_Year_2_Applications": x,
    "term_3_ENVTD_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_ENVTD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_ENVTD_INTERNATIONAL_Year_2_Denied": x,
    "term_3_ENVTD_INTERNATIONAL_Year_2_Declined": x,
    "term_3_ENVTD_INTERNATIONAL_Year_2_Pending": x,
    "term_3_ENVTD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_ENVTD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_ENVTD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_ENVNC_DOMESTIC_Year_1_Applications": x,
    "term_3_ENVNC_DOMESTIC_Year_1_Accepted": x,
    "term_3_ENVNC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_ENVNC_DOMESTIC_Year_1_Denied": x,
    "term_3_ENVNC_DOMESTIC_Year_1_Declined": x,
    "term_3_ENVNC_DOMESTIC_Year_1_Pending": x,
    "term_3_ENVNC_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_ENVNC_DOMESTIC_Year_1_Enrolled": x,
    "term_3_ENVNC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_ENVNC_INTERNATIONAL_Year_1_Applications": x,
    "term_3_ENVNC_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_ENVNC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_ENVNC_INTERNATIONAL_Year_1_Denied": x,
    "term_3_ENVNC_INTERNATIONAL_Year_1_Declined": x,
    "term_3_ENVNC_INTERNATIONAL_Year_1_Pending": x,
    "term_3_ENVNC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_ENVNC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_ENVNC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_ENVNC_DOMESTIC_Year_2_Applications": x,
    "term_3_ENVNC_DOMESTIC_Year_2_Accepted": x,
    "term_3_ENVNC_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_ENVNC_DOMESTIC_Year_2_Denied": x,
    "term_3_ENVNC_DOMESTIC_Year_2_Declined": x,
    "term_3_ENVNC_DOMESTIC_Year_2_Pending": x,
    "term_3_ENVNC_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_ENVNC_DOMESTIC_Year_2_Enrolled": x,
    "term_3_ENVNC_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_ENVNC_INTERNATIONAL_Year_2_Applications": x,
    "term_3_ENVNC_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_ENVNC_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_ENVNC_INTERNATIONAL_Year_2_Denied": x,
    "term_3_ENVNC_INTERNATIONAL_Year_2_Declined": x,
    "term_3_ENVNC_INTERNATIONAL_Year_2_Pending": x,
    "term_3_ENVNC_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_ENVNC_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_ENVNC_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_UTARTS_DOMESTIC_Year_1_Applications": x,
    "term_3_UTARTS_DOMESTIC_Year_1_Accepted": x,
    "term_3_UTARTS_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_UTARTS_DOMESTIC_Year_1_Denied": x,
    "term_3_UTARTS_DOMESTIC_Year_1_Declined": x,
    "term_3_UTARTS_DOMESTIC_Year_1_Pending": x,
    "term_3_UTARTS_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_UTARTS_DOMESTIC_Year_1_Enrolled": x,
    "term_3_UTARTS_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_UTARTS_INTERNATIONAL_Year_1_Applications": x,
    "term_3_UTARTS_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_UTARTS_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_UTARTS_INTERNATIONAL_Year_1_Denied": x,
    "term_3_UTARTS_INTERNATIONAL_Year_1_Declined": x,
    "term_3_UTARTS_INTERNATIONAL_Year_1_Pending": x,
    "term_3_UTARTS_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_UTARTS_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_UTARTS_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_UTARTS_DOMESTIC_Year_2_Applications": x,
    "term_3_UTARTS_DOMESTIC_Year_2_Accepted": x,
    "term_3_UTARTS_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_UTARTS_DOMESTIC_Year_2_Denied": x,
    "term_3_UTARTS_DOMESTIC_Year_2_Declined": x,
    "term_3_UTARTS_DOMESTIC_Year_2_Pending": x,
    "term_3_UTARTS_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_UTARTS_DOMESTIC_Year_2_Enrolled": x,
    "term_3_UTARTS_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_UTARTS_INTERNATIONAL_Year_2_Applications": x,
    "term_3_UTARTS_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_UTARTS_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_UTARTS_INTERNATIONAL_Year_2_Denied": x,
    "term_3_UTARTS_INTERNATIONAL_Year_2_Declined": x,
    "term_3_UTARTS_INTERNATIONAL_Year_2_Pending": x,
    "term_3_UTARTS_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_UTARTS_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_UTARTS_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_UTCOMM_DOMESTIC_Year_1_Applications": x,
    "term_3_UTCOMM_DOMESTIC_Year_1_Accepted": x,
    "term_3_UTCOMM_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_UTCOMM_DOMESTIC_Year_1_Denied": x,
    "term_3_UTCOMM_DOMESTIC_Year_1_Declined": x,
    "term_3_UTCOMM_DOMESTIC_Year_1_Pending": x,
    "term_3_UTCOMM_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_UTCOMM_DOMESTIC_Year_1_Enrolled": x,
    "term_3_UTCOMM_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_UTCOMM_INTERNATIONAL_Year_1_Applications": x,
    "term_3_UTCOMM_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_UTCOMM_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_UTCOMM_INTERNATIONAL_Year_1_Denied": x,
    "term_3_UTCOMM_INTERNATIONAL_Year_1_Declined": x,
    "term_3_UTCOMM_INTERNATIONAL_Year_1_Pending": x,
    "term_3_UTCOMM_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_UTCOMM_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_UTCOMM_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_UTCOMM_DOMESTIC_Year_2_Applications": x,
    "term_3_UTCOMM_DOMESTIC_Year_2_Accepted": x,
    "term_3_UTCOMM_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_UTCOMM_DOMESTIC_Year_2_Denied": x,
    "term_3_UTCOMM_DOMESTIC_Year_2_Declined": x,
    "term_3_UTCOMM_DOMESTIC_Year_2_Pending": x,
    "term_3_UTCOMM_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_UTCOMM_DOMESTIC_Year_2_Enrolled": x,
    "term_3_UTCOMM_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_UTCOMM_INTERNATIONAL_Year_2_Applications": x,
    "term_3_UTCOMM_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_UTCOMM_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_UTCOMM_INTERNATIONAL_Year_2_Denied": x,
    "term_3_UTCOMM_INTERNATIONAL_Year_2_Declined": x,
    "term_3_UTCOMM_INTERNATIONAL_Year_2_Pending": x,
    "term_3_UTCOMM_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_UTCOMM_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_UTCOMM_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_UTENG_DOMESTIC_Year_1_Applications": x,
    "term_3_UTENG_DOMESTIC_Year_1_Accepted": x,
    "term_3_UTENG_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_UTENG_DOMESTIC_Year_1_Denied": x,
    "term_3_UTENG_DOMESTIC_Year_1_Declined": x,
    "term_3_UTENG_DOMESTIC_Year_1_Pending": x,
    "term_3_UTENG_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_UTENG_DOMESTIC_Year_1_Enrolled": x,
    "term_3_UTENG_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_UTENG_INTERNATIONAL_Year_1_Applications": x,
    "term_3_UTENG_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_UTENG_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_UTENG_INTERNATIONAL_Year_1_Denied": x,
    "term_3_UTENG_INTERNATIONAL_Year_1_Declined": x,
    "term_3_UTENG_INTERNATIONAL_Year_1_Pending": x,
    "term_3_UTENG_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_UTENG_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_UTENG_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_UTSCGS_DOMESTIC_Year_1_Applications": x,
    "term_3_UTSCGS_DOMESTIC_Year_1_Accepted": x,
    "term_3_UTSCGS_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_UTSCGS_DOMESTIC_Year_1_Denied": x,
    "term_3_UTSCGS_DOMESTIC_Year_1_Declined": x,
    "term_3_UTSCGS_DOMESTIC_Year_1_Pending": x,
    "term_3_UTSCGS_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_UTSCGS_DOMESTIC_Year_1_Enrolled": x,
    "term_3_UTSCGS_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_UTSCGS_INTERNATIONAL_Year_1_Applications": x,
    "term_3_UTSCGS_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_UTSCGS_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_UTSCGS_INTERNATIONAL_Year_1_Denied": x,
    "term_3_UTSCGS_INTERNATIONAL_Year_1_Declined": x,
    "term_3_UTSCGS_INTERNATIONAL_Year_1_Pending": x,
    "term_3_UTSCGS_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_UTSCGS_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_UTSCGS_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_UTSCGS_DOMESTIC_Year_2_Applications": x,
    "term_3_UTSCGS_DOMESTIC_Year_2_Accepted": x,
    "term_3_UTSCGS_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_UTSCGS_DOMESTIC_Year_2_Denied": x,
    "term_3_UTSCGS_DOMESTIC_Year_2_Declined": x,
    "term_3_UTSCGS_DOMESTIC_Year_2_Pending": x,
    "term_3_UTSCGS_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_UTSCGS_DOMESTIC_Year_2_Enrolled": x,
    "term_3_UTSCGS_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_UTSCGS_INTERNATIONAL_Year_2_Applications": x,
    "term_3_UTSCGS_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_UTSCGS_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_UTSCGS_INTERNATIONAL_Year_2_Denied": x,
    "term_3_UTSCGS_INTERNATIONAL_Year_2_Declined": x,
    "term_3_UTSCGS_INTERNATIONAL_Year_2_Pending": x,
    "term_3_UTSCGS_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_UTSCGS_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_UTSCGS_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_UTSCWK_DOMESTIC_Year_1_Applications": x,
    "term_3_UTSCWK_DOMESTIC_Year_1_Accepted": x,
    "term_3_UTSCWK_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_UTSCWK_DOMESTIC_Year_1_Denied": x,
    "term_3_UTSCWK_DOMESTIC_Year_1_Declined": x,
    "term_3_UTSCWK_DOMESTIC_Year_1_Pending": x,
    "term_3_UTSCWK_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_UTSCWK_DOMESTIC_Year_1_Enrolled": x,
    "term_3_UTSCWK_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_UTSCWK_INTERNATIONAL_Year_1_Applications": x,
    "term_3_UTSCWK_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_UTSCWK_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_UTSCWK_INTERNATIONAL_Year_1_Denied": x,
    "term_3_UTSCWK_INTERNATIONAL_Year_1_Declined": x,
    "term_3_UTSCWK_INTERNATIONAL_Year_1_Pending": x,
    "term_3_UTSCWK_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_UTSCWK_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_UTSCWK_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_UTSCWK_DOMESTIC_Year_2_Applications": x,
    "term_3_UTSCWK_DOMESTIC_Year_2_Accepted": x,
    "term_3_UTSCWK_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_UTSCWK_DOMESTIC_Year_2_Denied": x,
    "term_3_UTSCWK_DOMESTIC_Year_2_Declined": x,
    "term_3_UTSCWK_DOMESTIC_Year_2_Pending": x,
    "term_3_UTSCWK_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_UTSCWK_DOMESTIC_Year_2_Enrolled": x,
    "term_3_UTSCWK_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_UTSCWK_INTERNATIONAL_Year_2_Applications": x,
    "term_3_UTSCWK_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_UTSCWK_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_UTSCWK_INTERNATIONAL_Year_2_Denied": x,
    "term_3_UTSCWK_INTERNATIONAL_Year_2_Declined": x,
    "term_3_UTSCWK_INTERNATIONAL_Year_2_Pending": x,
    "term_3_UTSCWK_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_UTSCWK_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_UTSCWK_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_OPEN_DOMESTIC_Year_1_Applications": x,
    "term_3_OPEN_DOMESTIC_Year_1_Accepted": x,
    "term_3_OPEN_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_OPEN_DOMESTIC_Year_1_Denied": x,
    "term_3_OPEN_DOMESTIC_Year_1_Declined": x,
    "term_3_OPEN_DOMESTIC_Year_1_Pending": x,
    "term_3_OPEN_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_OPEN_DOMESTIC_Year_1_Enrolled": x,
    "term_3_OPEN_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_OPEN_INTERNATIONAL_Year_1_Applications": x,
    "term_3_OPEN_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_OPEN_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_OPEN_INTERNATIONAL_Year_1_Denied": x,
    "term_3_OPEN_INTERNATIONAL_Year_1_Declined": x,
    "term_3_OPEN_INTERNATIONAL_Year_1_Pending": x,
    "term_3_OPEN_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_OPEN_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_OPEN_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_GOVNSC_DOMESTIC_Year_1_Applications": x,
    "term_3_GOVNSC_DOMESTIC_Year_1_Accepted": x,
    "term_3_GOVNSC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_GOVNSC_DOMESTIC_Year_1_Denied": x,
    "term_3_GOVNSC_DOMESTIC_Year_1_Declined": x,
    "term_3_GOVNSC_DOMESTIC_Year_1_Pending": x,
    "term_3_GOVNSC_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_GOVNSC_DOMESTIC_Year_1_Enrolled": x,
    "term_3_GOVNSC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_GOVNSC_INTERNATIONAL_Year_1_Applications": x,
    "term_3_GOVNSC_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_GOVNSC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_GOVNSC_INTERNATIONAL_Year_1_Denied": x,
    "term_3_GOVNSC_INTERNATIONAL_Year_1_Declined": x,
    "term_3_GOVNSC_INTERNATIONAL_Year_1_Pending": x,
    "term_3_GOVNSC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_GOVNSC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_GOVNSC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_GOVNSC_DOMESTIC_Year_2_Applications": x,
    "term_3_GOVNSC_DOMESTIC_Year_2_Accepted": x,
    "term_3_GOVNSC_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_GOVNSC_DOMESTIC_Year_2_Denied": x,
    "term_3_GOVNSC_DOMESTIC_Year_2_Declined": x,
    "term_3_GOVNSC_DOMESTIC_Year_2_Pending": x,
    "term_3_GOVNSC_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_GOVNSC_DOMESTIC_Year_2_Enrolled": x,
    "term_3_GOVNSC_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_GOVNSC_INTERNATIONAL_Year_2_Applications": x,
    "term_3_GOVNSC_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_GOVNSC_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_GOVNSC_INTERNATIONAL_Year_2_Denied": x,
    "term_3_GOVNSC_INTERNATIONAL_Year_2_Declined": x,
    "term_3_GOVNSC_INTERNATIONAL_Year_2_Pending": x,
    "term_3_GOVNSC_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_GOVNSC_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_GOVNSC_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_COMSCC_DOMESTIC_Year_1_Applications": x,
    "term_3_COMSCC_DOMESTIC_Year_1_Accepted": x,
    "term_3_COMSCC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_COMSCC_DOMESTIC_Year_1_Denied": x,
    "term_3_COMSCC_DOMESTIC_Year_1_Declined": x,
    "term_3_COMSCC_DOMESTIC_Year_1_Pending": x,
    "term_3_COMSCC_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_COMSCC_DOMESTIC_Year_1_Enrolled": x,
    "term_3_COMSCC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_COMSCC_INTERNATIONAL_Year_1_Applications": x,
    "term_3_COMSCC_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_COMSCC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_COMSCC_INTERNATIONAL_Year_1_Denied": x,
    "term_3_COMSCC_INTERNATIONAL_Year_1_Declined": x,
    "term_3_COMSCC_INTERNATIONAL_Year_1_Pending": x,
    "term_3_COMSCC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_COMSCC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_COMSCC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_COMSCD_DOMESTIC_Year_1_Applications": x,
    "term_3_COMSCD_DOMESTIC_Year_1_Accepted": x,
    "term_3_COMSCD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_COMSCD_DOMESTIC_Year_1_Denied": x,
    "term_3_COMSCD_DOMESTIC_Year_1_Declined": x,
    "term_3_COMSCD_DOMESTIC_Year_1_Pending": x,
    "term_3_COMSCD_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_COMSCD_DOMESTIC_Year_1_Enrolled": x,
    "term_3_COMSCD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_COMSCD_INTERNATIONAL_Year_1_Applications": x,
    "term_3_COMSCD_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_COMSCD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_COMSCD_INTERNATIONAL_Year_1_Denied": x,
    "term_3_COMSCD_INTERNATIONAL_Year_1_Declined": x,
    "term_3_COMSCD_INTERNATIONAL_Year_1_Pending": x,
    "term_3_COMSCD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_COMSCD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_COMSCD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_COMSCD_DOMESTIC_Year_2_Applications": x,
    "term_3_COMSCD_DOMESTIC_Year_2_Accepted": x,
    "term_3_COMSCD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_COMSCD_DOMESTIC_Year_2_Denied": x,
    "term_3_COMSCD_DOMESTIC_Year_2_Declined": x,
    "term_3_COMSCD_DOMESTIC_Year_2_Pending": x,
    "term_3_COMSCD_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_COMSCD_DOMESTIC_Year_2_Enrolled": x,
    "term_3_COMSCD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_COMSCD_INTERNATIONAL_Year_2_Applications": x,
    "term_3_COMSCD_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_COMSCD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_COMSCD_INTERNATIONAL_Year_2_Denied": x,
    "term_3_COMSCD_INTERNATIONAL_Year_2_Declined": x,
    "term_3_COMSCD_INTERNATIONAL_Year_2_Pending": x,
    "term_3_COMSCD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_COMSCD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_COMSCD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_GNARTC_DOMESTIC_Year_1_Applications": x,
    "term_3_GNARTC_DOMESTIC_Year_1_Accepted": x,
    "term_3_GNARTC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_GNARTC_DOMESTIC_Year_1_Denied": x,
    "term_3_GNARTC_DOMESTIC_Year_1_Declined": x,
    "term_3_GNARTC_DOMESTIC_Year_1_Pending": x,
    "term_3_GNARTC_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_GNARTC_DOMESTIC_Year_1_Enrolled": x,
    "term_3_GNARTC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_GNARTC_INTERNATIONAL_Year_1_Applications": x,
    "term_3_GNARTC_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_GNARTC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_GNARTC_INTERNATIONAL_Year_1_Denied": x,
    "term_3_GNARTC_INTERNATIONAL_Year_1_Declined": x,
    "term_3_GNARTC_INTERNATIONAL_Year_1_Pending": x,
    "term_3_GNARTC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_GNARTC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_GNARTC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_GNSCIC_DOMESTIC_Year_1_Applications": x,
    "term_3_GNSCIC_DOMESTIC_Year_1_Accepted": x,
    "term_3_GNSCIC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_GNSCIC_DOMESTIC_Year_1_Denied": x,
    "term_3_GNSCIC_DOMESTIC_Year_1_Declined": x,
    "term_3_GNSCIC_DOMESTIC_Year_1_Pending": x,
    "term_3_GNSCIC_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_GNSCIC_DOMESTIC_Year_1_Enrolled": x,
    "term_3_GNSCIC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_GNSCIC_INTERNATIONAL_Year_1_Applications": x,
    "term_3_GNSCIC_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_GNSCIC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_GNSCIC_INTERNATIONAL_Year_1_Denied": x,
    "term_3_GNSCIC_INTERNATIONAL_Year_1_Declined": x,
    "term_3_GNSCIC_INTERNATIONAL_Year_1_Pending": x,
    "term_3_GNSCIC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_GNSCIC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_GNSCIC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_ANCPDC_DOMESTIC_Year_1_Applications": x,
    "term_3_ANCPDC_DOMESTIC_Year_1_Accepted": x,
    "term_3_ANCPDC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_ANCPDC_DOMESTIC_Year_1_Denied": x,
    "term_3_ANCPDC_DOMESTIC_Year_1_Declined": x,
    "term_3_ANCPDC_DOMESTIC_Year_1_Pending": x,
    "term_3_ANCPDC_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_ANCPDC_DOMESTIC_Year_1_Enrolled": x,
    "term_3_ANCPDC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_ANCPDC_INTERNATIONAL_Year_1_Applications": x,
    "term_3_ANCPDC_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_ANCPDC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_ANCPDC_INTERNATIONAL_Year_1_Denied": x,
    "term_3_ANCPDC_INTERNATIONAL_Year_1_Declined": x,
    "term_3_ANCPDC_INTERNATIONAL_Year_1_Pending": x,
    "term_3_ANCPDC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_ANCPDC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_ANCPDC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_NSPDC_DOMESTIC_Year_1_Applications": x,
    "term_3_NSPDC_DOMESTIC_Year_1_Accepted": x,
    "term_3_NSPDC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_NSPDC_DOMESTIC_Year_1_Denied": x,
    "term_3_NSPDC_DOMESTIC_Year_1_Declined": x,
    "term_3_NSPDC_DOMESTIC_Year_1_Pending": x,
    "term_3_NSPDC_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_NSPDC_DOMESTIC_Year_1_Enrolled": x,
    "term_3_NSPDC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_NSPDC_INTERNATIONAL_Year_1_Applications": x,
    "term_3_NSPDC_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_NSPDC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_NSPDC_INTERNATIONAL_Year_1_Denied": x,
    "term_3_NSPDC_INTERNATIONAL_Year_1_Declined": x,
    "term_3_NSPDC_INTERNATIONAL_Year_1_Pending": x,
    "term_3_NSPDC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_NSPDC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_NSPDC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3__DOMESTIC_Year_1_Applications": x,
    "term_3__DOMESTIC_Year_1_Accepted": x,
    "term_3__DOMESTIC_Year_1_Payment_Pending": x,
    "term_3__DOMESTIC_Year_1_Denied": x,
    "term_3__DOMESTIC_Year_1_Declined": x,
    "term_3__DOMESTIC_Year_1_Pending": x,
    "term_3__DOMESTIC_Year_1_Waitlisted": x,
    "term_3__DOMESTIC_Year_1_Enrolled": x,
    "term_3__DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3__INTERNATIONAL_Year_1_Applications": x,
    "term_3__INTERNATIONAL_Year_1_Accepted": x,
    "term_3__INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3__INTERNATIONAL_Year_1_Denied": x,
    "term_3__INTERNATIONAL_Year_1_Declined": x,
    "term_3__INTERNATIONAL_Year_1_Pending": x,
    "term_3__INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3__INTERNATIONAL_Year_1_Enrolled": x,
    "term_3__INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_LINC13_DOMESTIC_Year_1_Applications": x,
    "term_3_LINC13_DOMESTIC_Year_1_Accepted": x,
    "term_3_LINC13_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_LINC13_DOMESTIC_Year_1_Denied": x,
    "term_3_LINC13_DOMESTIC_Year_1_Declined": x,
    "term_3_LINC13_DOMESTIC_Year_1_Pending": x,
    "term_3_LINC13_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_LINC13_DOMESTIC_Year_1_Enrolled": x,
    "term_3_LINC13_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_LINC13_INTERNATIONAL_Year_1_Applications": x,
    "term_3_LINC13_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_LINC13_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_LINC13_INTERNATIONAL_Year_1_Denied": x,
    "term_3_LINC13_INTERNATIONAL_Year_1_Declined": x,
    "term_3_LINC13_INTERNATIONAL_Year_1_Pending": x,
    "term_3_LINC13_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_LINC13_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_LINC13_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_AELCC_DOMESTIC_Year_1_Applications": x,
    "term_3_AELCC_DOMESTIC_Year_1_Accepted": x,
    "term_3_AELCC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_AELCC_DOMESTIC_Year_1_Denied": x,
    "term_3_AELCC_DOMESTIC_Year_1_Declined": x,
    "term_3_AELCC_DOMESTIC_Year_1_Pending": x,
    "term_3_AELCC_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_AELCC_DOMESTIC_Year_1_Enrolled": x,
    "term_3_AELCC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_AELCC_INTERNATIONAL_Year_1_Applications": x,
    "term_3_AELCC_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_AELCC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_AELCC_INTERNATIONAL_Year_1_Denied": x,
    "term_3_AELCC_INTERNATIONAL_Year_1_Declined": x,
    "term_3_AELCC_INTERNATIONAL_Year_1_Pending": x,
    "term_3_AELCC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_AELCC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_AELCC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_ECCERT_DOMESTIC_Year_1_Applications": x,
    "term_3_ECCERT_DOMESTIC_Year_1_Accepted": x,
    "term_3_ECCERT_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_ECCERT_DOMESTIC_Year_1_Denied": x,
    "term_3_ECCERT_DOMESTIC_Year_1_Declined": x,
    "term_3_ECCERT_DOMESTIC_Year_1_Pending": x,
    "term_3_ECCERT_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_ECCERT_DOMESTIC_Year_1_Enrolled": x,
    "term_3_ECCERT_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_ECCERT_INTERNATIONAL_Year_1_Applications": x,
    "term_3_ECCERT_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_ECCERT_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_ECCERT_INTERNATIONAL_Year_1_Denied": x,
    "term_3_ECCERT_INTERNATIONAL_Year_1_Declined": x,
    "term_3_ECCERT_INTERNATIONAL_Year_1_Pending": x,
    "term_3_ECCERT_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_ECCERT_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_ECCERT_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_ECDIP_DOMESTIC_Year_1_Applications": x,
    "term_3_ECDIP_DOMESTIC_Year_1_Accepted": x,
    "term_3_ECDIP_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_ECDIP_DOMESTIC_Year_1_Denied": x,
    "term_3_ECDIP_DOMESTIC_Year_1_Declined": x,
    "term_3_ECDIP_DOMESTIC_Year_1_Pending": x,
    "term_3_ECDIP_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_ECDIP_DOMESTIC_Year_1_Enrolled": x,
    "term_3_ECDIP_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_ECDIP_INTERNATIONAL_Year_1_Applications": x,
    "term_3_ECDIP_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_ECDIP_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_ECDIP_INTERNATIONAL_Year_1_Denied": x,
    "term_3_ECDIP_INTERNATIONAL_Year_1_Declined": x,
    "term_3_ECDIP_INTERNATIONAL_Year_1_Pending": x,
    "term_3_ECDIP_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_ECDIP_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_ECDIP_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_ECDIP_DOMESTIC_Year_2_Applications": x,
    "term_3_ECDIP_DOMESTIC_Year_2_Accepted": x,
    "term_3_ECDIP_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_ECDIP_DOMESTIC_Year_2_Denied": x,
    "term_3_ECDIP_DOMESTIC_Year_2_Declined": x,
    "term_3_ECDIP_DOMESTIC_Year_2_Pending": x,
    "term_3_ECDIP_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_ECDIP_DOMESTIC_Year_2_Enrolled": x,
    "term_3_ECDIP_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_ECDIP_INTERNATIONAL_Year_2_Applications": x,
    "term_3_ECDIP_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_ECDIP_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_ECDIP_INTERNATIONAL_Year_2_Denied": x,
    "term_3_ECDIP_INTERNATIONAL_Year_2_Declined": x,
    "term_3_ECDIP_INTERNATIONAL_Year_2_Pending": x,
    "term_3_ECDIP_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_ECDIP_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_ECDIP_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_CYCD_DOMESTIC_Year_1_Applications": x,
    "term_3_CYCD_DOMESTIC_Year_1_Accepted": x,
    "term_3_CYCD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_CYCD_DOMESTIC_Year_1_Denied": x,
    "term_3_CYCD_DOMESTIC_Year_1_Declined": x,
    "term_3_CYCD_DOMESTIC_Year_1_Pending": x,
    "term_3_CYCD_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_CYCD_DOMESTIC_Year_1_Enrolled": x,
    "term_3_CYCD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_CYCD_INTERNATIONAL_Year_1_Applications": x,
    "term_3_CYCD_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_CYCD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_CYCD_INTERNATIONAL_Year_1_Denied": x,
    "term_3_CYCD_INTERNATIONAL_Year_1_Declined": x,
    "term_3_CYCD_INTERNATIONAL_Year_1_Pending": x,
    "term_3_CYCD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_CYCD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_CYCD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_CYCD_DOMESTIC_Year_2_Applications": x,
    "term_3_CYCD_DOMESTIC_Year_2_Accepted": x,
    "term_3_CYCD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_CYCD_DOMESTIC_Year_2_Denied": x,
    "term_3_CYCD_DOMESTIC_Year_2_Declined": x,
    "term_3_CYCD_DOMESTIC_Year_2_Pending": x,
    "term_3_CYCD_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_CYCD_DOMESTIC_Year_2_Enrolled": x,
    "term_3_CYCD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_CYCD_INTERNATIONAL_Year_2_Applications": x,
    "term_3_CYCD_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_CYCD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_CYCD_INTERNATIONAL_Year_2_Denied": x,
    "term_3_CYCD_INTERNATIONAL_Year_2_Declined": x,
    "term_3_CYCD_INTERNATIONAL_Year_2_Pending": x,
    "term_3_CYCD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_CYCD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_CYCD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_ECEAC_DOMESTIC_Year_1_Applications": x,
    "term_3_ECEAC_DOMESTIC_Year_1_Accepted": x,
    "term_3_ECEAC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_ECEAC_DOMESTIC_Year_1_Denied": x,
    "term_3_ECEAC_DOMESTIC_Year_1_Declined": x,
    "term_3_ECEAC_DOMESTIC_Year_1_Pending": x,
    "term_3_ECEAC_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_ECEAC_DOMESTIC_Year_1_Enrolled": x,
    "term_3_ECEAC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_ECEAC_INTERNATIONAL_Year_1_Applications": x,
    "term_3_ECEAC_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_ECEAC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_ECEAC_INTERNATIONAL_Year_1_Denied": x,
    "term_3_ECEAC_INTERNATIONAL_Year_1_Declined": x,
    "term_3_ECEAC_INTERNATIONAL_Year_1_Pending": x,
    "term_3_ECEAC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_ECEAC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_ECEAC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_SOCWKD_DOMESTIC_Year_1_Applications": x,
    "term_3_SOCWKD_DOMESTIC_Year_1_Accepted": x,
    "term_3_SOCWKD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_SOCWKD_DOMESTIC_Year_1_Denied": x,
    "term_3_SOCWKD_DOMESTIC_Year_1_Declined": x,
    "term_3_SOCWKD_DOMESTIC_Year_1_Pending": x,
    "term_3_SOCWKD_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_SOCWKD_DOMESTIC_Year_1_Enrolled": x,
    "term_3_SOCWKD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_SOCWKD_INTERNATIONAL_Year_1_Applications": x,
    "term_3_SOCWKD_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_SOCWKD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_SOCWKD_INTERNATIONAL_Year_1_Denied": x,
    "term_3_SOCWKD_INTERNATIONAL_Year_1_Declined": x,
    "term_3_SOCWKD_INTERNATIONAL_Year_1_Pending": x,
    "term_3_SOCWKD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_SOCWKD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_SOCWKD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_SOCWKD_DOMESTIC_Year_2_Applications": x,
    "term_3_SOCWKD_DOMESTIC_Year_2_Accepted": x,
    "term_3_SOCWKD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_SOCWKD_DOMESTIC_Year_2_Denied": x,
    "term_3_SOCWKD_DOMESTIC_Year_2_Declined": x,
    "term_3_SOCWKD_DOMESTIC_Year_2_Pending": x,
    "term_3_SOCWKD_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_SOCWKD_DOMESTIC_Year_2_Enrolled": x,
    "term_3_SOCWKD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_SOCWKD_INTERNATIONAL_Year_2_Applications": x,
    "term_3_SOCWKD_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_SOCWKD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_SOCWKD_INTERNATIONAL_Year_2_Denied": x,
    "term_3_SOCWKD_INTERNATIONAL_Year_2_Declined": x,
    "term_3_SOCWKD_INTERNATIONAL_Year_2_Pending": x,
    "term_3_SOCWKD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_SOCWKD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_SOCWKD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_HCAC_DOMESTIC_Year_1_Applications": x,
    "term_3_HCAC_DOMESTIC_Year_1_Accepted": x,
    "term_3_HCAC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_HCAC_DOMESTIC_Year_1_Denied": x,
    "term_3_HCAC_DOMESTIC_Year_1_Declined": x,
    "term_3_HCAC_DOMESTIC_Year_1_Pending": x,
    "term_3_HCAC_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_HCAC_DOMESTIC_Year_1_Enrolled": x,
    "term_3_HCAC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_HCAC_INTERNATIONAL_Year_1_Applications": x,
    "term_3_HCAC_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_HCAC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_HCAC_INTERNATIONAL_Year_1_Denied": x,
    "term_3_HCAC_INTERNATIONAL_Year_1_Declined": x,
    "term_3_HCAC_INTERNATIONAL_Year_1_Pending": x,
    "term_3_HCAC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_HCAC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_HCAC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_PCPC_DOMESTIC_Year_1_Applications": x,
    "term_3_PCPC_DOMESTIC_Year_1_Accepted": x,
    "term_3_PCPC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_PCPC_DOMESTIC_Year_1_Denied": x,
    "term_3_PCPC_DOMESTIC_Year_1_Declined": x,
    "term_3_PCPC_DOMESTIC_Year_1_Pending": x,
    "term_3_PCPC_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_PCPC_DOMESTIC_Year_1_Enrolled": x,
    "term_3_PCPC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_PCPC_INTERNATIONAL_Year_1_Applications": x,
    "term_3_PCPC_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_PCPC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_PCPC_INTERNATIONAL_Year_1_Denied": x,
    "term_3_PCPC_INTERNATIONAL_Year_1_Declined": x,
    "term_3_PCPC_INTERNATIONAL_Year_1_Pending": x,
    "term_3_PCPC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_PCPC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_PCPC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_ACP_DOMESTIC_Year_1_Applications": x,
    "term_3_ACP_DOMESTIC_Year_1_Accepted": x,
    "term_3_ACP_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_ACP_DOMESTIC_Year_1_Denied": x,
    "term_3_ACP_DOMESTIC_Year_1_Declined": x,
    "term_3_ACP_DOMESTIC_Year_1_Pending": x,
    "term_3_ACP_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_ACP_DOMESTIC_Year_1_Enrolled": x,
    "term_3_ACP_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_ACP_INTERNATIONAL_Year_1_Applications": x,
    "term_3_ACP_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_ACP_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_ACP_INTERNATIONAL_Year_1_Denied": x,
    "term_3_ACP_INTERNATIONAL_Year_1_Declined": x,
    "term_3_ACP_INTERNATIONAL_Year_1_Pending": x,
    "term_3_ACP_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_ACP_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_ACP_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_ACP_DOMESTIC_Year_2_Applications": x,
    "term_3_ACP_DOMESTIC_Year_2_Accepted": x,
    "term_3_ACP_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_ACP_DOMESTIC_Year_2_Denied": x,
    "term_3_ACP_DOMESTIC_Year_2_Declined": x,
    "term_3_ACP_DOMESTIC_Year_2_Pending": x,
    "term_3_ACP_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_ACP_DOMESTIC_Year_2_Enrolled": x,
    "term_3_ACP_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_ACP_INTERNATIONAL_Year_2_Applications": x,
    "term_3_ACP_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_ACP_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_ACP_INTERNATIONAL_Year_2_Denied": x,
    "term_3_ACP_INTERNATIONAL_Year_2_Declined": x,
    "term_3_ACP_INTERNATIONAL_Year_2_Pending": x,
    "term_3_ACP_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_ACP_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_ACP_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_PNR_DOMESTIC_Year_1_Applications": x,
    "term_3_PNR_DOMESTIC_Year_1_Accepted": x,
    "term_3_PNR_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_PNR_DOMESTIC_Year_1_Denied": x,
    "term_3_PNR_DOMESTIC_Year_1_Declined": x,
    "term_3_PNR_DOMESTIC_Year_1_Pending": x,
    "term_3_PNR_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_PNR_DOMESTIC_Year_1_Enrolled": x,
    "term_3_PNR_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_PNR_INTERNATIONAL_Year_1_Applications": x,
    "term_3_PNR_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_PNR_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_PNR_INTERNATIONAL_Year_1_Denied": x,
    "term_3_PNR_INTERNATIONAL_Year_1_Declined": x,
    "term_3_PNR_INTERNATIONAL_Year_1_Pending": x,
    "term_3_PNR_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_PNR_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_PNR_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_PNR_DOMESTIC_Year_2_Applications": x,
    "term_3_PNR_DOMESTIC_Year_2_Accepted": x,
    "term_3_PNR_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_PNR_DOMESTIC_Year_2_Denied": x,
    "term_3_PNR_DOMESTIC_Year_2_Declined": x,
    "term_3_PNR_DOMESTIC_Year_2_Pending": x,
    "term_3_PNR_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_PNR_DOMESTIC_Year_2_Enrolled": x,
    "term_3_PNR_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_PNR_INTERNATIONAL_Year_2_Applications": x,
    "term_3_PNR_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_PNR_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_PNR_INTERNATIONAL_Year_2_Denied": x,
    "term_3_PNR_INTERNATIONAL_Year_2_Declined": x,
    "term_3_PNR_INTERNATIONAL_Year_2_Pending": x,
    "term_3_PNR_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_PNR_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_PNR_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_BSCN_DOMESTIC_Year_1_Applications": x,
    "term_3_BSCN_DOMESTIC_Year_1_Accepted": x,
    "term_3_BSCN_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_BSCN_DOMESTIC_Year_1_Denied": x,
    "term_3_BSCN_DOMESTIC_Year_1_Declined": x,
    "term_3_BSCN_DOMESTIC_Year_1_Pending": x,
    "term_3_BSCN_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_BSCN_DOMESTIC_Year_1_Enrolled": x,
    "term_3_BSCN_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_BSCN_INTERNATIONAL_Year_1_Applications": x,
    "term_3_BSCN_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_BSCN_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_BSCN_INTERNATIONAL_Year_1_Denied": x,
    "term_3_BSCN_INTERNATIONAL_Year_1_Declined": x,
    "term_3_BSCN_INTERNATIONAL_Year_1_Pending": x,
    "term_3_BSCN_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_BSCN_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_BSCN_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_BSCN_DOMESTIC_Year_2_Applications": x,
    "term_3_BSCN_DOMESTIC_Year_2_Accepted": x,
    "term_3_BSCN_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_BSCN_DOMESTIC_Year_2_Denied": x,
    "term_3_BSCN_DOMESTIC_Year_2_Declined": x,
    "term_3_BSCN_DOMESTIC_Year_2_Pending": x,
    "term_3_BSCN_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_BSCN_DOMESTIC_Year_2_Enrolled": x,
    "term_3_BSCN_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_BSCN_INTERNATIONAL_Year_2_Applications": x,
    "term_3_BSCN_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_BSCN_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_BSCN_INTERNATIONAL_Year_2_Denied": x,
    "term_3_BSCN_INTERNATIONAL_Year_2_Declined": x,
    "term_3_BSCN_INTERNATIONAL_Year_2_Pending": x,
    "term_3_BSCN_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_BSCN_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_BSCN_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_3_BSCN_DOMESTIC_Year_3_Applications": x,
    "term_3_BSCN_DOMESTIC_Year_3_Accepted": x,
    "term_3_BSCN_DOMESTIC_Year_3_Payment_Pending": x,
    "term_3_BSCN_DOMESTIC_Year_3_Denied": x,
    "term_3_BSCN_DOMESTIC_Year_3_Declined": x,
    "term_3_BSCN_DOMESTIC_Year_3_Pending": x,
    "term_3_BSCN_DOMESTIC_Year_3_Waitlisted": x,
    "term_3_BSCN_DOMESTIC_Year_3_Enrolled": x,
    "term_3_BSCN_DOMESTIC_Year_3_Conversion_Rate": x,
    "term_3_BSCN_INTERNATIONAL_Year_3_Applications": x,
    "term_3_BSCN_INTERNATIONAL_Year_3_Accepted": x,
    "term_3_BSCN_INTERNATIONAL_Year_3_Payment_Pending": x,
    "term_3_BSCN_INTERNATIONAL_Year_3_Denied": x,
    "term_3_BSCN_INTERNATIONAL_Year_3_Declined": x,
    "term_3_BSCN_INTERNATIONAL_Year_3_Pending": x,
    "term_3_BSCN_INTERNATIONAL_Year_3_Waitlisted": x,
    "term_3_BSCN_INTERNATIONAL_Year_3_Enrolled": x,
    "term_3_BSCN_INTERNATIONAL_Year_3_Conversion_Rate": x,
    "term_3_IENCN_DOMESTIC_Year_1_Applications": x,
    "term_3_IENCN_DOMESTIC_Year_1_Accepted": x,
    "term_3_IENCN_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_IENCN_DOMESTIC_Year_1_Denied": x,
    "term_3_IENCN_DOMESTIC_Year_1_Declined": x,
    "term_3_IENCN_DOMESTIC_Year_1_Pending": x,
    "term_3_IENCN_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_IENCN_DOMESTIC_Year_1_Enrolled": x,
    "term_3_IENCN_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_IENCN_INTERNATIONAL_Year_1_Applications": x,
    "term_3_IENCN_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_IENCN_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_IENCN_INTERNATIONAL_Year_1_Denied": x,
    "term_3_IENCN_INTERNATIONAL_Year_1_Declined": x,
    "term_3_IENCN_INTERNATIONAL_Year_1_Pending": x,
    "term_3_IENCN_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_IENCN_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_IENCN_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_PAELEC_DOMESTIC_Year_1_Applications": x,
    "term_3_PAELEC_DOMESTIC_Year_1_Accepted": x,
    "term_3_PAELEC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_PAELEC_DOMESTIC_Year_1_Denied": x,
    "term_3_PAELEC_DOMESTIC_Year_1_Declined": x,
    "term_3_PAELEC_DOMESTIC_Year_1_Pending": x,
    "term_3_PAELEC_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_PAELEC_DOMESTIC_Year_1_Enrolled": x,
    "term_3_PAELEC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_PAELEC_INTERNATIONAL_Year_1_Applications": x,
    "term_3_PAELEC_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_PAELEC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_PAELEC_INTERNATIONAL_Year_1_Denied": x,
    "term_3_PAELEC_INTERNATIONAL_Year_1_Declined": x,
    "term_3_PAELEC_INTERNATIONAL_Year_1_Pending": x,
    "term_3_PAELEC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_PAELEC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_PAELEC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_PAWELD_DOMESTIC_Year_1_Applications": x,
    "term_3_PAWELD_DOMESTIC_Year_1_Accepted": x,
    "term_3_PAWELD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_PAWELD_DOMESTIC_Year_1_Denied": x,
    "term_3_PAWELD_DOMESTIC_Year_1_Declined": x,
    "term_3_PAWELD_DOMESTIC_Year_1_Pending": x,
    "term_3_PAWELD_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_PAWELD_DOMESTIC_Year_1_Enrolled": x,
    "term_3_PAWELD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_PAWELD_INTERNATIONAL_Year_1_Applications": x,
    "term_3_PAWELD_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_PAWELD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_PAWELD_INTERNATIONAL_Year_1_Denied": x,
    "term_3_PAWELD_INTERNATIONAL_Year_1_Declined": x,
    "term_3_PAWELD_INTERNATIONAL_Year_1_Pending": x,
    "term_3_PAWELD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_PAWELD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_PAWELD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_PAHET_DOMESTIC_Year_1_Applications": x,
    "term_3_PAHET_DOMESTIC_Year_1_Accepted": x,
    "term_3_PAHET_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_PAHET_DOMESTIC_Year_1_Denied": x,
    "term_3_PAHET_DOMESTIC_Year_1_Declined": x,
    "term_3_PAHET_DOMESTIC_Year_1_Pending": x,
    "term_3_PAHET_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_PAHET_DOMESTIC_Year_1_Enrolled": x,
    "term_3_PAHET_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_PAHET_INTERNATIONAL_Year_1_Applications": x,
    "term_3_PAHET_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_PAHET_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_PAHET_INTERNATIONAL_Year_1_Denied": x,
    "term_3_PAHET_INTERNATIONAL_Year_1_Declined": x,
    "term_3_PAHET_INTERNATIONAL_Year_1_Pending": x,
    "term_3_PAHET_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_PAHET_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_PAHET_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_PAPLUM_DOMESTIC_Year_1_Applications": x,
    "term_3_PAPLUM_DOMESTIC_Year_1_Accepted": x,
    "term_3_PAPLUM_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_PAPLUM_DOMESTIC_Year_1_Denied": x,
    "term_3_PAPLUM_DOMESTIC_Year_1_Declined": x,
    "term_3_PAPLUM_DOMESTIC_Year_1_Pending": x,
    "term_3_PAPLUM_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_PAPLUM_DOMESTIC_Year_1_Enrolled": x,
    "term_3_PAPLUM_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_PAPLUM_INTERNATIONAL_Year_1_Applications": x,
    "term_3_PAPLUM_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_PAPLUM_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_PAPLUM_INTERNATIONAL_Year_1_Denied": x,
    "term_3_PAPLUM_INTERNATIONAL_Year_1_Declined": x,
    "term_3_PAPLUM_INTERNATIONAL_Year_1_Pending": x,
    "term_3_PAPLUM_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_PAPLUM_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_PAPLUM_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_POWCM4_DOMESTIC_Year_1_Applications": x,
    "term_3_POWCM4_DOMESTIC_Year_1_Accepted": x,
    "term_3_POWCM4_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_POWCM4_DOMESTIC_Year_1_Denied": x,
    "term_3_POWCM4_DOMESTIC_Year_1_Declined": x,
    "term_3_POWCM4_DOMESTIC_Year_1_Pending": x,
    "term_3_POWCM4_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_POWCM4_DOMESTIC_Year_1_Enrolled": x,
    "term_3_POWCM4_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_POWCM4_INTERNATIONAL_Year_1_Applications": x,
    "term_3_POWCM4_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_POWCM4_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_POWCM4_INTERNATIONAL_Year_1_Denied": x,
    "term_3_POWCM4_INTERNATIONAL_Year_1_Declined": x,
    "term_3_POWCM4_INTERNATIONAL_Year_1_Pending": x,
    "term_3_POWCM4_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_POWCM4_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_POWCM4_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_POWCM3_DOMESTIC_Year_1_Applications": x,
    "term_3_POWCM3_DOMESTIC_Year_1_Accepted": x,
    "term_3_POWCM3_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_POWCM3_DOMESTIC_Year_1_Denied": x,
    "term_3_POWCM3_DOMESTIC_Year_1_Declined": x,
    "term_3_POWCM3_DOMESTIC_Year_1_Pending": x,
    "term_3_POWCM3_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_POWCM3_DOMESTIC_Year_1_Enrolled": x,
    "term_3_POWCM3_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_POWCM3_INTERNATIONAL_Year_1_Applications": x,
    "term_3_POWCM3_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_POWCM3_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_POWCM3_INTERNATIONAL_Year_1_Denied": x,
    "term_3_POWCM3_INTERNATIONAL_Year_1_Declined": x,
    "term_3_POWCM3_INTERNATIONAL_Year_1_Pending": x,
    "term_3_POWCM3_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_POWCM3_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_POWCM3_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_POWCP4_DOMESTIC_Year_1_Applications": x,
    "term_3_POWCP4_DOMESTIC_Year_1_Accepted": x,
    "term_3_POWCP4_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_POWCP4_DOMESTIC_Year_1_Denied": x,
    "term_3_POWCP4_DOMESTIC_Year_1_Declined": x,
    "term_3_POWCP4_DOMESTIC_Year_1_Pending": x,
    "term_3_POWCP4_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_POWCP4_DOMESTIC_Year_1_Enrolled": x,
    "term_3_POWCP4_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_POWCP4_INTERNATIONAL_Year_1_Applications": x,
    "term_3_POWCP4_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_POWCP4_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_POWCP4_INTERNATIONAL_Year_1_Denied": x,
    "term_3_POWCP4_INTERNATIONAL_Year_1_Declined": x,
    "term_3_POWCP4_INTERNATIONAL_Year_1_Pending": x,
    "term_3_POWCP4_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_POWCP4_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_POWCP4_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_POWCO4_DOMESTIC_Year_1_Applications": x,
    "term_3_POWCO4_DOMESTIC_Year_1_Accepted": x,
    "term_3_POWCO4_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_POWCO4_DOMESTIC_Year_1_Denied": x,
    "term_3_POWCO4_DOMESTIC_Year_1_Declined": x,
    "term_3_POWCO4_DOMESTIC_Year_1_Pending": x,
    "term_3_POWCO4_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_POWCO4_DOMESTIC_Year_1_Enrolled": x,
    "term_3_POWCO4_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_POWCO4_INTERNATIONAL_Year_1_Applications": x,
    "term_3_POWCO4_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_POWCO4_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_POWCO4_INTERNATIONAL_Year_1_Denied": x,
    "term_3_POWCO4_INTERNATIONAL_Year_1_Declined": x,
    "term_3_POWCO4_INTERNATIONAL_Year_1_Pending": x,
    "term_3_POWCO4_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_POWCO4_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_POWCO4_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_POWCO3_DOMESTIC_Year_1_Applications": x,
    "term_3_POWCO3_DOMESTIC_Year_1_Accepted": x,
    "term_3_POWCO3_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_POWCO3_DOMESTIC_Year_1_Denied": x,
    "term_3_POWCO3_DOMESTIC_Year_1_Declined": x,
    "term_3_POWCO3_DOMESTIC_Year_1_Pending": x,
    "term_3_POWCO3_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_POWCO3_DOMESTIC_Year_1_Enrolled": x,
    "term_3_POWCO3_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_POWCO3_INTERNATIONAL_Year_1_Applications": x,
    "term_3_POWCO3_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_POWCO3_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_POWCO3_INTERNATIONAL_Year_1_Denied": x,
    "term_3_POWCO3_INTERNATIONAL_Year_1_Declined": x,
    "term_3_POWCO3_INTERNATIONAL_Year_1_Pending": x,
    "term_3_POWCO3_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_POWCO3_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_POWCO3_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_PPET_DOMESTIC_Year_1_Applications": x,
    "term_3_PPET_DOMESTIC_Year_1_Accepted": x,
    "term_3_PPET_DOMESTIC_Year_1_Payment_Pending": x,
    "term_3_PPET_DOMESTIC_Year_1_Denied": x,
    "term_3_PPET_DOMESTIC_Year_1_Declined": x,
    "term_3_PPET_DOMESTIC_Year_1_Pending": x,
    "term_3_PPET_DOMESTIC_Year_1_Waitlisted": x,
    "term_3_PPET_DOMESTIC_Year_1_Enrolled": x,
    "term_3_PPET_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_3_PPET_INTERNATIONAL_Year_1_Applications": x,
    "term_3_PPET_INTERNATIONAL_Year_1_Accepted": x,
    "term_3_PPET_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_3_PPET_INTERNATIONAL_Year_1_Denied": x,
    "term_3_PPET_INTERNATIONAL_Year_1_Declined": x,
    "term_3_PPET_INTERNATIONAL_Year_1_Pending": x,
    "term_3_PPET_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_3_PPET_INTERNATIONAL_Year_1_Enrolled": x,
    "term_3_PPET_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_3_PPET_DOMESTIC_Year_2_Applications": x,
    "term_3_PPET_DOMESTIC_Year_2_Accepted": x,
    "term_3_PPET_DOMESTIC_Year_2_Payment_Pending": x,
    "term_3_PPET_DOMESTIC_Year_2_Denied": x,
    "term_3_PPET_DOMESTIC_Year_2_Declined": x,
    "term_3_PPET_DOMESTIC_Year_2_Pending": x,
    "term_3_PPET_DOMESTIC_Year_2_Waitlisted": x,
    "term_3_PPET_DOMESTIC_Year_2_Enrolled": x,
    "term_3_PPET_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_3_PPET_INTERNATIONAL_Year_2_Applications": x,
    "term_3_PPET_INTERNATIONAL_Year_2_Accepted": x,
    "term_3_PPET_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_3_PPET_INTERNATIONAL_Year_2_Denied": x,
    "term_3_PPET_INTERNATIONAL_Year_2_Declined": x,
    "term_3_PPET_INTERNATIONAL_Year_2_Pending": x,
    "term_3_PPET_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_3_PPET_INTERNATIONAL_Year_2_Enrolled": x,
    "term_3_PPET_INTERNATIONAL_Year_2_Conversion_Rate": x,

    #Term4Yr YYYY Apps By School sheet
    "term_4_BABUSC_DOMESTIC_Year_1_Applications": x,
    "term_4_BABUSC_DOMESTIC_Year_1_Accepted": x,
    "term_4_BABUSC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_BABUSC_DOMESTIC_Year_1_Denied": x,
    "term_4_BABUSC_DOMESTIC_Year_1_Declined": x,
    "term_4_BABUSC_DOMESTIC_Year_1_Pending": x,
    "term_4_BABUSC_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_BABUSC_DOMESTIC_Year_1_Enrolled": x,
    "term_4_BABUSC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_BABUSC_INTERNATIONAL_Year_1_Applications": x,
    "term_4_BABUSC_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_BABUSC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_BABUSC_INTERNATIONAL_Year_1_Denied": x,
    "term_4_BABUSC_INTERNATIONAL_Year_1_Declined": x,
    "term_4_BABUSC_INTERNATIONAL_Year_1_Pending": x,
    "term_4_BABUSC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_BABUSC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_BABUSC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_BAACTD_DOMESTIC_Year_1_Applications": x,
    "term_4_BAACTD_DOMESTIC_Year_1_Accepted": x,
    "term_4_BAACTD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_BAACTD_DOMESTIC_Year_1_Denied": x,
    "term_4_BAACTD_DOMESTIC_Year_1_Declined": x,
    "term_4_BAACTD_DOMESTIC_Year_1_Pending": x,
    "term_4_BAACTD_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_BAACTD_DOMESTIC_Year_1_Enrolled": x,
    "term_4_BAACTD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_BAACTD_INTERNATIONAL_Year_1_Applications": x,
    "term_4_BAACTD_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_BAACTD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_BAACTD_INTERNATIONAL_Year_1_Denied": x,
    "term_4_BAACTD_INTERNATIONAL_Year_1_Declined": x,
    "term_4_BAACTD_INTERNATIONAL_Year_1_Pending": x,
    "term_4_BAACTD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_BAACTD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_BAACTD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_BAACTD_DOMESTIC_Year_2_Applications": x,
    "term_4_BAACTD_DOMESTIC_Year_2_Accepted": x,
    "term_4_BAACTD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_BAACTD_DOMESTIC_Year_2_Denied": x,
    "term_4_BAACTD_DOMESTIC_Year_2_Declined": x,
    "term_4_BAACTD_DOMESTIC_Year_2_Pending": x,
    "term_4_BAACTD_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_BAACTD_DOMESTIC_Year_2_Enrolled": x,
    "term_4_BAACTD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_BAACTD_INTERNATIONAL_Year_2_Applications": x,
    "term_4_BAACTD_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_BAACTD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_BAACTD_INTERNATIONAL_Year_2_Denied": x,
    "term_4_BAACTD_INTERNATIONAL_Year_2_Declined": x,
    "term_4_BAACTD_INTERNATIONAL_Year_2_Pending": x,
    "term_4_BAACTD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_BAACTD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_BAACTD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_BABUSD_DOMESTIC_Year_1_Applications": x,
    "term_4_BABUSD_DOMESTIC_Year_1_Accepted": x,
    "term_4_BABUSD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_BABUSD_DOMESTIC_Year_1_Denied": x,
    "term_4_BABUSD_DOMESTIC_Year_1_Declined": x,
    "term_4_BABUSD_DOMESTIC_Year_1_Pending": x,
    "term_4_BABUSD_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_BABUSD_DOMESTIC_Year_1_Enrolled": x,
    "term_4_BABUSD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_BABUSD_INTERNATIONAL_Year_1_Applications": x,
    "term_4_BABUSD_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_BABUSD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_BABUSD_INTERNATIONAL_Year_1_Denied": x,
    "term_4_BABUSD_INTERNATIONAL_Year_1_Declined": x,
    "term_4_BABUSD_INTERNATIONAL_Year_1_Pending": x,
    "term_4_BABUSD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_BABUSD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_BABUSD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_BABUSD_DOMESTIC_Year_2_Applications": x,
    "term_4_BABUSD_DOMESTIC_Year_2_Accepted": x,
    "term_4_BABUSD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_BABUSD_DOMESTIC_Year_2_Denied": x,
    "term_4_BABUSD_DOMESTIC_Year_2_Declined": x,
    "term_4_BABUSD_DOMESTIC_Year_2_Pending": x,
    "term_4_BABUSD_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_BABUSD_DOMESTIC_Year_2_Enrolled": x,
    "term_4_BABUSD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_BABUSD_INTERNATIONAL_Year_2_Applications": x,
    "term_4_BABUSD_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_BABUSD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_BABUSD_INTERNATIONAL_Year_2_Denied": x,
    "term_4_BABUSD_INTERNATIONAL_Year_2_Declined": x,
    "term_4_BABUSD_INTERNATIONAL_Year_2_Pending": x,
    "term_4_BABUSD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_BABUSD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_BABUSD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_BAESPM_DOMESTIC_Year_1_Applications": x,
    "term_4_BAESPM_DOMESTIC_Year_1_Accepted": x,
    "term_4_BAESPM_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_BAESPM_DOMESTIC_Year_1_Denied": x,
    "term_4_BAESPM_DOMESTIC_Year_1_Declined": x,
    "term_4_BAESPM_DOMESTIC_Year_1_Pending": x,
    "term_4_BAESPM_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_BAESPM_DOMESTIC_Year_1_Enrolled": x,
    "term_4_BAESPM_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_BAESPM_INTERNATIONAL_Year_1_Applications": x,
    "term_4_BAESPM_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_BAESPM_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_BAESPM_INTERNATIONAL_Year_1_Denied": x,
    "term_4_BAESPM_INTERNATIONAL_Year_1_Declined": x,
    "term_4_BAESPM_INTERNATIONAL_Year_1_Pending": x,
    "term_4_BAESPM_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_BAESPM_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_BAESPM_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_BAESPM_DOMESTIC_Year_2_Applications": x,
    "term_4_BAESPM_DOMESTIC_Year_2_Accepted": x,
    "term_4_BAESPM_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_BAESPM_DOMESTIC_Year_2_Denied": x,
    "term_4_BAESPM_DOMESTIC_Year_2_Declined": x,
    "term_4_BAESPM_DOMESTIC_Year_2_Pending": x,
    "term_4_BAESPM_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_BAESPM_DOMESTIC_Year_2_Enrolled": x,
    "term_4_BAESPM_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_BAESPM_INTERNATIONAL_Year_2_Applications": x,
    "term_4_BAESPM_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_BAESPM_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_BAESPM_INTERNATIONAL_Year_2_Denied": x,
    "term_4_BAESPM_INTERNATIONAL_Year_2_Declined": x,
    "term_4_BAESPM_INTERNATIONAL_Year_2_Pending": x,
    "term_4_BAESPM_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_BAESPM_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_BAESPM_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_BAHRMD_DOMESTIC_Year_1_Applications": x,
    "term_4_BAHRMD_DOMESTIC_Year_1_Accepted": x,
    "term_4_BAHRMD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_BAHRMD_DOMESTIC_Year_1_Denied": x,
    "term_4_BAHRMD_DOMESTIC_Year_1_Declined": x,
    "term_4_BAHRMD_DOMESTIC_Year_1_Pending": x,
    "term_4_BAHRMD_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_BAHRMD_DOMESTIC_Year_1_Enrolled": x,
    "term_4_BAHRMD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_BAHRMD_INTERNATIONAL_Year_1_Applications": x,
    "term_4_BAHRMD_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_BAHRMD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_BAHRMD_INTERNATIONAL_Year_1_Denied": x,
    "term_4_BAHRMD_INTERNATIONAL_Year_1_Declined": x,
    "term_4_BAHRMD_INTERNATIONAL_Year_1_Pending": x,
    "term_4_BAHRMD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_BAHRMD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_BAHRMD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_BAHRMD_DOMESTIC_Year_2_Applications": x,
    "term_4_BAHRMD_DOMESTIC_Year_2_Accepted": x,
    "term_4_BAHRMD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_BAHRMD_DOMESTIC_Year_2_Denied": x,
    "term_4_BAHRMD_DOMESTIC_Year_2_Declined": x,
    "term_4_BAHRMD_DOMESTIC_Year_2_Pending": x,
    "term_4_BAHRMD_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_BAHRMD_DOMESTIC_Year_2_Enrolled": x,
    "term_4_BAHRMD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_BAHRMD_INTERNATIONAL_Year_2_Applications": x,
    "term_4_BAHRMD_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_BAHRMD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_BAHRMD_INTERNATIONAL_Year_2_Denied": x,
    "term_4_BAHRMD_INTERNATIONAL_Year_2_Declined": x,
    "term_4_BAHRMD_INTERNATIONAL_Year_2_Pending": x,
    "term_4_BAHRMD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_BAHRMD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_BAHRMD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_GSCMD_DOMESTIC_Year_1_Applications": x,
    "term_4_GSCMD_DOMESTIC_Year_1_Accepted": x,
    "term_4_GSCMD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_GSCMD_DOMESTIC_Year_1_Denied": x,
    "term_4_GSCMD_DOMESTIC_Year_1_Declined": x,
    "term_4_GSCMD_DOMESTIC_Year_1_Pending": x,
    "term_4_GSCMD_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_GSCMD_DOMESTIC_Year_1_Enrolled": x,
    "term_4_GSCMD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_GSCMD_INTERNATIONAL_Year_1_Applications": x,
    "term_4_GSCMD_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_GSCMD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_GSCMD_INTERNATIONAL_Year_1_Denied": x,
    "term_4_GSCMD_INTERNATIONAL_Year_1_Declined": x,
    "term_4_GSCMD_INTERNATIONAL_Year_1_Pending": x,
    "term_4_GSCMD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_GSCMD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_GSCMD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_GSCMD_DOMESTIC_Year_2_Applications": x,
    "term_4_GSCMD_DOMESTIC_Year_2_Accepted": x,
    "term_4_GSCMD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_GSCMD_DOMESTIC_Year_2_Denied": x,
    "term_4_GSCMD_DOMESTIC_Year_2_Declined": x,
    "term_4_GSCMD_DOMESTIC_Year_2_Pending": x,
    "term_4_GSCMD_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_GSCMD_DOMESTIC_Year_2_Enrolled": x,
    "term_4_GSCMD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_GSCMD_INTERNATIONAL_Year_2_Applications": x,
    "term_4_GSCMD_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_GSCMD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_GSCMD_INTERNATIONAL_Year_2_Denied": x,
    "term_4_GSCMD_INTERNATIONAL_Year_2_Declined": x,
    "term_4_GSCMD_INTERNATIONAL_Year_2_Pending": x,
    "term_4_GSCMD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_GSCMD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_GSCMD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_DIGMD_DOMESTIC_Year_1_Applications": x,
    "term_4_DIGMD_DOMESTIC_Year_1_Accepted": x,
    "term_4_DIGMD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_DIGMD_DOMESTIC_Year_1_Denied": x,
    "term_4_DIGMD_DOMESTIC_Year_1_Declined": x,
    "term_4_DIGMD_DOMESTIC_Year_1_Pending": x,
    "term_4_DIGMD_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_DIGMD_DOMESTIC_Year_1_Enrolled": x,
    "term_4_DIGMD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_DIGMD_INTERNATIONAL_Year_1_Applications": x,
    "term_4_DIGMD_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_DIGMD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_DIGMD_INTERNATIONAL_Year_1_Denied": x,
    "term_4_DIGMD_INTERNATIONAL_Year_1_Declined": x,
    "term_4_DIGMD_INTERNATIONAL_Year_1_Pending": x,
    "term_4_DIGMD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_DIGMD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_DIGMD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_DIGMD_DOMESTIC_Year_2_Applications": x,
    "term_4_DIGMD_DOMESTIC_Year_2_Accepted": x,
    "term_4_DIGMD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_DIGMD_DOMESTIC_Year_2_Denied": x,
    "term_4_DIGMD_DOMESTIC_Year_2_Declined": x,
    "term_4_DIGMD_DOMESTIC_Year_2_Pending": x,
    "term_4_DIGMD_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_DIGMD_DOMESTIC_Year_2_Enrolled": x,
    "term_4_DIGMD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_DIGMD_INTERNATIONAL_Year_2_Applications": x,
    "term_4_DIGMD_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_DIGMD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_DIGMD_INTERNATIONAL_Year_2_Denied": x,
    "term_4_DIGMD_INTERNATIONAL_Year_2_Declined": x,
    "term_4_DIGMD_INTERNATIONAL_Year_2_Pending": x,
    "term_4_DIGMD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_DIGMD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_DIGMD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_BAATCO_DOMESTIC_Year_1_Applications": x,
    "term_4_BAATCO_DOMESTIC_Year_1_Accepted": x,
    "term_4_BAATCO_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_BAATCO_DOMESTIC_Year_1_Denied": x,
    "term_4_BAATCO_DOMESTIC_Year_1_Declined": x,
    "term_4_BAATCO_DOMESTIC_Year_1_Pending": x,
    "term_4_BAATCO_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_BAATCO_DOMESTIC_Year_1_Enrolled": x,
    "term_4_BAATCO_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_BAATCO_INTERNATIONAL_Year_1_Applications": x,
    "term_4_BAATCO_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_BAATCO_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_BAATCO_INTERNATIONAL_Year_1_Denied": x,
    "term_4_BAATCO_INTERNATIONAL_Year_1_Declined": x,
    "term_4_BAATCO_INTERNATIONAL_Year_1_Pending": x,
    "term_4_BAATCO_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_BAATCO_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_BAATCO_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_BAATCO_DOMESTIC_Year_2_Applications": x,
    "term_4_BAATCO_DOMESTIC_Year_2_Accepted": x,
    "term_4_BAATCO_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_BAATCO_DOMESTIC_Year_2_Denied": x,
    "term_4_BAATCO_DOMESTIC_Year_2_Declined": x,
    "term_4_BAATCO_DOMESTIC_Year_2_Pending": x,
    "term_4_BAATCO_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_BAATCO_DOMESTIC_Year_2_Enrolled": x,
    "term_4_BAATCO_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_BAATCO_INTERNATIONAL_Year_2_Applications": x,
    "term_4_BAATCO_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_BAATCO_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_BAATCO_INTERNATIONAL_Year_2_Denied": x,
    "term_4_BAATCO_INTERNATIONAL_Year_2_Declined": x,
    "term_4_BAATCO_INTERNATIONAL_Year_2_Pending": x,
    "term_4_BAATCO_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_BAATCO_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_BAATCO_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_BABSCO_DOMESTIC_Year_1_Applications": x,
    "term_4_BABSCO_DOMESTIC_Year_1_Accepted": x,
    "term_4_BABSCO_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_BABSCO_DOMESTIC_Year_1_Denied": x,
    "term_4_BABSCO_DOMESTIC_Year_1_Declined": x,
    "term_4_BABSCO_DOMESTIC_Year_1_Pending": x,
    "term_4_BABSCO_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_BABSCO_DOMESTIC_Year_1_Enrolled": x,
    "term_4_BABSCO_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_BABSCO_INTERNATIONAL_Year_1_Applications": x,
    "term_4_BABSCO_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_BABSCO_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_BABSCO_INTERNATIONAL_Year_1_Denied": x,
    "term_4_BABSCO_INTERNATIONAL_Year_1_Declined": x,
    "term_4_BABSCO_INTERNATIONAL_Year_1_Pending": x,
    "term_4_BABSCO_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_BABSCO_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_BABSCO_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_BABSCO_DOMESTIC_Year_2_Applications": x,
    "term_4_BABSCO_DOMESTIC_Year_2_Accepted": x,
    "term_4_BABSCO_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_BABSCO_DOMESTIC_Year_2_Denied": x,
    "term_4_BABSCO_DOMESTIC_Year_2_Declined": x,
    "term_4_BABSCO_DOMESTIC_Year_2_Pending": x,
    "term_4_BABSCO_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_BABSCO_DOMESTIC_Year_2_Enrolled": x,
    "term_4_BABSCO_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_BABSCO_INTERNATIONAL_Year_2_Applications": x,
    "term_4_BABSCO_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_BABSCO_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_BABSCO_INTERNATIONAL_Year_2_Denied": x,
    "term_4_BABSCO_INTERNATIONAL_Year_2_Declined": x,
    "term_4_BABSCO_INTERNATIONAL_Year_2_Pending": x,
    "term_4_BABSCO_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_BABSCO_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_BABSCO_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_NAO_DOMESTIC_Year_1_Applications": x,
    "term_4_NAO_DOMESTIC_Year_1_Accepted": x,
    "term_4_NAO_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_NAO_DOMESTIC_Year_1_Denied": x,
    "term_4_NAO_DOMESTIC_Year_1_Declined": x,
    "term_4_NAO_DOMESTIC_Year_1_Pending": x,
    "term_4_NAO_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_NAO_DOMESTIC_Year_1_Enrolled": x,
    "term_4_NAO_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_NAO_INTERNATIONAL_Year_1_Applications": x,
    "term_4_NAO_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_NAO_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_NAO_INTERNATIONAL_Year_1_Denied": x,
    "term_4_NAO_INTERNATIONAL_Year_1_Declined": x,
    "term_4_NAO_INTERNATIONAL_Year_1_Pending": x,
    "term_4_NAO_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_NAO_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_NAO_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_NAO_DOMESTIC_Year_2_Applications": x,
    "term_4_NAO_DOMESTIC_Year_2_Accepted": x,
    "term_4_NAO_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_NAO_DOMESTIC_Year_2_Denied": x,
    "term_4_NAO_DOMESTIC_Year_2_Declined": x,
    "term_4_NAO_DOMESTIC_Year_2_Pending": x,
    "term_4_NAO_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_NAO_DOMESTIC_Year_2_Enrolled": x,
    "term_4_NAO_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_NAO_INTERNATIONAL_Year_2_Applications": x,
    "term_4_NAO_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_NAO_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_NAO_INTERNATIONAL_Year_2_Denied": x,
    "term_4_NAO_INTERNATIONAL_Year_2_Declined": x,
    "term_4_NAO_INTERNATIONAL_Year_2_Pending": x,
    "term_4_NAO_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_NAO_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_NAO_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_ALO_DOMESTIC_Year_1_Applications": x,
    "term_4_ALO_DOMESTIC_Year_1_Accepted": x,
    "term_4_ALO_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_ALO_DOMESTIC_Year_1_Denied": x,
    "term_4_ALO_DOMESTIC_Year_1_Declined": x,
    "term_4_ALO_DOMESTIC_Year_1_Pending": x,
    "term_4_ALO_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_ALO_DOMESTIC_Year_1_Enrolled": x,
    "term_4_ALO_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_ALO_INTERNATIONAL_Year_1_Applications": x,
    "term_4_ALO_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_ALO_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_ALO_INTERNATIONAL_Year_1_Denied": x,
    "term_4_ALO_INTERNATIONAL_Year_1_Declined": x,
    "term_4_ALO_INTERNATIONAL_Year_1_Pending": x,
    "term_4_ALO_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_ALO_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_ALO_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_ALO_DOMESTIC_Year_2_Applications": x,
    "term_4_ALO_DOMESTIC_Year_2_Accepted": x,
    "term_4_ALO_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_ALO_DOMESTIC_Year_2_Denied": x,
    "term_4_ALO_DOMESTIC_Year_2_Declined": x,
    "term_4_ALO_DOMESTIC_Year_2_Pending": x,
    "term_4_ALO_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_ALO_DOMESTIC_Year_2_Enrolled": x,
    "term_4_ALO_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_ALO_INTERNATIONAL_Year_2_Applications": x,
    "term_4_ALO_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_ALO_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_ALO_INTERNATIONAL_Year_2_Denied": x,
    "term_4_ALO_INTERNATIONAL_Year_2_Declined": x,
    "term_4_ALO_INTERNATIONAL_Year_2_Pending": x,
    "term_4_ALO_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_ALO_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_ALO_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_HOSMNG_DOMESTIC_Year_1_Applications": x,
    "term_4_HOSMNG_DOMESTIC_Year_1_Accepted": x,
    "term_4_HOSMNG_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_HOSMNG_DOMESTIC_Year_1_Denied": x,
    "term_4_HOSMNG_DOMESTIC_Year_1_Declined": x,
    "term_4_HOSMNG_DOMESTIC_Year_1_Pending": x,
    "term_4_HOSMNG_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_HOSMNG_DOMESTIC_Year_1_Enrolled": x,
    "term_4_HOSMNG_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_HOSMNG_INTERNATIONAL_Year_1_Applications": x,
    "term_4_HOSMNG_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_HOSMNG_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_HOSMNG_INTERNATIONAL_Year_1_Denied": x,
    "term_4_HOSMNG_INTERNATIONAL_Year_1_Declined": x,
    "term_4_HOSMNG_INTERNATIONAL_Year_1_Pending": x,
    "term_4_HOSMNG_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_HOSMNG_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_HOSMNG_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_HOSMNG_DOMESTIC_Year_2_Applications": x,
    "term_4_HOSMNG_DOMESTIC_Year_2_Accepted": x,
    "term_4_HOSMNG_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_HOSMNG_DOMESTIC_Year_2_Denied": x,
    "term_4_HOSMNG_DOMESTIC_Year_2_Declined": x,
    "term_4_HOSMNG_DOMESTIC_Year_2_Pending": x,
    "term_4_HOSMNG_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_HOSMNG_DOMESTIC_Year_2_Enrolled": x,
    "term_4_HOSMNG_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_HOSMNG_INTERNATIONAL_Year_2_Applications": x,
    "term_4_HOSMNG_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_HOSMNG_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_HOSMNG_INTERNATIONAL_Year_2_Denied": x,
    "term_4_HOSMNG_INTERNATIONAL_Year_2_Declined": x,
    "term_4_HOSMNG_INTERNATIONAL_Year_2_Pending": x,
    "term_4_HOSMNG_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_HOSMNG_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_HOSMNG_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_BAHRMC_DOMESTIC_Year_1_Applications": x,
    "term_4_BAHRMC_DOMESTIC_Year_1_Accepted": x,
    "term_4_BAHRMC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_BAHRMC_DOMESTIC_Year_1_Denied": x,
    "term_4_BAHRMC_DOMESTIC_Year_1_Declined": x,
    "term_4_BAHRMC_DOMESTIC_Year_1_Pending": x,
    "term_4_BAHRMC_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_BAHRMC_DOMESTIC_Year_1_Enrolled": x,
    "term_4_BAHRMC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_BAHRMC_INTERNATIONAL_Year_1_Applications": x,
    "term_4_BAHRMC_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_BAHRMC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_BAHRMC_INTERNATIONAL_Year_1_Denied": x,
    "term_4_BAHRMC_INTERNATIONAL_Year_1_Declined": x,
    "term_4_BAHRMC_INTERNATIONAL_Year_1_Pending": x,
    "term_4_BAHRMC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_BAHRMC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_BAHRMC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_OFAC_DOMESTIC_Year_1_Applications": x,
    "term_4_OFAC_DOMESTIC_Year_1_Accepted": x,
    "term_4_OFAC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_OFAC_DOMESTIC_Year_1_Denied": x,
    "term_4_OFAC_DOMESTIC_Year_1_Declined": x,
    "term_4_OFAC_DOMESTIC_Year_1_Pending": x,
    "term_4_OFAC_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_OFAC_DOMESTIC_Year_1_Enrolled": x,
    "term_4_OFAC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_OFAC_INTERNATIONAL_Year_1_Applications": x,
    "term_4_OFAC_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_OFAC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_OFAC_INTERNATIONAL_Year_1_Denied": x,
    "term_4_OFAC_INTERNATIONAL_Year_1_Declined": x,
    "term_4_OFAC_INTERNATIONAL_Year_1_Pending": x,
    "term_4_OFAC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_OFAC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_OFAC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_EAP_DOMESTIC_Year_1_Applications": x,
    "term_4_EAP_DOMESTIC_Year_1_Accepted": x,
    "term_4_EAP_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_EAP_DOMESTIC_Year_1_Denied": x,
    "term_4_EAP_DOMESTIC_Year_1_Declined": x,
    "term_4_EAP_DOMESTIC_Year_1_Pending": x,
    "term_4_EAP_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_EAP_DOMESTIC_Year_1_Enrolled": x,
    "term_4_EAP_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_EAP_INTERNATIONAL_Year_1_Applications": x,
    "term_4_EAP_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_EAP_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_EAP_INTERNATIONAL_Year_1_Denied": x,
    "term_4_EAP_INTERNATIONAL_Year_1_Declined": x,
    "term_4_EAP_INTERNATIONAL_Year_1_Pending": x,
    "term_4_EAP_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_EAP_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_EAP_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_HSAFA_DOMESTIC_Year_1_Applications": x,
    "term_4_HSAFA_DOMESTIC_Year_1_Accepted": x,
    "term_4_HSAFA_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_HSAFA_DOMESTIC_Year_1_Denied": x,
    "term_4_HSAFA_DOMESTIC_Year_1_Declined": x,
    "term_4_HSAFA_DOMESTIC_Year_1_Pending": x,
    "term_4_HSAFA_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_HSAFA_DOMESTIC_Year_1_Enrolled": x,
    "term_4_HSAFA_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_HSAFA_INTERNATIONAL_Year_1_Applications": x,
    "term_4_HSAFA_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_HSAFA_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_HSAFA_INTERNATIONAL_Year_1_Denied": x,
    "term_4_HSAFA_INTERNATIONAL_Year_1_Declined": x,
    "term_4_HSAFA_INTERNATIONAL_Year_1_Pending": x,
    "term_4_HSAFA_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_HSAFA_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_HSAFA_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_HSAD_DOMESTIC_Year_1_Applications": x,
    "term_4_HSAD_DOMESTIC_Year_1_Accepted": x,
    "term_4_HSAD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_HSAD_DOMESTIC_Year_1_Denied": x,
    "term_4_HSAD_DOMESTIC_Year_1_Declined": x,
    "term_4_HSAD_DOMESTIC_Year_1_Pending": x,
    "term_4_HSAD_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_HSAD_DOMESTIC_Year_1_Enrolled": x,
    "term_4_HSAD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_HSAD_INTERNATIONAL_Year_1_Applications": x,
    "term_4_HSAD_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_HSAD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_HSAD_INTERNATIONAL_Year_1_Denied": x,
    "term_4_HSAD_INTERNATIONAL_Year_1_Declined": x,
    "term_4_HSAD_INTERNATIONAL_Year_1_Pending": x,
    "term_4_HSAD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_HSAD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_HSAD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_HSGD_DOMESTIC_Year_1_Applications": x,
    "term_4_HSGD_DOMESTIC_Year_1_Accepted": x,
    "term_4_HSGD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_HSGD_DOMESTIC_Year_1_Denied": x,
    "term_4_HSGD_DOMESTIC_Year_1_Declined": x,
    "term_4_HSGD_DOMESTIC_Year_1_Pending": x,
    "term_4_HSGD_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_HSGD_DOMESTIC_Year_1_Enrolled": x,
    "term_4_HSGD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_HSGD_INTERNATIONAL_Year_1_Applications": x,
    "term_4_HSGD_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_HSGD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_HSGD_INTERNATIONAL_Year_1_Denied": x,
    "term_4_HSGD_INTERNATIONAL_Year_1_Declined": x,
    "term_4_HSGD_INTERNATIONAL_Year_1_Pending": x,
    "term_4_HSGD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_HSGD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_HSGD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_HSNP_DOMESTIC_Year_1_Applications": x,
    "term_4_HSNP_DOMESTIC_Year_1_Accepted": x,
    "term_4_HSNP_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_HSNP_DOMESTIC_Year_1_Denied": x,
    "term_4_HSNP_DOMESTIC_Year_1_Declined": x,
    "term_4_HSNP_DOMESTIC_Year_1_Pending": x,
    "term_4_HSNP_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_HSNP_DOMESTIC_Year_1_Enrolled": x,
    "term_4_HSNP_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_HSNP_INTERNATIONAL_Year_1_Applications": x,
    "term_4_HSNP_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_HSNP_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_HSNP_INTERNATIONAL_Year_1_Denied": x,
    "term_4_HSNP_INTERNATIONAL_Year_1_Declined": x,
    "term_4_HSNP_INTERNATIONAL_Year_1_Pending": x,
    "term_4_HSNP_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_HSNP_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_HSNP_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_UTELEM_DOMESTIC_Year_1_Applications": x,
    "term_4_UTELEM_DOMESTIC_Year_1_Accepted": x,
    "term_4_UTELEM_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_UTELEM_DOMESTIC_Year_1_Denied": x,
    "term_4_UTELEM_DOMESTIC_Year_1_Declined": x,
    "term_4_UTELEM_DOMESTIC_Year_1_Pending": x,
    "term_4_UTELEM_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_UTELEM_DOMESTIC_Year_1_Enrolled": x,
    "term_4_UTELEM_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_UTELEM_INTERNATIONAL_Year_1_Applications": x,
    "term_4_UTELEM_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_UTELEM_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_UTELEM_INTERNATIONAL_Year_1_Denied": x,
    "term_4_UTELEM_INTERNATIONAL_Year_1_Declined": x,
    "term_4_UTELEM_INTERNATIONAL_Year_1_Pending": x,
    "term_4_UTELEM_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_UTELEM_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_UTELEM_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_UTELEM_DOMESTIC_Year_2_Applications": x,
    "term_4_UTELEM_DOMESTIC_Year_2_Accepted": x,
    "term_4_UTELEM_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_UTELEM_DOMESTIC_Year_2_Denied": x,
    "term_4_UTELEM_DOMESTIC_Year_2_Declined": x,
    "term_4_UTELEM_DOMESTIC_Year_2_Pending": x,
    "term_4_UTELEM_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_UTELEM_DOMESTIC_Year_2_Enrolled": x,
    "term_4_UTELEM_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_UTELEM_INTERNATIONAL_Year_2_Applications": x,
    "term_4_UTELEM_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_UTELEM_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_UTELEM_INTERNATIONAL_Year_2_Denied": x,
    "term_4_UTELEM_INTERNATIONAL_Year_2_Declined": x,
    "term_4_UTELEM_INTERNATIONAL_Year_2_Pending": x,
    "term_4_UTELEM_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_UTELEM_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_UTELEM_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_UTSEC_DOMESTIC_Year_1_Applications": x,
    "term_4_UTSEC_DOMESTIC_Year_1_Accepted": x,
    "term_4_UTSEC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_UTSEC_DOMESTIC_Year_1_Denied": x,
    "term_4_UTSEC_DOMESTIC_Year_1_Declined": x,
    "term_4_UTSEC_DOMESTIC_Year_1_Pending": x,
    "term_4_UTSEC_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_UTSEC_DOMESTIC_Year_1_Enrolled": x,
    "term_4_UTSEC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_UTSEC_INTERNATIONAL_Year_1_Applications": x,
    "term_4_UTSEC_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_UTSEC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_UTSEC_INTERNATIONAL_Year_1_Denied": x,
    "term_4_UTSEC_INTERNATIONAL_Year_1_Declined": x,
    "term_4_UTSEC_INTERNATIONAL_Year_1_Pending": x,
    "term_4_UTSEC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_UTSEC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_UTSEC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_UTSEC_DOMESTIC_Year_2_Applications": x,
    "term_4_UTSEC_DOMESTIC_Year_2_Accepted": x,
    "term_4_UTSEC_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_UTSEC_DOMESTIC_Year_2_Denied": x,
    "term_4_UTSEC_DOMESTIC_Year_2_Declined": x,
    "term_4_UTSEC_DOMESTIC_Year_2_Pending": x,
    "term_4_UTSEC_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_UTSEC_DOMESTIC_Year_2_Enrolled": x,
    "term_4_UTSEC_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_UTSEC_INTERNATIONAL_Year_2_Applications": x,
    "term_4_UTSEC_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_UTSEC_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_UTSEC_INTERNATIONAL_Year_2_Denied": x,
    "term_4_UTSEC_INTERNATIONAL_Year_2_Declined": x,
    "term_4_UTSEC_INTERNATIONAL_Year_2_Pending": x,
    "term_4_UTSEC_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_UTSEC_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_UTSEC_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_ENVTD_DOMESTIC_Year_1_Applications": x,
    "term_4_ENVTD_DOMESTIC_Year_1_Accepted": x,
    "term_4_ENVTD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_ENVTD_DOMESTIC_Year_1_Denied": x,
    "term_4_ENVTD_DOMESTIC_Year_1_Declined": x,
    "term_4_ENVTD_DOMESTIC_Year_1_Pending": x,
    "term_4_ENVTD_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_ENVTD_DOMESTIC_Year_1_Enrolled": x,
    "term_4_ENVTD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_ENVTD_INTERNATIONAL_Year_1_Applications": x,
    "term_4_ENVTD_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_ENVTD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_ENVTD_INTERNATIONAL_Year_1_Denied": x,
    "term_4_ENVTD_INTERNATIONAL_Year_1_Declined": x,
    "term_4_ENVTD_INTERNATIONAL_Year_1_Pending": x,
    "term_4_ENVTD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_ENVTD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_ENVTD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_ENVTD_DOMESTIC_Year_2_Applications": x,
    "term_4_ENVTD_DOMESTIC_Year_2_Accepted": x,
    "term_4_ENVTD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_ENVTD_DOMESTIC_Year_2_Denied": x,
    "term_4_ENVTD_DOMESTIC_Year_2_Declined": x,
    "term_4_ENVTD_DOMESTIC_Year_2_Pending": x,
    "term_4_ENVTD_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_ENVTD_DOMESTIC_Year_2_Enrolled": x,
    "term_4_ENVTD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_ENVTD_INTERNATIONAL_Year_2_Applications": x,
    "term_4_ENVTD_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_ENVTD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_ENVTD_INTERNATIONAL_Year_2_Denied": x,
    "term_4_ENVTD_INTERNATIONAL_Year_2_Declined": x,
    "term_4_ENVTD_INTERNATIONAL_Year_2_Pending": x,
    "term_4_ENVTD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_ENVTD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_ENVTD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_ENVNC_DOMESTIC_Year_1_Applications": x,
    "term_4_ENVNC_DOMESTIC_Year_1_Accepted": x,
    "term_4_ENVNC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_ENVNC_DOMESTIC_Year_1_Denied": x,
    "term_4_ENVNC_DOMESTIC_Year_1_Declined": x,
    "term_4_ENVNC_DOMESTIC_Year_1_Pending": x,
    "term_4_ENVNC_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_ENVNC_DOMESTIC_Year_1_Enrolled": x,
    "term_4_ENVNC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_ENVNC_INTERNATIONAL_Year_1_Applications": x,
    "term_4_ENVNC_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_ENVNC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_ENVNC_INTERNATIONAL_Year_1_Denied": x,
    "term_4_ENVNC_INTERNATIONAL_Year_1_Declined": x,
    "term_4_ENVNC_INTERNATIONAL_Year_1_Pending": x,
    "term_4_ENVNC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_ENVNC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_ENVNC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_ENVNC_DOMESTIC_Year_2_Applications": x,
    "term_4_ENVNC_DOMESTIC_Year_2_Accepted": x,
    "term_4_ENVNC_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_ENVNC_DOMESTIC_Year_2_Denied": x,
    "term_4_ENVNC_DOMESTIC_Year_2_Declined": x,
    "term_4_ENVNC_DOMESTIC_Year_2_Pending": x,
    "term_4_ENVNC_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_ENVNC_DOMESTIC_Year_2_Enrolled": x,
    "term_4_ENVNC_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_ENVNC_INTERNATIONAL_Year_2_Applications": x,
    "term_4_ENVNC_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_ENVNC_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_ENVNC_INTERNATIONAL_Year_2_Denied": x,
    "term_4_ENVNC_INTERNATIONAL_Year_2_Declined": x,
    "term_4_ENVNC_INTERNATIONAL_Year_2_Pending": x,
    "term_4_ENVNC_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_ENVNC_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_ENVNC_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_UTARTS_DOMESTIC_Year_1_Applications": x,
    "term_4_UTARTS_DOMESTIC_Year_1_Accepted": x,
    "term_4_UTARTS_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_UTARTS_DOMESTIC_Year_1_Denied": x,
    "term_4_UTARTS_DOMESTIC_Year_1_Declined": x,
    "term_4_UTARTS_DOMESTIC_Year_1_Pending": x,
    "term_4_UTARTS_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_UTARTS_DOMESTIC_Year_1_Enrolled": x,
    "term_4_UTARTS_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_UTARTS_INTERNATIONAL_Year_1_Applications": x,
    "term_4_UTARTS_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_UTARTS_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_UTARTS_INTERNATIONAL_Year_1_Denied": x,
    "term_4_UTARTS_INTERNATIONAL_Year_1_Declined": x,
    "term_4_UTARTS_INTERNATIONAL_Year_1_Pending": x,
    "term_4_UTARTS_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_UTARTS_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_UTARTS_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_UTARTS_DOMESTIC_Year_2_Applications": x,
    "term_4_UTARTS_DOMESTIC_Year_2_Accepted": x,
    "term_4_UTARTS_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_UTARTS_DOMESTIC_Year_2_Denied": x,
    "term_4_UTARTS_DOMESTIC_Year_2_Declined": x,
    "term_4_UTARTS_DOMESTIC_Year_2_Pending": x,
    "term_4_UTARTS_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_UTARTS_DOMESTIC_Year_2_Enrolled": x,
    "term_4_UTARTS_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_UTARTS_INTERNATIONAL_Year_2_Applications": x,
    "term_4_UTARTS_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_UTARTS_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_UTARTS_INTERNATIONAL_Year_2_Denied": x,
    "term_4_UTARTS_INTERNATIONAL_Year_2_Declined": x,
    "term_4_UTARTS_INTERNATIONAL_Year_2_Pending": x,
    "term_4_UTARTS_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_UTARTS_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_UTARTS_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_UTCOMM_DOMESTIC_Year_1_Applications": x,
    "term_4_UTCOMM_DOMESTIC_Year_1_Accepted": x,
    "term_4_UTCOMM_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_UTCOMM_DOMESTIC_Year_1_Denied": x,
    "term_4_UTCOMM_DOMESTIC_Year_1_Declined": x,
    "term_4_UTCOMM_DOMESTIC_Year_1_Pending": x,
    "term_4_UTCOMM_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_UTCOMM_DOMESTIC_Year_1_Enrolled": x,
    "term_4_UTCOMM_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_UTCOMM_INTERNATIONAL_Year_1_Applications": x,
    "term_4_UTCOMM_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_UTCOMM_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_UTCOMM_INTERNATIONAL_Year_1_Denied": x,
    "term_4_UTCOMM_INTERNATIONAL_Year_1_Declined": x,
    "term_4_UTCOMM_INTERNATIONAL_Year_1_Pending": x,
    "term_4_UTCOMM_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_UTCOMM_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_UTCOMM_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_UTCOMM_DOMESTIC_Year_2_Applications": x,
    "term_4_UTCOMM_DOMESTIC_Year_2_Accepted": x,
    "term_4_UTCOMM_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_UTCOMM_DOMESTIC_Year_2_Denied": x,
    "term_4_UTCOMM_DOMESTIC_Year_2_Declined": x,
    "term_4_UTCOMM_DOMESTIC_Year_2_Pending": x,
    "term_4_UTCOMM_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_UTCOMM_DOMESTIC_Year_2_Enrolled": x,
    "term_4_UTCOMM_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_UTCOMM_INTERNATIONAL_Year_2_Applications": x,
    "term_4_UTCOMM_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_UTCOMM_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_UTCOMM_INTERNATIONAL_Year_2_Denied": x,
    "term_4_UTCOMM_INTERNATIONAL_Year_2_Declined": x,
    "term_4_UTCOMM_INTERNATIONAL_Year_2_Pending": x,
    "term_4_UTCOMM_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_UTCOMM_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_UTCOMM_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_UTENG_DOMESTIC_Year_1_Applications": x,
    "term_4_UTENG_DOMESTIC_Year_1_Accepted": x,
    "term_4_UTENG_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_UTENG_DOMESTIC_Year_1_Denied": x,
    "term_4_UTENG_DOMESTIC_Year_1_Declined": x,
    "term_4_UTENG_DOMESTIC_Year_1_Pending": x,
    "term_4_UTENG_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_UTENG_DOMESTIC_Year_1_Enrolled": x,
    "term_4_UTENG_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_UTENG_INTERNATIONAL_Year_1_Applications": x,
    "term_4_UTENG_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_UTENG_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_UTENG_INTERNATIONAL_Year_1_Denied": x,
    "term_4_UTENG_INTERNATIONAL_Year_1_Declined": x,
    "term_4_UTENG_INTERNATIONAL_Year_1_Pending": x,
    "term_4_UTENG_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_UTENG_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_UTENG_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_UTSCGS_DOMESTIC_Year_1_Applications": x,
    "term_4_UTSCGS_DOMESTIC_Year_1_Accepted": x,
    "term_4_UTSCGS_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_UTSCGS_DOMESTIC_Year_1_Denied": x,
    "term_4_UTSCGS_DOMESTIC_Year_1_Declined": x,
    "term_4_UTSCGS_DOMESTIC_Year_1_Pending": x,
    "term_4_UTSCGS_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_UTSCGS_DOMESTIC_Year_1_Enrolled": x,
    "term_4_UTSCGS_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_UTSCGS_INTERNATIONAL_Year_1_Applications": x,
    "term_4_UTSCGS_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_UTSCGS_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_UTSCGS_INTERNATIONAL_Year_1_Denied": x,
    "term_4_UTSCGS_INTERNATIONAL_Year_1_Declined": x,
    "term_4_UTSCGS_INTERNATIONAL_Year_1_Pending": x,
    "term_4_UTSCGS_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_UTSCGS_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_UTSCGS_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_UTSCGS_DOMESTIC_Year_2_Applications": x,
    "term_4_UTSCGS_DOMESTIC_Year_2_Accepted": x,
    "term_4_UTSCGS_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_UTSCGS_DOMESTIC_Year_2_Denied": x,
    "term_4_UTSCGS_DOMESTIC_Year_2_Declined": x,
    "term_4_UTSCGS_DOMESTIC_Year_2_Pending": x,
    "term_4_UTSCGS_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_UTSCGS_DOMESTIC_Year_2_Enrolled": x,
    "term_4_UTSCGS_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_UTSCGS_INTERNATIONAL_Year_2_Applications": x,
    "term_4_UTSCGS_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_UTSCGS_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_UTSCGS_INTERNATIONAL_Year_2_Denied": x,
    "term_4_UTSCGS_INTERNATIONAL_Year_2_Declined": x,
    "term_4_UTSCGS_INTERNATIONAL_Year_2_Pending": x,
    "term_4_UTSCGS_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_UTSCGS_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_UTSCGS_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_UTSCWK_DOMESTIC_Year_1_Applications": x,
    "term_4_UTSCWK_DOMESTIC_Year_1_Accepted": x,
    "term_4_UTSCWK_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_UTSCWK_DOMESTIC_Year_1_Denied": x,
    "term_4_UTSCWK_DOMESTIC_Year_1_Declined": x,
    "term_4_UTSCWK_DOMESTIC_Year_1_Pending": x,
    "term_4_UTSCWK_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_UTSCWK_DOMESTIC_Year_1_Enrolled": x,
    "term_4_UTSCWK_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_UTSCWK_INTERNATIONAL_Year_1_Applications": x,
    "term_4_UTSCWK_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_UTSCWK_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_UTSCWK_INTERNATIONAL_Year_1_Denied": x,
    "term_4_UTSCWK_INTERNATIONAL_Year_1_Declined": x,
    "term_4_UTSCWK_INTERNATIONAL_Year_1_Pending": x,
    "term_4_UTSCWK_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_UTSCWK_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_UTSCWK_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_UTSCWK_DOMESTIC_Year_2_Applications": x,
    "term_4_UTSCWK_DOMESTIC_Year_2_Accepted": x,
    "term_4_UTSCWK_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_UTSCWK_DOMESTIC_Year_2_Denied": x,
    "term_4_UTSCWK_DOMESTIC_Year_2_Declined": x,
    "term_4_UTSCWK_DOMESTIC_Year_2_Pending": x,
    "term_4_UTSCWK_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_UTSCWK_DOMESTIC_Year_2_Enrolled": x,
    "term_4_UTSCWK_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_UTSCWK_INTERNATIONAL_Year_2_Applications": x,
    "term_4_UTSCWK_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_UTSCWK_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_UTSCWK_INTERNATIONAL_Year_2_Denied": x,
    "term_4_UTSCWK_INTERNATIONAL_Year_2_Declined": x,
    "term_4_UTSCWK_INTERNATIONAL_Year_2_Pending": x,
    "term_4_UTSCWK_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_UTSCWK_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_UTSCWK_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_OPEN_DOMESTIC_Year_1_Applications": x,
    "term_4_OPEN_DOMESTIC_Year_1_Accepted": x,
    "term_4_OPEN_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_OPEN_DOMESTIC_Year_1_Denied": x,
    "term_4_OPEN_DOMESTIC_Year_1_Declined": x,
    "term_4_OPEN_DOMESTIC_Year_1_Pending": x,
    "term_4_OPEN_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_OPEN_DOMESTIC_Year_1_Enrolled": x,
    "term_4_OPEN_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_OPEN_INTERNATIONAL_Year_1_Applications": x,
    "term_4_OPEN_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_OPEN_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_OPEN_INTERNATIONAL_Year_1_Denied": x,
    "term_4_OPEN_INTERNATIONAL_Year_1_Declined": x,
    "term_4_OPEN_INTERNATIONAL_Year_1_Pending": x,
    "term_4_OPEN_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_OPEN_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_OPEN_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_GOVNSC_DOMESTIC_Year_1_Applications": x,
    "term_4_GOVNSC_DOMESTIC_Year_1_Accepted": x,
    "term_4_GOVNSC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_GOVNSC_DOMESTIC_Year_1_Denied": x,
    "term_4_GOVNSC_DOMESTIC_Year_1_Declined": x,
    "term_4_GOVNSC_DOMESTIC_Year_1_Pending": x,
    "term_4_GOVNSC_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_GOVNSC_DOMESTIC_Year_1_Enrolled": x,
    "term_4_GOVNSC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_GOVNSC_INTERNATIONAL_Year_1_Applications": x,
    "term_4_GOVNSC_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_GOVNSC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_GOVNSC_INTERNATIONAL_Year_1_Denied": x,
    "term_4_GOVNSC_INTERNATIONAL_Year_1_Declined": x,
    "term_4_GOVNSC_INTERNATIONAL_Year_1_Pending": x,
    "term_4_GOVNSC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_GOVNSC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_GOVNSC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_GOVNSC_DOMESTIC_Year_2_Applications": x,
    "term_4_GOVNSC_DOMESTIC_Year_2_Accepted": x,
    "term_4_GOVNSC_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_GOVNSC_DOMESTIC_Year_2_Denied": x,
    "term_4_GOVNSC_DOMESTIC_Year_2_Declined": x,
    "term_4_GOVNSC_DOMESTIC_Year_2_Pending": x,
    "term_4_GOVNSC_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_GOVNSC_DOMESTIC_Year_2_Enrolled": x,
    "term_4_GOVNSC_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_GOVNSC_INTERNATIONAL_Year_2_Applications": x,
    "term_4_GOVNSC_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_GOVNSC_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_GOVNSC_INTERNATIONAL_Year_2_Denied": x,
    "term_4_GOVNSC_INTERNATIONAL_Year_2_Declined": x,
    "term_4_GOVNSC_INTERNATIONAL_Year_2_Pending": x,
    "term_4_GOVNSC_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_GOVNSC_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_GOVNSC_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_COMSCC_DOMESTIC_Year_1_Applications": x,
    "term_4_COMSCC_DOMESTIC_Year_1_Accepted": x,
    "term_4_COMSCC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_COMSCC_DOMESTIC_Year_1_Denied": x,
    "term_4_COMSCC_DOMESTIC_Year_1_Declined": x,
    "term_4_COMSCC_DOMESTIC_Year_1_Pending": x,
    "term_4_COMSCC_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_COMSCC_DOMESTIC_Year_1_Enrolled": x,
    "term_4_COMSCC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_COMSCC_INTERNATIONAL_Year_1_Applications": x,
    "term_4_COMSCC_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_COMSCC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_COMSCC_INTERNATIONAL_Year_1_Denied": x,
    "term_4_COMSCC_INTERNATIONAL_Year_1_Declined": x,
    "term_4_COMSCC_INTERNATIONAL_Year_1_Pending": x,
    "term_4_COMSCC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_COMSCC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_COMSCC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_COMSCD_DOMESTIC_Year_1_Applications": x,
    "term_4_COMSCD_DOMESTIC_Year_1_Accepted": x,
    "term_4_COMSCD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_COMSCD_DOMESTIC_Year_1_Denied": x,
    "term_4_COMSCD_DOMESTIC_Year_1_Declined": x,
    "term_4_COMSCD_DOMESTIC_Year_1_Pending": x,
    "term_4_COMSCD_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_COMSCD_DOMESTIC_Year_1_Enrolled": x,
    "term_4_COMSCD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_COMSCD_INTERNATIONAL_Year_1_Applications": x,
    "term_4_COMSCD_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_COMSCD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_COMSCD_INTERNATIONAL_Year_1_Denied": x,
    "term_4_COMSCD_INTERNATIONAL_Year_1_Declined": x,
    "term_4_COMSCD_INTERNATIONAL_Year_1_Pending": x,
    "term_4_COMSCD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_COMSCD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_COMSCD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_COMSCD_DOMESTIC_Year_2_Applications": x,
    "term_4_COMSCD_DOMESTIC_Year_2_Accepted": x,
    "term_4_COMSCD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_COMSCD_DOMESTIC_Year_2_Denied": x,
    "term_4_COMSCD_DOMESTIC_Year_2_Declined": x,
    "term_4_COMSCD_DOMESTIC_Year_2_Pending": x,
    "term_4_COMSCD_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_COMSCD_DOMESTIC_Year_2_Enrolled": x,
    "term_4_COMSCD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_COMSCD_INTERNATIONAL_Year_2_Applications": x,
    "term_4_COMSCD_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_COMSCD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_COMSCD_INTERNATIONAL_Year_2_Denied": x,
    "term_4_COMSCD_INTERNATIONAL_Year_2_Declined": x,
    "term_4_COMSCD_INTERNATIONAL_Year_2_Pending": x,
    "term_4_COMSCD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_COMSCD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_COMSCD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_GNARTC_DOMESTIC_Year_1_Applications": x,
    "term_4_GNARTC_DOMESTIC_Year_1_Accepted": x,
    "term_4_GNARTC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_GNARTC_DOMESTIC_Year_1_Denied": x,
    "term_4_GNARTC_DOMESTIC_Year_1_Declined": x,
    "term_4_GNARTC_DOMESTIC_Year_1_Pending": x,
    "term_4_GNARTC_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_GNARTC_DOMESTIC_Year_1_Enrolled": x,
    "term_4_GNARTC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_GNARTC_INTERNATIONAL_Year_1_Applications": x,
    "term_4_GNARTC_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_GNARTC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_GNARTC_INTERNATIONAL_Year_1_Denied": x,
    "term_4_GNARTC_INTERNATIONAL_Year_1_Declined": x,
    "term_4_GNARTC_INTERNATIONAL_Year_1_Pending": x,
    "term_4_GNARTC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_GNARTC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_GNARTC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_GNSCIC_DOMESTIC_Year_1_Applications": x,
    "term_4_GNSCIC_DOMESTIC_Year_1_Accepted": x,
    "term_4_GNSCIC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_GNSCIC_DOMESTIC_Year_1_Denied": x,
    "term_4_GNSCIC_DOMESTIC_Year_1_Declined": x,
    "term_4_GNSCIC_DOMESTIC_Year_1_Pending": x,
    "term_4_GNSCIC_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_GNSCIC_DOMESTIC_Year_1_Enrolled": x,
    "term_4_GNSCIC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_GNSCIC_INTERNATIONAL_Year_1_Applications": x,
    "term_4_GNSCIC_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_GNSCIC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_GNSCIC_INTERNATIONAL_Year_1_Denied": x,
    "term_4_GNSCIC_INTERNATIONAL_Year_1_Declined": x,
    "term_4_GNSCIC_INTERNATIONAL_Year_1_Pending": x,
    "term_4_GNSCIC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_GNSCIC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_GNSCIC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_ANCPDC_DOMESTIC_Year_1_Applications": x,
    "term_4_ANCPDC_DOMESTIC_Year_1_Accepted": x,
    "term_4_ANCPDC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_ANCPDC_DOMESTIC_Year_1_Denied": x,
    "term_4_ANCPDC_DOMESTIC_Year_1_Declined": x,
    "term_4_ANCPDC_DOMESTIC_Year_1_Pending": x,
    "term_4_ANCPDC_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_ANCPDC_DOMESTIC_Year_1_Enrolled": x,
    "term_4_ANCPDC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_ANCPDC_INTERNATIONAL_Year_1_Applications": x,
    "term_4_ANCPDC_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_ANCPDC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_ANCPDC_INTERNATIONAL_Year_1_Denied": x,
    "term_4_ANCPDC_INTERNATIONAL_Year_1_Declined": x,
    "term_4_ANCPDC_INTERNATIONAL_Year_1_Pending": x,
    "term_4_ANCPDC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_ANCPDC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_ANCPDC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_NSPDC_DOMESTIC_Year_1_Applications": x,
    "term_4_NSPDC_DOMESTIC_Year_1_Accepted": x,
    "term_4_NSPDC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_NSPDC_DOMESTIC_Year_1_Denied": x,
    "term_4_NSPDC_DOMESTIC_Year_1_Declined": x,
    "term_4_NSPDC_DOMESTIC_Year_1_Pending": x,
    "term_4_NSPDC_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_NSPDC_DOMESTIC_Year_1_Enrolled": x,
    "term_4_NSPDC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_NSPDC_INTERNATIONAL_Year_1_Applications": x,
    "term_4_NSPDC_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_NSPDC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_NSPDC_INTERNATIONAL_Year_1_Denied": x,
    "term_4_NSPDC_INTERNATIONAL_Year_1_Declined": x,
    "term_4_NSPDC_INTERNATIONAL_Year_1_Pending": x,
    "term_4_NSPDC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_NSPDC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_NSPDC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4__DOMESTIC_Year_1_Applications": x,
    "term_4__DOMESTIC_Year_1_Accepted": x,
    "term_4__DOMESTIC_Year_1_Payment_Pending": x,
    "term_4__DOMESTIC_Year_1_Denied": x,
    "term_4__DOMESTIC_Year_1_Declined": x,
    "term_4__DOMESTIC_Year_1_Pending": x,
    "term_4__DOMESTIC_Year_1_Waitlisted": x,
    "term_4__DOMESTIC_Year_1_Enrolled": x,
    "term_4__DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4__INTERNATIONAL_Year_1_Applications": x,
    "term_4__INTERNATIONAL_Year_1_Accepted": x,
    "term_4__INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4__INTERNATIONAL_Year_1_Denied": x,
    "term_4__INTERNATIONAL_Year_1_Declined": x,
    "term_4__INTERNATIONAL_Year_1_Pending": x,
    "term_4__INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4__INTERNATIONAL_Year_1_Enrolled": x,
    "term_4__INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_LINC13_DOMESTIC_Year_1_Applications": x,
    "term_4_LINC13_DOMESTIC_Year_1_Accepted": x,
    "term_4_LINC13_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_LINC13_DOMESTIC_Year_1_Denied": x,
    "term_4_LINC13_DOMESTIC_Year_1_Declined": x,
    "term_4_LINC13_DOMESTIC_Year_1_Pending": x,
    "term_4_LINC13_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_LINC13_DOMESTIC_Year_1_Enrolled": x,
    "term_4_LINC13_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_LINC13_INTERNATIONAL_Year_1_Applications": x,
    "term_4_LINC13_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_LINC13_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_LINC13_INTERNATIONAL_Year_1_Denied": x,
    "term_4_LINC13_INTERNATIONAL_Year_1_Declined": x,
    "term_4_LINC13_INTERNATIONAL_Year_1_Pending": x,
    "term_4_LINC13_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_LINC13_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_LINC13_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_AELCC_DOMESTIC_Year_1_Applications": x,
    "term_4_AELCC_DOMESTIC_Year_1_Accepted": x,
    "term_4_AELCC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_AELCC_DOMESTIC_Year_1_Denied": x,
    "term_4_AELCC_DOMESTIC_Year_1_Declined": x,
    "term_4_AELCC_DOMESTIC_Year_1_Pending": x,
    "term_4_AELCC_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_AELCC_DOMESTIC_Year_1_Enrolled": x,
    "term_4_AELCC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_AELCC_INTERNATIONAL_Year_1_Applications": x,
    "term_4_AELCC_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_AELCC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_AELCC_INTERNATIONAL_Year_1_Denied": x,
    "term_4_AELCC_INTERNATIONAL_Year_1_Declined": x,
    "term_4_AELCC_INTERNATIONAL_Year_1_Pending": x,
    "term_4_AELCC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_AELCC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_AELCC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_ECCERT_DOMESTIC_Year_1_Applications": x,
    "term_4_ECCERT_DOMESTIC_Year_1_Accepted": x,
    "term_4_ECCERT_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_ECCERT_DOMESTIC_Year_1_Denied": x,
    "term_4_ECCERT_DOMESTIC_Year_1_Declined": x,
    "term_4_ECCERT_DOMESTIC_Year_1_Pending": x,
    "term_4_ECCERT_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_ECCERT_DOMESTIC_Year_1_Enrolled": x,
    "term_4_ECCERT_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_ECCERT_INTERNATIONAL_Year_1_Applications": x,
    "term_4_ECCERT_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_ECCERT_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_ECCERT_INTERNATIONAL_Year_1_Denied": x,
    "term_4_ECCERT_INTERNATIONAL_Year_1_Declined": x,
    "term_4_ECCERT_INTERNATIONAL_Year_1_Pending": x,
    "term_4_ECCERT_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_ECCERT_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_ECCERT_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_ECDIP_DOMESTIC_Year_1_Applications": x,
    "term_4_ECDIP_DOMESTIC_Year_1_Accepted": x,
    "term_4_ECDIP_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_ECDIP_DOMESTIC_Year_1_Denied": x,
    "term_4_ECDIP_DOMESTIC_Year_1_Declined": x,
    "term_4_ECDIP_DOMESTIC_Year_1_Pending": x,
    "term_4_ECDIP_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_ECDIP_DOMESTIC_Year_1_Enrolled": x,
    "term_4_ECDIP_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_ECDIP_INTERNATIONAL_Year_1_Applications": x,
    "term_4_ECDIP_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_ECDIP_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_ECDIP_INTERNATIONAL_Year_1_Denied": x,
    "term_4_ECDIP_INTERNATIONAL_Year_1_Declined": x,
    "term_4_ECDIP_INTERNATIONAL_Year_1_Pending": x,
    "term_4_ECDIP_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_ECDIP_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_ECDIP_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_ECDIP_DOMESTIC_Year_2_Applications": x,
    "term_4_ECDIP_DOMESTIC_Year_2_Accepted": x,
    "term_4_ECDIP_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_ECDIP_DOMESTIC_Year_2_Denied": x,
    "term_4_ECDIP_DOMESTIC_Year_2_Declined": x,
    "term_4_ECDIP_DOMESTIC_Year_2_Pending": x,
    "term_4_ECDIP_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_ECDIP_DOMESTIC_Year_2_Enrolled": x,
    "term_4_ECDIP_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_ECDIP_INTERNATIONAL_Year_2_Applications": x,
    "term_4_ECDIP_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_ECDIP_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_ECDIP_INTERNATIONAL_Year_2_Denied": x,
    "term_4_ECDIP_INTERNATIONAL_Year_2_Declined": x,
    "term_4_ECDIP_INTERNATIONAL_Year_2_Pending": x,
    "term_4_ECDIP_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_ECDIP_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_ECDIP_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_CYCD_DOMESTIC_Year_1_Applications": x,
    "term_4_CYCD_DOMESTIC_Year_1_Accepted": x,
    "term_4_CYCD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_CYCD_DOMESTIC_Year_1_Denied": x,
    "term_4_CYCD_DOMESTIC_Year_1_Declined": x,
    "term_4_CYCD_DOMESTIC_Year_1_Pending": x,
    "term_4_CYCD_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_CYCD_DOMESTIC_Year_1_Enrolled": x,
    "term_4_CYCD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_CYCD_INTERNATIONAL_Year_1_Applications": x,
    "term_4_CYCD_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_CYCD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_CYCD_INTERNATIONAL_Year_1_Denied": x,
    "term_4_CYCD_INTERNATIONAL_Year_1_Declined": x,
    "term_4_CYCD_INTERNATIONAL_Year_1_Pending": x,
    "term_4_CYCD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_CYCD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_CYCD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_CYCD_DOMESTIC_Year_2_Applications": x,
    "term_4_CYCD_DOMESTIC_Year_2_Accepted": x,
    "term_4_CYCD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_CYCD_DOMESTIC_Year_2_Denied": x,
    "term_4_CYCD_DOMESTIC_Year_2_Declined": x,
    "term_4_CYCD_DOMESTIC_Year_2_Pending": x,
    "term_4_CYCD_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_CYCD_DOMESTIC_Year_2_Enrolled": x,
    "term_4_CYCD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_CYCD_INTERNATIONAL_Year_2_Applications": x,
    "term_4_CYCD_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_CYCD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_CYCD_INTERNATIONAL_Year_2_Denied": x,
    "term_4_CYCD_INTERNATIONAL_Year_2_Declined": x,
    "term_4_CYCD_INTERNATIONAL_Year_2_Pending": x,
    "term_4_CYCD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_CYCD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_CYCD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_ECEAC_DOMESTIC_Year_1_Applications": x,
    "term_4_ECEAC_DOMESTIC_Year_1_Accepted": x,
    "term_4_ECEAC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_ECEAC_DOMESTIC_Year_1_Denied": x,
    "term_4_ECEAC_DOMESTIC_Year_1_Declined": x,
    "term_4_ECEAC_DOMESTIC_Year_1_Pending": x,
    "term_4_ECEAC_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_ECEAC_DOMESTIC_Year_1_Enrolled": x,
    "term_4_ECEAC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_ECEAC_INTERNATIONAL_Year_1_Applications": x,
    "term_4_ECEAC_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_ECEAC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_ECEAC_INTERNATIONAL_Year_1_Denied": x,
    "term_4_ECEAC_INTERNATIONAL_Year_1_Declined": x,
    "term_4_ECEAC_INTERNATIONAL_Year_1_Pending": x,
    "term_4_ECEAC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_ECEAC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_ECEAC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_SOCWKD_DOMESTIC_Year_1_Applications": x,
    "term_4_SOCWKD_DOMESTIC_Year_1_Accepted": x,
    "term_4_SOCWKD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_SOCWKD_DOMESTIC_Year_1_Denied": x,
    "term_4_SOCWKD_DOMESTIC_Year_1_Declined": x,
    "term_4_SOCWKD_DOMESTIC_Year_1_Pending": x,
    "term_4_SOCWKD_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_SOCWKD_DOMESTIC_Year_1_Enrolled": x,
    "term_4_SOCWKD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_SOCWKD_INTERNATIONAL_Year_1_Applications": x,
    "term_4_SOCWKD_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_SOCWKD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_SOCWKD_INTERNATIONAL_Year_1_Denied": x,
    "term_4_SOCWKD_INTERNATIONAL_Year_1_Declined": x,
    "term_4_SOCWKD_INTERNATIONAL_Year_1_Pending": x,
    "term_4_SOCWKD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_SOCWKD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_SOCWKD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_SOCWKD_DOMESTIC_Year_2_Applications": x,
    "term_4_SOCWKD_DOMESTIC_Year_2_Accepted": x,
    "term_4_SOCWKD_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_SOCWKD_DOMESTIC_Year_2_Denied": x,
    "term_4_SOCWKD_DOMESTIC_Year_2_Declined": x,
    "term_4_SOCWKD_DOMESTIC_Year_2_Pending": x,
    "term_4_SOCWKD_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_SOCWKD_DOMESTIC_Year_2_Enrolled": x,
    "term_4_SOCWKD_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_SOCWKD_INTERNATIONAL_Year_2_Applications": x,
    "term_4_SOCWKD_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_SOCWKD_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_SOCWKD_INTERNATIONAL_Year_2_Denied": x,
    "term_4_SOCWKD_INTERNATIONAL_Year_2_Declined": x,
    "term_4_SOCWKD_INTERNATIONAL_Year_2_Pending": x,
    "term_4_SOCWKD_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_SOCWKD_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_SOCWKD_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_HCAC_DOMESTIC_Year_1_Applications": x,
    "term_4_HCAC_DOMESTIC_Year_1_Accepted": x,
    "term_4_HCAC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_HCAC_DOMESTIC_Year_1_Denied": x,
    "term_4_HCAC_DOMESTIC_Year_1_Declined": x,
    "term_4_HCAC_DOMESTIC_Year_1_Pending": x,
    "term_4_HCAC_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_HCAC_DOMESTIC_Year_1_Enrolled": x,
    "term_4_HCAC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_HCAC_INTERNATIONAL_Year_1_Applications": x,
    "term_4_HCAC_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_HCAC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_HCAC_INTERNATIONAL_Year_1_Denied": x,
    "term_4_HCAC_INTERNATIONAL_Year_1_Declined": x,
    "term_4_HCAC_INTERNATIONAL_Year_1_Pending": x,
    "term_4_HCAC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_HCAC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_HCAC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_PCPC_DOMESTIC_Year_1_Applications": x,
    "term_4_PCPC_DOMESTIC_Year_1_Accepted": x,
    "term_4_PCPC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_PCPC_DOMESTIC_Year_1_Denied": x,
    "term_4_PCPC_DOMESTIC_Year_1_Declined": x,
    "term_4_PCPC_DOMESTIC_Year_1_Pending": x,
    "term_4_PCPC_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_PCPC_DOMESTIC_Year_1_Enrolled": x,
    "term_4_PCPC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_PCPC_INTERNATIONAL_Year_1_Applications": x,
    "term_4_PCPC_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_PCPC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_PCPC_INTERNATIONAL_Year_1_Denied": x,
    "term_4_PCPC_INTERNATIONAL_Year_1_Declined": x,
    "term_4_PCPC_INTERNATIONAL_Year_1_Pending": x,
    "term_4_PCPC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_PCPC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_PCPC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_ACP_DOMESTIC_Year_1_Applications": x,
    "term_4_ACP_DOMESTIC_Year_1_Accepted": x,
    "term_4_ACP_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_ACP_DOMESTIC_Year_1_Denied": x,
    "term_4_ACP_DOMESTIC_Year_1_Declined": x,
    "term_4_ACP_DOMESTIC_Year_1_Pending": x,
    "term_4_ACP_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_ACP_DOMESTIC_Year_1_Enrolled": x,
    "term_4_ACP_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_ACP_INTERNATIONAL_Year_1_Applications": x,
    "term_4_ACP_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_ACP_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_ACP_INTERNATIONAL_Year_1_Denied": x,
    "term_4_ACP_INTERNATIONAL_Year_1_Declined": x,
    "term_4_ACP_INTERNATIONAL_Year_1_Pending": x,
    "term_4_ACP_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_ACP_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_ACP_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_ACP_DOMESTIC_Year_2_Applications": x,
    "term_4_ACP_DOMESTIC_Year_2_Accepted": x,
    "term_4_ACP_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_ACP_DOMESTIC_Year_2_Denied": x,
    "term_4_ACP_DOMESTIC_Year_2_Declined": x,
    "term_4_ACP_DOMESTIC_Year_2_Pending": x,
    "term_4_ACP_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_ACP_DOMESTIC_Year_2_Enrolled": x,
    "term_4_ACP_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_ACP_INTERNATIONAL_Year_2_Applications": x,
    "term_4_ACP_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_ACP_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_ACP_INTERNATIONAL_Year_2_Denied": x,
    "term_4_ACP_INTERNATIONAL_Year_2_Declined": x,
    "term_4_ACP_INTERNATIONAL_Year_2_Pending": x,
    "term_4_ACP_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_ACP_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_ACP_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_PNR_DOMESTIC_Year_1_Applications": x,
    "term_4_PNR_DOMESTIC_Year_1_Accepted": x,
    "term_4_PNR_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_PNR_DOMESTIC_Year_1_Denied": x,
    "term_4_PNR_DOMESTIC_Year_1_Declined": x,
    "term_4_PNR_DOMESTIC_Year_1_Pending": x,
    "term_4_PNR_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_PNR_DOMESTIC_Year_1_Enrolled": x,
    "term_4_PNR_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_PNR_INTERNATIONAL_Year_1_Applications": x,
    "term_4_PNR_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_PNR_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_PNR_INTERNATIONAL_Year_1_Denied": x,
    "term_4_PNR_INTERNATIONAL_Year_1_Declined": x,
    "term_4_PNR_INTERNATIONAL_Year_1_Pending": x,
    "term_4_PNR_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_PNR_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_PNR_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_PNR_DOMESTIC_Year_2_Applications": x,
    "term_4_PNR_DOMESTIC_Year_2_Accepted": x,
    "term_4_PNR_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_PNR_DOMESTIC_Year_2_Denied": x,
    "term_4_PNR_DOMESTIC_Year_2_Declined": x,
    "term_4_PNR_DOMESTIC_Year_2_Pending": x,
    "term_4_PNR_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_PNR_DOMESTIC_Year_2_Enrolled": x,
    "term_4_PNR_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_PNR_INTERNATIONAL_Year_2_Applications": x,
    "term_4_PNR_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_PNR_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_PNR_INTERNATIONAL_Year_2_Denied": x,
    "term_4_PNR_INTERNATIONAL_Year_2_Declined": x,
    "term_4_PNR_INTERNATIONAL_Year_2_Pending": x,
    "term_4_PNR_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_PNR_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_PNR_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_BSCN_DOMESTIC_Year_1_Applications": x,
    "term_4_BSCN_DOMESTIC_Year_1_Accepted": x,
    "term_4_BSCN_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_BSCN_DOMESTIC_Year_1_Denied": x,
    "term_4_BSCN_DOMESTIC_Year_1_Declined": x,
    "term_4_BSCN_DOMESTIC_Year_1_Pending": x,
    "term_4_BSCN_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_BSCN_DOMESTIC_Year_1_Enrolled": x,
    "term_4_BSCN_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_BSCN_INTERNATIONAL_Year_1_Applications": x,
    "term_4_BSCN_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_BSCN_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_BSCN_INTERNATIONAL_Year_1_Denied": x,
    "term_4_BSCN_INTERNATIONAL_Year_1_Declined": x,
    "term_4_BSCN_INTERNATIONAL_Year_1_Pending": x,
    "term_4_BSCN_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_BSCN_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_BSCN_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_BSCN_DOMESTIC_Year_2_Applications": x,
    "term_4_BSCN_DOMESTIC_Year_2_Accepted": x,
    "term_4_BSCN_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_BSCN_DOMESTIC_Year_2_Denied": x,
    "term_4_BSCN_DOMESTIC_Year_2_Declined": x,
    "term_4_BSCN_DOMESTIC_Year_2_Pending": x,
    "term_4_BSCN_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_BSCN_DOMESTIC_Year_2_Enrolled": x,
    "term_4_BSCN_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_BSCN_INTERNATIONAL_Year_2_Applications": x,
    "term_4_BSCN_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_BSCN_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_BSCN_INTERNATIONAL_Year_2_Denied": x,
    "term_4_BSCN_INTERNATIONAL_Year_2_Declined": x,
    "term_4_BSCN_INTERNATIONAL_Year_2_Pending": x,
    "term_4_BSCN_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_BSCN_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_BSCN_INTERNATIONAL_Year_2_Conversion_Rate": x,
    "term_4_BSCN_DOMESTIC_Year_3_Applications": x,
    "term_4_BSCN_DOMESTIC_Year_3_Accepted": x,
    "term_4_BSCN_DOMESTIC_Year_3_Payment_Pending": x,
    "term_4_BSCN_DOMESTIC_Year_3_Denied": x,
    "term_4_BSCN_DOMESTIC_Year_3_Declined": x,
    "term_4_BSCN_DOMESTIC_Year_3_Pending": x,
    "term_4_BSCN_DOMESTIC_Year_3_Waitlisted": x,
    "term_4_BSCN_DOMESTIC_Year_3_Enrolled": x,
    "term_4_BSCN_DOMESTIC_Year_3_Conversion_Rate": x,
    "term_4_BSCN_INTERNATIONAL_Year_3_Applications": x,
    "term_4_BSCN_INTERNATIONAL_Year_3_Accepted": x,
    "term_4_BSCN_INTERNATIONAL_Year_3_Payment_Pending": x,
    "term_4_BSCN_INTERNATIONAL_Year_3_Denied": x,
    "term_4_BSCN_INTERNATIONAL_Year_3_Declined": x,
    "term_4_BSCN_INTERNATIONAL_Year_3_Pending": x,
    "term_4_BSCN_INTERNATIONAL_Year_3_Waitlisted": x,
    "term_4_BSCN_INTERNATIONAL_Year_3_Enrolled": x,
    "term_4_BSCN_INTERNATIONAL_Year_3_Conversion_Rate": x,
    "term_4_IENCN_DOMESTIC_Year_1_Applications": x,
    "term_4_IENCN_DOMESTIC_Year_1_Accepted": x,
    "term_4_IENCN_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_IENCN_DOMESTIC_Year_1_Denied": x,
    "term_4_IENCN_DOMESTIC_Year_1_Declined": x,
    "term_4_IENCN_DOMESTIC_Year_1_Pending": x,
    "term_4_IENCN_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_IENCN_DOMESTIC_Year_1_Enrolled": x,
    "term_4_IENCN_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_IENCN_INTERNATIONAL_Year_1_Applications": x,
    "term_4_IENCN_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_IENCN_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_IENCN_INTERNATIONAL_Year_1_Denied": x,
    "term_4_IENCN_INTERNATIONAL_Year_1_Declined": x,
    "term_4_IENCN_INTERNATIONAL_Year_1_Pending": x,
    "term_4_IENCN_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_IENCN_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_IENCN_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_PAELEC_DOMESTIC_Year_1_Applications": x,
    "term_4_PAELEC_DOMESTIC_Year_1_Accepted": x,
    "term_4_PAELEC_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_PAELEC_DOMESTIC_Year_1_Denied": x,
    "term_4_PAELEC_DOMESTIC_Year_1_Declined": x,
    "term_4_PAELEC_DOMESTIC_Year_1_Pending": x,
    "term_4_PAELEC_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_PAELEC_DOMESTIC_Year_1_Enrolled": x,
    "term_4_PAELEC_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_PAELEC_INTERNATIONAL_Year_1_Applications": x,
    "term_4_PAELEC_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_PAELEC_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_PAELEC_INTERNATIONAL_Year_1_Denied": x,
    "term_4_PAELEC_INTERNATIONAL_Year_1_Declined": x,
    "term_4_PAELEC_INTERNATIONAL_Year_1_Pending": x,
    "term_4_PAELEC_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_PAELEC_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_PAELEC_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_PAWELD_DOMESTIC_Year_1_Applications": x,
    "term_4_PAWELD_DOMESTIC_Year_1_Accepted": x,
    "term_4_PAWELD_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_PAWELD_DOMESTIC_Year_1_Denied": x,
    "term_4_PAWELD_DOMESTIC_Year_1_Declined": x,
    "term_4_PAWELD_DOMESTIC_Year_1_Pending": x,
    "term_4_PAWELD_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_PAWELD_DOMESTIC_Year_1_Enrolled": x,
    "term_4_PAWELD_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_PAWELD_INTERNATIONAL_Year_1_Applications": x,
    "term_4_PAWELD_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_PAWELD_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_PAWELD_INTERNATIONAL_Year_1_Denied": x,
    "term_4_PAWELD_INTERNATIONAL_Year_1_Declined": x,
    "term_4_PAWELD_INTERNATIONAL_Year_1_Pending": x,
    "term_4_PAWELD_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_PAWELD_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_PAWELD_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_PAHET_DOMESTIC_Year_1_Applications": x,
    "term_4_PAHET_DOMESTIC_Year_1_Accepted": x,
    "term_4_PAHET_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_PAHET_DOMESTIC_Year_1_Denied": x,
    "term_4_PAHET_DOMESTIC_Year_1_Declined": x,
    "term_4_PAHET_DOMESTIC_Year_1_Pending": x,
    "term_4_PAHET_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_PAHET_DOMESTIC_Year_1_Enrolled": x,
    "term_4_PAHET_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_PAHET_INTERNATIONAL_Year_1_Applications": x,
    "term_4_PAHET_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_PAHET_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_PAHET_INTERNATIONAL_Year_1_Denied": x,
    "term_4_PAHET_INTERNATIONAL_Year_1_Declined": x,
    "term_4_PAHET_INTERNATIONAL_Year_1_Pending": x,
    "term_4_PAHET_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_PAHET_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_PAHET_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_PAPLUM_DOMESTIC_Year_1_Applications": x,
    "term_4_PAPLUM_DOMESTIC_Year_1_Accepted": x,
    "term_4_PAPLUM_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_PAPLUM_DOMESTIC_Year_1_Denied": x,
    "term_4_PAPLUM_DOMESTIC_Year_1_Declined": x,
    "term_4_PAPLUM_DOMESTIC_Year_1_Pending": x,
    "term_4_PAPLUM_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_PAPLUM_DOMESTIC_Year_1_Enrolled": x,
    "term_4_PAPLUM_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_PAPLUM_INTERNATIONAL_Year_1_Applications": x,
    "term_4_PAPLUM_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_PAPLUM_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_PAPLUM_INTERNATIONAL_Year_1_Denied": x,
    "term_4_PAPLUM_INTERNATIONAL_Year_1_Declined": x,
    "term_4_PAPLUM_INTERNATIONAL_Year_1_Pending": x,
    "term_4_PAPLUM_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_PAPLUM_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_PAPLUM_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_POWCM4_DOMESTIC_Year_1_Applications": x,
    "term_4_POWCM4_DOMESTIC_Year_1_Accepted": x,
    "term_4_POWCM4_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_POWCM4_DOMESTIC_Year_1_Denied": x,
    "term_4_POWCM4_DOMESTIC_Year_1_Declined": x,
    "term_4_POWCM4_DOMESTIC_Year_1_Pending": x,
    "term_4_POWCM4_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_POWCM4_DOMESTIC_Year_1_Enrolled": x,
    "term_4_POWCM4_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_POWCM4_INTERNATIONAL_Year_1_Applications": x,
    "term_4_POWCM4_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_POWCM4_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_POWCM4_INTERNATIONAL_Year_1_Denied": x,
    "term_4_POWCM4_INTERNATIONAL_Year_1_Declined": x,
    "term_4_POWCM4_INTERNATIONAL_Year_1_Pending": x,
    "term_4_POWCM4_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_POWCM4_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_POWCM4_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_POWCM3_DOMESTIC_Year_1_Applications": x,
    "term_4_POWCM3_DOMESTIC_Year_1_Accepted": x,
    "term_4_POWCM3_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_POWCM3_DOMESTIC_Year_1_Denied": x,
    "term_4_POWCM3_DOMESTIC_Year_1_Declined": x,
    "term_4_POWCM3_DOMESTIC_Year_1_Pending": x,
    "term_4_POWCM3_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_POWCM3_DOMESTIC_Year_1_Enrolled": x,
    "term_4_POWCM3_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_POWCM3_INTERNATIONAL_Year_1_Applications": x,
    "term_4_POWCM3_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_POWCM3_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_POWCM3_INTERNATIONAL_Year_1_Denied": x,
    "term_4_POWCM3_INTERNATIONAL_Year_1_Declined": x,
    "term_4_POWCM3_INTERNATIONAL_Year_1_Pending": x,
    "term_4_POWCM3_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_POWCM3_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_POWCM3_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_POWCP4_DOMESTIC_Year_1_Applications": x,
    "term_4_POWCP4_DOMESTIC_Year_1_Accepted": x,
    "term_4_POWCP4_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_POWCP4_DOMESTIC_Year_1_Denied": x,
    "term_4_POWCP4_DOMESTIC_Year_1_Declined": x,
    "term_4_POWCP4_DOMESTIC_Year_1_Pending": x,
    "term_4_POWCP4_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_POWCP4_DOMESTIC_Year_1_Enrolled": x,
    "term_4_POWCP4_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_POWCP4_INTERNATIONAL_Year_1_Applications": x,
    "term_4_POWCP4_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_POWCP4_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_POWCP4_INTERNATIONAL_Year_1_Denied": x,
    "term_4_POWCP4_INTERNATIONAL_Year_1_Declined": x,
    "term_4_POWCP4_INTERNATIONAL_Year_1_Pending": x,
    "term_4_POWCP4_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_POWCP4_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_POWCP4_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_POWCO4_DOMESTIC_Year_1_Applications": x,
    "term_4_POWCO4_DOMESTIC_Year_1_Accepted": x,
    "term_4_POWCO4_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_POWCO4_DOMESTIC_Year_1_Denied": x,
    "term_4_POWCO4_DOMESTIC_Year_1_Declined": x,
    "term_4_POWCO4_DOMESTIC_Year_1_Pending": x,
    "term_4_POWCO4_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_POWCO4_DOMESTIC_Year_1_Enrolled": x,
    "term_4_POWCO4_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_POWCO4_INTERNATIONAL_Year_1_Applications": x,
    "term_4_POWCO4_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_POWCO4_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_POWCO4_INTERNATIONAL_Year_1_Denied": x,
    "term_4_POWCO4_INTERNATIONAL_Year_1_Declined": x,
    "term_4_POWCO4_INTERNATIONAL_Year_1_Pending": x,
    "term_4_POWCO4_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_POWCO4_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_POWCO4_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_POWCO3_DOMESTIC_Year_1_Applications": x,
    "term_4_POWCO3_DOMESTIC_Year_1_Accepted": x,
    "term_4_POWCO3_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_POWCO3_DOMESTIC_Year_1_Denied": x,
    "term_4_POWCO3_DOMESTIC_Year_1_Declined": x,
    "term_4_POWCO3_DOMESTIC_Year_1_Pending": x,
    "term_4_POWCO3_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_POWCO3_DOMESTIC_Year_1_Enrolled": x,
    "term_4_POWCO3_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_POWCO3_INTERNATIONAL_Year_1_Applications": x,
    "term_4_POWCO3_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_POWCO3_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_POWCO3_INTERNATIONAL_Year_1_Denied": x,
    "term_4_POWCO3_INTERNATIONAL_Year_1_Declined": x,
    "term_4_POWCO3_INTERNATIONAL_Year_1_Pending": x,
    "term_4_POWCO3_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_POWCO3_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_POWCO3_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_PPET_DOMESTIC_Year_1_Applications": x,
    "term_4_PPET_DOMESTIC_Year_1_Accepted": x,
    "term_4_PPET_DOMESTIC_Year_1_Payment_Pending": x,
    "term_4_PPET_DOMESTIC_Year_1_Denied": x,
    "term_4_PPET_DOMESTIC_Year_1_Declined": x,
    "term_4_PPET_DOMESTIC_Year_1_Pending": x,
    "term_4_PPET_DOMESTIC_Year_1_Waitlisted": x,
    "term_4_PPET_DOMESTIC_Year_1_Enrolled": x,
    "term_4_PPET_DOMESTIC_Year_1_Conversion_Rate": x,
    "term_4_PPET_INTERNATIONAL_Year_1_Applications": x,
    "term_4_PPET_INTERNATIONAL_Year_1_Accepted": x,
    "term_4_PPET_INTERNATIONAL_Year_1_Payment_Pending": x,
    "term_4_PPET_INTERNATIONAL_Year_1_Denied": x,
    "term_4_PPET_INTERNATIONAL_Year_1_Declined": x,
    "term_4_PPET_INTERNATIONAL_Year_1_Pending": x,
    "term_4_PPET_INTERNATIONAL_Year_1_Waitlisted": x,
    "term_4_PPET_INTERNATIONAL_Year_1_Enrolled": x,
    "term_4_PPET_INTERNATIONAL_Year_1_Conversion_Rate": x,
    "term_4_PPET_DOMESTIC_Year_2_Applications": x,
    "term_4_PPET_DOMESTIC_Year_2_Accepted": x,
    "term_4_PPET_DOMESTIC_Year_2_Payment_Pending": x,
    "term_4_PPET_DOMESTIC_Year_2_Denied": x,
    "term_4_PPET_DOMESTIC_Year_2_Declined": x,
    "term_4_PPET_DOMESTIC_Year_2_Pending": x,
    "term_4_PPET_DOMESTIC_Year_2_Waitlisted": x,
    "term_4_PPET_DOMESTIC_Year_2_Enrolled": x,
    "term_4_PPET_DOMESTIC_Year_2_Conversion_Rate": x,
    "term_4_PPET_INTERNATIONAL_Year_2_Applications": x,
    "term_4_PPET_INTERNATIONAL_Year_2_Accepted": x,
    "term_4_PPET_INTERNATIONAL_Year_2_Payment_Pending": x,
    "term_4_PPET_INTERNATIONAL_Year_2_Denied": x,
    "term_4_PPET_INTERNATIONAL_Year_2_Declined": x,
    "term_4_PPET_INTERNATIONAL_Year_2_Pending": x,
    "term_4_PPET_INTERNATIONAL_Year_2_Waitlisted": x,
    "term_4_PPET_INTERNATIONAL_Year_2_Enrolled": x,
    "term_4_PPET_INTERNATIONAL_Year_2_Conversion_Rate": x,
}

# Regular expression to detect {{variable}} patterns
pattern = re.compile(r"{{(.*?)}}")

# Loop through all worksheets and replace variables
for ws in wb.worksheets:
    for row in ws.iter_rows():
        for cell in row:
            if isinstance(cell.value, str):
                matches = pattern.findall(cell.value)
                for match in matches:
                    if match in variable_map:
                        replacement = variable_map[match]
                        if cell.value.strip() == f"{{{{{match}}}}}":  # Entire cell is just the variable
                            cell.value = replacement  # Assign number directly if numeric
                        else:
                            cell.value = cell.value.replace(f"{{{{{match}}}}}", str(replacement))

# Rename sheets by replacing placeholders in their names
for ws in wb.worksheets:
    matches = pattern.findall(ws.title)
    new_title = ws.title
    for match in matches:
        if match in variable_map:
            new_title = new_title.replace(f"{{{{{match}}}}}", str(variable_map[match]))
    if new_title != ws.title:
        ws.title = new_title

# Convert text to numbers in column P for specific sheets
target_sheets = [f"Summer {term_1_year_current} Apps By School", f"Fall {term_2_year_current} Apps By School", f"Winter {term_3_year_current} Apps By School", f"Spring {term_4_year_current} Apps By School"]
column_p_index = openpyxl.utils.column_index_from_string('P')

for sheet_name in target_sheets:
    if sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        for row in ws.iter_rows(min_col=column_p_index, max_col=column_p_index):
            cell = row[0]
            if isinstance(cell.value, str) and cell.value.strip().replace(',', '').replace('.', '', 1).isdigit():
                try:
                    cell.value = float(cell.value.replace(',', '')) if '.' in cell.value else int(cell.value.replace(',', ''))
                except ValueError:
                    pass  # Skip if conversion fails

# Save to new file
output_filename = f"KC {variable_Current_applications_academic_year} Applications KPI as of {variable_Extraction_date}.xlsx"
wb.save(output_filename)
